In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# I. Importing the required libraries"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 106,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "        <script type=\"text/javascript\">\n",
       "        window.PlotlyConfig = {MathJaxConfig: 'local'};\n",
       "        if (window.MathJax && window.MathJax.Hub && window.MathJax.Hub.Config) {window.MathJax.Hub.Config({SVG: {font: \"STIX-Web\"}});}\n",
       "        if (typeof require !== 'undefined') {\n",
       "        require.undef(\"plotly\");\n",
       "        requirejs.config({\n",
       "            paths: {\n",
       "                'plotly': ['https://cdn.plot.ly/plotly-2.12.1.min']\n",
       "            }\n",
       "        });\n",
       "        require(['plotly'], function(Plotly) {\n",
       "            window._Plotly = Plotly;\n",
       "        });\n",
       "        }\n",
       "        </script>\n",
       "        "
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "## Python\n",
    "import os\n",
    "import random\n",
    "import sys\n",
    "\n",
    "## Package\n",
    "import glob \n",
    "import keras\n",
    "import IPython.display as ipd\n",
    "import librosa\n",
    "import librosa.display\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import plotly.graph_objs as go\n",
    "import plotly.offline as py\n",
    "import plotly.tools as tls\n",
    "import seaborn as sns\n",
    "import scipy.io.wavfile\n",
    "import tensorflow as tf\n",
    "import pickle\n",
    "py.init_notebook_mode(connected=True)\n",
    "\n",
    "\n",
    "## Keras\n",
    "from keras import regularizers\n",
    "from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping\n",
    "from keras.callbacks import  History, ReduceLROnPlateau, CSVLogger\n",
    "from keras.models import Model, Sequential\n",
    "from keras.layers import Dense, Embedding, LSTM\n",
    "from keras.layers import Input, Flatten, Dropout, Activation, BatchNormalization\n",
    "from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D\n",
    "from keras.preprocessing import sequence\n",
    "from keras.preprocessing.sequence import pad_sequences\n",
    "from keras.preprocessing.text import Tokenizer\n",
    "from keras.utils import np_utils\n",
    "from tensorflow.keras.utils import to_categorical\n",
    "from tensorflow.keras.optimizers import SGD\n",
    "\n",
    "## Sklearn\n",
    "from sklearn.model_selection import StratifiedShuffleSplit\n",
    "from sklearn.metrics import confusion_matrix\n",
    "from sklearn.preprocessing import LabelEncoder\n",
    "\n",
    "\n",
    "## Rest\n",
    "from scipy.fftpack import fft\n",
    "from scipy import signal\n",
    "from scipy.io import wavfile\n",
    "from tqdm import tqdm\n",
    "\n",
    "# to read audio file  \n",
    "import soundfile        \n",
    "\n",
    "input_duration=3\n",
    "# % pylab inline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 107,
   "metadata": {},
   "outputs": [],
   "source": [
    "# II. Reading the data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 108,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']\n"
     ]
    }
   ],
   "source": [
    "# Data Directory\n",
    "# Please edit according to your directory change.\n",
    "dir_list = os.listdir(\"D:\\\\ravdess data\\\\\")\n",
    "dir_list.sort()\n",
    "print (dir_list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 109,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create DataFrame for Data intel\n",
    "data_df = pd.DataFrame(columns=['path', 'source', 'actor', 'gender',\n",
    "                                'intensity', 'statement', 'repetition', 'emotion'])\n",
    "count = 0\n",
    "for i in dir_list:\n",
    "    file_list = os.listdir(\"D://ravdess data//\" + i)\n",
    "    for f in file_list:\n",
    "        nm = f.split('.')[0].split('-')\n",
    "        path = \"D://ravdess data//\" + i + '/' + f\n",
    "        src = int(nm[1])\n",
    "        actor = int(nm[-1])\n",
    "        emotion = int(nm[2])\n",
    "        \n",
    "        if int(actor)%2 == 0:\n",
    "            gender = \"female\"\n",
    "        else:\n",
    "            gender = \"male\"\n",
    "        \n",
    "        if nm[3] == '01':\n",
    "            intensity = 0\n",
    "        else:\n",
    "            intensity = 1\n",
    "        \n",
    "        if nm[4] == '01':\n",
    "            statement = 0\n",
    "        else:\n",
    "            statement = 1\n",
    "        \n",
    "        if nm[5] == '01':\n",
    "            repeat = 0\n",
    "        else:\n",
    "            repeat = 1\n",
    "            \n",
    "        data_df.loc[count] = [path, src, actor, gender, intensity, statement, repeat, emotion]\n",
    "        count += 1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 110,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "1438\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>path</th>\n",
       "      <th>source</th>\n",
       "      <th>actor</th>\n",
       "      <th>gender</th>\n",
       "      <th>intensity</th>\n",
       "      <th>statement</th>\n",
       "      <th>repetition</th>\n",
       "      <th>emotion</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-01-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-01-02-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-02-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-02-02-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-02-01-01-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                                                path  source  actor gender  \\\n",
       "0  D://ravdess data//Actor_01/03-01-01-01-01-01-0...       1      1   male   \n",
       "1  D://ravdess data//Actor_01/03-01-01-01-01-02-0...       1      1   male   \n",
       "2  D://ravdess data//Actor_01/03-01-01-01-02-01-0...       1      1   male   \n",
       "3  D://ravdess data//Actor_01/03-01-01-01-02-02-0...       1      1   male   \n",
       "4  D://ravdess data//Actor_01/03-01-02-01-01-01-0...       1      1   male   \n",
       "\n",
       "   intensity  statement  repetition  emotion  \n",
       "0          0          0           0        1  \n",
       "1          0          0           1        1  \n",
       "2          0          1           0        1  \n",
       "3          0          1           1        1  \n",
       "4          0          0           0        2  "
      ]
     },
     "execution_count": 110,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "print (len(data_df))\n",
    "data_df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# III. Plotting the audio file's waveform and its spectrogram"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 111,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "D://ravdess data//Actor_01/03-01-01-01-01-01-01.wav\n"
     ]
    }
   ],
   "source": [
    "print(data_df.path[0])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 112,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "D://ravdess data//Actor_17/03-01-07-01-01-01-17.wav\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "(22050,\n",
       " array([-6.0857743e-05, -6.8510170e-05, -5.2941483e-05, ...,\n",
       "         2.5669031e-05,  3.8714046e-05,  0.0000000e+00], dtype=float32))"
      ]
     },
     "execution_count": 112,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "filename = data_df.path[1002]\n",
    "print (filename)\n",
    "\n",
    "samples, sample_rate = librosa.load(filename)\n",
    "sample_rate, samples"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 113,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(84610, 22050)"
      ]
     },
     "execution_count": 113,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(samples), sample_rate"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 114,
   "metadata": {},
   "outputs": [],
   "source": [
    "def log_specgram(audio, sample_rate, window_size=20,\n",
    "                 step_size=10, eps=1e-10):\n",
    "    nperseg = int(round(window_size * sample_rate / 1e3))\n",
    "    noverlap = int(round(step_size * sample_rate / 1e3))\n",
    "    freqs, times, spec = signal.spectrogram(audio,\n",
    "                                    fs=sample_rate,\n",
    "                                    window='hann',\n",
    "                                    nperseg=nperseg,\n",
    "                                    noverlap=noverlap,\n",
    "                                    detrend=False)\n",
    "    return freqs, times, np.log(spec.T.astype(np.float32) + eps)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 115,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0.26060749320411297"
      ]
     },
     "execution_count": 115,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "sample_rate/ len(samples)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 116,
   "metadata": {
    "scrolled": False
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "Text(0.5, 0, 'Seconds')"
      ]
     },
     "execution_count": 116,
     "metadata": {},
     "output_type": "execute_result"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1008x576 with 2 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Plotting Wave Form and Spectrogram\n",
    "freqs, times, spectrogram = log_specgram(samples, sample_rate)\n",
    "\n",
    "fig = plt.figure(figsize=(14, 8))\n",
    "ax1 = fig.add_subplot(211)\n",
    "ax1.set_title('Raw wave of ' + filename)\n",
    "ax1.set_ylabel('Amplitude')\n",
    "librosa.display.waveshow(samples, sr=sample_rate)\n",
    "\n",
    "ax2 = fig.add_subplot(212)\n",
    "ax2.imshow(spectrogram.T, aspect='auto', origin='lower', \n",
    "           extent=[times.min(), times.max(), freqs.min(), freqs.max()])\n",
    "ax2.set_yticks(freqs[::16])\n",
    "ax2.set_xticks(times[::16])\n",
    "ax2.set_title('Spectrogram of ' + filename)\n",
    "ax2.set_ylabel('Freqs in Hz')\n",
    "ax2.set_xlabel('Seconds')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 117,
   "metadata": {},
   "outputs": [],
   "source": [
    "mean = np.mean(spectrogram, axis=0)\n",
    "std = np.std(spectrogram, axis=0)\n",
    "spectrogram = (spectrogram - mean) / std"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 118,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(array([ 0.00070844, -0.00016367, -0.00107828, ...,  0.00064844,\n",
       "         0.00094032,  0.0009506 ], dtype=float32),\n",
       " array([22528, 58880]))"
      ]
     },
     "execution_count": 118,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Trim the silence voice\n",
    "aa , bb = librosa.effects.trim(samples, top_db=30)\n",
    "aa, bb"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 119,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 864x288 with 2 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Plotting Mel Power Spectrogram\n",
    "S = librosa.feature.melspectrogram(aa, sr=sample_rate, n_mels=128)\n",
    "\n",
    "# Convert to log scale (dB). We'll use the peak power (max) as reference.\n",
    "log_S = librosa.power_to_db(S, ref=np.max)\n",
    "\n",
    "plt.figure(figsize=(12, 4))\n",
    "librosa.display.specshow(log_S, sr=sample_rate, x_axis='time', y_axis='mel')\n",
    "plt.title('Mel power spectrogram ')\n",
    "plt.colorbar(format='%+02.0f dB')\n",
    "plt.tight_layout()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 120,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "iVBORw0KGgoAAAANSUhEUgAAAwQAAAEYCAYAAADicDErAAAAOXRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjUuMiwgaHR0cHM6Ly9tYXRwbG90bGliLm9yZy8qNh9FAAAACXBIWXMAAAsTAAALEwEAmpwYAAAikElEQVR4nO3de5AlZ3nf8d9zbnPmvtqLVpfVagUIYS7lMhLGMqkKNggwuCyMAYMDoQiVSqVIgNihQhxCIGVShGCXwTHBChAE2GAgEJQAIkjhGmMiycEIQYRkpEUSu1r2OjuXc+t+8sc5C2Oxe553dnpmzpn+fqpO7cyZ33a/3f32e+adt99uc3cBAAAAKKfKVhcAAAAAwNahQwAAAACUGB0CAAAAoMToEAAAAAAlRocAAAAAKDE6BAAAAECJ0SEAAAAASowOAQAUwMzuN7OOme1+xPv/18zczA6Y2QcGmcVVr99clf0tM7t98P4hM/ucmf2dVT9/rJl93MyOmtkpM/uWmf22mVU3c1sBANsLHQIAKM59kl565hsze5KkqUdk3u7uM6tefz7I/rakP5T07yTtlbRf0rslXT/4+aMlfUPSA5Ke5O7zkl4k6RpJsxu5UQCA7c14UjEArJ+Z3S/pvZKud/enDN57h6QTkn5P0hWS3izpQXd/4yP+77ykhyS90t0/fo7lf1jSBe7+vI3aBgBAOTFCAADF+UtJc2b2M4PLeF4i6cMJ/+9aSU1JnxqSeaakT6y/iAAA/G10CACgWB+S9PclXSfpu+r/5X+1f25mJwevo4P3dkk66u69IcvdJelQ4aUFAJRebasLAADbzIckfUX9S4Q+eJafv+ORlwxJOiZpt5nVhnQKjkm6uLhiAgDQxwgBABTI3Q+qP7n4uZI+mfjfvi6pLen5QzK3SPqNdRUOAICzoEMAAMV7laRfdvellLC7n5L0Jkl/bGbPN7MpM6ub2a+Y2dsHsX8j6RfN7D+Y2UWSZGaPMbMPm9mOjdgIAEA5cMkQABTM3f/mPP7P75vZYUlvlPSnkk5LukPSW88s08yuVf+ORXeZWU3S/ZL+yyALAMB54bajAAAAQIlxyRAAAABQYnQIAAAAgBKjQwAAAACUGB0CAAAAoMQ25S5D8xfs9osu2b8Zq9p0LgszpmImbsdrSg1JRc0lt4T1FbX97mkblyfshDxhWSn7KGX7q5ay/Zs7uT+l3qZIObYp+yh1WUWVezOlnmsp25bnCfU2YV0pe9ES6m0lqW6n5VLKlHYmFXNuF1nXCtvfytdfGKVvW0o7mZLJEuptikpKe1tJ20cp7VLF4mVVktqtWO7x32ezhIxUXDsxir7//+446u57trocw1xdmfYFz5Lz96r9eXd/zgYWKcmmdAguumS/bvjY1zZjVYVKOal6eXyC1hIaqMIap8R2t5sVMzg0UTvXQ1V/IqXcKb/sd/NqUplavXqc6cbLavfiMk3U4mZ1eqIbZtI6DWm/NKToZmn7MlKvxo1evZLWMKbUk5QPzRSpnctIyvFIrbftXtwcL7XjZfUS2q1qwm5s1OLjMTvRiRckabIanwNJ7UTCL7KdLN6PnYT6n9K2p5Yp5ZfUiYRzabLWTipTJGUfSWlt6WInzqTU25ROWrMe78cdk2l1MuWza6rWCjMNi+t2pnj7V3rNMLPQiTOSdLoVH9+UdmIUvfja6sGtLkNkQZneOXkgOf+8lbt3b1xp0vEcAgAAAKAAZqZKbfw6XHQIAAAAgCKYZPXxm6JLhwAAAAAogokRAgAAAKC0TLI6HQIAAACglJhDAAAAAJQZIwQAAABAiTGHAAAAACgvM6na4C5DAAAAQEmZLPUpsSOEDgEAAABQBJMs5XHwI4YOAQAAAFAAk1SpMkIAAAAAlJOJS4YAAACA8jJGCAAAAICyMpOMDgEAAABQXlZhUjEAAABQTswhOLeKuSar3c1YVaGq9SzMdPNqmGln9TCz0k05FPG6mrW4zJI0We+EmZrlYSb3uBe8UtD253naCWYJsVol3rZKI15Qoxovp16Jj0lKRpLMPMy4x+U2xctJqdudXpzpWpyR0o5Jyn5yxdvfy+N6m5LpZnGmWon3tSRN1eM2csfESlymhOO21G2EmeVOvJwjp5thRpKqlTg3mdDeTjfidqtZ7SWUJ15X6t/3KgntZNK5lMdt4PGV6XhdCXUypY3s5+K6m/KZMzsR1+2kNjlhX6e0f5LUyeL9vdKbjcuU0JYW9Zk0WUv7PWpyZvx+39pemEMAAAAAlJYxQgAAAACUmEmVWtrI+CihQwAAAAAUgkuGAAAAgNLikiEAAACg5LjtKAAAAFBWjBAAAAAAZWZ0CAAAAIAyo0MAAAAAlFR/UjFzCAAAAIDS4rajAAAAQFkZcwgAAACA0jKeVAwAAACUGyMEAAAAQGkZk4oBAACA0uLBZAAAAECZMUIAAAAAlJsxQnBWZq56tbsZqypUzbIw4x4f9MzyMDNRjdeVUr9qlXhdktTL40PfyuMernvS6kJT9bh+5J7W4+5mca7Vi+8A0MtSTuh4XRUr7jRrVHthxiw+KPVKXN9SRjx7Vsy+lqQTyxPx+hKOyWQjPgd2THbCzAXNpTAzXV0OM660D4aVrBlmlntxppdw3qYc/5mJuB6l1DVJqibkljvxeXJ0aTJpfZF6Na4jKRlJatYS9mW9lbCcuE72avE+amf1hEzaOdlJOHdbCe1tyjmQ8tlliutRJfEzMEv47O4k7MulbryPUrZtqhG37Sl1RJKa1TiX8vsNzo9xyRAAAABQblwyBAAAAJQVDyYDAAAAyo0RAgAAAKCk+k8qpkMAAAAAlJRJjBAAAAAA5WVjeNvR8evCAAAAAKPI+nMIUl9DF2V2mZl90cy+Y2Z3mdlrB+/vNLMvmNk9g38vWG+x6RAAAAAAhejfZSj1FehJ+h13f7ykX5D0ajN7vKQ3SLrV3a+UdOvg+3WhQwAAAAAUwdSfQ5D6GsLdD7n7Xw2+Pi3pu5IulXS9pBsHsRslPX+9xWYOAQAAAFCQNT6HYLeZ3b7q+xvc/YafWqbZAUk/J+kbkva6+6HBjw5L2nueRf0xOgQAAABAAUwmszVdgHPU3a8ZukyzGUn/VdLr3H1h9aRld3cz8/Mq7Cp0CAAAAIAimKQCn1RsZnX1OwN/6u6fHLz9sJld7O6HzOxiSUfWux7mEAAAAAAFKfAuQybpfZK+6+5/sOpHN0l6xeDrV0j69HrLzAgBAAAAUAQzWbVa1NKeJunlku40s28O3vtdSW+T9DEze5Wkg5JevN4V0SEAAAAACrLGScXn5O5fU/8ipLN5RiErGaBDAAAAABQluBRoFNEhAAAAAApgZlp9F6BxsSkdgiyvaKEztRmr2nSm+E5P1YS7QU3XO2Gm53GPc7nbCDOS1OoVc33bZL0XZqbr7TBTr2RhJsvTyryseB+sdEerL9xN3LaUUchaJT4mmcfrW+7Ww0wviwvUy9P+UlKvxufJ/GS8bTubS2FmstoKM+08rkfHO/NhppOl1bXM431Zq+RhplGN91FKfVtpx8et3Us7trWEY5vy+TkzEW9bs9aNMwn7KFXKcWtlCe1y3ASq1YvrUp5QnpTzVpLyc16p8BMpn28pullcl2oJnzdF/l02y4s5J6cacblTztuaxetK1fXR+gzcdhghAAAAAMqrqDkEm4kOAQAAAFAEM2ltDyYbCXQIAAAAgKIwQgAAAACUlzFCAAAAAJSUiRECAAAAoLwKfVLxpqFDAAAAABTBxG1HAQAAgPKytAerjBg6BAAAAEBBjBECAAAAoKRMPIcAAAAAKC/jLkMAAABAWZl4DgEAAABQXjyHAAAAACgzYw4BAAAAUGrcdhQAAAAoKTOpOn6/Xo9fiQEAAIBRxRyCs6uYa6rW2YxVFcrkYaaTx7twpVcvJGMWl2eqnrafdzbjnCuu0AvtyTBzZHEmzPTyeF2T9SzMSNLsRLxtl8wuhJmU7V/qToSZ0+342LqnNR7dWnxdYjMhU6/E+3Km0Q4znSyu/92suGspa5U8zCz3GmGmncXHJEs4JsvdeDmtbjXMSFK1Ep/fKedAzeJ9lNIez9RbYSZlP0rSYic+T1L2U0dxXWpU40xKW1q1tPbGPC53lsWZxU68L4+ejjN5Qls61YzriCTtmOyGmel63E40qr0wU9Tn7emENjnVZD0u90Q13kcpsoR6dKoTf95KaW3uGP6+Ol6YQwAAAACUGHMIAAAAgJIykyqMEAAAAADlxQgBAAAAUGLMIQAAAABKikuGAAAAgJLjkiEAAACgxLhkCAAAACgpM6ma9vyZUUKHAAAAACiAS3IuGQIAAADKyrhkCAAAACg1OgQAAABAeXHJEAAAAFBWxiVDAAAAQLkxQgAAAACU2Bg+qTgssZm91szmrO99ZvZXZvaszSgcAAAAMD5MbumvUZHShfkH7r4g6VmSLpD0cklv29BSAQAAAOPG1J9DkPoaESmXDJ3pvjxX0ofc/S6ztXVpzFy1Sm/NhdtqdSumzIv5RJhZaMWHopKw17uNtMqVJ+Smaq0ws6O5FGYm650wc7rdDDOtXtqT/xZajXhZtXh/Vy2PM5U4M9eMt7+TuG3dLD5uK92EutRIKHd9Jcw0q3F5Wll8PCRpoR2fJ8dOx/Wkl8XrmpuMt3/XVLz9F02fCjOZpx3bdlYPM8vdeF8eW54MM5WKh5mJarwjZxrtMCNJF07F+8kVN3ArvbiOJO2jlakwk+dpH3Mpn4bzzbgt3dmM61vV4uOW0k7WEo6/JDVr8WdgLaENdI93Up7w98lWLz62Ke2flHZ869WUbYvXlVJHUtr2LGE/SlIvj5eVUm6cL5NXintSsZm9X9KvSjri7k8cvLdT0p9LOiDpfkkvdvcT61lPym+Pd5jZ/1S/Q/B5M5uVFJ8lAAAAQMm4VZJfCT4g6TmPeO8Nkm519ysl3Tr4fl3OWRIze9rgy1cPVvQUd1+W1JD0yvWuGAAAANh2zNJfAXf/iqTjj3j7ekk3Dr6+UdLz11vkYWNr75J0taSvufuTVxXsmKRj610xAAAAsK1sznMI9rr7ocHXhyXtXe8Ch3UIumZ2g6R9ZvauR/7Q3V+z3pUDAAAA24VrzU8q3m1mt6/6/gZ3vyF5fe5uljDBKDCsQ/Crkp4p6dmS7ljvigAAAIBtb20jBEfd/Zo1ruFhM7vY3Q+Z2cWSjqzx//+UYR2C17v7vzCz/e5+45AcAAAAAKXdNW2dbpL0CvUfA/AKSZ9e7wKHdWGeO7i96EvWuxIAAABg+7NC7zJkZh+R9HVJV5nZg2b2KvU7AteZ2T3qX82z7ueDDRshuFnSCUkzZrag/vMI/My/7j633pUDAAAA20qBk4rd/aXn+NEzCluJhowQuPvr3X2HpM+4+5y7z67+t8hCAAAAAGPP+pOKU1+jInykn7tfb2aXS7rS3W8xs0lJNXc/vfHFAwAAAMaDF/yk4s0SjmmY2T+U9AlJfzJ4a5+k/7aBZQIAAADGUsFPKt4UKSV5taSnSVqQJHe/R9KFG1koAAAAYPxYf5Qg8TUqwkuGJLXdvWOD65zMrKb+5GIAAAAAq4zSX/5TpZT4y2b2u5Imzew6SR+X9N83tlgAAADAmDFJZumvEZHSIXiDpB9JulPSP5L0WUlv3MhCAQAAAOPH5Kokv0ZFyl2GcjO7Uf2HIkjS3e7OJUMAAADAKi6N1O1EU4UdAjN7uqQbJd2v/kDIZWb2Cnf/yoaWDAAAABgz4ziHIGVS8e9Lepa73y1JZvZYSR+RdHXqSnKvaLnbPL8SbqFuHt9H1iweLGnWumFmajbOZB73OGuWhxlJalR7YSZl9nvKcW314mrWy4s7eVKWVUsY5Jqux8ckZT+m1KPU45bV4mOSJWx/q1cPMwvtC5LKFEkdT6xV4uDe2VaYmax3wsxENeF8SzhuJ1ozYSal/qdq1LIw06zHmV6WUI8S2pvTnbR2fbnXCDP1SlzulMz8xEqYSWkjUs5tSapaXKZWFm//UjfOHD41EWYWl8OIpifjjCT5bJy5YLKY45aiWYvP7SI/SyYTPrtTtq3ncZnqlfh8S/l9Q+r/zhVpZ+N3n/xxMkp3D0qV8klVP9MZkCR3/56Zxb9NAAAAAKVi23aE4HYze6+kDw++/3uSbt+4IgEAAADjxyXl27RD8I/VfzjZawbff1XSuzesRAAAAMA4su07QlCT9E53/wNJMrOqpPhCRgAAAKBkxnEOQUoX5lZJq6cgTUq6ZWOKAwAAAIwvt0rya1SkjBA03X3xzDfuvmhmUxtYJgAAAGAsbdcRgiUze/KZb8zsaknxfd0AAACAEvHBXYa24wjB6yR93Mx+qP6DyS6S9JsbWSgAAABgHI3jCEHYIXD328zscZKuGrx1t7vHT+sAAAAASsZtG3YIJGnQAfj2BpcFAAAAGGue8KT3UZPUIQAAAAAQMXnSFN3RQocAAAAAKIBLysewQ3DOEpvZs83shWd5/4Vmdt3GFgsAAAAYPy5Lfo2KYV2YN0n68lne/5Kkf7shpQEAAADGVnpnYJQ6BMMuGZpw9x898k13P2pm0xtYJgAAAGAsbbdJxXNmVnP33uo3zawuaXJjiwUAAACMF9d4Podg2CVDn5T0n1ePBpjZjKT3DH4GAAAAYJXtdsnQGyX9nqSDZnZw8N5+Se+T9K/XtBLradfEyfMq4FaqWyfMrORTYebBxZ1h5oGjjTCzvOJhZteOMCJJ2jndizNTy3GmcSrM5I14tv1CN74K7eRKM8xI0nIvXt+x0/H+Xlish5kL5uKTef+ulTCzd/J4mJGk+d6xMFPxLMys1GfDzNHe7jBzohXX/+VONcxIUjeLj1svj/d3O4vXV7F4kLNi8fmWUh6PFzNYVrz9llCmnc24vjWr7ULWlTos3snjc6mdxZlT7bgNaHXj459yTFI/qOvVPMzMTsTP8mzW4szjL4qPbe5xPVrpxftakk61JsLMXT+ci5dzOt7htYRmIuXz7aoL488kSdpbORRmJrunw0ymeF8uN+J9tOQzYaaVxcdDklYSPgPbCecJzt8o/aKf6pwdgsGlQm8ws7dIeszg7XvdPW6RAAAAgNKx7TWHwMxeJsnc/UOS7lz1/sslZe7+Z5tQPgAAAGAs9J9DMH4dgmHjSv9U0qfO8v4nJf3OxhQHAAAAGF/bbQ5B3d0XH/mmuy8N7jQEAAAA4AxPm8szaoZ1CCbNbNrdl1a/aWazkuIZmQAAAECpjOccgmFdmPdJ+oSZXX7mDTM7IOmjg58BAAAAGDjzHIJtc8mQu7/DzBYlfWXw/AFJWpT0Nnf/T5tSOgAAAGCMjOMIwbBLhuTu75H0nsFlQnL3+Ka8AAAAQEnFTycZPee8ZMjMPrDq2xfQGQAAAACGc7fk16gYNofgZ1d9/dqNLggAAAAwztYyf2As5hCoPy8CAAAAQKJR+st/qmEdgn1m9i5JturrH3P312xoyQAAAIAxU+Rf/s3sOZLeKakq6b3u/rbCFr7KsA7B61d9fftGrBwAAADYNlzKC7rGxsyqkv5Y0nWSHpR0m5nd5O7fKWYNPzHstqM3Fr0yAAAAYLs68xyCgvy8pHvd/fuSZGYflXS9pM3rEJjZTcP+o7v/WtGFAQAAAMZZnq+pQ7DbzFZfiXODu98w+PpSSQ+s+tmDkp66zuKd1bBLhq4dFOIjkr4hjdBUaAAAAGDkmPK1/cp81N2v2ajSpBrWIbhI/WuWXirptyR9RtJH3P2uzSgYAAAAME5chd5l6CFJl636ft/gvcINm0OQSbpZ0s1mNqF+x+BLZvYWd/+Pa1nJYqehr92/b30l3QLHT8bPmpueGvYoh779F/bCzNMO/DDM7OoeCjO1rB1mJGmlMRdmvrPy2DDzxW/NhJkTxzthZna+HmauOhDva0l69K6TYWZ+96kwU1UWZk7mO+JMazrM/HB5V5iRpOPV+TAzVW+FmbrHdTJLaNDmJlYSMmGkv768mhYMmBUzm2ulF9fJ+x+ON+7EqbgeSVK9Fu/vvbsbYWa2EZ9vu+vxcyZn2ifCTKse121Juq93eZhZaMXbVq/GbfLu6bj+Vy0+Jq2E49/PDfu7Wt/pdrysQ61mmMkSHn/abMT1v1FNO0dS9veBPfFnztyl8TFpVuPlZEMfndTXztKO2wPdy8JMz+P1NRLa0kYWZ6zAO71XE9rAlGOL8+fFHc7bJF1pZleo3xF4ifp/pC/c0JZs0BF4nvqdgQOS3iXpUxtREAAAAGDcFTWp2N17ZvZPJH1e/duOvn+jrtQZNqn4g5KeKOmzkt7i7t/eiAIAAAAA20KBtx2VJHf/rPq/i2+oYSMEL5O0JOm1kl5j9uPejklyd4+vOQEAAABKouA5BJtm2ByCtAu2AQAAAEgqdA7BpolnQwEAAABIssbbjo4EOgQAAABAQRghAAAAAErK3ZSt7UnFI4EOAQAAAFAQRggAAACAEivqOQSbiQ4BAAAAUABXsc8h2Cx0CAAAAICCcMkQAAAAUGJ0CAAAAICScpfy7fSkYgAAAABrwwgBAAAAUGJ0CAAAAIAS4y5DAAAAQEm5SzlPKj7HSirSrtlsM1ZVqMfs7YSZXc2FMFNVvO2nujNh5lDvCWEmyythRpKWT1XDTKOah5mnPK4XZurVuKu80AojOrmUtm33ndgRZpr1uTAzWY+37YKJpTCzoxlnWr1GmOnn6nEmi+vSbL0dZuZqcbln/WSYqXjauf9w5ZIwc/DkjjBzdCGu2612XCcb9bhB3zkXnyNXXRK3I5LUrHXDTObxtmUJk9lO5fNhpjeRUNfyZpiRpHolrgO7plbCTCeLP7IWO3G58zw+32Ym0o7bVCM+bseW4v3UrMd1ac9M3FA2q3G5T7anwowkHT4V7ydPqG/H6/ExmZucDDOXTJ8MMxfXDocZSVrSbJi5f2FPmPne8bR9GdkxEx//3dNxuy1J8xPLYWauEbfvOH+MEAAAAAAl5WIOAQAAAFBqdAgAAACAEuOSIQAAAKCsnBECAAAAoLRcUh7PER85dAgAAACAgjBCAAAAAJQYHQIAAACgpNyZVAwAAACUWj6GPQI6BAAAAEBBmFQMAAAAlJRz21EAAACg3MbwiiE6BAAAAEBRGCEAAAAASszHcIiADgEAAABQAG47CgAAAJQclwwBAAAAJcZzCM6hXs20b/70ZqyqULVKL8yY4oO+2JsOMw+cnAkzxxcqYWZuOq0SXrErPh4z9ZUwc6oTl/vI4mSYefh4vG29LG3bZppx7lFzR8LMfH4szCxUd4aZHy7vCjMr3bRTsZJQ3+ab7TAzVy/mfHyod2mYafUaScvqZnEdmGxkYeay3XGmXo1vEt2sxuf/UjfethMrE2FGkpbbzTDT6liYmajH67p4R1zfpqeXw8y+1vfilUnq1OM28ET9wjDzo9aOMNPN4h1Qq8TnUbuXdk7WKnFd2j0dt6VLnbgu3fE3U2FmcSmuRzvmq2FGki7bE59Lu6bibZutx3VpohK3Wx2P99G9K5eHGSntuM01O2Fm57542xoJv0tUrLgb13fz+Bw4vhx/duP8uBghAAAAAMrLXRkjBAAAAEB5OU8qBgAAAMqpf8kQIwQAAABAObmUM0IAAAAAlNc4jhDEt/UAAAAAEHL1H0yW+loPM3uRmd1lZrmZXfOIn/1LM7vXzO42s2dHy2KEAAAAACiCS755dxn6tqQXSPqT1W+a2eMlvUTSEyRdIukWM3usu5/zXsJ0CAAAAICCbNYVQ+7+XUky+6ln1Fwv6aPu3pZ0n5ndK+nnJX39XMuiQwAAAAAUZI1PKt5tZrev+v4Gd79hnUW4VNJfrvr+wcF750SHAAAAACiAu691UvFRd7/mXD80s1skXXSWH/0rd//0Wst3LnQIAAAAgIJkWXHXDLn7M8/jvz0k6bJV3+8bvHdO3GUIAAAAKMiZUYKU1wa5SdJLzGzCzK6QdKWk/zPsPzBCAAAAABTAfc1zCM6bmf26pD+StEfSZ8zsm+7+bHe/y8w+Juk7knqSXj3sDkMSHQIAAACgMJt4l6FPSfrUOX72VklvTV0WHQIAAACgIJv4HILC0CEAAAAACuDuyjdriKBAm9IhOHZK+uDnxm/+8vGH22Fm3xU7w8wznloNM780d1uYmWicDDMn5g+EGUk61DvbHaz+tm8c3Btmjp3Iw8zcTHzsH3/Zcph5VOP+MCNJs0tHwkzt6EKYyRuTYebwzNDb+kqSji02wszxhbTzo9uLG5ksi0/r3KfDTC9hXdXqTz0M5adcuDPOSNITLjoWZg60vxtmGssnwky3ORdmDk89JsxkviPMpGrU4jrQnYj35UonXs59RybCzL3Z/jDTqMeZfi6uS/NTQy9vlSTtmV4KM4+aj89tV7wfT3VmwowkLbTjfbnSjT8DWp24TI/d1wszB+bi9u/i0/eEGUlqnDocZjKPz6WTOw6EmYPty8PMXx+M261DD3fCjCTt2RUft0ddEtfJXVPxZ+ByrxlmFlrx58SxxXqYkaSllTjTSFsUzhMjBAAAAECJ0SEAAAAAysqlMewP0CEAAAAAiuBihAAAAAAoL3dlWTy3ZNTQIQAAAAAKsoFPIN4wdAgAAACAAnDJEAAAAFBmTocAAAAAKDEeTAYAAACUGiMEAAAAQEm5mFQMAAAAlJdLOSMEAAAAQHlxyRAAAABQWs4lQwAAAEBZuUtZL9vqYqwZHQIAAACgIIwQAAAAAGXlzhwCAAAAoKxcTCo+JzNTtVrZjFUV6slPvSTM7L8oPugXNBfCzLHapWFmofq4MHPbPfNhRpJu+cw9YebCffE1cP/sRcthZv/X3h9mFr/6/TAz98SfCTOSdOzq54aZzx9/epj531/4UZi5dP9MmPm7T+7Ey5nvhRlJOtWaCDPfPxyf1g882Aoz09P1MHNgX7yuK3YvhRlJmqquhJnTk3vCzMrUgTDzg8ULw8xDP2iEmaft/0GY2f/9/xVmJEntePvbl1wZZu6ee0qY+cp3ZsPMX3wpPieP3PdQmJGkp/7KNWHml34hrtsT1W6YaeXxco4sxdt/ulUNM5K0sBx/tnXjYusJl8XnyZNuen2Y+Ys33RpmFq7ZERdI0s++7jfCzPee/Mow89Evxfv7tlu/FWaq9bhNesHLfi7MSNJjLm7H67M8zBw8Hn8GPHA4/j3h+PH4c6LXS6hIkuZ3xG3XVQfG73eycZJ7XHdGDSMEAAAAQBGcEQIAAACgtFzMIQAAAABKjbsMAQAAAGXlUp4zhwAAAAAoLS4ZAgAAAErK3ZVnPKkYAAAAKC1GCAAAAIDScp5DAAAAAJSV8xwCAAAAoNycuwwBAAAAJcUIAQAAAFBmLmcOAQAAAFBOLilnhAAAAAAoKWcOAQAAAFBizhwCAAAAoLRcPKkYAAAAKCuXj+UlQ+a+8cMaZvYjSQc3fEUAAADYri539z1bXYhhzOxmSbvX8F+OuvtzNqo8qTalQwAAAABgNFW2ugAAAAAAtg4dAgAAAKDE6BAAAAAAJUaHAAAAACgxOgQAAABAidEhAAAAAEqMDgEAAABQYnQIAKBgZrbLzL45eB02s4cGXy+a2bu3unwAAKzGg8kAYAOZ2ZslLbr7O7a6LAAAnA0jBACwSczs6Wb2PwZfv9nMbjSzr5rZQTN7gZm93czuNLObzaw+yF1tZl82szvM7PNmdvHWbgUAYLuhQwAAW+fRkn5Z0q9J+rCkL7r7kyStSHreoFPwR5Je6O5XS3q/pLduVWEBANtTbasLAAAl9jl375rZnZKqkm4evH+npAOSrpL0RElfMDMNMoe2oJwAgG2MDgEAbJ22JLl7bmZd/8mkrlz99tkk3eXu125VAQEA2x+XDAHA6Lpb0h4zu1aSzKxuZk/Y4jIBALYZOgQAMKLcvSPphZL+vZn9taRvSvrFLS0UAGDb4bajAAAAQIkxQgAAAACUGB0CAAAAoMToEAAAAAAlRocAAAAAKDE6BAAAAECJ0SEAAAAASowOAQAAAFBi/x+CxzlvVG286AAAAABJRU5ErkJggg==\n",
      "text/plain": [
       "<Figure size 864x288 with 2 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Plotting MFCC\n",
    "mfcc = librosa.feature.mfcc(S=log_S, n_mfcc=13)\n",
    "\n",
    "# Let's pad on the first and second deltas while we're at it\n",
    "delta2_mfcc = librosa.feature.delta(mfcc, order=2)\n",
    "\n",
    "plt.figure(figsize=(12, 4))\n",
    "librosa.display.specshow(delta2_mfcc)\n",
    "plt.ylabel('MFCC coeffs')\n",
    "plt.xlabel('Time')\n",
    "plt.title('MFCC')\n",
    "plt.colorbar()\n",
    "plt.tight_layout()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 121,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "                <audio  controls=\"controls\" >\n",
       "                    <source src=\"...\" type=\"audio/wav\" />\n",
       "                    Your browser does not support the audio element.\n",
       "                </audio>\n",
       "              "
      ],
      "text/plain": [
       "<IPython.lib.display.Audio object>"
      ]
     },
     "execution_count": 121,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Original Sound\n",
    "ipd.Audio(samples, rate=sample_rate)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 122,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "                <audio  controls=\"controls\" >\n",
       "                    <source src=\"...\" type=\"audio/wav\" />\n",
       "                    Your browser does not support the audio element.\n",
       "                </audio>\n",
       "              "
      ],
      "text/plain": [
       "<IPython.lib.display.Audio object>"
      ]
     },
     "execution_count": 122,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Silence trimmed Sound by librosa.effects.trim()\n",
    "ipd.Audio(aa, rate=sample_rate)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 123,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "\n",
       "                <audio  controls=\"controls\" >\n",
       "                    <source src=\"...\" type=\"audio/wav\" />\n",
       "                    Your browser does not support the audio element.\n",
       "                </audio>\n",
       "              "
      ],
      "text/plain": [
       "<IPython.lib.display.Audio object>"
      ]
     },
     "execution_count": 123,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Silence trimmed Sound by manuel trimming\n",
    "samples_cut = samples[10000:-12500]\n",
    "ipd.Audio(samples_cut, rate=sample_rate)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# IV. Defining the truth label"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 124,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "1438"
      ]
     },
     "execution_count": 124,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# 2 class: Positive & Negative\n",
    "\n",
    "# Positive: Calm, Happy\n",
    "# Negative: Angry, Fearful, Sad\n",
    "\n",
    "label2_list = []\n",
    "for i in range(len(data_df)):\n",
    "    if data_df.emotion[i] == 2: # Calm\n",
    "        lb = \"_positive\"\n",
    "    elif data_df.emotion[i] == 3: # Happy\n",
    "        lb = \"_positive\"\n",
    "    elif data_df.emotion[i] == 4: # Sad\n",
    "        lb = \"_negative\"\n",
    "    elif data_df.emotion[i] == 5: # Angry\n",
    "        lb = \"_negative\"\n",
    "    elif data_df.emotion[i] == 6: # Fearful\n",
    "        lb = \"_negative\"\n",
    "    else:\n",
    "        lb = \"_none\"\n",
    "        \n",
    "    # Add gender to the label    \n",
    "    label2_list.append(data_df.gender[i] + lb)\n",
    "    \n",
    "len(label2_list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 125,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "1438"
      ]
     },
     "execution_count": 125,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "#3 class: Positive, Neutral & Negative\n",
    "\n",
    "# Positive:  Happy\n",
    "# Negative: Angry, Fearful, Sad\n",
    "# Neutral: Calm, Neutral\n",
    "\n",
    "label3_list = []\n",
    "for i in range(len(data_df)):\n",
    "    if data_df.emotion[i] == 1: # Neutral\n",
    "        lb = \"_neutral\"\n",
    "    elif data_df.emotion[i] == 2: # Calm\n",
    "        lb = \"_neutral\"\n",
    "    elif data_df.emotion[i] == 3: # Happy\n",
    "        lb = \"_positive\"\n",
    "    elif data_df.emotion[i] == 4: # Sad\n",
    "        lb = \"_negative\"\n",
    "    elif data_df.emotion[i] == 5: # Angry\n",
    "        lb = \"_negative\"\n",
    "    elif data_df.emotion[i] == 6: # Fearful\n",
    "        lb = \"_negative\"\n",
    "    else:\n",
    "        lb = \"_none\"\n",
    "    \n",
    "    # Add gender to the label  \n",
    "    label3_list.append(data_df.gender[i] + lb)\n",
    "    \n",
    "len(label3_list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 126,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "1438"
      ]
     },
     "execution_count": 126,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# 5 class: angry, calm, sad, happy & fearful\n",
    "label5_list = []\n",
    "for i in range(len(data_df)):\n",
    "    if data_df.emotion[i] == 2:\n",
    "        lb = \"_calm\"\n",
    "    elif data_df.emotion[i] == 3:\n",
    "        lb = \"_happy\"\n",
    "    elif data_df.emotion[i] == 4:\n",
    "        lb = \"_sad\"\n",
    "    elif data_df.emotion[i] == 5:\n",
    "        lb = \"_angry\"\n",
    "    elif data_df.emotion[i] == 6:\n",
    "        lb = \"_fearful\"    \n",
    "    else:\n",
    "        lb = \"_none\"\n",
    "    \n",
    "    # Add gender to the label  \n",
    "    label5_list.append(data_df.gender[i] + lb)\n",
    "    \n",
    "len(label5_list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 127,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "1438"
      ]
     },
     "execution_count": 127,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# All class\n",
    "\n",
    "label8_list = []\n",
    "for i in range(len(data_df)):\n",
    "    if data_df.emotion[i] == 1:\n",
    "        lb = \"_neutral\"\n",
    "    elif data_df.emotion[i] == 2:\n",
    "        lb = \"_calm\"\n",
    "    elif data_df.emotion[i] == 3:\n",
    "        lb = \"_happy\"\n",
    "    elif data_df.emotion[i] == 4:\n",
    "        lb = \"_sad\"\n",
    "    elif data_df.emotion[i] == 5:\n",
    "        lb = \"_angry\"\n",
    "    elif data_df.emotion[i] == 6:\n",
    "        lb = \"_fearful\"\n",
    "    elif data_df.emotion[i] == 7:\n",
    "        lb = \"_disgust\"\n",
    "    elif data_df.emotion[i] == 8:\n",
    "        lb = \"_surprised\"\n",
    "    else:\n",
    "        lb = \"_none\"\n",
    "        \n",
    "    # Add gender to the label \n",
    "    label8_list.append(data_df.gender[i]  + lb)\n",
    "    \n",
    "len(label8_list)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 128,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>path</th>\n",
       "      <th>source</th>\n",
       "      <th>actor</th>\n",
       "      <th>gender</th>\n",
       "      <th>intensity</th>\n",
       "      <th>statement</th>\n",
       "      <th>repetition</th>\n",
       "      <th>emotion</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-01-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>male_none</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-01-02-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male_none</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-02-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>male_none</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-01-01-02-02-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male_none</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-02-01-01-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                                                path  source  actor gender  \\\n",
       "0  D://ravdess data//Actor_01/03-01-01-01-01-01-0...       1      1   male   \n",
       "1  D://ravdess data//Actor_01/03-01-01-01-01-02-0...       1      1   male   \n",
       "2  D://ravdess data//Actor_01/03-01-01-01-02-01-0...       1      1   male   \n",
       "3  D://ravdess data//Actor_01/03-01-01-01-02-02-0...       1      1   male   \n",
       "4  D://ravdess data//Actor_01/03-01-02-01-01-01-0...       1      1   male   \n",
       "\n",
       "   intensity  statement  repetition  emotion          label  \n",
       "0          0          0           0        1      male_none  \n",
       "1          0          0           1        1      male_none  \n",
       "2          0          1           0        1      male_none  \n",
       "3          0          1           1        1      male_none  \n",
       "4          0          0           0        2  male_positive  "
      ]
     },
     "execution_count": 128,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Select the label set you want by commenting the unwanteds.\n",
    "\n",
    "data_df['label'] = label2_list\n",
    "#data_df['label'] = label3_list\n",
    "#data_df['label'] = label5_list\n",
    "#data_df['label'] = label8_list\n",
    "data_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 129,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Index(['male_negative', 'female_negative', 'male_none', 'female_none',\n",
      "       'male_positive', 'female_positive'],\n",
      "      dtype='object')\n"
     ]
    }
   ],
   "source": [
    "print (data_df.label.value_counts().keys())"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 130,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Plotting the emotion distribution\n",
    "\n",
    "def plot_emotion_dist(dist, color_code='#C2185B', title=\"Plot\"):\n",
    "    \"\"\"\n",
    "    To plot the data distributioin by class.\n",
    "    Arg:\n",
    "      dist: pandas series of label count. \n",
    "    \"\"\"\n",
    "    tmp_df = pd.DataFrame()\n",
    "    tmp_df['Emotion'] = list(dist.keys())\n",
    "    tmp_df['Count'] = list(dist)\n",
    "    fig, ax = plt.subplots(figsize=(14, 7))\n",
    "    ax = sns.barplot(x=\"Emotion\", y='Count', color=color_code, data=tmp_df)\n",
    "    ax.set_title(title)\n",
    "    ax.set_xticklabels(ax.get_xticklabels(),rotation=45)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 131,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1008x504 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "a = data_df.label.value_counts()\n",
    "plot_emotion_dist(a, \"#2962FF\", \"Emotion Distribution\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# V. Data Splitting"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 132,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Female Data Set\n",
    "\n",
    "## Uncomment all below to use Female set \n",
    "\n",
    "# data2_df = data_df.copy()\n",
    "# data2_df = data2_df[data2_df.label != \"male_none\"]\n",
    "# data2_df = data2_df[data2_df.label != \"female_none\"]\n",
    "# data2_df = data2_df[data2_df.label != \"male_happy\"]\n",
    "# data2_df = data2_df[data2_df.label != \"male_angry\"]\n",
    "# data2_df = data2_df[data2_df.label != \"male_sad\"]\n",
    "# data2_df = data2_df[data2_df.label != \"male_fearful\"]\n",
    "# data2_df = data2_df[data2_df.label != \"male_calm\"]\n",
    "# data2_df = data2_df[data2_df.label != \"male_positive\"]\n",
    "# data2_df = data2_df[data2_df.label != \"male_negative\"].reset_index(drop=True)\n",
    "\n",
    "# tmp1 = data2_df[data2_df.actor == 22]\n",
    "# tmp2 = data2_df[data2_df.actor == 24]\n",
    "# data3_df = pd.concat([tmp1, tmp2],ignore_index=True).reset_index(drop=True)\n",
    "# data2_df = data2_df[data2_df.actor != 22]\n",
    "# data2_df = data2_df[data2_df.actor != 24].reset_index(drop=True)\n",
    "# print (len(data2_df))\n",
    "# data2_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 133,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "400\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>path</th>\n",
       "      <th>source</th>\n",
       "      <th>actor</th>\n",
       "      <th>gender</th>\n",
       "      <th>intensity</th>\n",
       "      <th>statement</th>\n",
       "      <th>repetition</th>\n",
       "      <th>emotion</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-02-01-01-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-02-01-01-02-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-02-01-02-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-02-01-02-02-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>D://ravdess data//Actor_01/03-01-02-02-01-01-0...</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>male</td>\n",
       "      <td>1</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                                                path  source  actor gender  \\\n",
       "0  D://ravdess data//Actor_01/03-01-02-01-01-01-0...       1      1   male   \n",
       "1  D://ravdess data//Actor_01/03-01-02-01-01-02-0...       1      1   male   \n",
       "2  D://ravdess data//Actor_01/03-01-02-01-02-01-0...       1      1   male   \n",
       "3  D://ravdess data//Actor_01/03-01-02-01-02-02-0...       1      1   male   \n",
       "4  D://ravdess data//Actor_01/03-01-02-02-01-01-0...       1      1   male   \n",
       "\n",
       "   intensity  statement  repetition  emotion          label  \n",
       "0          0          0           0        2  male_positive  \n",
       "1          0          0           1        2  male_positive  \n",
       "2          0          1           0        2  male_positive  \n",
       "3          0          1           1        2  male_positive  \n",
       "4          1          0           0        2  male_positive  "
      ]
     },
     "execution_count": 133,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Male Data Set\n",
    "\n",
    "## Uncomment all below to use Male set \n",
    "\n",
    "data2_df = data_df.copy()\n",
    "data2_df = data2_df[data2_df.label != \"male_none\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_none\"].reset_index(drop=True)\n",
    "data2_df = data2_df[data2_df.label != \"female_neutral\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_happy\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_angry\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_sad\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_fearful\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_calm\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_positive\"]\n",
    "data2_df = data2_df[data2_df.label != \"female_negative\"].reset_index(drop=True)\n",
    "\n",
    "tmp1 = data2_df[data2_df.actor == 21]\n",
    "tmp2 = data2_df[data2_df.actor == 22]\n",
    "tmp3 = data2_df[data2_df.actor == 23]\n",
    "tmp4 = data2_df[data2_df.actor == 24]\n",
    "data3_df = pd.concat([tmp1, tmp3],ignore_index=True).reset_index(drop=True)\n",
    "data2_df = data2_df[data2_df.actor != 21]\n",
    "data2_df = data2_df[data2_df.actor != 22]\n",
    "data2_df = data2_df[data2_df.actor != 23].reset_index(drop=True)\n",
    "data2_df = data2_df[data2_df.actor != 24].reset_index(drop=True)\n",
    "print (len(data2_df))\n",
    "data2_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 134,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "80\n"
     ]
    },
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>path</th>\n",
       "      <th>source</th>\n",
       "      <th>actor</th>\n",
       "      <th>gender</th>\n",
       "      <th>intensity</th>\n",
       "      <th>statement</th>\n",
       "      <th>repetition</th>\n",
       "      <th>emotion</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>D://ravdess data//Actor_21/03-01-02-01-01-01-2...</td>\n",
       "      <td>1</td>\n",
       "      <td>21</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>D://ravdess data//Actor_21/03-01-02-01-01-02-2...</td>\n",
       "      <td>1</td>\n",
       "      <td>21</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>D://ravdess data//Actor_21/03-01-02-01-02-01-2...</td>\n",
       "      <td>1</td>\n",
       "      <td>21</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>D://ravdess data//Actor_21/03-01-02-01-02-02-2...</td>\n",
       "      <td>1</td>\n",
       "      <td>21</td>\n",
       "      <td>male</td>\n",
       "      <td>0</td>\n",
       "      <td>1</td>\n",
       "      <td>1</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>D://ravdess data//Actor_21/03-01-02-02-01-01-2...</td>\n",
       "      <td>1</td>\n",
       "      <td>21</td>\n",
       "      <td>male</td>\n",
       "      <td>1</td>\n",
       "      <td>0</td>\n",
       "      <td>0</td>\n",
       "      <td>2</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                                                path  source  actor gender  \\\n",
       "0  D://ravdess data//Actor_21/03-01-02-01-01-01-2...       1     21   male   \n",
       "1  D://ravdess data//Actor_21/03-01-02-01-01-02-2...       1     21   male   \n",
       "2  D://ravdess data//Actor_21/03-01-02-01-02-01-2...       1     21   male   \n",
       "3  D://ravdess data//Actor_21/03-01-02-01-02-02-2...       1     21   male   \n",
       "4  D://ravdess data//Actor_21/03-01-02-02-01-01-2...       1     21   male   \n",
       "\n",
       "   intensity  statement  repetition  emotion          label  \n",
       "0          0          0           0        2  male_positive  \n",
       "1          0          0           1        2  male_positive  \n",
       "2          0          1           0        2  male_positive  \n",
       "3          0          1           1        2  male_positive  \n",
       "4          1          0           0        2  male_positive  "
      ]
     },
     "execution_count": 134,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "print (len(data3_df))\n",
    "data3_df.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# VI. Getting the features of audio files using librosa"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 135,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:34<00:00, 11.67it/s]\n"
     ]
    }
   ],
   "source": [
    "data = pd.DataFrame(columns=['feature'])\n",
    "for i in tqdm(range(len(data2_df))):\n",
    "    X, sample_rate = librosa.load(data2_df.path[i], res_type='kaiser_fast',duration=input_duration,sr=22050*2,offset=0.5)\n",
    "#     X = X[10000:90000]\n",
    "    sample_rate = np.array(sample_rate)\n",
    "    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)\n",
    "    feature = mfccs\n",
    "    data.loc[i] = [feature]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 136,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>feature</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>[-70.26908, -70.26908, -70.26908, -70.26908, -...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>[-67.558426, -67.558426, -67.558426, -67.55842...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>[-69.69399, -69.69399, -69.69399, -69.69399, -...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>[-69.05305, -69.05305, -69.05305, -69.05305, -...</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>[-73.841805, -73.841805, -73.841805, -73.84180...</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                                             feature\n",
       "0  [-70.26908, -70.26908, -70.26908, -70.26908, -...\n",
       "1  [-67.558426, -67.558426, -67.558426, -67.55842...\n",
       "2  [-69.69399, -69.69399, -69.69399, -69.69399, -...\n",
       "3  [-69.05305, -69.05305, -69.05305, -69.05305, -...\n",
       "4  [-73.841805, -73.841805, -73.841805, -73.84180..."
      ]
     },
     "execution_count": 136,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "data.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 137,
   "metadata": {},
   "outputs": [],
   "source": [
    "df3 = pd.DataFrame(data['feature'].values.tolist())\n",
    "labels = data2_df.label"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 138,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "      <th>4</th>\n",
       "      <th>5</th>\n",
       "      <th>6</th>\n",
       "      <th>7</th>\n",
       "      <th>8</th>\n",
       "      <th>9</th>\n",
       "      <th>...</th>\n",
       "      <th>249</th>\n",
       "      <th>250</th>\n",
       "      <th>251</th>\n",
       "      <th>252</th>\n",
       "      <th>253</th>\n",
       "      <th>254</th>\n",
       "      <th>255</th>\n",
       "      <th>256</th>\n",
       "      <th>257</th>\n",
       "      <th>258</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-65.434235</td>\n",
       "      <td>-65.635078</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>...</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>...</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.005867</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-68.918213</td>\n",
       "      <td>...</td>\n",
       "      <td>-65.829712</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>...</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>5 rows × 259 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "         0          1          2          3          4          5    \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047 -69.053047 -69.005867   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "\n",
       "         6          7          8          9    ...        249        250  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081  ... -70.269081 -70.269081   \n",
       "1 -65.434235 -65.635078 -67.558426 -67.558426  ... -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993  ... -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -68.918213  ... -65.829712 -69.053047   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805  ... -73.841805 -73.841805   \n",
       "\n",
       "         251        252        253        254        255        256  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047 -69.053047        NaN   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "\n",
       "         257        258  \n",
       "0 -70.269081 -70.269081  \n",
       "1 -67.558426 -67.558426  \n",
       "2 -69.693993 -69.693993  \n",
       "3        NaN        NaN  \n",
       "4 -73.841805 -73.841805  \n",
       "\n",
       "[5 rows x 259 columns]"
      ]
     },
     "execution_count": 138,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df3.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 139,
   "metadata": {},
   "outputs": [],
   "source": [
    "newdf = pd.concat([df3,labels], axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 140,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "400"
      ]
     },
     "execution_count": 140,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "rnewdf = newdf.rename(index=str, columns={\"0\": \"label\"})\n",
    "len(rnewdf)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 141,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "      <th>4</th>\n",
       "      <th>5</th>\n",
       "      <th>6</th>\n",
       "      <th>7</th>\n",
       "      <th>8</th>\n",
       "      <th>9</th>\n",
       "      <th>...</th>\n",
       "      <th>250</th>\n",
       "      <th>251</th>\n",
       "      <th>252</th>\n",
       "      <th>253</th>\n",
       "      <th>254</th>\n",
       "      <th>255</th>\n",
       "      <th>256</th>\n",
       "      <th>257</th>\n",
       "      <th>258</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-65.434235</td>\n",
       "      <td>-65.635078</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>...</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>...</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.005867</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-68.918213</td>\n",
       "      <td>...</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>...</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.112907</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.166939</td>\n",
       "      <td>-66.261154</td>\n",
       "      <td>-68.245354</td>\n",
       "      <td>...</td>\n",
       "      <td>-57.991135</td>\n",
       "      <td>-61.884342</td>\n",
       "      <td>-67.038483</td>\n",
       "      <td>-68.177948</td>\n",
       "      <td>-67.128456</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>-69.243927</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.180733</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>-73.255638</td>\n",
       "      <td>...</td>\n",
       "      <td>-50.809422</td>\n",
       "      <td>-55.574772</td>\n",
       "      <td>-54.543968</td>\n",
       "      <td>-53.226849</td>\n",
       "      <td>-56.153770</td>\n",
       "      <td>-57.413322</td>\n",
       "      <td>-58.656780</td>\n",
       "      <td>-59.756897</td>\n",
       "      <td>-58.867874</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.747940</td>\n",
       "      <td>-70.030136</td>\n",
       "      <td>-69.341812</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>-63.797665</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>-59.036747</td>\n",
       "      <td>-57.874493</td>\n",
       "      <td>-61.433617</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>...</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>-63.658138</td>\n",
       "      <td>-63.679913</td>\n",
       "      <td>-63.451157</td>\n",
       "      <td>-63.448044</td>\n",
       "      <td>-63.797665</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.255657</td>\n",
       "      <td>-60.949566</td>\n",
       "      <td>-60.959072</td>\n",
       "      <td>-60.982517</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.360420</td>\n",
       "      <td>...</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>-60.982449</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>NaN</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>10 rows × 260 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "           0          1          2          3          4          5  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047 -69.053047 -69.005867   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "5 -69.243927 -69.243927 -69.243927 -69.243927 -69.243927 -69.112907   \n",
       "6 -73.255638 -73.255638 -73.255638 -73.255638 -73.180733 -73.255638   \n",
       "7 -70.747940 -70.747940 -70.747940 -70.747940 -70.747940 -70.747940   \n",
       "8 -63.797665 -63.797665 -63.797665 -63.797665 -63.797665 -59.036747   \n",
       "9 -60.982449 -60.255657 -60.949566 -60.959072 -60.982517 -60.982449   \n",
       "\n",
       "           6          7          8          9  ...        250        251  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081  ... -70.269081 -70.269081   \n",
       "1 -65.434235 -65.635078 -67.558426 -67.558426  ... -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993  ... -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -68.918213  ... -69.053047 -69.053047   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805  ... -73.841805 -73.841805   \n",
       "5 -69.243927 -69.166939 -66.261154 -68.245354  ... -57.991135 -61.884342   \n",
       "6 -73.255638 -73.255638 -73.255638 -73.255638  ... -50.809422 -55.574772   \n",
       "7 -70.747940 -70.747940 -70.747940 -70.747940  ... -70.747940 -70.747940   \n",
       "8 -57.874493 -61.433617 -63.797665 -63.797665  ... -63.797665 -63.658138   \n",
       "9 -60.982449 -60.982449 -60.982449 -60.360420  ... -60.982449 -60.982449   \n",
       "\n",
       "         252        253        254        255        256        257  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047        NaN        NaN   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "5 -67.038483 -68.177948 -67.128456 -69.243927 -69.243927 -69.243927   \n",
       "6 -54.543968 -53.226849 -56.153770 -57.413322 -58.656780 -59.756897   \n",
       "7 -70.747940 -70.747940 -70.747940 -70.747940 -70.747940 -70.030136   \n",
       "8 -63.679913 -63.451157 -63.448044 -63.797665        NaN        NaN   \n",
       "9 -60.982449 -60.982449 -60.982449 -60.982449        NaN        NaN   \n",
       "\n",
       "         258          label  \n",
       "0 -70.269081  male_positive  \n",
       "1 -67.558426  male_positive  \n",
       "2 -69.693993  male_positive  \n",
       "3        NaN  male_positive  \n",
       "4 -73.841805  male_positive  \n",
       "5 -69.243927  male_positive  \n",
       "6 -58.867874  male_positive  \n",
       "7 -69.341812  male_positive  \n",
       "8        NaN  male_positive  \n",
       "9        NaN  male_positive  \n",
       "\n",
       "[10 rows x 260 columns]"
      ]
     },
     "execution_count": 141,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "rnewdf.head(10)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 142,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "2284"
      ]
     },
     "execution_count": 142,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "rnewdf.isNone().sum().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 143,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "      <th>4</th>\n",
       "      <th>5</th>\n",
       "      <th>6</th>\n",
       "      <th>7</th>\n",
       "      <th>8</th>\n",
       "      <th>9</th>\n",
       "      <th>...</th>\n",
       "      <th>250</th>\n",
       "      <th>251</th>\n",
       "      <th>252</th>\n",
       "      <th>253</th>\n",
       "      <th>254</th>\n",
       "      <th>255</th>\n",
       "      <th>256</th>\n",
       "      <th>257</th>\n",
       "      <th>258</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-65.434235</td>\n",
       "      <td>-65.635078</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>...</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>...</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.005867</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-68.918213</td>\n",
       "      <td>...</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>...</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>5 rows × 260 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "           0          1          2          3          4          5  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047 -69.053047 -69.005867   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "\n",
       "           6          7          8          9  ...        250        251  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081  ... -70.269081 -70.269081   \n",
       "1 -65.434235 -65.635078 -67.558426 -67.558426  ... -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993  ... -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -68.918213  ... -69.053047 -69.053047   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805  ... -73.841805 -73.841805   \n",
       "\n",
       "         252        253        254        255        256        257  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047   0.000000   0.000000   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "\n",
       "         258          label  \n",
       "0 -70.269081  male_positive  \n",
       "1 -67.558426  male_positive  \n",
       "2 -69.693993  male_positive  \n",
       "3   0.000000  male_positive  \n",
       "4 -73.841805  male_positive  \n",
       "\n",
       "[5 rows x 260 columns]"
      ]
     },
     "execution_count": 143,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "rnewdf = rnewdf.fillna(0)\n",
    "rnewdf.head()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# VII. Data Augmentation"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 144,
   "metadata": {},
   "outputs": [],
   "source": [
    "def plot_time_series(data):\n",
    "    \"\"\"\n",
    "    Plot the Audio Frequency.\n",
    "    \"\"\"\n",
    "    fig = plt.figure(figsize=(14, 8))\n",
    "    plt.title('Raw wave ')\n",
    "    plt.ylabel('Amplitude')\n",
    "    plt.plot(np.linspace(0, 1, len(data)), data)\n",
    "    plt.show()\n",
    "\n",
    "\n",
    "def noise(data):\n",
    "    \"\"\"\n",
    "    Adding White Noise.\n",
    "    \"\"\"\n",
    "    # you can take any distribution from https://docs.scipy.org/doc/numpy-1.13.0/reference/routines.random.html\n",
    "    noise_amp = 0.005*np.random.uniform()*np.amax(data)\n",
    "    data = data.astype('float64') + noise_amp * np.random.normal(size=data.shape[0])\n",
    "    return data\n",
    "    \n",
    "def shift(data):\n",
    "    \"\"\"\n",
    "    Random Shifting.\n",
    "    \"\"\"\n",
    "    s_range = int(np.random.uniform(low=-5, high = 5)*500)\n",
    "    return np.roll(data, s_range)\n",
    "    \n",
    "def stretch(data, rate=0.8):\n",
    "    \"\"\"\n",
    "    Streching the Sound.\n",
    "    \"\"\"\n",
    "    data = librosa.effects.time_stretch(data, rate)\n",
    "    return data\n",
    "    \n",
    "def pitch(data, sample_rate):\n",
    "    \"\"\"\n",
    "    Pitch Tuning.\n",
    "    \"\"\"\n",
    "    bins_per_octave = 12\n",
    "    pitch_pm = 2\n",
    "    pitch_change =  pitch_pm * 2*(np.random.uniform())   \n",
    "    data = librosa.effects.pitch_shift(data.astype('float64'), \n",
    "                                      sample_rate, n_steps=pitch_change, \n",
    "                                      bins_per_octave=bins_per_octave)\n",
    "    return data\n",
    "    \n",
    "def dyn_change(data):\n",
    "    \"\"\"\n",
    "    Random Value Change.\n",
    "    \"\"\"\n",
    "    dyn_change = np.random.uniform(low=1.5,high=3)\n",
    "    return (data * dyn_change)\n",
    "    \n",
    "def speedNpitch(data):\n",
    "    \"\"\"\n",
    "    peed and Pitch Tuning.\n",
    "    \"\"\"\n",
    "    # you can change low and high here\n",
    "    length_change = np.random.uniform(low=0.8, high = 1)\n",
    "    speed_fac = 1.0  / length_change\n",
    "    tmp = np.interp(np.arange(0,len(data),speed_fac),np.arange(0,len(data)),data)\n",
    "    minlen = min(data.shape[0], tmp.shape[0])\n",
    "    data *= 0\n",
    "    data[0:minlen] = tmp[0:minlen]\n",
    "    return data\n",
    "\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 145,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1008x576 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "text/html": [
       "\n",
       "                <audio  controls=\"controls\" >\n",
       "                    <source src=\"...\" type=\"audio/wav\" />\n",
       "                    Your browser does not support the audio element.\n",
       "                </audio>\n",
       "              "
      ],
      "text/plain": [
       "<IPython.lib.display.Audio object>"
      ]
     },
     "execution_count": 145,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "X, sample_rate = librosa.load(data2_df.path[216], res_type='kaiser_fast',duration=4,sr=22050*2,offset=0.5)\n",
    "plot_time_series(X)\n",
    "ipd.Audio(X, rate=sample_rate)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 146,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1008x576 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "text/html": [
       "\n",
       "                <audio  controls=\"controls\" >\n",
       "                    <source src=\"...\" type=\"audio/wav\" />\n",
       "                    Your browser does not support the audio element.\n",
       "                </audio>\n",
       "              "
      ],
      "text/plain": [
       "<IPython.lib.display.Audio object>"
      ]
     },
     "execution_count": 146,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "x = pitch(X, sample_rate)\n",
    "plot_time_series(x)\n",
    "ipd.Audio(x, rate=sample_rate)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 147,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:18<00:00, 21.62it/s]\n"
     ]
    }
   ],
   "source": [
    "# Augmentation Method 1\n",
    "\n",
    "syn_data1 = pd.DataFrame(columns=['feature', 'label'])\n",
    "for i in tqdm(range(len(data2_df))):\n",
    "    X, sample_rate = librosa.load(data2_df.path[i], res_type='kaiser_fast',duration=input_duration,sr=22050*2,offset=0.5)\n",
    "    if data2_df.label[i]:\n",
    "#     if data2_df.label[i] == \"male_positive\":\n",
    "        X = noise(X)\n",
    "        sample_rate = np.array(sample_rate)\n",
    "        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)\n",
    "        feature = mfccs\n",
    "        a = random.uniform(0, 1)\n",
    "        syn_data1.loc[i] = [feature, data2_df.label[i]]\n",
    "        "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 148,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [01:37<00:00,  4.09it/s]\n"
     ]
    }
   ],
   "source": [
    "# Augmentation Method 2\n",
    "\n",
    "syn_data2 = pd.DataFrame(columns=['feature', 'label'])\n",
    "for i in tqdm(range(len(data2_df))):\n",
    "    X, sample_rate = librosa.load(data2_df.path[i], res_type='kaiser_fast',duration=input_duration,sr=22050*2,offset=0.5)\n",
    "    if data2_df.label[i]:\n",
    "#     if data2_df.label[i] == \"male_positive\":\n",
    "        X = pitch(X, sample_rate)\n",
    "        sample_rate = np.array(sample_rate)\n",
    "        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)\n",
    "        feature = mfccs\n",
    "        a = random.uniform(0, 1)\n",
    "        syn_data2.loc[i] = [feature, data2_df.label[i]]\n",
    "        "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 149,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(400, 400)"
      ]
     },
     "execution_count": 149,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(syn_data1), len(syn_data2) "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 150,
   "metadata": {},
   "outputs": [],
   "source": [
    "syn_data1 = syn_data1.reset_index(drop=True)\n",
    "syn_data2 = syn_data2.reset_index(drop=True)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 151,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "400"
      ]
     },
     "execution_count": 151,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df4 = pd.DataFrame(syn_data1['feature'].values.tolist())\n",
    "labels4 = syn_data1.label\n",
    "syndf1 = pd.concat([df4,labels4], axis=1)\n",
    "syndf1 = syndf1.rename(index=str, columns={\"0\": \"label\"})\n",
    "syndf1 = syndf1.fillna(0)\n",
    "len(syndf1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 152,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "      <th>4</th>\n",
       "      <th>5</th>\n",
       "      <th>6</th>\n",
       "      <th>7</th>\n",
       "      <th>8</th>\n",
       "      <th>9</th>\n",
       "      <th>...</th>\n",
       "      <th>250</th>\n",
       "      <th>251</th>\n",
       "      <th>252</th>\n",
       "      <th>253</th>\n",
       "      <th>254</th>\n",
       "      <th>255</th>\n",
       "      <th>256</th>\n",
       "      <th>257</th>\n",
       "      <th>258</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>-67.501613</td>\n",
       "      <td>-63.247109</td>\n",
       "      <td>-60.202636</td>\n",
       "      <td>-58.132352</td>\n",
       "      <td>-57.917084</td>\n",
       "      <td>-58.285566</td>\n",
       "      <td>-61.120950</td>\n",
       "      <td>-60.799884</td>\n",
       "      <td>-60.491254</td>\n",
       "      <td>-61.095390</td>\n",
       "      <td>...</td>\n",
       "      <td>-59.443395</td>\n",
       "      <td>-59.629200</td>\n",
       "      <td>-59.527165</td>\n",
       "      <td>-60.428864</td>\n",
       "      <td>-61.115954</td>\n",
       "      <td>-61.716086</td>\n",
       "      <td>-59.677856</td>\n",
       "      <td>-61.786687</td>\n",
       "      <td>-63.391250</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>-64.076151</td>\n",
       "      <td>-62.829781</td>\n",
       "      <td>-62.264875</td>\n",
       "      <td>-62.097716</td>\n",
       "      <td>-62.016164</td>\n",
       "      <td>-63.622768</td>\n",
       "      <td>-63.665668</td>\n",
       "      <td>-62.513625</td>\n",
       "      <td>-60.993501</td>\n",
       "      <td>-60.211762</td>\n",
       "      <td>...</td>\n",
       "      <td>-60.013881</td>\n",
       "      <td>-62.850593</td>\n",
       "      <td>-62.527192</td>\n",
       "      <td>-59.482334</td>\n",
       "      <td>-61.075535</td>\n",
       "      <td>-61.528257</td>\n",
       "      <td>-62.377972</td>\n",
       "      <td>-60.319339</td>\n",
       "      <td>-60.720926</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>-57.647941</td>\n",
       "      <td>-56.390651</td>\n",
       "      <td>-56.085176</td>\n",
       "      <td>-53.520781</td>\n",
       "      <td>-54.228542</td>\n",
       "      <td>-53.513820</td>\n",
       "      <td>-53.803102</td>\n",
       "      <td>-53.998886</td>\n",
       "      <td>-54.392238</td>\n",
       "      <td>-55.106582</td>\n",
       "      <td>...</td>\n",
       "      <td>-54.246286</td>\n",
       "      <td>-53.616619</td>\n",
       "      <td>-55.444986</td>\n",
       "      <td>-57.842706</td>\n",
       "      <td>-56.390067</td>\n",
       "      <td>-55.533582</td>\n",
       "      <td>-56.032392</td>\n",
       "      <td>-56.051366</td>\n",
       "      <td>-55.172123</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>-68.010831</td>\n",
       "      <td>-64.964521</td>\n",
       "      <td>-65.247758</td>\n",
       "      <td>-65.133780</td>\n",
       "      <td>-65.192203</td>\n",
       "      <td>-65.097121</td>\n",
       "      <td>-65.132199</td>\n",
       "      <td>-64.912027</td>\n",
       "      <td>-66.273033</td>\n",
       "      <td>-64.252885</td>\n",
       "      <td>...</td>\n",
       "      <td>-64.061722</td>\n",
       "      <td>-63.510034</td>\n",
       "      <td>-63.297921</td>\n",
       "      <td>-64.747269</td>\n",
       "      <td>-64.829299</td>\n",
       "      <td>-64.668101</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>-62.637717</td>\n",
       "      <td>-60.878367</td>\n",
       "      <td>-59.162805</td>\n",
       "      <td>-58.216770</td>\n",
       "      <td>-57.695962</td>\n",
       "      <td>-58.656503</td>\n",
       "      <td>-57.597098</td>\n",
       "      <td>-57.381009</td>\n",
       "      <td>-56.167200</td>\n",
       "      <td>-55.130039</td>\n",
       "      <td>...</td>\n",
       "      <td>-59.639604</td>\n",
       "      <td>-60.266822</td>\n",
       "      <td>-60.552137</td>\n",
       "      <td>-59.019028</td>\n",
       "      <td>-56.329578</td>\n",
       "      <td>-56.644446</td>\n",
       "      <td>-58.176731</td>\n",
       "      <td>-59.439603</td>\n",
       "      <td>-60.342576</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>5 rows × 260 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "           0          1          2          3          4          5  \\\n",
       "0 -67.501613 -63.247109 -60.202636 -58.132352 -57.917084 -58.285566   \n",
       "1 -64.076151 -62.829781 -62.264875 -62.097716 -62.016164 -63.622768   \n",
       "2 -57.647941 -56.390651 -56.085176 -53.520781 -54.228542 -53.513820   \n",
       "3 -68.010831 -64.964521 -65.247758 -65.133780 -65.192203 -65.097121   \n",
       "4 -62.637717 -60.878367 -59.162805 -58.216770 -57.695962 -58.656503   \n",
       "\n",
       "           6          7          8          9  ...        250        251  \\\n",
       "0 -61.120950 -60.799884 -60.491254 -61.095390  ... -59.443395 -59.629200   \n",
       "1 -63.665668 -62.513625 -60.993501 -60.211762  ... -60.013881 -62.850593   \n",
       "2 -53.803102 -53.998886 -54.392238 -55.106582  ... -54.246286 -53.616619   \n",
       "3 -65.132199 -64.912027 -66.273033 -64.252885  ... -64.061722 -63.510034   \n",
       "4 -57.597098 -57.381009 -56.167200 -55.130039  ... -59.639604 -60.266822   \n",
       "\n",
       "         252        253        254        255        256        257  \\\n",
       "0 -59.527165 -60.428864 -61.115954 -61.716086 -59.677856 -61.786687   \n",
       "1 -62.527192 -59.482334 -61.075535 -61.528257 -62.377972 -60.319339   \n",
       "2 -55.444986 -57.842706 -56.390067 -55.533582 -56.032392 -56.051366   \n",
       "3 -63.297921 -64.747269 -64.829299 -64.668101   0.000000   0.000000   \n",
       "4 -60.552137 -59.019028 -56.329578 -56.644446 -58.176731 -59.439603   \n",
       "\n",
       "         258          label  \n",
       "0 -63.391250  male_positive  \n",
       "1 -60.720926  male_positive  \n",
       "2 -55.172123  male_positive  \n",
       "3   0.000000  male_positive  \n",
       "4 -60.342576  male_positive  \n",
       "\n",
       "[5 rows x 260 columns]"
      ]
     },
     "execution_count": 152,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "syndf1.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 153,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "400"
      ]
     },
     "execution_count": 153,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df4 = pd.DataFrame(syn_data2['feature'].values.tolist())\n",
    "labels4 = syn_data2.label\n",
    "syndf2 = pd.concat([df4,labels4], axis=1)\n",
    "syndf2 = syndf2.rename(index=str, columns={\"0\": \"label\"})\n",
    "syndf2 = syndf2.fillna(0)\n",
    "len(syndf2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 154,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "      <th>4</th>\n",
       "      <th>5</th>\n",
       "      <th>6</th>\n",
       "      <th>7</th>\n",
       "      <th>8</th>\n",
       "      <th>9</th>\n",
       "      <th>...</th>\n",
       "      <th>250</th>\n",
       "      <th>251</th>\n",
       "      <th>252</th>\n",
       "      <th>253</th>\n",
       "      <th>254</th>\n",
       "      <th>255</th>\n",
       "      <th>256</th>\n",
       "      <th>257</th>\n",
       "      <th>258</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>-70.996518</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-65.984394</td>\n",
       "      <td>-66.281247</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>-70.423599</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.165288</td>\n",
       "      <td>...</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>-72.421590</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.918336</td>\n",
       "      <td>-71.427715</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.269480</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.914767</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>-71.923342</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>...</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>-73.776266</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>5 rows × 260 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "           0          1          2          3          4          5  \\\n",
       "0 -70.996518 -70.996518 -70.996518 -70.996518 -70.996518 -70.996518   \n",
       "1 -70.423599 -70.423599 -70.423599 -70.423599 -70.423599 -70.423599   \n",
       "2 -72.421590 -72.421590 -72.421590 -72.421590 -72.421590 -72.421590   \n",
       "3 -71.923342 -71.923342 -71.923342 -71.923342 -71.918336 -71.427715   \n",
       "4 -73.776266 -73.776266 -73.776266 -73.776266 -73.776266 -73.776266   \n",
       "\n",
       "           6          7          8          9  ...        250        251  \\\n",
       "0 -70.996518 -70.996518 -70.996518 -70.996518  ... -70.996518 -70.996518   \n",
       "1 -65.984394 -66.281247 -70.423599 -70.423599  ... -70.423599 -70.423599   \n",
       "2 -72.421590 -72.421590 -72.421590 -72.165288  ... -72.421590 -72.421590   \n",
       "3 -71.923342 -71.923342 -71.923342 -71.269480  ... -70.914767 -71.923342   \n",
       "4 -73.776266 -73.776266 -73.776266 -73.776266  ... -73.776266 -73.776266   \n",
       "\n",
       "         252        253        254        255        256        257  \\\n",
       "0 -70.996518 -70.996518 -70.996518 -70.996518 -70.996518 -70.996518   \n",
       "1 -70.423599 -70.423599 -70.423599 -70.423599 -70.423599 -70.423599   \n",
       "2 -72.421590 -72.421590 -72.421590 -72.421590 -72.421590 -72.421590   \n",
       "3 -71.923342 -71.923342 -71.923342 -71.923342   0.000000   0.000000   \n",
       "4 -73.776266 -73.776266 -73.776266 -73.776266 -73.776266 -73.776266   \n",
       "\n",
       "         258          label  \n",
       "0 -70.996518  male_positive  \n",
       "1 -70.423599  male_positive  \n",
       "2 -72.421590  male_positive  \n",
       "3   0.000000  male_positive  \n",
       "4 -73.776266  male_positive  \n",
       "\n",
       "[5 rows x 260 columns]"
      ]
     },
     "execution_count": 154,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "syndf2.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 155,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>0</th>\n",
       "      <th>1</th>\n",
       "      <th>2</th>\n",
       "      <th>3</th>\n",
       "      <th>4</th>\n",
       "      <th>5</th>\n",
       "      <th>6</th>\n",
       "      <th>7</th>\n",
       "      <th>8</th>\n",
       "      <th>9</th>\n",
       "      <th>...</th>\n",
       "      <th>250</th>\n",
       "      <th>251</th>\n",
       "      <th>252</th>\n",
       "      <th>253</th>\n",
       "      <th>254</th>\n",
       "      <th>255</th>\n",
       "      <th>256</th>\n",
       "      <th>257</th>\n",
       "      <th>258</th>\n",
       "      <th>label</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>...</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>-70.269081</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-65.434235</td>\n",
       "      <td>-65.635078</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>...</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>-67.558426</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>...</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>-69.693993</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.005867</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-68.918213</td>\n",
       "      <td>...</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>-69.053047</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>0.000000</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>...</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>-73.841805</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>5 rows × 260 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "           0          1          2          3          4          5  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047 -69.053047 -69.005867   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "\n",
       "           6          7          8          9  ...        250        251  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081  ... -70.269081 -70.269081   \n",
       "1 -65.434235 -65.635078 -67.558426 -67.558426  ... -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993  ... -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -68.918213  ... -69.053047 -69.053047   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805  ... -73.841805 -73.841805   \n",
       "\n",
       "         252        253        254        255        256        257  \\\n",
       "0 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081 -70.269081   \n",
       "1 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426 -67.558426   \n",
       "2 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993 -69.693993   \n",
       "3 -69.053047 -69.053047 -69.053047 -69.053047   0.000000   0.000000   \n",
       "4 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805 -73.841805   \n",
       "\n",
       "         258          label  \n",
       "0 -70.269081  male_positive  \n",
       "1 -67.558426  male_positive  \n",
       "2 -69.693993  male_positive  \n",
       "3   0.000000  male_positive  \n",
       "4 -73.841805  male_positive  \n",
       "\n",
       "[5 rows x 260 columns]"
      ]
     },
     "execution_count": 155,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "# Combining the Augmented data with original\n",
    "combined_df = pd.concat([rnewdf, syndf1, syndf2], ignore_index=True)\n",
    "combined_df = combined_df.fillna(0)\n",
    "combined_df.head()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 156,
   "metadata": {},
   "outputs": [],
   "source": [
    "#  Stratified Shuffle Split\n",
    "\n",
    "X = combined_df.drop(['label'], axis=1)\n",
    "y = combined_df.label\n",
    "xxx = StratifiedShuffleSplit(1, test_size=0.2, random_state=12)\n",
    "for train_index, test_index in xxx.split(X, y):\n",
    "    X_train, X_test = X.iloc[train_index], X.iloc[test_index]\n",
    "    y_train, y_test = y.iloc[train_index], y.iloc[test_index]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 157,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "male_negative    576\n",
       "male_positive    384\n",
       "Name: label, dtype: int64"
      ]
     },
     "execution_count": 157,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "y_train.value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 158,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "male_negative    144\n",
       "male_positive     96\n",
       "Name: label, dtype: int64"
      ]
     },
     "execution_count": 158,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "y_test.value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 159,
   "metadata": {
    "scrolled": True
   },
   "outputs": [
    {
     "data": {
      "text/plain": [
       "0"
      ]
     },
     "execution_count": 159,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "X_train.isna().sum().sum()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 160,
   "metadata": {},
   "outputs": [],
   "source": [
    "X_train = np.array(X_train)\n",
    "y_train = np.array(y_train)\n",
    "X_test = np.array(X_test)\n",
    "y_test = np.array(y_test)\n",
    "lb = LabelEncoder()\n",
    "y_train = np_utils.to_categorical(lb.fit_transform(y_train))\n",
    "y_test = np_utils.to_categorical(lb.fit_transform(y_test))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 161,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[1., 0.],\n",
       "       [1., 0.],\n",
       "       [0., 1.],\n",
       "       ...,\n",
       "       [1., 0.],\n",
       "       [1., 0.],\n",
       "       [0., 1.]], dtype=float32)"
      ]
     },
     "execution_count": 161,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "y_train"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 162,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "(960, 259)"
      ]
     },
     "execution_count": 162,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "X_train.shape"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# VIII. Changing dimension for CNN model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 163,
   "metadata": {},
   "outputs": [],
   "source": [
    "x_traincnn = np.expand_dims(X_train, axis=2)\n",
    "x_testcnn = np.expand_dims(X_test, axis=2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 164,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Set up Keras util functions\n",
    "\n",
    "from keras import backend as K\n",
    "\n",
    "def precision(y_True, y_pred):\n",
    "    True_positives = K.sum(K.round(K.clip(y_True * y_pred, 0, 1)))\n",
    "    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))\n",
    "    precision = True_positives / (predicted_positives + K.epsilon())\n",
    "    return precision\n",
    "\n",
    "\n",
    "def recall(y_True, y_pred):\n",
    "    True_positives = K.sum(K.round(K.clip(y_True * y_pred, 0, 1)))\n",
    "    possible_positives = (K.sum(K.round(K.clip(y_True, 0, 1))))\n",
    "    recall = True_positives / (possible_positives + K.epsilon())\n",
    "    return recall\n",
    "\n",
    "\n",
    "def fscore(y_True, y_pred):\n",
    "    if K.sum(K.round(K.clip(y_True, 0, 1))) == 0:\n",
    "        return 0.0\n",
    "\n",
    "    p = precision(y_True, y_pred)\n",
    "    r = recall(y_True, y_pred)\n",
    "    f_score = 2 * (p * r) / (p + r + K.epsilon())\n",
    "    return float(f_score)\n",
    "\n",
    "def get_lr_metric(optimizer):\n",
    "    def lr(y_True, y_pred):\n",
    "        return optimizer.lr\n",
    "    return lr\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 165,
   "metadata": {},
   "outputs": [],
   "source": [
    "# New model\n",
    "model = Sequential()\n",
    "model.add(Conv1D(256, 8, padding='same',input_shape=(X_train.shape[1],1)))\n",
    "model.add(Activation('relu'))\n",
    "model.add(Conv1D(256, 8, padding='same'))\n",
    "model.add(BatchNormalization())\n",
    "model.add(Activation('relu'))\n",
    "model.add(Dropout(0.25))\n",
    "model.add(MaxPooling1D(pool_size=(8)))\n",
    "model.add(Conv1D(128, 8, padding='same'))\n",
    "model.add(Activation('relu'))\n",
    "model.add(Conv1D(128, 8, padding='same'))\n",
    "model.add(Activation('relu'))\n",
    "model.add(Conv1D(128, 8, padding='same'))\n",
    "model.add(Activation('relu'))\n",
    "model.add(Conv1D(128, 8, padding='same'))\n",
    "model.add(BatchNormalization())\n",
    "model.add(Activation('relu'))\n",
    "model.add(Dropout(0.25))\n",
    "model.add(MaxPooling1D(pool_size=(8)))\n",
    "model.add(Conv1D(64, 8, padding='same'))\n",
    "model.add(Activation('relu'))\n",
    "model.add(Conv1D(64, 8, padding='same'))\n",
    "model.add(Activation('relu'))\n",
    "model.add(Flatten())\n",
    "# Edit according to target class no.\n",
    "model.add(Dense(2))\n",
    "model.add(Activation('softmax'))\n",
    "opt = tf.keras.optimizers.SGD(lr=0.0001, momentum=0.0, decay=0.0, nesterov=False)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 166,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Original Model\n",
    "\n",
    "# model = Sequential()\n",
    "# model.add(Conv1D(256, 5,padding='same', input_shape=(X_train.shape[1],1)))\n",
    "# model.add(Activation('relu'))\n",
    "# model.add(Conv1D(128, 5,padding='same'))\n",
    "# model.add(Activation('relu'))\n",
    "# model.add(Dropout(0.1))\n",
    "# model.add(MaxPooling1D(pool_size=(8)))\n",
    "# model.add(Conv1D(128, 5,padding='same',))\n",
    "# model.add(Activation('relu'))\n",
    "# model.add(Conv1D(128, 5,padding='same',))\n",
    "# model.add(Activation('relu'))\n",
    "# model.add(Conv1D(128, 5,padding='same',))\n",
    "# model.add(Activation('relu'))\n",
    "# model.add(Dropout(0.2))\n",
    "# model.add(Conv1D(128, 5,padding='same',))\n",
    "# model.add(Activation('relu'))\n",
    "# model.add(Flatten())\n",
    "# model.add(Dense(5))\n",
    "# model.add(Activation('softmax'))\n",
    "# opt = keras.optimizers.rmsprop(lr=0.00001, decay=1e-6)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 167,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Model: \"sequential_1\"\n",
      "_________________________________________________________________\n",
      " Layer (type)                Output Shape              Param #   \n",
      "=================================================================\n",
      " conv1d_8 (Conv1D)           (None, 259, 256)          2304      \n",
      "                                                                 \n",
      " activation_9 (Activation)   (None, 259, 256)          0         \n",
      "                                                                 \n",
      " conv1d_9 (Conv1D)           (None, 259, 256)          524544    \n",
      "                                                                 \n",
      " batch_normalization_2 (Batc  (None, 259, 256)         1024      \n",
      " hNormalization)                                                 \n",
      "                                                                 \n",
      " activation_10 (Activation)  (None, 259, 256)          0         \n",
      "                                                                 \n",
      " dropout_2 (Dropout)         (None, 259, 256)          0         \n",
      "                                                                 \n",
      " max_pooling1d_2 (MaxPooling  (None, 32, 256)          0         \n",
      " 1D)                                                             \n",
      "                                                                 \n",
      " conv1d_10 (Conv1D)          (None, 32, 128)           262272    \n",
      "                                                                 \n",
      " activation_11 (Activation)  (None, 32, 128)           0         \n",
      "                                                                 \n",
      " conv1d_11 (Conv1D)          (None, 32, 128)           131200    \n",
      "                                                                 \n",
      " activation_12 (Activation)  (None, 32, 128)           0         \n",
      "                                                                 \n",
      " conv1d_12 (Conv1D)          (None, 32, 128)           131200    \n",
      "                                                                 \n",
      " activation_13 (Activation)  (None, 32, 128)           0         \n",
      "                                                                 \n",
      " conv1d_13 (Conv1D)          (None, 32, 128)           131200    \n",
      "                                                                 \n",
      " batch_normalization_3 (Batc  (None, 32, 128)          512       \n",
      " hNormalization)                                                 \n",
      "                                                                 \n",
      " activation_14 (Activation)  (None, 32, 128)           0         \n",
      "                                                                 \n",
      " dropout_3 (Dropout)         (None, 32, 128)           0         \n",
      "                                                                 \n",
      " max_pooling1d_3 (MaxPooling  (None, 4, 128)           0         \n",
      " 1D)                                                             \n",
      "                                                                 \n",
      " conv1d_14 (Conv1D)          (None, 4, 64)             65600     \n",
      "                                                                 \n",
      " activation_15 (Activation)  (None, 4, 64)             0         \n",
      "                                                                 \n",
      " conv1d_15 (Conv1D)          (None, 4, 64)             32832     \n",
      "                                                                 \n",
      " activation_16 (Activation)  (None, 4, 64)             0         \n",
      "                                                                 \n",
      " flatten_1 (Flatten)         (None, 256)               0         \n",
      "                                                                 \n",
      " dense_1 (Dense)             (None, 2)                 514       \n",
      "                                                                 \n",
      " activation_17 (Activation)  (None, 2)                 0         \n",
      "                                                                 \n",
      "=================================================================\n",
      "Total params: 1,283,202\n",
      "Trainable params: 1,282,434\n",
      "Non-trainable params: 768\n",
      "_________________________________________________________________\n"
     ]
    }
   ],
   "source": [
    "# Plotting Model Summary\n",
    "\n",
    "model.summary()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 168,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Compile your model\n",
    "\n",
    "model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', fscore])"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# IX. Removed the whole training part for avoiding unnecessary long epochs list"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 169,
   "metadata": {
    "scrolled": True
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch 1/25\n",
      "60/60 [==============================] - 14s 184ms/step - loss: 0.6997 - accuracy: 0.5833 - fscore: 0.5833 - val_loss: 0.6864 - val_accuracy: 0.6000 - val_fscore: 0.6000 - lr: 1.0000e-04\n",
      "Epoch 2/25\n",
      "60/60 [==============================] - 11s 176ms/step - loss: 0.6515 - accuracy: 0.6156 - fscore: 0.6156 - val_loss: 0.6877 - val_accuracy: 0.5875 - val_fscore: 0.5875 - lr: 1.0000e-04\n",
      "Epoch 3/25\n",
      "60/60 [==============================] - 11s 185ms/step - loss: 0.6389 - accuracy: 0.6271 - fscore: 0.6271 - val_loss: 0.6786 - val_accuracy: 0.6208 - val_fscore: 0.6208 - lr: 1.0000e-04\n",
      "Epoch 4/25\n",
      "60/60 [==============================] - 11s 192ms/step - loss: 0.6277 - accuracy: 0.6417 - fscore: 0.6417 - val_loss: 0.6660 - val_accuracy: 0.6917 - val_fscore: 0.6917 - lr: 1.0000e-04\n",
      "Epoch 5/25\n",
      "60/60 [==============================] - 11s 182ms/step - loss: 0.6210 - accuracy: 0.6490 - fscore: 0.6490 - val_loss: 0.6539 - val_accuracy: 0.6667 - val_fscore: 0.6667 - lr: 1.0000e-04\n",
      "Epoch 6/25\n",
      "60/60 [==============================] - 13s 217ms/step - loss: 0.6122 - accuracy: 0.6521 - fscore: 0.6521 - val_loss: 0.6364 - val_accuracy: 0.6792 - val_fscore: 0.6792 - lr: 1.0000e-04\n",
      "Epoch 7/25\n",
      "60/60 [==============================] - 12s 199ms/step - loss: 0.6081 - accuracy: 0.6760 - fscore: 0.6760 - val_loss: 0.6280 - val_accuracy: 0.6875 - val_fscore: 0.6875 - lr: 1.0000e-04\n",
      "Epoch 8/25\n",
      "60/60 [==============================] - 12s 195ms/step - loss: 0.6045 - accuracy: 0.6771 - fscore: 0.6771 - val_loss: 0.6183 - val_accuracy: 0.7042 - val_fscore: 0.7042 - lr: 1.0000e-04\n",
      "Epoch 9/25\n",
      "60/60 [==============================] - 12s 192ms/step - loss: 0.5952 - accuracy: 0.6885 - fscore: 0.6885 - val_loss: 0.6123 - val_accuracy: 0.7167 - val_fscore: 0.7167 - lr: 1.0000e-04\n",
      "Epoch 10/25\n",
      "60/60 [==============================] - 11s 180ms/step - loss: 0.5976 - accuracy: 0.6687 - fscore: 0.6687 - val_loss: 0.6112 - val_accuracy: 0.7125 - val_fscore: 0.7125 - lr: 1.0000e-04\n",
      "Epoch 11/25\n",
      "60/60 [==============================] - 12s 194ms/step - loss: 0.5893 - accuracy: 0.7073 - fscore: 0.7073 - val_loss: 0.6047 - val_accuracy: 0.7167 - val_fscore: 0.7167 - lr: 1.0000e-04\n",
      "Epoch 12/25\n",
      "60/60 [==============================] - 11s 181ms/step - loss: 0.5759 - accuracy: 0.7073 - fscore: 0.7073 - val_loss: 0.6037 - val_accuracy: 0.6958 - val_fscore: 0.6958 - lr: 1.0000e-04\n",
      "Epoch 13/25\n",
      "60/60 [==============================] - 12s 204ms/step - loss: 0.5772 - accuracy: 0.7115 - fscore: 0.7115 - val_loss: 0.6011 - val_accuracy: 0.7292 - val_fscore: 0.7292 - lr: 1.0000e-04\n",
      "Epoch 14/25\n",
      "60/60 [==============================] - 11s 186ms/step - loss: 0.5749 - accuracy: 0.6917 - fscore: 0.6917 - val_loss: 0.5983 - val_accuracy: 0.7167 - val_fscore: 0.7167 - lr: 1.0000e-04\n",
      "Epoch 15/25\n",
      "60/60 [==============================] - 12s 199ms/step - loss: 0.5712 - accuracy: 0.7063 - fscore: 0.7062 - val_loss: 0.5959 - val_accuracy: 0.7083 - val_fscore: 0.7083 - lr: 1.0000e-04\n",
      "Epoch 16/25\n",
      "60/60 [==============================] - 11s 189ms/step - loss: 0.5677 - accuracy: 0.7135 - fscore: 0.7135 - val_loss: 0.5953 - val_accuracy: 0.7000 - val_fscore: 0.7000 - lr: 1.0000e-04\n",
      "Epoch 17/25\n",
      "60/60 [==============================] - 11s 190ms/step - loss: 0.5636 - accuracy: 0.7125 - fscore: 0.7125 - val_loss: 0.5919 - val_accuracy: 0.7417 - val_fscore: 0.7417 - lr: 1.0000e-04\n",
      "Epoch 18/25\n",
      "60/60 [==============================] - 13s 224ms/step - loss: 0.5608 - accuracy: 0.7115 - fscore: 0.7115 - val_loss: 0.5960 - val_accuracy: 0.7083 - val_fscore: 0.7083 - lr: 1.0000e-04\n",
      "Epoch 19/25\n",
      "60/60 [==============================] - 11s 183ms/step - loss: 0.5600 - accuracy: 0.7229 - fscore: 0.7229 - val_loss: 0.5949 - val_accuracy: 0.6958 - val_fscore: 0.6958 - lr: 1.0000e-04\n",
      "Epoch 20/25\n",
      "60/60 [==============================] - 11s 180ms/step - loss: 0.5639 - accuracy: 0.7104 - fscore: 0.7104 - val_loss: 0.5897 - val_accuracy: 0.7167 - val_fscore: 0.7167 - lr: 1.0000e-04\n",
      "Epoch 21/25\n",
      "60/60 [==============================] - 13s 224ms/step - loss: 0.5557 - accuracy: 0.7333 - fscore: 0.7333 - val_loss: 0.5894 - val_accuracy: 0.7292 - val_fscore: 0.7292 - lr: 1.0000e-04\n",
      "Epoch 22/25\n",
      "60/60 [==============================] - 14s 235ms/step - loss: 0.5579 - accuracy: 0.7188 - fscore: 0.7187 - val_loss: 0.5867 - val_accuracy: 0.7417 - val_fscore: 0.7417 - lr: 1.0000e-04\n",
      "Epoch 23/25\n",
      "60/60 [==============================] - 11s 185ms/step - loss: 0.5539 - accuracy: 0.7312 - fscore: 0.7312 - val_loss: 0.5824 - val_accuracy: 0.7458 - val_fscore: 0.7458 - lr: 1.0000e-04\n",
      "Epoch 24/25\n",
      "60/60 [==============================] - 11s 178ms/step - loss: 0.5488 - accuracy: 0.7292 - fscore: 0.7292 - val_loss: 0.5754 - val_accuracy: 0.7333 - val_fscore: 0.7333 - lr: 1.0000e-04\n",
      "Epoch 25/25\n",
      "60/60 [==============================] - 10s 174ms/step - loss: 0.5506 - accuracy: 0.7427 - fscore: 0.7427 - val_loss: 0.5814 - val_accuracy: 0.7500 - val_fscore: 0.7500 - lr: 1.0000e-04\n"
     ]
    }
   ],
   "source": [
    "# Model Training\n",
    "\n",
    "lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=20, min_lr=0.000001)\n",
    "mcp_save = ModelCheckpoint('model/aug_noiseNshift_2class2_np.h5', save_best_only=True, monitor='val_loss', mode='min')\n",
    "cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=25,\n",
    "                     validation_data=(x_testcnn, y_test), callbacks=[mcp_save, lr_reduce])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 170,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    },
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 432x288 with 1 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Plotting the Train Valid Loss Graph & accuracy graph\n",
    "\n",
    "plt.plot(cnnhistory.history['loss'])\n",
    "plt.plot(cnnhistory.history['val_loss'])\n",
    "plt.title('model loss')\n",
    "plt.ylabel('loss')\n",
    "plt.xlabel('epoch')\n",
    "plt.legend(['train', 'test'], loc='upper left')\n",
    "plt.show()\n",
    "\n",
    "plt.plot(cnnhistory.history['accuracy'])\n",
    "plt.plot(cnnhistory.history['val_accuracy'])\n",
    "plt.title('model accuracy')\n",
    "plt.ylabel('accuracy')\n",
    "plt.xlabel('epoch')\n",
    "plt.legend(['train', 'test'], loc='upper right')\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Saving the model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 171,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Saving the model.json\n",
    "\n",
    "import json\n",
    "model_json = model.to_json()\n",
    "with open(\"model.json\", \"w\") as json_file:\n",
    "    json_file.write(model_json)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Loading the model"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 172,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Loaded model from disk\n",
      "accuracy: 73.33%\n"
     ]
    }
   ],
   "source": [
    "# loading json and creating model\n",
    "from keras.models import model_from_json\n",
    "json_file = open('model.json', 'r')\n",
    "loaded_model_json = json_file.read()\n",
    "json_file.close()\n",
    "loaded_model = model_from_json(loaded_model_json)\n",
    "\n",
    "# load weights into new model\n",
    "loaded_model.load_weights(\"model/aug_noiseNshift_2class2_np.h5\")\n",
    "print(\"Loaded model from disk\")\n",
    " \n",
    "# evaluate loaded model on test data\n",
    "loaded_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])\n",
    "score = loaded_model.evaluate(x_testcnn, y_test, verbose=0)\n",
    "print(\"%s: %.2f%%\" % (loaded_model.metrics_names[1], score[1]*100))"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# X. Predicting emotions on the test data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 173,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "80"
      ]
     },
     "execution_count": 173,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "len(data3_df)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 174,
   "metadata": {},
   "outputs": [
    {
     "name": "stderr",
     "output_type": "stream",
     "text": [
      "100%|██████████████████████████████████████████████████████████████████████████████████| 80/80 [00:05<00:00, 15.72it/s]\n"
     ]
    }
   ],
   "source": [
    "data_test = pd.DataFrame(columns=['feature'])\n",
    "for i in tqdm(range(len(data3_df))):\n",
    "    X, sample_rate = librosa.load(data3_df.path[i], res_type='kaiser_fast',duration=input_duration,sr=22050*2,offset=0.5)\n",
    "#     X = X[10000:90000]\n",
    "    sample_rate = np.array(sample_rate)\n",
    "    mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=13), axis=0)\n",
    "    feature = mfccs\n",
    "    data_test.loc[i] = [feature]\n",
    "    \n",
    "test_valid = pd.DataFrame(data_test['feature'].values.tolist())\n",
    "test_valid = np.array(test_valid)\n",
    "test_valid_lb = np.array(data3_df.label)\n",
    "lb = LabelEncoder()\n",
    "test_valid_lb = np_utils.to_categorical(lb.fit_transform(test_valid_lb))\n",
    "test_valid = np.expand_dims(test_valid, axis=2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 175,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "5/5 [==============================] - 1s 33ms/step\n"
     ]
    }
   ],
   "source": [
    "preds = loaded_model.predict(test_valid, \n",
    "                         batch_size=16, \n",
    "                         verbose=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 176,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[0.39799744, 0.60200256],\n",
       "       [0.4317867 , 0.5682133 ],\n",
       "       [0.52339804, 0.47660193],\n",
       "       [0.39828992, 0.6017101 ],\n",
       "       [0.43527666, 0.5647234 ],\n",
       "       [0.44211674, 0.55788326],\n",
       "       [0.41547117, 0.5845288 ],\n",
       "       [0.37163547, 0.62836456],\n",
       "       [0.41603145, 0.58396846],\n",
       "       [0.42191023, 0.5780897 ],\n",
       "       [0.55916715, 0.44083282],\n",
       "       [0.5546409 , 0.44535914],\n",
       "       [0.5332749 , 0.46672508],\n",
       "       [0.517651  , 0.48234904],\n",
       "       [0.5643516 , 0.43564835],\n",
       "       [0.60008967, 0.39991036],\n",
       "       [0.45038965, 0.5496103 ],\n",
       "       [0.49098086, 0.50901914],\n",
       "       [0.37377146, 0.6262285 ],\n",
       "       [0.2736539 , 0.7263461 ],\n",
       "       [0.52166045, 0.47833955],\n",
       "       [0.37754875, 0.6224513 ],\n",
       "       [0.39858   , 0.60142   ],\n",
       "       [0.3820519 , 0.61794806],\n",
       "       [0.5351277 , 0.4648723 ],\n",
       "       [0.58111125, 0.41888872],\n",
       "       [0.64711994, 0.35288006],\n",
       "       [0.6692375 , 0.33076254],\n",
       "       [0.7639404 , 0.23605959],\n",
       "       [0.7414318 , 0.2585683 ],\n",
       "       [0.64928675, 0.35071322],\n",
       "       [0.6931477 , 0.3068523 ],\n",
       "       [0.58336604, 0.41663387],\n",
       "       [0.5118737 , 0.48812628],\n",
       "       [0.3573279 , 0.6426721 ],\n",
       "       [0.3928843 , 0.60711575],\n",
       "       [0.46603563, 0.5339643 ],\n",
       "       [0.51263624, 0.48736382],\n",
       "       [0.6386045 , 0.36139554],\n",
       "       [0.56631863, 0.43368143],\n",
       "       [0.38200304, 0.617997  ],\n",
       "       [0.4073559 , 0.5926441 ],\n",
       "       [0.35481623, 0.64518374],\n",
       "       [0.3544502 , 0.64554983],\n",
       "       [0.43035275, 0.56964725],\n",
       "       [0.4450774 , 0.55492264],\n",
       "       [0.36310178, 0.6368982 ],\n",
       "       [0.3981318 , 0.6018682 ],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [0.55990183, 0.44009823],\n",
       "       [0.47230104, 0.527699  ],\n",
       "       [0.56867594, 0.43132406],\n",
       "       [0.477071  , 0.52292895],\n",
       "       [0.49770266, 0.50229734],\n",
       "       [0.40737018, 0.59262985],\n",
       "       [0.44578326, 0.55421674],\n",
       "       [0.38358104, 0.61641896],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [       nan,        nan],\n",
       "       [0.5993174 , 0.4006827 ],\n",
       "       [       nan,        nan],\n",
       "       [0.58776104, 0.412239  ],\n",
       "       [0.58861846, 0.41138157]], dtype=float32)"
      ]
     },
     "execution_count": 176,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "preds"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 177,
   "metadata": {},
   "outputs": [],
   "source": [
    "preds1=preds.argmax(axis=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 178,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1,\n",
       "       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,\n",
       "       1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,\n",
       "       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)"
      ]
     },
     "execution_count": 178,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "preds1"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 179,
   "metadata": {},
   "outputs": [],
   "source": [
    "abc = preds1.astype(int).flatten()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 180,
   "metadata": {},
   "outputs": [],
   "source": [
    "predictions = (lb.inverse_transform((abc)))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 181,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>predictedvalues</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "  predictedvalues\n",
       "0   male_positive\n",
       "1   male_positive\n",
       "2   male_negative\n",
       "3   male_positive\n",
       "4   male_positive\n",
       "5   male_positive\n",
       "6   male_positive\n",
       "7   male_positive\n",
       "8   male_positive\n",
       "9   male_positive"
      ]
     },
     "execution_count": 181,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "preddf = pd.DataFrame({'predictedvalues': predictions})\n",
    "preddf[:10]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 182,
   "metadata": {},
   "outputs": [],
   "source": [
    "actual=test_valid_lb.argmax(axis=1)\n",
    "abc123 = actual.astype(int).flatten()\n",
    "actualvalues = (lb.inverse_transform((abc123)))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 183,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>actualvalues</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>2</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>3</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>4</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>5</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>6</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>7</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>8</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>9</th>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "    actualvalues\n",
       "0  male_positive\n",
       "1  male_positive\n",
       "2  male_positive\n",
       "3  male_positive\n",
       "4  male_positive\n",
       "5  male_positive\n",
       "6  male_positive\n",
       "7  male_positive\n",
       "8  male_positive\n",
       "9  male_positive"
      ]
     },
     "execution_count": 183,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "actualdf = pd.DataFrame({'actualvalues': actualvalues})\n",
    "actualdf[:10]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 184,
   "metadata": {},
   "outputs": [],
   "source": [
    "finaldf = actualdf.join(preddf)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## Actual v/s Predicted emotions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 185,
   "metadata": {
    "scrolled": True
   },
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>actualvalues</th>\n",
       "      <th>predictedvalues</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>20</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>21</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>22</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>23</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>24</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>25</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>26</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>27</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>28</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>29</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>30</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>31</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>32</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>33</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>34</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>35</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>36</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_positive</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>37</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>38</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>39</th>\n",
       "      <td>male_negative</td>\n",
       "      <td>male_negative</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "     actualvalues predictedvalues\n",
       "20  male_negative   male_negative\n",
       "21  male_negative   male_positive\n",
       "22  male_negative   male_positive\n",
       "23  male_negative   male_positive\n",
       "24  male_negative   male_negative\n",
       "25  male_negative   male_negative\n",
       "26  male_negative   male_negative\n",
       "27  male_negative   male_negative\n",
       "28  male_negative   male_negative\n",
       "29  male_negative   male_negative\n",
       "30  male_negative   male_negative\n",
       "31  male_negative   male_negative\n",
       "32  male_negative   male_negative\n",
       "33  male_negative   male_negative\n",
       "34  male_negative   male_positive\n",
       "35  male_negative   male_positive\n",
       "36  male_negative   male_positive\n",
       "37  male_negative   male_negative\n",
       "38  male_negative   male_negative\n",
       "39  male_negative   male_negative"
      ]
     },
     "execution_count": 185,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "finaldf[20:40]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 186,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>predictedvalues</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>actualvalues</th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>male_negative</th>\n",
       "      <td>48</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>male_positive</th>\n",
       "      <td>32</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "               predictedvalues\n",
       "actualvalues                  \n",
       "male_negative               48\n",
       "male_positive               32"
      ]
     },
     "execution_count": 186,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "finaldf.groupby('actualvalues').count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 187,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>actualvalues</th>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>predictedvalues</th>\n",
       "      <th></th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>male_negative</th>\n",
       "      <td>47</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>male_positive</th>\n",
       "      <td>33</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "</div>"
      ],
      "text/plain": [
       "                 actualvalues\n",
       "predictedvalues              \n",
       "male_negative              47\n",
       "male_positive              33"
      ]
     },
     "execution_count": 187,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "finaldf.groupby('predictedvalues').count()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 188,
   "metadata": {},
   "outputs": [],
   "source": [
    "finaldf.to_csv('Predictions.csv', index=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 189,
   "metadata": {},
   "outputs": [],
   "source": [
    "def print_confusion_matrix(confusion_matrix, class_names, figsize = (10,7), fontsize=14):\n",
    "    \"\"\"Prints a confusion matrix, as returned by sklearn.metrics.confusion_matrix, as a heatmap.\n",
    "    \n",
    "    Arguments\n",
    "    ---------\n",
    "    confusion_matrix: numpy.ndarray\n",
    "        The numpy.ndarray object returned from a call to sklearn.metrics.confusion_matrix. \n",
    "        Similarly constructed ndarrays can also be used.\n",
    "    class_names: list\n",
    "        An ordered list of class names, in the order they index the given confusion matrix.\n",
    "    figsize: tuple\n",
    "        A 2-long tuple, the first value determining the horizontal size of the ouputted figure,\n",
    "        the second determining the vertical size. Defaults to (10,7).\n",
    "    fontsize: int\n",
    "        Font size for axes labels. Defaults to 14.\n",
    "        \n",
    "    Returns\n",
    "    -------\n",
    "    matplotlib.figure.Figure\n",
    "        The resulting confusion matrix figure\n",
    "    \"\"\"\n",
    "    df_cm = pd.DataFrame(\n",
    "        confusion_matrix, index=class_names, columns=class_names, \n",
    "    )\n",
    "    fig = plt.figure(figsize=figsize)\n",
    "    try:\n",
    "        heatmap = sns.heatmap(df_cm, annot=True, fmt=\"d\")\n",
    "    except ValueError:\n",
    "        raise ValueError(\"Confusion matrix values must be integers.\")\n",
    "        \n",
    "    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)\n",
    "    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)\n",
    "    plt.ylabel('True label')\n",
    "    plt.xlabel('Predicted label')\n",
    "    "
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 190,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "61.25000000000001"
      ]
     },
     "execution_count": 190,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import accuracy_score\n",
    "y_True = finaldf.actualvalues\n",
    "y_pred = finaldf.predictedvalues\n",
    "accuracy_score(y_True, y_pred)*100"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 191,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "59.83805668016194"
      ]
     },
     "execution_count": 191,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import f1_score\n",
    "f1_score(y_True, y_pred, average='macro') *100"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 192,
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[32, 16],\n",
       "       [15, 17]], dtype=int64)"
      ]
     },
     "execution_count": 192,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "from sklearn.metrics import confusion_matrix\n",
    "c = confusion_matrix(y_True, y_pred)\n",
    "c"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 193,
   "metadata": {
    "scrolled": False
   },
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 720x504 with 2 Axes>"
      ]
     },
     "metadata": {
      "needs_background": "light"
     },
     "output_type": "display_data"
    }
   ],
   "source": [
    "# Visualize Confusion Matrix \n",
    "\n",
    "class_names = ['male_negative', 'male_positive']\n",
    "#class_names = ['female_angry', 'female_calm', 'female_fearful', 'female_happy', 'female_sad', 'male_angry', 'male_calm', 'male_fearful', 'male_happy', 'male_sad']\n",
    "\n",
    "\n",
    "print_confusion_matrix(c, class_names)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "metadata": {
    "scrolled": True
   },
   "outputs": [],
   "source": [
    "import librosa\n",
    "import soundfile\n",
    "import os, glob, pickle\n",
    "import numpy as np\n",
    "from sklearn.model_selection import train_test_split\n",
    "from sklearn.neural_network import MLPClassifier\n",
    "from sklearn.metrics import accuracy_score\n",
    "from playsound import playsound\n",
    "import warnings"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "metadata": {},
   "outputs": [],
   "source": [
    "#Extract features (mfcc, chroma, mel) from a sound file\n",
    "def extract_feature(file_name, mfcc, chroma, mel):\n",
    "    with soundfile.SoundFile(file_name) as sound_file:\n",
    "        X = sound_file.read(dtype=\"float32\")\n",
    "        sample_rate=sound_file.samplerate\n",
    "        if chroma:\n",
    "            stft=np.abs(librosa.stft(X))\n",
    "        result=np.array([])\n",
    "        if mfcc:\n",
    "            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)\n",
    "            result=np.hstack((result, mfccs))\n",
    "        if chroma:\n",
    "            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)\n",
    "            result=np.hstack((result, chroma))\n",
    "        if mel:\n",
    "            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)\n",
    "            result=np.hstack((result, mel))\n",
    "        print(result)\n",
    "    return result"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Emotions in the RAVDESS dataset\n",
    "emotions={\n",
    "  '01':'neutral',\n",
    "  '02':'calm',\n",
    "  '03':'happy',\n",
    "  '04':'sad',\n",
    "  '05':'angry',\n",
    "  '06':'fearful',\n",
    "  '07':'disgust',\n",
    "  '08':'surprised'\n",
    "}\n",
    "\n",
    "#Emotions to observe\n",
    "observed_emotions=['calm', 'happy', 'fearful', 'disgust','angry','sad','surprised','neutral']\n",
    "\n",
    "def get_user_feat():\n",
    "    print(\"Which Feature Extraction Processes do you want to apply: \")\n",
    "    print(\"1. MFCC \\n2. MEL \\n3. Chroma \\n\")\n",
    "    fext_userinp = input(\"Enter your options(e.g. 23): \")\n",
    "    print(\"Processing. Please wait.....\")\n",
    "    mel = False;\n",
    "    mfcc = False;\n",
    "    chroma = False\n",
    "    for i in range(0, len(fext_userinp)):\n",
    "        if fext_userinp[i] == \"1\":\n",
    "            mfcc = True\n",
    "        elif fext_userinp[i] == \"2\":\n",
    "            mel = True\n",
    "        elif fext_userinp[i] == \"3\":\n",
    "            chroma = True\n",
    "    return mfcc,mel,chroma\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Which Feature Extraction Processes do you want to apply: \n",
      "1. MFCC \n",
      "2. MEL \n",
      "3. Chroma \n",
      "\n",
      "Enter your options(e.g. 23): 3\n",
      "Processing. Please wait.....\n",
      "[0.6337347  0.64876068 0.62174428 0.63455319 0.66034561 0.66046399\n",
      " 0.70094901 0.73155636 0.74692154 0.72342044 0.71611965 0.63060284]\n",
      "[0.66000104 0.7082442  0.70675522 0.67540741 0.69795352 0.73283887\n",
      " 0.71803045 0.76160073 0.79046834 0.78951305 0.77539867 0.72026587]\n",
      "[0.68981159 0.72004443 0.70266551 0.72014254 0.71627665 0.69108003\n",
      " 0.70406407 0.74857765 0.79880881 0.80049765 0.72979522 0.70465845]\n",
      "[0.72479993 0.71111047 0.70021063 0.71980739 0.74475932 0.7496016\n",
      " 0.74401945 0.79119521 0.80600274 0.7612341  0.7644465  0.74188924]\n",
      "[0.60386842 0.62637341 0.69648367 0.68891484 0.67997438 0.69295287\n",
      " 0.69974959 0.72074366 0.7345677  0.74788439 0.77408779 0.73281467]\n",
      "[0.60688192 0.63046306 0.63783842 0.69573522 0.67423135 0.66124982\n",
      " 0.70877624 0.71637642 0.73657078 0.72540027 0.74662089 0.69657117]\n",
      "[0.6277107  0.69338095 0.71361119 0.7187683  0.71151292 0.71573341\n",
      " 0.71919817 0.7311964  0.75823474 0.74410146 0.74314469 0.66763973]\n",
      "[0.6274929  0.6712541  0.69413429 0.70163113 0.6871776  0.69193155\n",
      " 0.70459872 0.72326016 0.76635569 0.77499324 0.76484644 0.72234231]\n",
      "[0.5888747  0.61102414 0.67147291 0.65463889 0.64390057 0.6896559\n",
      " 0.71216637 0.75549132 0.73418349 0.67258751 0.68163365 0.69798791]\n",
      "[0.64442837 0.66010571 0.69409651 0.71849006 0.70866841 0.71344179\n",
      " 0.7425856  0.7660073  0.72927469 0.69927877 0.69376123 0.64285427]\n",
      "[0.59160298 0.66827893 0.69214046 0.65013409 0.67390394 0.68067795\n",
      " 0.6619606  0.67551708 0.68657535 0.72454584 0.80686957 0.69637108]\n",
      "[0.55395836 0.60931635 0.68325448 0.66698784 0.66708988 0.70213389\n",
      " 0.70875317 0.72603399 0.71370959 0.69051856 0.68708944 0.63936937]\n",
      "[0.6905458  0.72373724 0.70779043 0.71284133 0.74131238 0.75586176\n",
      " 0.7048285  0.73658335 0.77391326 0.77448422 0.76698959 0.7417891 ]\n",
      "[0.69527841 0.68325353 0.6790117  0.71335149 0.71457744 0.71963495\n",
      " 0.72717953 0.76714855 0.77246058 0.76713926 0.79183453 0.73862869]\n",
      "[0.63080919 0.62992609 0.68586016 0.67405289 0.63655806 0.65969455\n",
      " 0.67243505 0.71933019 0.74774539 0.73506707 0.7446292  0.7395944 ]\n",
      "[0.63865471 0.67745131 0.70689702 0.68515337 0.64585602 0.70013368\n",
      " 0.69700009 0.71790439 0.74375135 0.72808695 0.74754077 0.70980412]\n",
      "[0.59327769 0.58256108 0.67871302 0.64895511 0.63720524 0.68340844\n",
      " 0.71144927 0.75122505 0.69043803 0.64949757 0.67339337 0.71169001]\n",
      "[0.55770916 0.62408054 0.59184957 0.58198285 0.64362216 0.68688178\n",
      " 0.69232637 0.74284983 0.75375378 0.65333861 0.62883085 0.56794524]\n",
      "[0.57173324 0.59325218 0.60087228 0.64458972 0.6885162  0.74456757\n",
      " 0.71807516 0.75377792 0.76758897 0.71758771 0.68473881 0.60907143]\n",
      "[0.62921399 0.63652527 0.66574401 0.66432714 0.67557371 0.72553223\n",
      " 0.70059854 0.73236847 0.76279873 0.73008233 0.65967453 0.65718007]\n",
      "[0.61556202 0.64998639 0.63677377 0.62328255 0.58870965 0.55158782\n",
      " 0.54390883 0.60455358 0.63967937 0.63589585 0.65421009 0.63353473]\n",
      "[0.66775841 0.68687487 0.70141649 0.72897625 0.70038497 0.68853849\n",
      " 0.68884373 0.7593351  0.78010982 0.730865   0.68501103 0.64404088]\n",
      "[0.61304379 0.61869001 0.63779813 0.65808523 0.62046176 0.62620115\n",
      " 0.65625131 0.69073915 0.71961027 0.70911396 0.6876229  0.64600962]\n",
      "[0.58216619 0.5985173  0.6144008  0.6587851  0.63617116 0.64446265\n",
      " 0.65539134 0.69600159 0.69813883 0.68840683 0.71022958 0.67259932]\n",
      "[0.69379753 0.65250629 0.61391968 0.63101393 0.66950858 0.67091745\n",
      " 0.67067736 0.71075702 0.71359885 0.74334478 0.77927458 0.74372762]\n",
      "[0.62451625 0.63217366 0.6271897  0.64676595 0.67460948 0.65361989\n",
      " 0.67121798 0.74932992 0.71399283 0.68268347 0.72537625 0.71777427]\n",
      "[0.59208834 0.62992096 0.61296052 0.60103971 0.61196721 0.66461724\n",
      " 0.65149838 0.69470143 0.71865398 0.68234605 0.67188114 0.61958492]\n",
      "[0.60246819 0.62528485 0.63330293 0.64527452 0.63379717 0.61177069\n",
      " 0.6387375  0.6627835  0.65470916 0.65626448 0.66552299 0.63868272]\n",
      "[0.5711301  0.62103891 0.64242733 0.67514795 0.66209233 0.67866331\n",
      " 0.67563015 0.6568923  0.64210451 0.71161467 0.71281368 0.62134868]\n",
      "[0.59955066 0.6452052  0.64362907 0.6538142  0.66863966 0.69121534\n",
      " 0.67831719 0.70642793 0.69524139 0.68473846 0.68950021 0.64645886]\n",
      "[0.65955335 0.67896569 0.65806991 0.7134276  0.72517937 0.69798547\n",
      " 0.70459592 0.77275801 0.78935665 0.77453917 0.77248734 0.6945442 ]\n",
      "[0.59931749 0.67123127 0.69397199 0.68009883 0.67005342 0.65747964\n",
      " 0.65955997 0.70015317 0.726785   0.7427752  0.72522056 0.65371579]\n",
      "[0.53335255 0.59826243 0.6315034  0.68050599 0.72024524 0.71256447\n",
      " 0.68592465 0.7466771  0.70063961 0.63091636 0.64361757 0.60198593]\n",
      "[0.5364809  0.57390553 0.59037322 0.55070359 0.65430206 0.71600938\n",
      " 0.60424846 0.55289567 0.60762942 0.63797873 0.6396907  0.59933543]\n",
      "[0.59748793 0.64339232 0.60793799 0.66241592 0.69893193 0.77259552\n",
      " 0.72908461 0.71392632 0.75385696 0.74310058 0.7264421  0.62957579]\n",
      "[0.48414832 0.52534282 0.56325287 0.64931434 0.67933327 0.65916955\n",
      " 0.67967594 0.70872122 0.70638585 0.62887347 0.62517172 0.52751911]\n",
      "[0.72580451 0.68263781 0.65538353 0.66904324 0.71390307 0.67042464\n",
      " 0.6553126  0.69234622 0.69765449 0.69453102 0.71799761 0.71828592]\n",
      "[0.6037367  0.61125034 0.6493057  0.68788832 0.66605014 0.6866014\n",
      " 0.71795183 0.78552806 0.77679223 0.71843046 0.66680908 0.67214596]\n",
      "[0.69015831 0.73309934 0.70469701 0.6573773  0.68248636 0.69510663\n",
      " 0.67152941 0.75386786 0.81704283 0.8154763  0.74222147 0.72788769]\n",
      "[0.70048571 0.67917955 0.65882075 0.66264158 0.6610114  0.66726416\n",
      " 0.73236477 0.78450924 0.73605609 0.69586325 0.71404386 0.74210912]\n",
      "[0.567864   0.60017896 0.59518874 0.64323097 0.77181041 0.70910561\n",
      " 0.65645486 0.72768766 0.72722667 0.67804319 0.63324857 0.56189066]\n",
      "[0.51881868 0.59877419 0.58345544 0.51047432 0.59103268 0.67301178\n",
      " 0.61115223 0.66531354 0.72158653 0.65068495 0.63615888 0.5596503 ]\n",
      "[0.5800361  0.61329573 0.59822232 0.56988919 0.68784779 0.73033261\n",
      " 0.6418854  0.67077345 0.66976255 0.64703578 0.62060201 0.55743843]\n",
      "[0.54122776 0.59509182 0.59679788 0.57033128 0.63453507 0.72137153\n",
      " 0.67464364 0.67671448 0.70031649 0.64786476 0.60646135 0.58816022]\n",
      "[0.5813036  0.57134157 0.59702659 0.61408979 0.61599576 0.66092598\n",
      " 0.66507781 0.66909111 0.66800714 0.67436993 0.67477393 0.65397781]\n",
      "[0.57265568 0.62276703 0.67964327 0.71271372 0.70800984 0.69026726\n",
      " 0.72627974 0.76330388 0.7139613  0.68384379 0.71666956 0.66665852]\n",
      "[0.61427772 0.678496   0.72652221 0.72969806 0.67842722 0.65217763\n",
      " 0.6780557  0.73342335 0.74403334 0.69138116 0.66854537 0.63667643]\n",
      "[0.63570565 0.62562186 0.66587538 0.68702805 0.70332676 0.70783579\n",
      " 0.72746903 0.73410743 0.72407544 0.72143292 0.67921054 0.63720965]\n",
      "[0.50229502 0.53472638 0.57944632 0.63746679 0.67398638 0.63622576\n",
      " 0.63868147 0.70273817 0.72058427 0.69444519 0.66144043 0.60174549]\n",
      "[0.52216023 0.54392779 0.61493337 0.6097514  0.58727622 0.65663904\n",
      " 0.65242976 0.6871922  0.70269561 0.69229883 0.68600142 0.63714892]\n",
      "[0.63567722 0.62790847 0.61404252 0.63324851 0.70952058 0.69438225\n",
      " 0.69622076 0.67749906 0.60984647 0.64048779 0.69087696 0.67052567]\n",
      "[0.59168851 0.62681836 0.63892198 0.67662185 0.71142679 0.70964098\n",
      " 0.74847686 0.77182525 0.76175457 0.74725223 0.73336548 0.59386319]\n",
      "[0.65596801 0.68442976 0.70582867 0.69253612 0.7052539  0.71977454\n",
      " 0.72411448 0.79653126 0.78118461 0.75715107 0.75263149 0.72786427]\n",
      "[0.64360231 0.65440893 0.68167567 0.66862875 0.64796191 0.65298003\n",
      " 0.68099391 0.76615155 0.76956189 0.74676687 0.7543838  0.70808721]\n",
      "[0.64970696 0.7070744  0.76059622 0.74013793 0.72518408 0.74802035\n",
      " 0.74411881 0.77302068 0.76840538 0.74127239 0.79242569 0.74276912]\n",
      "[0.58868968 0.6447826  0.66041142 0.61217588 0.58941752 0.58440804\n",
      " 0.61962414 0.67240864 0.65475702 0.64380765 0.66151178 0.65171474]\n",
      "[0.54504204 0.56329489 0.58326501 0.59544855 0.6025418  0.59575146\n",
      " 0.61974597 0.66650826 0.66825461 0.64472663 0.6416328  0.53281188]\n",
      "[0.51733452 0.52571255 0.53770393 0.5592829  0.62347335 0.5939979\n",
      " 0.52782232 0.56876671 0.61281198 0.63406163 0.67661393 0.59215575]\n",
      "[0.58414006 0.60315311 0.64943534 0.66405743 0.67040962 0.68642992\n",
      " 0.64576644 0.67496711 0.67865914 0.6992411  0.67944419 0.63020658]\n",
      "[0.62359011 0.61524528 0.60672772 0.6374709  0.68983603 0.67488354\n",
      " 0.68875355 0.7685371  0.78831148 0.72661036 0.68048942 0.63086218]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.52099603 0.50921887 0.52978176 0.55762213 0.57547152 0.58669639\n",
      " 0.61310184 0.64144576 0.62371892 0.59318095 0.57584149 0.52479351]\n",
      "[0.49976036 0.50981432 0.52086496 0.55366254 0.5649358  0.5670346\n",
      " 0.59668702 0.65035456 0.63745224 0.60176861 0.59719557 0.54527283]\n",
      "[0.51649255 0.52611279 0.57321656 0.56248492 0.59635663 0.62109369\n",
      " 0.64531142 0.66699648 0.63084716 0.62170541 0.66210449 0.58680695]\n",
      "[0.53835279 0.54729927 0.53374624 0.54764849 0.62705213 0.65782911\n",
      " 0.66613519 0.66592824 0.64095521 0.63232225 0.65468603 0.5756191 ]\n",
      "[0.59469259 0.51913482 0.51604855 0.55766129 0.56931865 0.64773637\n",
      " 0.61893398 0.63734812 0.68575734 0.67865127 0.73100728 0.676305  ]\n",
      "[0.44613594 0.39687371 0.3695423  0.38801074 0.4192867  0.49201399\n",
      " 0.45563859 0.47728339 0.51080304 0.52933532 0.56252897 0.4284339 ]\n",
      "[0.44476917 0.42043892 0.40013599 0.37440744 0.45309839 0.52415782\n",
      " 0.53716558 0.55467099 0.54629201 0.58271724 0.58378249 0.48478651]\n",
      "[0.49252388 0.48734722 0.45228225 0.45010477 0.51007229 0.57616425\n",
      " 0.6165781  0.65510303 0.64936221 0.66485202 0.63794839 0.53200662]\n",
      "[0.51218158 0.53460538 0.53105634 0.51890898 0.5793556  0.62830764\n",
      " 0.74911618 0.73261434 0.66452998 0.6565119  0.66511935 0.57003224]\n",
      "[0.43448547 0.48264429 0.46279508 0.40225488 0.38342318 0.46711817\n",
      " 0.5402084  0.61106849 0.60376799 0.52999187 0.50922197 0.49083161]\n",
      "[0.48187676 0.46026617 0.42622125 0.39641404 0.42603898 0.49826664\n",
      " 0.57580107 0.60877323 0.63327742 0.59701151 0.60294127 0.56208664]\n",
      "[0.37463412 0.3640404  0.37989387 0.33513096 0.36092469 0.4287546\n",
      " 0.54795629 0.6540038  0.57137722 0.50358784 0.50734246 0.47852555]\n",
      "[0.63543534 0.6086722  0.57380062 0.5896976  0.59279102 0.65215212\n",
      " 0.74218035 0.79219329 0.74175763 0.73037457 0.69005287 0.68203551]\n",
      "[0.56140476 0.5600009  0.54777038 0.56524968 0.58722186 0.63231456\n",
      " 0.70013422 0.77518594 0.71170723 0.63301241 0.65179265 0.6233719 ]\n",
      "[0.51025462 0.49448755 0.49910459 0.4782007  0.46675068 0.57853258\n",
      " 0.60988176 0.631751   0.61032319 0.53976023 0.52048427 0.5365811 ]\n",
      "[0.62749141 0.59944785 0.58733487 0.60920894 0.56966406 0.65002292\n",
      " 0.73875278 0.74103492 0.75134194 0.72959697 0.70372957 0.70528954]\n",
      "[0.52387768 0.49754649 0.48932591 0.47565418 0.54387718 0.53985035\n",
      " 0.58785212 0.6442169  0.56826037 0.56706929 0.60506237 0.54929423]\n",
      "[0.57758367 0.55543864 0.50545418 0.49772075 0.54112345 0.57363367\n",
      " 0.60301441 0.65627885 0.65856349 0.67014337 0.61029673 0.60217237]\n",
      "[0.62903023 0.55304122 0.51048243 0.52784085 0.56147742 0.64857149\n",
      " 0.64312553 0.65826529 0.70235699 0.6476723  0.62618291 0.58863378]\n",
      "[0.51270902 0.46606004 0.42331532 0.41206196 0.44944912 0.50825912\n",
      " 0.57247865 0.57543474 0.61278427 0.55579334 0.52165824 0.49345347]\n",
      "[0.40031481 0.36000606 0.4071193  0.41078079 0.37830147 0.40349242\n",
      " 0.42605919 0.52562094 0.6037972  0.50007254 0.50292808 0.44071141]\n",
      "[0.41461384 0.35720867 0.41336107 0.45942587 0.45138243 0.45927539\n",
      " 0.46278414 0.5181452  0.5895229  0.55704319 0.53549385 0.45412481]\n",
      "[0.40112203 0.34370571 0.41522956 0.41101906 0.38942072 0.41566542\n",
      " 0.49244061 0.64293873 0.61721569 0.52090853 0.47069344 0.45513022]\n",
      "[0.44618571 0.37376603 0.35032898 0.43268964 0.42301604 0.41768822\n",
      " 0.46147606 0.50315624 0.6393407  0.65478373 0.54872108 0.50837451]\n",
      "[0.61002231 0.61399889 0.50923097 0.52322888 0.51194543 0.5002225\n",
      " 0.5247488  0.56324506 0.589001   0.5350163  0.54587209 0.5807637 ]\n",
      "[0.62223464 0.53113836 0.46243423 0.48196632 0.48869804 0.50099754\n",
      " 0.54259276 0.55411977 0.53434819 0.54788631 0.60023761 0.64389491]\n",
      "[0.5641011  0.56805736 0.4655256  0.44160068 0.45321107 0.51486075\n",
      " 0.51534569 0.50745773 0.5596754  0.54224616 0.56120318 0.54668754]\n",
      "[0.56083643 0.52510995 0.48566315 0.4626568  0.48735937 0.47576147\n",
      " 0.49011075 0.52540278 0.55291486 0.50674957 0.517349   0.51425534]\n",
      "[0.54124463 0.49674448 0.49272031 0.51722664 0.57894129 0.60188895\n",
      " 0.58345693 0.58378321 0.59250057 0.57622278 0.56987411 0.54033005]\n",
      "[0.61107892 0.54657471 0.54605883 0.56055564 0.65065211 0.66037977\n",
      " 0.67852372 0.70397311 0.69871265 0.62284487 0.62670332 0.60476583]\n",
      "[0.5675599  0.54339123 0.52061343 0.51685947 0.56144118 0.60034746\n",
      " 0.6369099  0.6274634  0.6240648  0.61616224 0.63611513 0.55013514]\n",
      "[0.5723874  0.53034818 0.5187754  0.50051177 0.54688323 0.62159652\n",
      " 0.67529923 0.61614686 0.59609753 0.60907179 0.63215643 0.60108882]\n",
      "[0.57356632 0.52258933 0.50008178 0.51236898 0.56000656 0.60452223\n",
      " 0.60483539 0.63030416 0.59656745 0.56522232 0.52400219 0.49213764]\n",
      "[0.50957209 0.51381814 0.51551861 0.50462651 0.52294844 0.58237427\n",
      " 0.63336664 0.63109577 0.60815161 0.56343794 0.53112239 0.54447955]\n",
      "[0.53276604 0.52426529 0.54997844 0.58935285 0.66345155 0.66514814\n",
      " 0.7272225  0.67771566 0.65789181 0.64786309 0.60461742 0.56299877]\n",
      "[0.50699896 0.51181215 0.53391016 0.54866695 0.55299813 0.55367702\n",
      " 0.61191845 0.65089124 0.59203655 0.53394157 0.56969345 0.51115698]\n",
      "[0.55668867 0.51944137 0.51489615 0.57223386 0.54614651 0.5662117\n",
      " 0.52457684 0.57900304 0.59083086 0.52392274 0.57551187 0.57275921]\n",
      "[0.48786011 0.44540152 0.48667246 0.55929637 0.54379475 0.51241183\n",
      " 0.54715794 0.5910542  0.56569558 0.49568647 0.55571163 0.54117936]\n",
      "[0.51580971 0.48010525 0.51252216 0.5411706  0.58107483 0.55725527\n",
      " 0.58332717 0.62749177 0.6047591  0.56624144 0.59883082 0.55534488]\n",
      "[0.54878259 0.44558412 0.45435122 0.58453655 0.64537275 0.63649678\n",
      " 0.62665081 0.62776273 0.62675238 0.58855975 0.66049075 0.66279536]\n",
      "[0.519476   0.45355892 0.42710888 0.42927328 0.4315339  0.47341964\n",
      " 0.55956417 0.66550916 0.79292721 0.69358337 0.63554627 0.60420567]\n",
      "[0.48426476 0.41485596 0.42852709 0.41191903 0.42644414 0.46153319\n",
      " 0.56860363 0.71228158 0.6664899  0.60791343 0.63533342 0.57105815]\n",
      "[0.56619471 0.52259707 0.42241967 0.40153757 0.43564057 0.46473598\n",
      " 0.5408116  0.61751819 0.63810396 0.65998894 0.62741864 0.60806203]\n",
      "[0.55509937 0.51581478 0.44101036 0.42213762 0.49107957 0.53965616\n",
      " 0.57244718 0.66785747 0.71231723 0.74067259 0.69068718 0.66409039]\n",
      "[0.49756467 0.47353137 0.43143016 0.46544155 0.48494378 0.51571971\n",
      " 0.52647454 0.56512558 0.56668079 0.5694775  0.5117346  0.51047307]\n",
      "[0.52712101 0.47636023 0.43602344 0.47804385 0.4990088  0.55821341\n",
      " 0.59762549 0.62666464 0.63111186 0.60751158 0.58661574 0.56661773]\n",
      "[0.43335557 0.37057024 0.3528811  0.39402696 0.47693002 0.54442281\n",
      " 0.54302448 0.51604545 0.54698718 0.58194453 0.54338223 0.45573601]\n",
      "[0.47820809 0.44420138 0.40836442 0.45297411 0.48376644 0.56103551\n",
      " 0.6380049  0.59575689 0.62363887 0.64235979 0.58704871 0.52131319]\n",
      "[0.49936318 0.43147001 0.44180316 0.48416203 0.50678897 0.52638614\n",
      " 0.55286598 0.57677454 0.61865371 0.58339345 0.58224928 0.55122602]\n",
      "[0.6096558  0.56832254 0.52928978 0.55947644 0.62137794 0.65879345\n",
      " 0.68303555 0.65589899 0.66416872 0.65900832 0.68326735 0.62067521]\n",
      "[0.52495468 0.41630939 0.38807476 0.41995364 0.4332861  0.47595447\n",
      " 0.5318867  0.5396046  0.54051131 0.55604583 0.57936394 0.55727214]\n",
      "[0.60717028 0.57075202 0.51309401 0.56400692 0.6054582  0.63125843\n",
      " 0.64472336 0.66409737 0.73112565 0.76340753 0.75275427 0.65159982]\n",
      "[0.54325402 0.52923185 0.50157237 0.48636717 0.53947788 0.60253668\n",
      " 0.61728114 0.61761612 0.61806726 0.62870109 0.65579975 0.59086436]\n",
      "[0.66606855 0.58119464 0.52326602 0.51831961 0.53739405 0.61376369\n",
      " 0.62672514 0.61401743 0.64401668 0.63824749 0.6460793  0.69608963]\n",
      "[0.55198056 0.50947219 0.4871538  0.49359456 0.56029528 0.62461001\n",
      " 0.63527089 0.64426076 0.68895215 0.67835218 0.67508692 0.6057058 ]\n",
      "[0.59142768 0.5143041  0.49624664 0.52023226 0.56483948 0.60990739\n",
      " 0.62525159 0.62101793 0.64063221 0.62978595 0.68180501 0.66543919]\n",
      "[0.60677457 0.55229121 0.4950242  0.48495573 0.54434478 0.6064781\n",
      " 0.64282292 0.63571942 0.6262942  0.64269423 0.69222772 0.65934885]\n",
      "[0.71711719 0.6436885  0.5798595  0.58788913 0.61363441 0.66618913\n",
      " 0.71393496 0.73169279 0.75283563 0.73070288 0.7648145  0.75488573]\n",
      "[0.60284001 0.55391675 0.52962619 0.54337996 0.53790098 0.56109893\n",
      " 0.62387156 0.65025258 0.65695846 0.66125375 0.67457277 0.63030565]\n",
      "[0.53414994 0.5026589  0.49587607 0.51276922 0.53313684 0.59055495\n",
      " 0.66646993 0.68527019 0.67254978 0.62732273 0.63885003 0.59672272]\n",
      "[0.53632933 0.55683976 0.5637334  0.5954299  0.63411337 0.57834357\n",
      " 0.5539192  0.60238433 0.64136928 0.5980342  0.61676264 0.58505726]\n",
      "[0.68992329 0.72352135 0.67710751 0.697824   0.71859896 0.71022129\n",
      " 0.74892122 0.75145859 0.76732254 0.77643222 0.78275061 0.7308023 ]\n",
      "[0.64131665 0.61725718 0.65652841 0.67888886 0.69624627 0.68747693\n",
      " 0.66686767 0.631845   0.66751993 0.72860998 0.71267736 0.71869308]\n",
      "[0.65869749 0.6572246  0.67604917 0.72789329 0.67514592 0.68671262\n",
      " 0.68420637 0.69846827 0.73675454 0.76182783 0.7773971  0.72040188]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.58964062 0.55465198 0.58016056 0.60237777 0.59298521 0.6135112\n",
      " 0.65337586 0.62979388 0.62809265 0.63262486 0.61627024 0.64592272]\n",
      "[0.52492338 0.53197956 0.53762072 0.53084797 0.56686944 0.57685018\n",
      " 0.63287246 0.6202004  0.61339706 0.59772277 0.58429235 0.60369438]\n",
      "[0.6305626  0.60918492 0.59710103 0.63180244 0.65131861 0.66959155\n",
      " 0.70139831 0.76890653 0.80284679 0.72516537 0.69418156 0.66524482]\n",
      "[0.53005141 0.52476001 0.56119037 0.66302931 0.71373898 0.68253386\n",
      " 0.6558575  0.64262831 0.63918203 0.62037593 0.64462072 0.65755373]\n",
      "[0.58616608 0.63638538 0.65176451 0.69490272 0.69074541 0.6778518\n",
      " 0.67774349 0.71856827 0.68069726 0.56821769 0.56968027 0.54675537]\n",
      "[0.56712449 0.55999714 0.63111192 0.67584485 0.67273617 0.63004506\n",
      " 0.65251017 0.66864169 0.6598413  0.66693383 0.65342891 0.62489152]\n",
      "[0.4950923  0.60909021 0.72913164 0.69839209 0.66614223 0.6687007\n",
      " 0.65518463 0.66940624 0.72152281 0.73569715 0.63675541 0.52172625]\n",
      "[0.53540212 0.50536644 0.60264599 0.65644914 0.73940921 0.66489089\n",
      " 0.67008352 0.66813791 0.69862986 0.69241881 0.67212075 0.66643226]\n",
      "[0.54047143 0.53130764 0.60748804 0.71197844 0.68819702 0.65653312\n",
      " 0.67736775 0.67786586 0.74990004 0.7286489  0.63296688 0.61821294]\n",
      "[0.52082425 0.56738806 0.63493484 0.68388265 0.66115117 0.64830691\n",
      " 0.63492882 0.6780588  0.75817609 0.71919441 0.67363602 0.58575541]\n",
      "[0.48615777 0.52845734 0.52332085 0.52765369 0.52559596 0.54546279\n",
      " 0.61624694 0.61402667 0.64064014 0.61616075 0.56895679 0.55925018]\n",
      "[0.38174039 0.41640252 0.46564311 0.45990565 0.48894897 0.48310751\n",
      " 0.53590751 0.60276854 0.58423263 0.56185895 0.50283194 0.4566403 ]\n",
      "[0.62191087 0.68198323 0.62665266 0.6178571  0.65208381 0.63140488\n",
      " 0.59017795 0.62731469 0.70545262 0.62194675 0.54449821 0.57659972]\n",
      "[0.58794802 0.60145313 0.6321488  0.6609109  0.63726991 0.57074594\n",
      " 0.58733904 0.60688168 0.603333   0.59871089 0.56736892 0.59839463]\n",
      "[0.5178932  0.56803882 0.64849454 0.66305524 0.68278098 0.64643383\n",
      " 0.60277534 0.56509763 0.58932722 0.62862426 0.64146942 0.62913656]\n",
      "[0.53852433 0.53148723 0.6187259  0.6484589  0.57838494 0.58103812\n",
      " 0.63569462 0.67009223 0.70255178 0.68117738 0.63911188 0.61082011]\n",
      "[0.59209251 0.65999293 0.6549474  0.65351152 0.66237378 0.66863155\n",
      " 0.67475432 0.67391324 0.70934772 0.73365057 0.7023412  0.66931802]\n",
      "[0.64818019 0.65577888 0.68044478 0.69604206 0.67006636 0.65207243\n",
      " 0.64902771 0.67326832 0.67271799 0.70712978 0.75045204 0.70341784]\n",
      "[0.61265391 0.66877717 0.66342098 0.66178721 0.67512745 0.59213638\n",
      " 0.56200415 0.63429821 0.74312431 0.71244347 0.65347683 0.64934903]\n",
      "[0.7018652  0.69247478 0.69780141 0.70375317 0.71455085 0.70715934\n",
      " 0.66441303 0.66229022 0.70579726 0.69847989 0.68255633 0.74707925]\n",
      "[0.53758961 0.59536749 0.65018749 0.67842323 0.69189179 0.67201358\n",
      " 0.66489196 0.69236225 0.67542398 0.6517545  0.63208598 0.57963383]\n",
      "[0.48415816 0.51244992 0.55904227 0.64088845 0.66902298 0.64371258\n",
      " 0.63202101 0.58484882 0.61608958 0.65574336 0.61470693 0.56370533]\n",
      "[0.49637479 0.50751829 0.53449869 0.55133712 0.61596084 0.64178157\n",
      " 0.67337894 0.70283258 0.73262453 0.71438783 0.66749078 0.57054836]\n",
      "[0.41110864 0.47219023 0.50852978 0.49178469 0.49070361 0.52069896\n",
      " 0.56454396 0.61891037 0.6413638  0.63464999 0.57271934 0.47474644]\n",
      "[0.68526161 0.68559241 0.6660592  0.68161571 0.7292679  0.77614659\n",
      " 0.77122462 0.80449164 0.84242678 0.76380903 0.76064259 0.71578169]\n",
      "[0.68905222 0.67916471 0.69437379 0.6233772  0.63428038 0.64615667\n",
      " 0.68533653 0.71542394 0.77716905 0.74977148 0.68469489 0.71105349]\n",
      "[0.65692151 0.61470515 0.58346921 0.60374516 0.63223094 0.6223287\n",
      " 0.62397426 0.62903112 0.67324919 0.71310443 0.7459597  0.73023993]\n",
      "[0.64454883 0.64847857 0.64714676 0.68317294 0.69820935 0.68871921\n",
      " 0.66821736 0.65618187 0.71003646 0.7249105  0.68348211 0.6809625 ]\n",
      "[0.6488204  0.68672878 0.64867783 0.62656307 0.63327008 0.63718736\n",
      " 0.60799628 0.64480114 0.70165557 0.70154119 0.73007101 0.71925384]\n",
      "[0.59820956 0.66777098 0.62225461 0.59488934 0.61413234 0.65823019\n",
      " 0.64929897 0.66624612 0.72560084 0.71917295 0.71158576 0.68617225]\n",
      "[0.62923354 0.64124268 0.60231405 0.65710872 0.65423769 0.62861115\n",
      " 0.65527117 0.74740374 0.81130058 0.76021755 0.71064991 0.63161719]\n",
      "[0.55234909 0.58044004 0.63006568 0.62950069 0.6355235  0.66428959\n",
      " 0.63902372 0.63999534 0.73093915 0.7570011  0.71950597 0.68045413]\n",
      "[0.66840929 0.66378278 0.63918906 0.6615867  0.73176336 0.70958042\n",
      " 0.70203304 0.72994804 0.77661347 0.76908052 0.75828218 0.72702456]\n",
      "[0.58462888 0.58282399 0.64049941 0.66207427 0.64985245 0.63849002\n",
      " 0.60674763 0.6052056  0.69225407 0.75628895 0.72408742 0.67338479]\n",
      "[0.6100083  0.59042996 0.61989748 0.71796757 0.73901314 0.71969271\n",
      " 0.68487602 0.66095597 0.72036541 0.71791404 0.70415652 0.70320117]\n",
      "[0.55279183 0.61871982 0.70180267 0.72682256 0.70028687 0.69263965\n",
      " 0.68772823 0.69144237 0.75554144 0.74035823 0.73175454 0.6680252 ]\n",
      "[0.51586699 0.59531873 0.66424435 0.72771156 0.77742016 0.76683944\n",
      " 0.72404259 0.73249573 0.73859751 0.70817649 0.61877644 0.57000089]\n",
      "[0.54487121 0.62146509 0.5774464  0.57668781 0.62180567 0.63433188\n",
      " 0.64475739 0.68698013 0.66395032 0.62894064 0.56038582 0.498981  ]\n",
      "[0.55743611 0.62972003 0.67925227 0.66207635 0.64553583 0.64517689\n",
      " 0.66981727 0.70258492 0.76561451 0.68750161 0.63506252 0.57820076]\n",
      "[0.54105937 0.6862188  0.74824446 0.6787383  0.64201075 0.65110475\n",
      " 0.63288236 0.62843657 0.66245085 0.59472775 0.52092141 0.49496743]\n",
      "[0.59111637 0.65730363 0.6647225  0.63682574 0.68760407 0.73267949\n",
      " 0.79419243 0.78710401 0.77998596 0.73745483 0.69704854 0.61342406]\n",
      "[0.634247   0.66952652 0.64730471 0.65808481 0.71049386 0.73173028\n",
      " 0.72580564 0.80864406 0.83241314 0.76753664 0.73097098 0.66508597]\n",
      "[0.62313884 0.65235966 0.64173436 0.61833251 0.64872348 0.67585051\n",
      " 0.71897525 0.78407353 0.84348357 0.8440823  0.77866918 0.68895215]\n",
      "[0.56349015 0.62721926 0.66841716 0.662597   0.64815879 0.69681406\n",
      " 0.74391323 0.73723274 0.76224911 0.76690388 0.70352662 0.60328066]\n",
      "[0.63312006 0.63853669 0.70892823 0.74561596 0.73076636 0.74067414\n",
      " 0.75290871 0.74872863 0.77357405 0.76564872 0.74713582 0.71313876]\n",
      "[0.6025371  0.70035619 0.71552134 0.70592296 0.69923693 0.68701816\n",
      " 0.72976273 0.77556127 0.70891923 0.65564263 0.66580427 0.59564263]\n",
      "[0.60486042 0.66236955 0.69685292 0.66168106 0.67711645 0.66973966\n",
      " 0.65223241 0.66909015 0.67655945 0.70053363 0.70810527 0.68002617]\n",
      "[0.56567812 0.62678635 0.64980555 0.60847908 0.64426154 0.69829172\n",
      " 0.68414658 0.7165733  0.71519387 0.72895569 0.73741829 0.64416665]\n",
      "[0.53422648 0.55710977 0.56659234 0.59619045 0.64347434 0.57971793\n",
      " 0.5753389  0.54814732 0.59686565 0.63036674 0.5948956  0.60851938]\n",
      "[0.61511034 0.63201177 0.68877453 0.7437039  0.74799526 0.73895538\n",
      " 0.73909795 0.72967345 0.72030264 0.7400713  0.70844817 0.69088995]\n",
      "[0.5905121  0.55875313 0.6307587  0.67824018 0.75827873 0.71329308\n",
      " 0.6383512  0.59601593 0.59550381 0.5856204  0.60219502 0.64477116]\n",
      "[0.51408362 0.53492707 0.57780379 0.62070292 0.59314793 0.59674901\n",
      " 0.5946992  0.58524144 0.57468325 0.56106824 0.55994189 0.53798312]\n",
      "[0.66721791 0.71103424 0.7259016  0.7694006  0.76349455 0.75476772\n",
      " 0.69276583 0.66664022 0.69329166 0.73496497 0.73678142 0.72655815]\n",
      "[0.56596005 0.63271755 0.65655982 0.62276167 0.56966329 0.54206049\n",
      " 0.60673475 0.60710901 0.59800732 0.60785437 0.61005169 0.5898279 ]\n",
      "[0.63983846 0.64470834 0.64022905 0.63632143 0.60758078 0.60403234\n",
      " 0.66845113 0.6421048  0.64596236 0.6646924  0.63839519 0.66964823]\n",
      "[0.58291167 0.59934539 0.67276794 0.7048173  0.70195997 0.68378979\n",
      " 0.69706142 0.68822068 0.65063357 0.63937151 0.64140844 0.63200712]\n",
      "[0.52565897 0.46003994 0.42447382 0.46752071 0.49396351 0.57796079\n",
      " 0.64685196 0.6282711  0.57450956 0.55067527 0.5223788  0.57254106]\n",
      "[0.52629554 0.46944526 0.43261421 0.45771113 0.5046407  0.6159845\n",
      " 0.56169325 0.52264708 0.52900076 0.56001294 0.50966227 0.54763687]\n",
      "[0.4526141  0.42864349 0.40457249 0.39717636 0.42746502 0.52101511\n",
      " 0.51349729 0.54267669 0.51050675 0.48261347 0.47557035 0.45623302]\n",
      "[0.44560996 0.4138836  0.40948841 0.42670307 0.51006591 0.55786538\n",
      " 0.53782851 0.52443802 0.52918971 0.51936352 0.50755179 0.45860183]\n",
      "[0.48762685 0.51400185 0.49667916 0.46646866 0.48456174 0.51187021\n",
      " 0.5781635  0.65726441 0.63822228 0.60114551 0.53823876 0.52293152]\n",
      "[0.49580526 0.43913427 0.42370889 0.55368334 0.53465974 0.50609159\n",
      " 0.53185004 0.56612861 0.66821527 0.64016944 0.56302071 0.57399416]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.46931645 0.39718831 0.38071805 0.41117743 0.38678741 0.41574571\n",
      " 0.43248492 0.49141711 0.58339524 0.50107479 0.49571696 0.48170853]\n",
      "[0.63306922 0.52296579 0.48028082 0.46242794 0.47090372 0.52490348\n",
      " 0.62096018 0.69162732 0.6656664  0.66199607 0.67502135 0.64336574]\n",
      "[0.40844893 0.41417626 0.38586378 0.39064303 0.42996028 0.45498303\n",
      " 0.45434463 0.46681058 0.55254644 0.64960343 0.56722659 0.50891972]\n",
      "[0.60012895 0.59781128 0.53704584 0.51722491 0.52141464 0.554389\n",
      " 0.56804103 0.64306492 0.72001261 0.72328502 0.75426894 0.69712216]\n",
      "[0.49165866 0.42823383 0.43436977 0.46360552 0.37761334 0.41166773\n",
      " 0.49218154 0.55679685 0.65124077 0.61415094 0.63239962 0.57370621]\n",
      "[0.54884589 0.50795931 0.48408091 0.49406427 0.50288886 0.55203003\n",
      " 0.5557282  0.65446514 0.75225574 0.71903354 0.7228862  0.63124633]\n",
      "[0.42161858 0.43080726 0.41894507 0.43025407 0.44644141 0.48312551\n",
      " 0.49633682 0.54111505 0.57758069 0.57783097 0.56557047 0.52302784]\n",
      "[0.55042613 0.52302849 0.5635038  0.58794236 0.6002875  0.62202507\n",
      " 0.68201011 0.72790778 0.70482737 0.65037906 0.63978559 0.59179592]\n",
      "[0.53146994 0.51467383 0.51394212 0.5027048  0.50635105 0.55022663\n",
      " 0.56843835 0.63811123 0.60259432 0.55692136 0.58922315 0.5240767 ]\n",
      "[0.59313619 0.56999558 0.62062496 0.60214573 0.56268352 0.58948272\n",
      " 0.62408906 0.63585025 0.67264211 0.68535691 0.67303187 0.65560877]\n",
      "[0.51346821 0.4932991  0.46915084 0.48103976 0.49313137 0.55442584\n",
      " 0.61499447 0.65564597 0.69140452 0.60987389 0.58123368 0.5701533 ]\n",
      "[0.40700832 0.36991367 0.40782285 0.4142997  0.45463467 0.47100684\n",
      " 0.5207575  0.60599715 0.58095676 0.53365093 0.4828535  0.46134418]\n",
      "[0.48956224 0.53929156 0.50119704 0.50876951 0.48732856 0.54132068\n",
      " 0.59279692 0.57188666 0.55922604 0.53319138 0.52427226 0.48737428]\n",
      "[0.57556099 0.53471434 0.53883368 0.55986446 0.59679383 0.64673513\n",
      " 0.6751228  0.76849318 0.79436791 0.72631127 0.71453154 0.66600722]\n",
      "[0.66268462 0.66139799 0.63825423 0.65912777 0.6467455  0.6950264\n",
      " 0.73212737 0.74828213 0.76838654 0.75012976 0.76808447 0.75596654]\n",
      "[0.46539304 0.44596189 0.49435237 0.56141514 0.53787374 0.52735114\n",
      " 0.56122619 0.62900317 0.61912501 0.58124661 0.56378114 0.51999861]\n",
      "[0.51592201 0.48904955 0.47360197 0.52557129 0.58897865 0.59657216\n",
      " 0.56242567 0.57210302 0.55813664 0.51273704 0.55834889 0.55281466]\n",
      "[0.55387104 0.54176801 0.50280112 0.53287739 0.542463   0.63994884\n",
      " 0.63653034 0.63416457 0.60502946 0.57925415 0.58167553 0.53853792]\n",
      "[0.49700376 0.42469859 0.43707058 0.46974644 0.48417792 0.50840682\n",
      " 0.56448478 0.63592947 0.60906613 0.52405125 0.53046006 0.50778329]\n",
      "[0.59500331 0.55832684 0.52537674 0.5620293  0.57434624 0.60729343\n",
      " 0.67725539 0.71685773 0.74701351 0.67771447 0.62064618 0.63270384]\n",
      "[0.58136922 0.57580501 0.55331123 0.45605114 0.43517238 0.47350207\n",
      " 0.50491285 0.53844821 0.58762419 0.59820771 0.55050105 0.54184717]\n",
      "[0.61947656 0.51247507 0.51775956 0.51508898 0.50757921 0.53357196\n",
      " 0.57438308 0.71931159 0.72789401 0.63421887 0.68203449 0.68477315]\n",
      "[0.4763771  0.46656901 0.48632747 0.53042555 0.54947966 0.54804373\n",
      " 0.58307195 0.57114458 0.57359391 0.58944255 0.56424463 0.48237249]\n",
      "[0.48220038 0.44318357 0.44508535 0.48108956 0.50532109 0.51342016\n",
      " 0.52846873 0.56184042 0.54169875 0.51839519 0.51216775 0.51925623]\n",
      "[0.50816339 0.50504959 0.50198239 0.55294901 0.55292755 0.54271901\n",
      " 0.57075125 0.60528576 0.6464808  0.61132693 0.53284937 0.51664722]\n",
      "[0.57813513 0.53976411 0.57254606 0.63073605 0.59531492 0.6365726\n",
      " 0.70572448 0.72102356 0.70464718 0.69698668 0.74140489 0.67700988]\n",
      "[0.45874554 0.47083828 0.49499959 0.56785524 0.57849985 0.53116316\n",
      " 0.51939982 0.58426237 0.63287371 0.59532207 0.58823198 0.52719826]\n",
      "[0.53124958 0.53864545 0.50664204 0.47431406 0.48485199 0.5640195\n",
      " 0.53443986 0.54045397 0.58400744 0.5569272  0.54375899 0.51472735]\n",
      "[0.54915321 0.55273974 0.61217761 0.69469118 0.65217596 0.66708446\n",
      " 0.70703167 0.72968751 0.7435773  0.71745163 0.67762762 0.61044097]\n",
      "[0.53557676 0.52426833 0.53157187 0.59150016 0.61166513 0.59055007\n",
      " 0.61058384 0.6421234  0.62870091 0.62144476 0.61755413 0.57559186]\n",
      "[0.47890198 0.45915794 0.45388466 0.49449396 0.60380006 0.59676814\n",
      " 0.57729989 0.60400444 0.59982485 0.54510599 0.52853113 0.50480074]\n",
      "[0.47359657 0.46911782 0.46497548 0.45355666 0.49242237 0.55925679\n",
      " 0.58914828 0.62617916 0.58884567 0.54093307 0.52245969 0.48253018]\n",
      "[0.40795135 0.4121576  0.44891283 0.483794   0.58170235 0.61679411\n",
      " 0.52511376 0.52505684 0.51258433 0.50017625 0.46677294 0.42251399]\n",
      "[0.42584702 0.42101669 0.4409171  0.48074698 0.53968304 0.59453189\n",
      " 0.49272317 0.48379421 0.50050497 0.49094859 0.46377262 0.42558947]\n",
      "[0.42861915 0.42874131 0.48554513 0.47353867 0.39931622 0.40268394\n",
      " 0.4696824  0.59674329 0.65228671 0.58576167 0.50909698 0.43320054]\n",
      "[0.58842951 0.55169946 0.48033789 0.47162423 0.47997531 0.48270035\n",
      " 0.54089785 0.60208851 0.61580294 0.5669803  0.55472702 0.55447233]\n",
      "[0.4787356  0.44377723 0.48386577 0.49043781 0.43862888 0.45881775\n",
      " 0.51985162 0.58968198 0.59480917 0.56796694 0.57257712 0.53462452]\n",
      "[0.58633441 0.54679102 0.54035437 0.58931315 0.62780744 0.64876932\n",
      " 0.67705834 0.76611847 0.77055871 0.70100051 0.6763438  0.62784851]\n",
      "[0.42804608 0.40773183 0.44140175 0.48914513 0.48735023 0.48779359\n",
      " 0.49056801 0.53621179 0.54064327 0.49150378 0.48959932 0.44799894]\n",
      "[0.51735085 0.52324963 0.55423576 0.5627237  0.54500347 0.58780527\n",
      " 0.59040469 0.61218226 0.61866874 0.61913568 0.64052671 0.56507105]\n",
      "[0.56136066 0.55768985 0.55179995 0.548172   0.55618745 0.55839723\n",
      " 0.57032037 0.63083494 0.71104199 0.68383706 0.65177244 0.58404082]\n",
      "[0.60102922 0.62773627 0.60347539 0.60193115 0.58914703 0.61151004\n",
      " 0.62943763 0.67887545 0.76314282 0.75214809 0.67982012 0.60509813]\n",
      "[0.47860911 0.48355889 0.49317816 0.47038475 0.43560246 0.484191\n",
      " 0.47770265 0.48837972 0.49220216 0.47784111 0.46009472 0.43520078]\n",
      "[0.54520339 0.53566933 0.54486418 0.52057964 0.47966191 0.56817645\n",
      " 0.57377017 0.55913275 0.57554841 0.58024764 0.57560635 0.53251815]\n",
      "[0.54668832 0.53395742 0.49276099 0.45126879 0.44989425 0.48711127\n",
      " 0.54079878 0.5928331  0.64359015 0.59376001 0.5632121  0.55507207]\n",
      "[0.43987375 0.43283603 0.44616553 0.48324242 0.50687426 0.52740675\n",
      " 0.56062138 0.58857626 0.56012005 0.5518719  0.54827923 0.55449373]\n",
      "[0.62917179 0.58727944 0.60151958 0.70763832 0.71554267 0.70900995\n",
      " 0.70406115 0.73754406 0.78317052 0.7393961  0.66775775 0.67022479]\n",
      "[0.47623301 0.4346267  0.45405832 0.49928951 0.55856985 0.6061402\n",
      " 0.55704832 0.54450798 0.57797849 0.56833506 0.57491034 0.5267235 ]\n",
      "[0.51760751 0.47751617 0.51694822 0.53849208 0.52138507 0.56428325\n",
      " 0.61023366 0.65349162 0.66054088 0.62734401 0.62540174 0.57776058]\n",
      "[0.52322108 0.50373489 0.46979636 0.49529994 0.51153553 0.57869828\n",
      " 0.58925766 0.60681814 0.65275156 0.6158604  0.59751773 0.56416714]\n",
      "[0.44254148 0.47356775 0.50164878 0.52969199 0.46401715 0.46414214\n",
      " 0.50809819 0.56999272 0.60988873 0.5337882  0.51460612 0.44353351]\n",
      "[0.5081386  0.51476622 0.57283437 0.5573175  0.53109699 0.56952441\n",
      " 0.56339568 0.60288662 0.61821216 0.61226594 0.55044872 0.50688517]\n",
      "[0.47933272 0.46559671 0.46662235 0.50219089 0.53388906 0.57229501\n",
      " 0.58689588 0.61999148 0.629076   0.57229024 0.54916853 0.51570714]\n",
      "[0.49436957 0.49304721 0.46999583 0.47975606 0.51053864 0.52252966\n",
      " 0.53837574 0.57584119 0.59996063 0.55568308 0.54694676 0.51278496]\n",
      "[0.67119616 0.71482104 0.71609896 0.68022144 0.69869161 0.7174722\n",
      " 0.67681015 0.77824306 0.81262851 0.76591766 0.7285127  0.69350821]\n",
      "[0.63776475 0.62869197 0.6515668  0.68157417 0.72103733 0.71910679\n",
      " 0.69526279 0.79357934 0.79593295 0.71199393 0.68635774 0.67529696]\n",
      "[0.59223711 0.62393618 0.66909915 0.67398643 0.66714764 0.72721446\n",
      " 0.72201079 0.74440694 0.78542703 0.73223627 0.71338958 0.68816257]\n",
      "[0.6826359  0.67426968 0.67964977 0.70056874 0.70117348 0.68847901\n",
      " 0.75384945 0.82389307 0.77841288 0.74492222 0.72205669 0.69785053]\n",
      "[0.65041035 0.62434715 0.65356624 0.68201613 0.69309121 0.7588951\n",
      " 0.76106709 0.76327342 0.75029325 0.71450573 0.77534652 0.72546405]\n",
      "[0.64318305 0.64055747 0.66836983 0.70865893 0.73784214 0.7316246\n",
      " 0.72706032 0.77561438 0.75939709 0.78637165 0.77324313 0.74194634]\n",
      "[0.72599578 0.62325102 0.62756693 0.68060416 0.73775828 0.72729832\n",
      " 0.72530204 0.74624145 0.73995924 0.71784025 0.72909635 0.76165867]\n",
      "[0.65113878 0.57679236 0.58080369 0.62910831 0.6785447  0.6975826\n",
      " 0.71742433 0.72481406 0.68200183 0.66731417 0.69347531 0.7071892 ]\n",
      "[0.6207214  0.63175839 0.64969617 0.66396302 0.70929813 0.74721193\n",
      " 0.74996042 0.73015243 0.73309964 0.73527807 0.7412163  0.70305341]\n",
      "[0.69615221 0.71999437 0.72809231 0.74638659 0.78888613 0.78211081\n",
      " 0.77798307 0.81646705 0.80949843 0.77252603 0.76028687 0.69528896]\n",
      "[0.72118831 0.61994398 0.59710282 0.64227277 0.69323337 0.6965549\n",
      " 0.69266605 0.74321687 0.6702587  0.64580095 0.6845324  0.72789425]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.62488604 0.63332427 0.64288104 0.73355877 0.78596705 0.72010797\n",
      " 0.7086193  0.75676239 0.80055445 0.82357252 0.76346064 0.68031406]\n",
      "[0.62797463 0.6634059  0.69254148 0.68871289 0.74201661 0.7638275\n",
      " 0.78435147 0.75189549 0.69075298 0.68817377 0.72634333 0.67760003]\n",
      "[0.62270886 0.65027499 0.68046242 0.74480349 0.75276047 0.77614623\n",
      " 0.80368078 0.76593256 0.77709234 0.7552864  0.73765802 0.70615864]\n",
      "[0.66297495 0.6414119  0.69221151 0.69942778 0.72045767 0.74771369\n",
      " 0.78364813 0.777394   0.80348063 0.78011274 0.78520489 0.79702306]\n",
      "[0.57842791 0.59688342 0.66819173 0.70405167 0.74281543 0.77563429\n",
      " 0.78356403 0.76489747 0.73700792 0.70488316 0.68427229 0.64828485]\n",
      "[0.62249327 0.68291414 0.64377576 0.62917131 0.65192229 0.66370547\n",
      " 0.66734946 0.69897056 0.66933715 0.67723173 0.66474116 0.62795866]\n",
      "[0.61807799 0.63841695 0.69530165 0.69364583 0.63334095 0.65230727\n",
      " 0.69989049 0.66170806 0.71872348 0.69815433 0.68809873 0.67185128]\n",
      "[0.5268656  0.52938086 0.60546535 0.65336519 0.64819342 0.68379545\n",
      " 0.70581293 0.71564394 0.71666259 0.67247903 0.65789926 0.63658261]\n",
      "[0.60531533 0.61642879 0.61799473 0.63346547 0.63914013 0.66340899\n",
      " 0.67252523 0.677315   0.70592356 0.7322638  0.73697966 0.69412571]\n",
      "[0.72339785 0.73683006 0.71923989 0.72060734 0.75014681 0.78746939\n",
      " 0.79833096 0.82589453 0.84698266 0.83142793 0.81898904 0.76867437]\n",
      "[0.65599859 0.65164584 0.66183156 0.68506187 0.68717444 0.73192555\n",
      " 0.75177819 0.79135412 0.85963535 0.80264008 0.79683542 0.71862537]\n",
      "[0.61214846 0.62941605 0.71810192 0.7314207  0.6953811  0.76499814\n",
      " 0.76892233 0.80301952 0.85611695 0.809223   0.79646766 0.71313065]\n",
      "[0.66866547 0.69544488 0.70818716 0.70007682 0.72487026 0.76257014\n",
      " 0.79344124 0.82561737 0.81408763 0.7878952  0.81977981 0.75694466]\n",
      "[0.74408716 0.69748133 0.70860487 0.708606   0.69275528 0.69923347\n",
      " 0.72041923 0.78239673 0.8185218  0.78779101 0.75882298 0.78926921]\n",
      "[0.65937954 0.67420125 0.71297663 0.70739466 0.69869012 0.76329154\n",
      " 0.78405797 0.75821775 0.76643932 0.79960513 0.78454256 0.71300071]\n",
      "[0.71418083 0.6647855  0.66466403 0.71835726 0.73147613 0.74215573\n",
      " 0.77389342 0.78146422 0.75813252 0.73900396 0.7778942  0.80390298]\n",
      "[0.65156764 0.66200119 0.66658819 0.65216696 0.67920333 0.78478336\n",
      " 0.8323012  0.72605217 0.71202856 0.71782231 0.73060161 0.70985001]\n",
      "[0.65895098 0.68535066 0.69662356 0.68704766 0.67757821 0.7186926\n",
      " 0.71298569 0.72588331 0.69836426 0.70062333 0.731534   0.72828275]\n",
      "[0.64676923 0.69798362 0.75457227 0.76852471 0.74036407 0.72569102\n",
      " 0.76332742 0.7721768  0.75016886 0.7339918  0.74036992 0.70188785]\n",
      "[0.59025651 0.67280436 0.69328356 0.71667677 0.70918894 0.71460205\n",
      " 0.68515062 0.68556994 0.72298038 0.74328774 0.69498038 0.62326264]\n",
      "[0.55894911 0.61234504 0.70503938 0.74802643 0.72582257 0.688815\n",
      " 0.69871557 0.71589327 0.71957195 0.77108943 0.74894255 0.65980196]\n",
      "[0.5894646  0.67605096 0.75924212 0.81899166 0.73257893 0.67160082\n",
      " 0.69848818 0.70986307 0.72698516 0.70944601 0.71916693 0.64661235]\n",
      "[0.67813683 0.66285849 0.68179905 0.76672971 0.82846981 0.76120609\n",
      " 0.70765465 0.72777265 0.73825312 0.74340779 0.73803914 0.73441392]\n",
      "[0.55979675 0.65512156 0.6621334  0.74731326 0.75152624 0.81160212\n",
      " 0.75115532 0.75611556 0.72394419 0.69767505 0.6605047  0.58744955]\n",
      "[0.58636981 0.6990543  0.69608068 0.71546477 0.71679193 0.72062266\n",
      " 0.7814374  0.71687502 0.62461632 0.62037176 0.6220603  0.57764155]\n",
      "[0.69409335 0.75673181 0.71747041 0.74460924 0.70443827 0.6941855\n",
      " 0.74165082 0.83733636 0.82752448 0.77721846 0.74334568 0.68169886]\n",
      "[0.66123372 0.70487326 0.70108449 0.70847189 0.71390957 0.73653871\n",
      " 0.75803816 0.75476342 0.77290261 0.76948202 0.72449398 0.68070006]\n",
      "[0.69765919 0.72814119 0.70989966 0.74566686 0.73012441 0.71790153\n",
      " 0.80128026 0.82487071 0.79862499 0.81223583 0.8068164  0.77376825]\n",
      "[0.68591326 0.69086391 0.71481985 0.78968334 0.79205322 0.73480105\n",
      " 0.77199429 0.7823562  0.77272534 0.75704509 0.75971287 0.73431104]\n",
      "[0.59718919 0.64247435 0.60998088 0.63103485 0.68524677 0.69066471\n",
      " 0.72767544 0.71026635 0.70207882 0.70040953 0.67116874 0.61341506]\n",
      "[0.68109447 0.64006144 0.57949966 0.61417353 0.62530893 0.6419369\n",
      " 0.69711131 0.65379536 0.65427428 0.63970041 0.5765993  0.65959638]\n",
      "[0.65045524 0.6151166  0.5966208  0.62405479 0.67270011 0.66093355\n",
      " 0.67406601 0.70762098 0.652583   0.68691027 0.63447952 0.61715239]\n",
      "[0.6065312  0.6694569  0.7215513  0.71803612 0.65704918 0.57773721\n",
      " 0.57214975 0.53862929 0.60753208 0.60237807 0.55935502 0.57731783]\n",
      "[0.66604936 0.64827418 0.66052222 0.72290766 0.73545742 0.79125899\n",
      " 0.79091859 0.76326102 0.77128357 0.7606436  0.74573779 0.73169756]\n",
      "[0.67997897 0.68547273 0.71337622 0.72166342 0.76219052 0.80895233\n",
      " 0.79322654 0.76033151 0.7998153  0.78375721 0.73782212 0.72003663]\n",
      "[0.65770781 0.64300889 0.71343213 0.75836641 0.75703776 0.80690253\n",
      " 0.81222194 0.71863961 0.73937547 0.75291342 0.73998088 0.70339334]\n",
      "[0.69684988 0.68404174 0.70248127 0.72513986 0.75300109 0.79366934\n",
      " 0.84288055 0.77886111 0.7722497  0.77953428 0.76515627 0.75634861]\n",
      "[0.70306146 0.76221979 0.73828256 0.74329764 0.73949307 0.75107354\n",
      " 0.80277401 0.76239896 0.73792833 0.70855284 0.74156809 0.72741681]\n",
      "[0.70773214 0.74453348 0.73116094 0.73505896 0.73448163 0.72058028\n",
      " 0.73285979 0.74887228 0.76284671 0.73289788 0.72388959 0.73022324]\n",
      "[0.64352453 0.68459707 0.7105937  0.68449628 0.66357964 0.72917241\n",
      " 0.7569679  0.76071918 0.7273528  0.73723179 0.76833278 0.73591745]\n",
      "[0.70604897 0.74205333 0.71151435 0.71438992 0.71605223 0.73587292\n",
      " 0.71809351 0.67824316 0.70540923 0.73409688 0.74854892 0.7317462 ]\n",
      "[0.68850112 0.7082935  0.7133711  0.73098356 0.75254184 0.72853971\n",
      " 0.70625508 0.70724171 0.75013471 0.765876   0.74513716 0.73554736]\n",
      "[0.67659283 0.66037709 0.69476211 0.72882259 0.73089379 0.71669745\n",
      " 0.67213792 0.68936723 0.7450977  0.78107512 0.78027856 0.74307799]\n",
      "[0.72482103 0.69823927 0.67660737 0.71175879 0.74100375 0.74690765\n",
      " 0.76161224 0.76295179 0.78731793 0.79230183 0.76479465 0.73698527]\n",
      "[0.61898631 0.6194973  0.64936185 0.6752516  0.69880348 0.72329473\n",
      " 0.67771482 0.65423548 0.69643039 0.73254448 0.78916407 0.72835869]\n",
      "[0.58140755 0.64947718 0.69920361 0.73770738 0.77790499 0.7126537\n",
      " 0.73203319 0.70119315 0.69405085 0.69775444 0.70261937 0.65423948]\n",
      "[0.59712183 0.63367897 0.67007977 0.68987489 0.68373334 0.71614772\n",
      " 0.7804172  0.76389521 0.71014374 0.6550861  0.65299541 0.64830273]\n",
      "[0.61562711 0.68318915 0.69704503 0.7582643  0.75944388 0.73736149\n",
      " 0.75393242 0.74730825 0.74964333 0.73604125 0.6829406  0.64903378]\n",
      "[0.63528621 0.66283554 0.68793809 0.66849333 0.6759181  0.67870301\n",
      " 0.67931765 0.67543209 0.68289024 0.70640266 0.74276048 0.70934039]\n",
      "[0.57127196 0.55684966 0.62964207 0.70322973 0.66774023 0.71430016\n",
      " 0.6724534  0.676489   0.73802054 0.75087267 0.78687596 0.64247239]\n",
      "[0.600582   0.53536677 0.62004179 0.64625633 0.63902903 0.70498109\n",
      " 0.70961571 0.64847815 0.67440766 0.67102987 0.71981615 0.69537455]\n",
      "[0.5840807  0.52738547 0.53974438 0.59534705 0.60941535 0.60795838\n",
      " 0.65649873 0.6701988  0.68304104 0.69774765 0.65377879 0.66354591]\n",
      "[0.65315461 0.55910212 0.54745126 0.61190939 0.58810085 0.63141888\n",
      " 0.6639148  0.66733295 0.69306415 0.69253606 0.76715863 0.74165678]\n",
      "[0.51359516 0.49257952 0.4886246  0.50475019 0.54575145 0.64088976\n",
      " 0.70553291 0.70685214 0.67560786 0.6619547  0.71431178 0.62711442]\n",
      "[0.50876117 0.45919788 0.49773738 0.47555664 0.47470781 0.56270546\n",
      " 0.65008193 0.72421372 0.70275694 0.65964192 0.70083553 0.64387715]\n",
      "[0.44265619 0.47429439 0.46701917 0.43555182 0.49882022 0.5495621\n",
      " 0.67260635 0.67760825 0.68407083 0.71311057 0.69598079 0.54544276]\n",
      "[0.47457111 0.44169194 0.41533926 0.45335346 0.51942223 0.56598294\n",
      " 0.59306717 0.57635027 0.65720516 0.65399033 0.74959713 0.61271495]\n",
      "[0.66020089 0.59423411 0.55319208 0.5629428  0.57433581 0.64703864\n",
      " 0.73823565 0.69562972 0.68360358 0.67804891 0.68133527 0.64979941]\n",
      "[0.56155545 0.48330477 0.49617824 0.54143983 0.59912944 0.67418826\n",
      " 0.6717484  0.66347772 0.6715675  0.66839468 0.71354926 0.67837012]\n",
      "[0.45873275 0.41920584 0.44435513 0.46307257 0.51227349 0.61322165\n",
      " 0.61787271 0.65028322 0.6698873  0.6653077  0.73012233 0.58292091]\n",
      "[0.41446868 0.44722503 0.42158872 0.42947829 0.47663715 0.57924259\n",
      " 0.65731955 0.63772064 0.59527338 0.66011333 0.62563902 0.50409782]\n",
      "[0.55641943 0.45762807 0.4911778  0.49238089 0.53166562 0.58649141\n",
      " 0.54012799 0.47765982 0.56719548 0.67244816 0.68675935 0.58327925]\n",
      "[0.54988712 0.48480326 0.46692914 0.5078252  0.52124649 0.53931016\n",
      " 0.57797045 0.58215892 0.62769842 0.64043468 0.6691547  0.63737291]\n",
      "[0.70415574 0.6034925  0.55397779 0.60553133 0.62884533 0.63222957\n",
      " 0.62725067 0.65281761 0.69900399 0.71511233 0.71622384 0.69235069]\n",
      "[0.62035358 0.56791574 0.52200967 0.56169975 0.59901899 0.61013156\n",
      " 0.56521267 0.58700305 0.6778264  0.70428222 0.75396752 0.68852997]\n",
      "[0.41350818 0.4505443  0.51757509 0.55425954 0.51414651 0.53888887\n",
      " 0.47400886 0.48192006 0.57539022 0.65659076 0.62876582 0.48297665]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.59504414 0.61122262 0.53061527 0.44999582 0.4699862  0.52405059\n",
      " 0.52385795 0.62467635 0.7192561  0.69987541 0.65770566 0.56704527]\n",
      "[0.59399939 0.5869875  0.48267826 0.49577332 0.46979395 0.48368698\n",
      " 0.4729611  0.53414518 0.56930304 0.58371794 0.69817209 0.64624012]\n",
      "[0.58782709 0.51373255 0.52256858 0.4861151  0.45022586 0.48464593\n",
      " 0.50377405 0.54330629 0.59160662 0.63567078 0.68181628 0.6174795 ]\n",
      "[0.57253069 0.52018529 0.561068   0.63435215 0.62669802 0.59942108\n",
      " 0.61240113 0.67337215 0.80317312 0.79765242 0.69202524 0.61608011]\n",
      "[0.59646636 0.4823876  0.52078795 0.54570723 0.4948425  0.48601326\n",
      " 0.49286458 0.56763047 0.69470561 0.72288322 0.70567167 0.64315248]\n",
      "[0.63140804 0.58361524 0.52608621 0.55950844 0.60383487 0.59902233\n",
      " 0.5800063  0.62367517 0.71280551 0.74729609 0.72173154 0.70677012]\n",
      "[0.48556727 0.44550806 0.47939318 0.54296565 0.51371223 0.57021463\n",
      " 0.60125285 0.59922487 0.7331996  0.70110202 0.71769637 0.60154998]\n",
      "[0.49954796 0.50103533 0.47995138 0.52580887 0.53869188 0.48151976\n",
      " 0.49056163 0.56042004 0.7336598  0.80686396 0.65221995 0.56878406]\n",
      "[0.53782099 0.51063722 0.52476186 0.51620597 0.55320126 0.57519352\n",
      " 0.61500448 0.67283827 0.74757093 0.74997467 0.70937002 0.63112181]\n",
      "[0.58419865 0.49584007 0.46215481 0.49291283 0.52029747 0.57409382\n",
      " 0.60556936 0.64029133 0.70762992 0.74996257 0.7887131  0.73491597]\n",
      "[0.45373583 0.42848414 0.49855572 0.50003999 0.57458556 0.54585528\n",
      " 0.57367432 0.61840457 0.61592042 0.65697396 0.65027899 0.61071557]\n",
      "[0.59761375 0.50833422 0.53487134 0.62869805 0.66923964 0.66790646\n",
      " 0.60521591 0.6017319  0.67891479 0.72600895 0.73086876 0.69172132]\n",
      "[0.58876306 0.59419346 0.5873664  0.59261507 0.66588455 0.74099958\n",
      " 0.69494867 0.64718705 0.67278481 0.72537804 0.73977304 0.64627177]\n",
      "[0.57026136 0.49081704 0.47845766 0.54169977 0.56287211 0.59483624\n",
      " 0.63361526 0.60308731 0.61505294 0.65077853 0.63447005 0.65293121]\n",
      "[0.55664599 0.55282176 0.5481953  0.54536796 0.54665411 0.60698724\n",
      " 0.59720844 0.58277094 0.66673875 0.63049692 0.64280939 0.6786235 ]\n",
      "[0.49862766 0.5933336  0.63627166 0.59208763 0.59347695 0.62682897\n",
      " 0.59603047 0.60259235 0.63766479 0.60380024 0.59655768 0.54729021]\n",
      "[0.61028057 0.57221162 0.58229744 0.60477352 0.54703319 0.55948156\n",
      " 0.57653111 0.5895071  0.58617789 0.60963345 0.64603913 0.61065578]\n",
      "[0.5731405  0.49641785 0.47533959 0.51796466 0.47744554 0.5164451\n",
      " 0.57908213 0.57053214 0.54664439 0.62898016 0.67284119 0.68737453]\n",
      "[0.62840396 0.48137841 0.40484199 0.40970531 0.46838999 0.4983927\n",
      " 0.54414135 0.62390995 0.61726046 0.59482783 0.64801741 0.68330956]\n",
      "[0.66301727 0.5916937  0.63455731 0.61817008 0.57129139 0.58499914\n",
      " 0.58637059 0.63380957 0.68609595 0.71422958 0.76354313 0.70317024]\n",
      "[0.60511941 0.54317468 0.54206514 0.56406128 0.5437091  0.58558077\n",
      " 0.5783183  0.62025082 0.70584232 0.74848264 0.81357968 0.71582347]\n",
      "[0.63893795 0.61662668 0.63692999 0.61348695 0.53896016 0.51066029\n",
      " 0.51915377 0.54856175 0.67125273 0.73462594 0.7311691  0.68110228]\n",
      "[0.69167346 0.67895257 0.61679953 0.57645494 0.5829109  0.59764826\n",
      " 0.59139961 0.65070313 0.72712266 0.74907291 0.76969188 0.71544349]\n",
      "[0.43774644 0.33247754 0.36214969 0.45286989 0.58098787 0.68002862\n",
      " 0.52242815 0.41126114 0.44287023 0.51700616 0.60431594 0.57995933]\n",
      "[0.32462811 0.27876323 0.31370586 0.38278902 0.52804559 0.67450142\n",
      " 0.62723434 0.5438168  0.49872673 0.53697991 0.56377786 0.42477554]\n",
      "[0.41188574 0.35510916 0.40646064 0.44084305 0.55768079 0.63519788\n",
      " 0.5460394  0.46577412 0.51921177 0.59187329 0.72750026 0.62221503]\n",
      "[0.33063966 0.30067196 0.25342968 0.29962599 0.49424291 0.69036114\n",
      " 0.67270774 0.53288203 0.53244436 0.66316152 0.67572641 0.42367458]\n",
      "[0.51267087 0.54817492 0.54880023 0.54211456 0.56397831 0.60656881\n",
      " 0.6274702  0.60346937 0.64138919 0.71548295 0.75291365 0.60141271]\n",
      "[0.56132346 0.5115841  0.49726406 0.54096502 0.56523961 0.60192168\n",
      " 0.59341019 0.62574446 0.70909095 0.72290981 0.72895235 0.62457985]\n",
      "[0.52224213 0.47734007 0.48038134 0.48471135 0.5049805  0.52651435\n",
      " 0.57850313 0.61560798 0.64417452 0.6647951  0.62169534 0.56660855]\n",
      "[0.42661291 0.37595299 0.39563012 0.45714834 0.41292137 0.4073557\n",
      " 0.45874771 0.48373336 0.60598385 0.58439046 0.54141957 0.49714151]\n",
      "[0.42948946 0.43853381 0.43038324 0.49918598 0.53216255 0.57230008\n",
      " 0.65285516 0.70052475 0.68293345 0.62885147 0.57037991 0.47342491]\n",
      "[0.56508738 0.50528669 0.4758279  0.55973428 0.54147595 0.54402095\n",
      " 0.54357314 0.54893678 0.51407313 0.52457172 0.53359181 0.52782422]\n",
      "[0.60899484 0.59893751 0.65239578 0.64186203 0.61132109 0.60195488\n",
      " 0.59683222 0.6215722  0.6633184  0.67844594 0.6668157  0.62379849]\n",
      "[0.67032194 0.7131995  0.66704524 0.62141865 0.60843855 0.61872804\n",
      " 0.60693377 0.59856361 0.65949297 0.65967423 0.67731285 0.62193906]\n",
      "[0.55927825 0.51727533 0.56666917 0.64223355 0.59394008 0.61274874\n",
      " 0.57634306 0.58030742 0.60363966 0.63262743 0.65011096 0.58894366]\n",
      "[0.56559819 0.57802022 0.51953197 0.53931373 0.57710296 0.57143289\n",
      " 0.57286733 0.55581397 0.57416582 0.62588996 0.64108396 0.6178177 ]\n",
      "[0.6443823  0.59715945 0.59136784 0.67644477 0.70805311 0.73287624\n",
      " 0.71184546 0.67049128 0.69492286 0.73862487 0.75254822 0.73127353]\n",
      "[0.57423353 0.53548288 0.55943304 0.61756808 0.66482484 0.67509747\n",
      " 0.65390885 0.65518117 0.67034602 0.75257301 0.75332034 0.67468154]\n",
      "[0.71743339 0.63584101 0.61036247 0.64757651 0.70751309 0.72733188\n",
      " 0.69350845 0.64924347 0.64609677 0.6792115  0.75491506 0.76007557]\n",
      "[0.56587833 0.52927583 0.56314462 0.66217339 0.73123485 0.72955984\n",
      " 0.72092164 0.72207439 0.74962723 0.75519055 0.71656716 0.65559757]\n",
      "[0.56760919 0.53824466 0.50822109 0.55816102 0.58296478 0.65276802\n",
      " 0.69725007 0.67916799 0.71112353 0.70942348 0.69700056 0.61142892]\n",
      "[0.46411651 0.43605283 0.49046144 0.54755449 0.60550183 0.62106442\n",
      " 0.64471042 0.70521718 0.77162486 0.76776284 0.70324165 0.57706374]\n",
      "[0.65886301 0.64673233 0.62427807 0.71160531 0.75299031 0.64980334\n",
      " 0.64191878 0.66440207 0.71983439 0.77876806 0.77478921 0.70415241]\n",
      "[0.68472499 0.65471309 0.63632256 0.6737417  0.70047432 0.68333077\n",
      " 0.62754095 0.64219981 0.66650766 0.75816786 0.82799143 0.716609  ]\n",
      "[0.71360105 0.70791322 0.64322925 0.67639244 0.66919947 0.66852593\n",
      " 0.69240803 0.69081736 0.71657801 0.75921035 0.79151911 0.74717557]\n",
      "[0.72385573 0.74053669 0.6345644  0.64642632 0.70170915 0.69194949\n",
      " 0.64155877 0.6695118  0.75541264 0.75434339 0.73196971 0.69644344]\n",
      "[0.66279602 0.63540566 0.62764668 0.59857029 0.62607604 0.57319683\n",
      " 0.58053136 0.66359633 0.7029618  0.6625073  0.64023322 0.66260487]\n",
      "[0.7052514  0.63221282 0.62103868 0.68284523 0.71268696 0.68472314\n",
      " 0.70118177 0.71829408 0.75505781 0.75698721 0.7455641  0.74800169]\n",
      "[0.69361043 0.68501794 0.63161862 0.66723955 0.69546038 0.71403664\n",
      " 0.6962564  0.72029215 0.69386244 0.74032414 0.82810479 0.77488726]\n",
      "[0.73640662 0.6540668  0.65868932 0.7044794  0.70120782 0.68111229\n",
      " 0.69847101 0.75808054 0.78726876 0.78405589 0.78860295 0.79763108]\n",
      "[0.71240461 0.68519324 0.63994902 0.61893284 0.69337404 0.68861949\n",
      " 0.66809464 0.68274885 0.66092604 0.694058   0.78776103 0.78556395]\n",
      "[0.61613202 0.64382249 0.54288715 0.5902968  0.64254129 0.60908765\n",
      " 0.52253634 0.59522486 0.69882959 0.75282389 0.73925513 0.64406723]\n",
      "[0.64553362 0.63020754 0.53162527 0.62740397 0.7052682  0.68681473\n",
      " 0.59399873 0.57913214 0.6908282  0.7095958  0.6322332  0.59152144]\n",
      "[0.66443247 0.64279252 0.54523689 0.60998958 0.63234943 0.6773392\n",
      " 0.70925522 0.69298488 0.7387861  0.80716157 0.75864309 0.61581242]\n",
      "[0.74774212 0.72584409 0.60804629 0.64576989 0.71232784 0.72240913\n",
      " 0.70446306 0.75174123 0.81096697 0.80679643 0.83338892 0.74435753]\n",
      "[0.76416981 0.73072743 0.67741662 0.69938213 0.73988944 0.66805035\n",
      " 0.60943305 0.63248956 0.72697163 0.763928   0.73131859 0.72399354]\n",
      "[0.7156533  0.79468584 0.66115475 0.64700866 0.69270426 0.7254675\n",
      " 0.74064368 0.70259178 0.66872936 0.74562001 0.785092   0.6768111 ]\n",
      "[0.7156533  0.79468584 0.66115475 0.64700866 0.69270426 0.7254675\n",
      " 0.74064368 0.70259178 0.66872936 0.74562001 0.785092   0.6768111 ]\n",
      "[0.67297292 0.68614769 0.63935983 0.64709324 0.58895171 0.57018405\n",
      " 0.60424918 0.58785444 0.66074038 0.71576238 0.69667107 0.65049946]\n",
      "[0.69263482 0.68360454 0.63475037 0.62239957 0.61166376 0.59977585\n",
      " 0.60695618 0.65878212 0.71757853 0.75043124 0.71531051 0.71028244]\n",
      "[0.63818586 0.62253332 0.60918945 0.6695025  0.62549686 0.5424704\n",
      " 0.58391398 0.65295208 0.76974308 0.78428042 0.70382845 0.64360482]\n",
      "[0.72839016 0.76560634 0.63848871 0.56090462 0.52830374 0.57005209\n",
      " 0.58349699 0.57152599 0.61190116 0.63396889 0.66988635 0.68767184]\n",
      "[0.65601605 0.6226899  0.57492644 0.59642345 0.62699199 0.62939554\n",
      " 0.62863994 0.66057628 0.70868772 0.70938551 0.69664073 0.67620456]\n",
      "[0.66199905 0.62437731 0.58486837 0.6257509  0.64552337 0.62262481\n",
      " 0.6553539  0.72052091 0.76102185 0.80713797 0.78697044 0.7518788 ]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.61460155 0.58651167 0.58050191 0.64429164 0.66734552 0.63280225\n",
      " 0.64766228 0.68851572 0.73899299 0.77575189 0.7310217  0.64675272]\n",
      "[0.68192405 0.71160781 0.66330642 0.67865855 0.65545398 0.60588747\n",
      " 0.64370495 0.70065135 0.70956832 0.71397877 0.73175889 0.69795436]\n",
      "[0.71657521 0.69165838 0.6324895  0.66120511 0.64453471 0.63487959\n",
      " 0.66374314 0.71813184 0.7423054  0.7611196  0.69721061 0.67106706]\n",
      "[0.67638493 0.64120352 0.58283955 0.63381886 0.6968016  0.7349295\n",
      " 0.72286725 0.72964537 0.73485214 0.70348734 0.70858532 0.70662755]\n",
      "[0.64948159 0.64568841 0.58734894 0.63977605 0.6240111  0.6703493\n",
      " 0.74603379 0.77039814 0.76787555 0.7463277  0.67667896 0.63645196]\n",
      "[0.57736325 0.60728508 0.63582009 0.7262817  0.75482446 0.70499462\n",
      " 0.67272305 0.68039715 0.70538837 0.71108967 0.67341399 0.61782283]\n",
      "[0.5906018  0.59451652 0.61212999 0.60429269 0.62748963 0.65419328\n",
      " 0.65570265 0.70194113 0.74646997 0.78264612 0.73101377 0.68585885]\n",
      "[0.72698003 0.70667607 0.70114738 0.64875823 0.61682945 0.62918007\n",
      " 0.63648671 0.66948789 0.64182413 0.660276   0.70463222 0.74575531]\n",
      "[0.51038611 0.51552296 0.5004012  0.52352989 0.54238141 0.54520553\n",
      " 0.53827685 0.60864961 0.61969751 0.68089581 0.67919385 0.59658432]\n",
      "[0.58361304 0.617823   0.64963233 0.65232772 0.6113739  0.59765297\n",
      " 0.65013719 0.67202365 0.70352077 0.6909858  0.66197914 0.603378  ]\n",
      "[0.75971752 0.72561944 0.66781479 0.65493464 0.67414868 0.66134137\n",
      " 0.66189826 0.68623525 0.71938914 0.74410141 0.76467556 0.80739105]\n",
      "[0.69009656 0.68717325 0.67236781 0.66700339 0.69089127 0.67665833\n",
      " 0.65748    0.71432108 0.75148237 0.77743632 0.79387617 0.73466629]\n",
      "[0.56883281 0.59535426 0.58520132 0.59286696 0.68135971 0.68550038\n",
      " 0.65071791 0.68631631 0.75306469 0.74844849 0.71829414 0.64148337]\n",
      "[0.59629452 0.6991958  0.62502426 0.61169732 0.60440016 0.66635942\n",
      " 0.63756055 0.63481456 0.71968752 0.71587723 0.72439373 0.67913175]\n",
      "[0.61171997 0.60045856 0.55679077 0.57517421 0.63541704 0.66325736\n",
      " 0.69507509 0.72775066 0.75728333 0.70539105 0.65264696 0.64626741]\n",
      "[0.60991359 0.58464259 0.5493809  0.59803379 0.58901608 0.59342182\n",
      " 0.66633284 0.72060114 0.79785669 0.7521053  0.68587226 0.61648208]\n",
      "[0.61604238 0.57264739 0.59719759 0.60602355 0.63144863 0.66963017\n",
      " 0.66783869 0.68922168 0.65144974 0.58875042 0.57099545 0.61828762]\n",
      "[0.64895445 0.63231027 0.61676133 0.60190427 0.71437043 0.6967299\n",
      " 0.67752826 0.71980596 0.71994394 0.72475159 0.66677278 0.65829843]\n",
      "[0.59704745 0.54683954 0.57343233 0.66172796 0.64047587 0.55254108\n",
      " 0.53215539 0.56699306 0.61493814 0.64975864 0.62727696 0.60759193]\n",
      "[0.68294269 0.71392959 0.57735074 0.5442059  0.55597603 0.60639858\n",
      " 0.61608773 0.61292148 0.6784187  0.67169553 0.60360765 0.57798797]\n",
      "[0.60672045 0.59335101 0.55497509 0.627056   0.69115001 0.66265005\n",
      " 0.65631855 0.60476881 0.62322319 0.66221023 0.67372543 0.61554003]\n",
      "[0.64121044 0.5944463  0.60995692 0.68516076 0.65171176 0.6463024\n",
      " 0.7284447  0.65704513 0.66947657 0.71493155 0.69390273 0.63388991]\n",
      "[0.59611022 0.63215309 0.58853233 0.60248023 0.58001846 0.55619854\n",
      " 0.58623242 0.59531027 0.64505529 0.61939049 0.60883099 0.5888108 ]\n",
      "[0.75460011 0.77735281 0.72447485 0.75233376 0.71051353 0.66380835\n",
      " 0.73479378 0.77089649 0.79078579 0.76875103 0.73526174 0.72568417]\n",
      "[0.68830246 0.63739157 0.62705064 0.66976023 0.6643827  0.72491252\n",
      " 0.7590304  0.76891935 0.74602789 0.74545366 0.79565686 0.72858202]\n",
      "[0.5194397  0.52638549 0.53760427 0.58701456 0.53079772 0.52119058\n",
      " 0.53425997 0.5429458  0.56718773 0.55306333 0.56601375 0.52625114]\n",
      "[0.6125496  0.6268006  0.63029182 0.69067627 0.73057866 0.69916689\n",
      " 0.68597299 0.72833622 0.74731553 0.71523833 0.69133776 0.66173196]\n",
      "[0.64083093 0.67427373 0.69220769 0.78699696 0.78680152 0.75252575\n",
      " 0.77469718 0.79324669 0.78744161 0.75978303 0.72755271 0.67558604]\n",
      "[0.6289593  0.63500434 0.66817319 0.78512383 0.7481308  0.73468792\n",
      " 0.74699861 0.75499189 0.77620471 0.76507539 0.71422774 0.61213541]\n",
      "[0.54159898 0.58450717 0.59081048 0.66256291 0.68802881 0.68658835\n",
      " 0.68232554 0.71284747 0.76212221 0.70272118 0.65980899 0.57066453]\n",
      "[0.63419223 0.59882027 0.5428111  0.58934891 0.59052324 0.58112562\n",
      " 0.61477435 0.65248936 0.66972488 0.65717661 0.63738227 0.61551374]\n",
      "[0.63002288 0.63799369 0.62312633 0.65284002 0.63569254 0.6089167\n",
      " 0.6091913  0.63986784 0.67930549 0.7001068  0.68046701 0.64674056]\n",
      "[0.70570642 0.69948477 0.65248352 0.65732348 0.69952762 0.68047291\n",
      " 0.68781024 0.77016228 0.74978244 0.70638222 0.71102905 0.70770156]\n",
      "[0.66839194 0.7145279  0.71547407 0.73631352 0.7140007  0.68398803\n",
      " 0.71055979 0.73593354 0.75122833 0.77124268 0.78181756 0.71393788]\n",
      "[0.60275126 0.63363403 0.57193065 0.66754276 0.64397061 0.59056318\n",
      " 0.57264352 0.55273235 0.60278678 0.66099876 0.69504142 0.60883588]\n",
      "[0.74353236 0.71400201 0.62849218 0.69425833 0.73171413 0.66945529\n",
      " 0.65277803 0.70458639 0.75836539 0.76669967 0.72152048 0.72305357]\n",
      "[0.75642341 0.74872923 0.6368379  0.64233756 0.67545265 0.66986984\n",
      " 0.66284281 0.64099658 0.64905828 0.70225108 0.73058647 0.72080994]\n",
      "[0.4227103  0.44755331 0.45193106 0.50444144 0.49147201 0.42822507\n",
      " 0.44204971 0.4969517  0.51962084 0.51224387 0.49466348 0.44746733]\n",
      "[0.61055624 0.58587211 0.57603163 0.62009245 0.67201746 0.71347922\n",
      " 0.78673559 0.73977309 0.74460977 0.69252074 0.69196695 0.65617037]\n",
      "[0.57353616 0.57800955 0.57264632 0.58577979 0.61868393 0.71582335\n",
      " 0.75959885 0.75477457 0.74669927 0.72840619 0.72077435 0.65755385]\n",
      "[0.55066502 0.55283558 0.58172911 0.6356712  0.69838703 0.69265509\n",
      " 0.71703762 0.75992209 0.80576187 0.71380025 0.65585053 0.56599212]\n",
      "[0.56860799 0.5892415  0.5974111  0.64025915 0.64494908 0.72713214\n",
      " 0.76144922 0.76140922 0.77611196 0.70587277 0.68065077 0.64878881]\n",
      "[0.61487621 0.56493139 0.57281548 0.60897529 0.62657917 0.71797836\n",
      " 0.77016801 0.83965546 0.79822999 0.72052288 0.70290011 0.71882117]\n",
      "[0.57306182 0.54414469 0.5487271  0.59320265 0.62307048 0.66686732\n",
      " 0.73144531 0.84489584 0.84651864 0.79053885 0.70701611 0.66217077]\n",
      "[0.58762717 0.60980338 0.6025092  0.62194473 0.686418   0.76335078\n",
      " 0.76132643 0.83288443 0.80868578 0.77994424 0.73574466 0.67261308]\n",
      "[0.52737933 0.57507634 0.5814923  0.56564349 0.59021574 0.63603717\n",
      " 0.66593838 0.77850246 0.83751351 0.81912524 0.71865553 0.62593955]\n",
      "[0.53624296 0.5513016  0.59867376 0.56621444 0.5827083  0.68498981\n",
      " 0.79257405 0.80986726 0.76101232 0.71543348 0.69614136 0.66177988]\n",
      "[0.47886598 0.51413691 0.55364203 0.5429402  0.57767802 0.65531451\n",
      " 0.76892573 0.81821012 0.76667905 0.67833143 0.65524185 0.57893038]\n",
      "[0.52965522 0.55070662 0.56557697 0.56013602 0.58236068 0.65640557\n",
      " 0.72784871 0.83517921 0.82808226 0.76502424 0.72509903 0.64952862]\n",
      "[0.49355969 0.56947249 0.54723883 0.55001265 0.5905906  0.65823132\n",
      " 0.80647415 0.85517448 0.76137066 0.69858235 0.65276539 0.55298734]\n",
      "[0.66621    0.67294681 0.63601595 0.63969237 0.7053194  0.74840826\n",
      " 0.75103843 0.71933204 0.75289452 0.70138639 0.69917846 0.64659709]\n",
      "[0.66044271 0.61214107 0.5466938  0.59298873 0.67068017 0.71733677\n",
      " 0.74415284 0.73935306 0.73057526 0.72318786 0.70733345 0.72480953]\n",
      "[0.58603168 0.55858529 0.56866962 0.60989696 0.65350705 0.72898597\n",
      " 0.73224396 0.72849089 0.77062315 0.74448317 0.76767939 0.73207945]\n",
      "[0.59540254 0.53750634 0.5276382  0.56919867 0.6145916  0.6646201\n",
      " 0.69399714 0.75994253 0.73355907 0.70547283 0.68903303 0.70340055]\n",
      "[0.56104779 0.53100979 0.54774684 0.60050786 0.56405896 0.60625404\n",
      " 0.6347059  0.68271154 0.68235976 0.65628433 0.64598143 0.64833122]\n",
      "[0.64736861 0.63419497 0.60241705 0.60305083 0.62561476 0.72132838\n",
      " 0.70483631 0.69961911 0.715312   0.72495085 0.67410845 0.65499216]\n",
      "[0.64800102 0.66110301 0.60601526 0.51798224 0.54941362 0.63092494\n",
      " 0.63235122 0.64847714 0.68202484 0.69737113 0.68902469 0.67047775]\n",
      "[0.61801744 0.6050725  0.52433503 0.54872221 0.5643267  0.61874551\n",
      " 0.65036315 0.68950915 0.70679402 0.68904316 0.643269   0.63761705]\n",
      "[0.5649392  0.5770883  0.57102507 0.59792072 0.58962721 0.63571244\n",
      " 0.68723112 0.74199069 0.82628995 0.82236695 0.74612069 0.67460304]\n",
      "[0.58422309 0.57200074 0.60082394 0.63283068 0.59370714 0.66181809\n",
      " 0.74409556 0.83629256 0.79215747 0.67214394 0.66119248 0.65090638]\n",
      "[0.53349257 0.53361851 0.57610512 0.63429385 0.64308685 0.67766434\n",
      " 0.69612581 0.76346534 0.86470622 0.83601588 0.74664748 0.63107246]\n",
      "[0.48020369 0.46495295 0.52484679 0.58967376 0.5630635  0.62272143\n",
      " 0.69030976 0.82149094 0.89772063 0.76338708 0.60836017 0.57520747]\n",
      "[0.670973   0.61240768 0.48571357 0.48394433 0.51453489 0.55080158\n",
      " 0.56245184 0.64368796 0.66844767 0.64630049 0.63691503 0.67007238]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.56743085 0.59719998 0.61771494 0.63810837 0.57838333 0.57017386\n",
      " 0.59893495 0.67803681 0.65124446 0.61889762 0.61497378 0.61970198]\n",
      "[0.65165365 0.65091902 0.59762216 0.62323457 0.64122355 0.6328882\n",
      " 0.62727612 0.68144888 0.71613383 0.69597429 0.7253657  0.72077876]\n",
      "[0.57534617 0.51448089 0.50053358 0.48588708 0.52737629 0.55588293\n",
      " 0.63663995 0.67221385 0.63677663 0.60487473 0.59214365 0.56275481]\n",
      "[0.62295228 0.60984457 0.61109334 0.60778689 0.60225487 0.63211966\n",
      " 0.68112051 0.71014422 0.75865495 0.78801644 0.73002517 0.65742612]\n",
      "[0.56518006 0.55768692 0.6250062  0.64831543 0.64832401 0.68477052\n",
      " 0.71483886 0.77013922 0.71649325 0.67405093 0.67528111 0.63536245]\n",
      "[0.57600486 0.5790956  0.61703002 0.63610518 0.62348008 0.70479047\n",
      " 0.74759579 0.78988558 0.72531921 0.70115787 0.72500175 0.69711208]\n",
      "[0.58570325 0.5649606  0.55736768 0.58097231 0.58876008 0.63456404\n",
      " 0.66140336 0.72299069 0.7526021  0.71769583 0.68376034 0.65032041]\n",
      "[0.58598107 0.5648697  0.60008031 0.62090552 0.69921231 0.69412762\n",
      " 0.6867311  0.64959925 0.63753068 0.63137126 0.68605936 0.62100214]\n",
      "[0.55006176 0.55719835 0.59176183 0.59873182 0.64594978 0.66537136\n",
      " 0.75127429 0.76322389 0.70088071 0.60435569 0.58575147 0.57222569]\n",
      "[0.67648298 0.62830657 0.63567239 0.67671764 0.65951788 0.63255227\n",
      " 0.66885442 0.7161243  0.64527744 0.61778867 0.61832929 0.67984217]\n",
      "[0.59271002 0.57846957 0.63544023 0.63578582 0.64109498 0.68696642\n",
      " 0.72249579 0.721322   0.67216867 0.60477877 0.59319121 0.58586341]\n",
      "[0.54551435 0.56847727 0.59739083 0.55824035 0.56177372 0.58174819\n",
      " 0.70596409 0.75010329 0.67940056 0.60564399 0.58673799 0.5444451 ]\n",
      "[0.67224663 0.54886788 0.48508605 0.52099127 0.58068669 0.6287812\n",
      " 0.70222616 0.76686919 0.68828106 0.61260122 0.61485881 0.63585991]\n",
      "[0.54096621 0.50830829 0.45683655 0.44016388 0.47860014 0.53287601\n",
      " 0.68188345 0.74452132 0.68755627 0.61122751 0.59382319 0.57285911]\n",
      "[0.63438171 0.62724268 0.58896869 0.55319685 0.58041543 0.59776551\n",
      " 0.60416937 0.71186101 0.70800358 0.66940528 0.63921249 0.64795339]\n",
      "[0.4993766  0.46700346 0.47537959 0.50602996 0.54631305 0.60850239\n",
      " 0.73910713 0.82463074 0.78918815 0.63932639 0.54306841 0.52466398]\n",
      "[0.42510438 0.42014706 0.43069515 0.44629386 0.56085753 0.65115488\n",
      " 0.7712934  0.79571265 0.72413373 0.62927288 0.52909666 0.46876267]\n",
      "[0.50554478 0.46620888 0.43464082 0.44721895 0.47242081 0.57683468\n",
      " 0.65902996 0.77882522 0.80334508 0.65360427 0.5557487  0.53512603]\n",
      "[0.44298708 0.43411982 0.46848848 0.5352971  0.58846503 0.65924442\n",
      " 0.77106863 0.81998289 0.63170511 0.5308283  0.51762205 0.47210822]\n",
      "[0.61418265 0.53795129 0.57467932 0.64488041 0.65344882 0.66962481\n",
      " 0.66320032 0.69829005 0.74062234 0.75941825 0.73833466 0.74931741]\n",
      "[0.61646968 0.58300138 0.63382608 0.6734466  0.72531241 0.75889587\n",
      " 0.7222057  0.72077298 0.77528012 0.76708758 0.7533887  0.7056517 ]\n",
      "[0.53483456 0.51911914 0.56158251 0.60885894 0.60771728 0.66527218\n",
      " 0.70929837 0.72960019 0.70365739 0.71763217 0.65536755 0.62476099]\n",
      "[0.56214619 0.592453   0.62432688 0.63233984 0.65680349 0.69598585\n",
      " 0.70842075 0.7525748  0.75600713 0.73105389 0.71940386 0.64513761]\n",
      "[0.56958103 0.56098795 0.5589571  0.60249871 0.64137149 0.68449873\n",
      " 0.6687603  0.7008394  0.72416824 0.73973089 0.72551489 0.66938895]\n",
      "[0.616328   0.62912625 0.65732259 0.68199253 0.68534595 0.67543334\n",
      " 0.6732018  0.66716176 0.67992198 0.71214068 0.7624349  0.66256535]\n",
      "[0.61283666 0.61342299 0.60207051 0.65536702 0.69033194 0.70722455\n",
      " 0.68432826 0.68559194 0.74782991 0.75339395 0.69701338 0.69282091]\n",
      "[0.59458929 0.60162282 0.65098596 0.65221959 0.65814096 0.67071861\n",
      " 0.68371558 0.73001355 0.75839996 0.72355336 0.7280587  0.63973868]\n",
      "[0.60155427 0.59764755 0.59518218 0.60963064 0.63915408 0.68955857\n",
      " 0.74353385 0.78396678 0.79243082 0.75526845 0.73316938 0.67135978]\n",
      "[0.62610728 0.58157694 0.58591825 0.61352211 0.64975715 0.69358271\n",
      " 0.70683485 0.73306417 0.80191267 0.75057948 0.72454649 0.69654936]\n",
      "[0.5978958  0.55444866 0.5219385  0.57238311 0.61020267 0.65966618\n",
      " 0.66537601 0.74098629 0.76950395 0.74743336 0.69740385 0.65955645]\n",
      "[0.54715788 0.56192911 0.58110124 0.58841342 0.6268943  0.65778583\n",
      " 0.70560127 0.73884386 0.75810635 0.71084923 0.72683442 0.66509652]\n",
      "[0.60072231 0.58364302 0.55130768 0.57493311 0.59217334 0.70289505\n",
      " 0.72615743 0.75270855 0.76867747 0.72581327 0.67854702 0.63131243]\n",
      "[0.59130514 0.59201193 0.56759012 0.59345043 0.60939467 0.67186689\n",
      " 0.69813818 0.76255119 0.75678521 0.73105383 0.70226878 0.67067277]\n",
      "[0.57342863 0.58192724 0.62711847 0.64612532 0.71561801 0.75119132\n",
      " 0.78474867 0.77204239 0.73497283 0.70091045 0.67117172 0.5883891 ]\n",
      "[0.53262478 0.52606148 0.553541   0.59313494 0.61738682 0.67174906\n",
      " 0.71183032 0.74055582 0.75519502 0.71939003 0.72596365 0.64454979]\n",
      "[0.78171581 0.7312215  0.72191608 0.72469187 0.74321324 0.79724616\n",
      " 0.81143463 0.8063944  0.81668568 0.79674399 0.816477   0.82665217]\n",
      "[0.7460435  0.72388661 0.69565976 0.69112056 0.70418435 0.73461199\n",
      " 0.70950758 0.75635976 0.741485   0.75197244 0.81234819 0.80672628]\n",
      "[0.68462801 0.67501277 0.68987918 0.65754724 0.64356124 0.68376923\n",
      " 0.71398175 0.77126116 0.74834502 0.7316336  0.75929558 0.7552129 ]\n",
      "[0.63643491 0.64213198 0.63362879 0.63624364 0.6611411  0.65340066\n",
      " 0.6870966  0.72455621 0.66898137 0.64045823 0.71097571 0.73880202]\n",
      "[0.74214101 0.70604551 0.68841082 0.70932275 0.72622705 0.76706111\n",
      " 0.75398976 0.77475739 0.79222828 0.75040126 0.76549822 0.77761978]\n",
      "[0.79689825 0.72955847 0.70797569 0.67753834 0.61662    0.66171128\n",
      " 0.67395228 0.72595358 0.72631311 0.70391905 0.71859276 0.78816116]\n",
      "[0.70945698 0.68197948 0.69316339 0.70340693 0.66237384 0.66332573\n",
      " 0.68887776 0.70101094 0.68742836 0.66370386 0.67683804 0.77560264]\n",
      "[0.72419816 0.63777083 0.65254658 0.6642583  0.64895475 0.68812293\n",
      " 0.73634589 0.7787984  0.77128261 0.71371096 0.72521263 0.74685603]\n",
      "[0.73123431 0.7430104  0.72166741 0.74286354 0.72493571 0.75997186\n",
      " 0.73717928 0.73214757 0.70843601 0.67162013 0.71792358 0.76862127]\n",
      "[0.69427383 0.73048592 0.73813766 0.82498384 0.80636984 0.7808857\n",
      " 0.72958744 0.71653944 0.69074702 0.62733382 0.69595444 0.73680168]\n",
      "[0.66624045 0.67931223 0.70914096 0.73301107 0.69201541 0.66596991\n",
      " 0.64488393 0.67700255 0.71188098 0.65311062 0.73640448 0.73058522]\n",
      "[0.58543134 0.51549917 0.47005647 0.47737283 0.49007043 0.52657682\n",
      " 0.56696254 0.57499933 0.56106812 0.54810327 0.53026766 0.56631202]\n",
      "[0.6477626  0.65075308 0.61715347 0.65199339 0.68773049 0.68760556\n",
      " 0.75451106 0.73380375 0.6937508  0.71943808 0.77113652 0.69156355]\n",
      "[0.69508457 0.6772182  0.67504072 0.65432757 0.67940235 0.72356433\n",
      " 0.73536396 0.76243109 0.76293093 0.80277044 0.81972086 0.80200779]\n",
      "[0.58893365 0.56526494 0.51640141 0.51360005 0.557845   0.56671613\n",
      " 0.57576954 0.6367889  0.63094276 0.67305815 0.6567198  0.63209605]\n",
      "[0.70366347 0.64509755 0.61476451 0.56685334 0.6071555  0.66053814\n",
      " 0.6966114  0.7434774  0.75594336 0.7578764  0.80841976 0.78812915]\n",
      "[0.55078787 0.49341059 0.46841753 0.49807152 0.59592819 0.6484673\n",
      " 0.64174825 0.68078554 0.67148644 0.64481497 0.60424304 0.56852847]\n",
      "[0.63386315 0.54358971 0.54013145 0.54072338 0.52138734 0.53920341\n",
      " 0.61222976 0.64688367 0.61932856 0.58765358 0.6323055  0.70673001]\n",
      "[0.70821911 0.64542252 0.59314114 0.55104846 0.58551061 0.64413261\n",
      " 0.64388597 0.70739573 0.72175139 0.76013499 0.79574001 0.81422168]\n",
      "[0.53173292 0.54644793 0.56616867 0.57318723 0.57149559 0.57246691\n",
      " 0.6309703  0.74032015 0.7430132  0.66363841 0.62815052 0.55791491]\n",
      "[0.72125298 0.73070043 0.68578798 0.66342795 0.72322613 0.78055477\n",
      " 0.75750095 0.79358059 0.8385433  0.80132681 0.79500538 0.7673133 ]\n",
      "[0.70214474 0.72670609 0.74711001 0.72404736 0.74538183 0.76933706\n",
      " 0.78260612 0.79221874 0.80043989 0.78062385 0.77827847 0.77693599]\n",
      "[0.57514364 0.60742396 0.60162801 0.65078807 0.70174581 0.68577009\n",
      " 0.6881817  0.71348113 0.70548862 0.71468204 0.7256161  0.63683945]\n",
      "[0.66875088 0.62666988 0.66249639 0.69649422 0.77607656 0.84349597\n",
      " 0.80923963 0.8055352  0.7753793  0.77075768 0.79017192 0.78886557]\n",
      "[0.60346264 0.59841543 0.61761987 0.65405184 0.65412396 0.6474154\n",
      " 0.63311827 0.70186341 0.80703562 0.78950793 0.74139661 0.68511742]\n",
      "[0.70641309 0.73709983 0.71197486 0.66767794 0.65797156 0.69887197\n",
      " 0.70457077 0.75987113 0.76835096 0.78677028 0.82028699 0.74038976]\n",
      "[0.64224631 0.63812995 0.651214   0.68837863 0.72223771 0.73467422\n",
      " 0.68220395 0.6983813  0.70710188 0.69681275 0.68703318 0.67892361]\n",
      "[0.65792733 0.64645213 0.62196523 0.63693583 0.69053972 0.7269184\n",
      " 0.74776417 0.80082327 0.82492685 0.79592305 0.76325339 0.69837594]\n",
      "[0.61890268 0.6228646  0.61291629 0.59901607 0.6396451  0.68513787\n",
      " 0.72702658 0.81509936 0.79269087 0.74547005 0.71095335 0.69028527]\n",
      "[0.58202285 0.53056186 0.5018006  0.51116818 0.56206214 0.60354787\n",
      " 0.62118083 0.68317556 0.67881757 0.68173885 0.68045241 0.68695503]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.66119468 0.64830351 0.63187408 0.64306837 0.65181118 0.68907732\n",
      " 0.71618712 0.7603184  0.75621158 0.71870518 0.71573114 0.72179186]\n",
      "[0.69595456 0.63859051 0.59256703 0.62701029 0.66869849 0.71295404\n",
      " 0.76100552 0.75755006 0.74967605 0.77936387 0.81804699 0.78285712]\n",
      "[0.56974626 0.63460559 0.64698261 0.64615542 0.68224394 0.6863569\n",
      " 0.69228876 0.73053205 0.75842088 0.73705006 0.66178232 0.57952678]\n",
      "[0.61411256 0.6442892  0.69439095 0.67832863 0.63594943 0.64903688\n",
      " 0.67659408 0.7589578  0.74816889 0.73175746 0.64792484 0.60239196]\n",
      "[0.60792524 0.67809606 0.65209359 0.64721626 0.70045865 0.73870862\n",
      " 0.72537714 0.74395221 0.67788517 0.60492063 0.56526941 0.56864637]\n",
      "[0.58315957 0.61867392 0.62778413 0.69363374 0.72457993 0.67464453\n",
      " 0.7403515  0.75570053 0.68011951 0.61384737 0.57887775 0.54459816]\n",
      "[0.71119964 0.6871202  0.6941672  0.68894762 0.72350174 0.72199494\n",
      " 0.71899086 0.70559275 0.68793237 0.70637149 0.75433588 0.76936066]\n",
      "[0.72835517 0.7257303  0.70556587 0.68167144 0.71229863 0.76784027\n",
      " 0.77790231 0.83997053 0.82916778 0.78460121 0.79604065 0.77897781]\n",
      "[0.75875598 0.67729235 0.66873914 0.69805753 0.78868717 0.83209324\n",
      " 0.82164419 0.78204948 0.77848798 0.79365516 0.79657805 0.78098303]\n",
      "[0.72492498 0.73365575 0.70557553 0.6637916  0.71270734 0.80189335\n",
      " 0.83230793 0.83918101 0.79442245 0.75910693 0.7837835  0.78102171]\n",
      "[0.69465601 0.67837048 0.70099288 0.6858905  0.64652097 0.65199655\n",
      " 0.71140736 0.75838625 0.69789028 0.65200388 0.6787073  0.70200408]\n",
      "[0.73235971 0.71060038 0.65133721 0.6526643  0.66533959 0.71226561\n",
      " 0.77332407 0.82188195 0.75580317 0.70616293 0.71482635 0.73256236]\n",
      "[0.59575522 0.60090011 0.62416178 0.67633849 0.6598193  0.68072808\n",
      " 0.73875368 0.80910856 0.73971695 0.64978284 0.63740128 0.61350989]\n",
      "[0.53678185 0.497215   0.50178665 0.55067056 0.62693167 0.72849387\n",
      " 0.71278274 0.79441977 0.76449615 0.62846208 0.54848361 0.53595412]\n",
      "[0.74997443 0.71054661 0.67541355 0.63095623 0.63839567 0.67570871\n",
      " 0.745103   0.78821349 0.80395365 0.80938828 0.75641894 0.76372272]\n",
      "[0.75121891 0.74617755 0.72066128 0.68687469 0.69767195 0.71499461\n",
      " 0.73290431 0.80901086 0.82980043 0.78786612 0.7874077  0.74674153]\n",
      "[0.69804466 0.69996971 0.74955612 0.7271198  0.70819724 0.74721211\n",
      " 0.75622964 0.77583867 0.77693856 0.79434693 0.78145319 0.75272143]\n",
      "[0.74972945 0.77660108 0.73824883 0.63448656 0.60167378 0.6310153\n",
      " 0.63471878 0.70670086 0.74094343 0.76112121 0.79483384 0.79513246]\n",
      "[0.70021081 0.68898493 0.69934392 0.74547011 0.75271004 0.75282198\n",
      " 0.75699574 0.78881854 0.75110251 0.74522412 0.77788234 0.76407826]\n",
      "[0.69577789 0.67924237 0.7149449  0.70213759 0.71924633 0.75024563\n",
      " 0.74390894 0.77594548 0.78130114 0.77767992 0.78701812 0.76045263]\n",
      "[0.70093834 0.65202278 0.65804499 0.68302345 0.73319334 0.72984028\n",
      " 0.77206659 0.7899915  0.77304929 0.7780512  0.77463901 0.73505658]\n",
      "[0.64807707 0.63795108 0.63876539 0.65975714 0.66971689 0.69421154\n",
      " 0.72972959 0.75274622 0.73548961 0.7565726  0.76441973 0.7588228 ]\n",
      "[0.74976891 0.7532739  0.73619127 0.72840983 0.74923426 0.79198408\n",
      " 0.81130695 0.84175092 0.81687909 0.79582125 0.82916451 0.78494626]\n",
      "[0.60928655 0.60266662 0.57023025 0.57640868 0.56420547 0.55368656\n",
      " 0.60454237 0.6483916  0.63553119 0.62798709 0.61261749 0.60180265]\n",
      "[0.77728224 0.75728637 0.70505363 0.710989   0.70129764 0.71447986\n",
      " 0.75382316 0.8082099  0.82690042 0.79363507 0.78173637 0.78353757]\n",
      "[0.66233486 0.64045995 0.6282329  0.61623198 0.62494785 0.66530234\n",
      " 0.68884093 0.73505408 0.74932742 0.73351794 0.72683549 0.72160566]\n",
      "[0.69332069 0.73717648 0.74920642 0.72348475 0.73070782 0.76186216\n",
      " 0.77079272 0.80007184 0.77476281 0.77043146 0.7830767  0.73851937]\n",
      "[0.76948601 0.75562066 0.74097407 0.75714672 0.75579774 0.73721254\n",
      " 0.77892119 0.79409915 0.79345733 0.78470057 0.76986504 0.75352508]\n",
      "[0.62235707 0.6432029  0.63758993 0.61383313 0.59558654 0.59938097\n",
      " 0.58457559 0.65923315 0.68617058 0.69641906 0.66897607 0.64909667]\n",
      "[0.68926346 0.66403288 0.66120327 0.67106539 0.69480062 0.68250901\n",
      " 0.74020576 0.78095019 0.80052221 0.77753299 0.74779552 0.7174263 ]\n",
      "[0.61268544 0.54638815 0.55015689 0.53964108 0.55204505 0.59078199\n",
      " 0.61705256 0.65135968 0.68317556 0.68250817 0.65620571 0.68274993]\n",
      "[0.49731863 0.51080739 0.53861016 0.54885536 0.59541053 0.63744843\n",
      " 0.70163804 0.72265631 0.69774634 0.65397722 0.69390148 0.60083091]\n",
      "[0.52467805 0.48895413 0.43857116 0.43605751 0.51314008 0.53043848\n",
      " 0.52547252 0.55990422 0.63725352 0.70841372 0.72635192 0.62084681]\n",
      "[0.55051959 0.49970138 0.4703674  0.49305019 0.54556066 0.63707066\n",
      " 0.63835382 0.66642135 0.69086325 0.71451956 0.62677717 0.62995476]\n",
      "[0.59150022 0.59524649 0.58078265 0.54710245 0.55887568 0.65680641\n",
      " 0.72189677 0.77475554 0.7382313  0.71349132 0.64663941 0.60006118]\n",
      "[0.49788466 0.40029547 0.43858781 0.49648154 0.48045087 0.52533567\n",
      " 0.56864923 0.6172269  0.7317335  0.71533376 0.71210474 0.68812066]\n",
      "[0.44121531 0.37397531 0.37383199 0.43743545 0.43255761 0.44654834\n",
      " 0.5593031  0.63073915 0.73480654 0.67072248 0.65766078 0.58164525]\n",
      "[0.5214467  0.44630682 0.41129854 0.45244879 0.44773498 0.47098586\n",
      " 0.56243956 0.60338187 0.652619   0.7395584  0.74332535 0.66913563]\n",
      "[0.44142145 0.44245461 0.43786094 0.44376263 0.47997117 0.55513829\n",
      " 0.55146599 0.50772858 0.58321154 0.64686835 0.70803821 0.61845541]\n",
      "[0.39862609 0.39135578 0.41722262 0.41158924 0.47741616 0.56730533\n",
      " 0.5605582  0.60372782 0.68896222 0.68687332 0.77621448 0.54174101]\n",
      "[0.48977298 0.49328545 0.47429153 0.51409239 0.56106228 0.6183812\n",
      " 0.60176432 0.67225093 0.77993661 0.80581975 0.64811581 0.55941296]\n",
      "[0.59655505 0.58218247 0.56654364 0.60491914 0.64456928 0.6731317\n",
      " 0.66031444 0.72196364 0.70477456 0.74935871 0.68900079 0.6300683 ]\n",
      "[0.59052467 0.59724998 0.6409353  0.65287727 0.61646539 0.62467819\n",
      " 0.63465869 0.67593729 0.67703724 0.70253646 0.69634366 0.5996716 ]\n",
      "[0.50064987 0.53507215 0.56690633 0.52255702 0.50402677 0.5201903\n",
      " 0.65568626 0.69032139 0.64775026 0.64059818 0.60918242 0.51359415]\n",
      "[0.58058238 0.59586072 0.64810675 0.63278288 0.55437261 0.51912051\n",
      " 0.54397076 0.64254004 0.74156821 0.70879292 0.64344317 0.60137147]\n",
      "[0.45604691 0.44262254 0.45006704 0.46153384 0.52598161 0.55776423\n",
      " 0.65988189 0.72242707 0.73444158 0.70359135 0.59579033 0.50885236]\n",
      "[0.46240804 0.42114708 0.4312489  0.44899932 0.48467615 0.51810336\n",
      " 0.55017358 0.6544233  0.76940036 0.83769935 0.76627696 0.57851911]\n",
      "[0.42954844 0.39909595 0.42414451 0.42456222 0.48152056 0.54132819\n",
      " 0.56884807 0.62259138 0.73150897 0.74388963 0.74250013 0.5805586 ]\n",
      "[0.51404572 0.54788679 0.54724997 0.53297317 0.57097262 0.61752218\n",
      " 0.65179437 0.69534773 0.76249927 0.75946772 0.70450902 0.56621963]\n",
      "[0.47692898 0.48755553 0.56608075 0.51230299 0.46655539 0.50418222\n",
      " 0.59453934 0.84036815 0.80676299 0.68532431 0.5850181  0.54638153]\n",
      "[0.4747583  0.419898   0.42023963 0.4597939  0.49461222 0.60465807\n",
      " 0.6572057  0.74727404 0.7273559  0.69201231 0.62657815 0.57993346]\n",
      "[0.50598568 0.49811277 0.50603086 0.55147493 0.54197973 0.5683195\n",
      " 0.60292178 0.69501543 0.82358968 0.73397732 0.66532797 0.55018675]\n",
      "[0.61471897 0.61451179 0.57466924 0.51411968 0.55774361 0.72028512\n",
      " 0.80455697 0.78761315 0.72703189 0.67739087 0.65287775 0.61336261]\n",
      "[0.63494241 0.60021108 0.61061692 0.59401798 0.55525416 0.60386205\n",
      " 0.59044743 0.63516545 0.67335474 0.68113196 0.70843518 0.72268307]\n",
      "[0.55589181 0.48814636 0.51099461 0.5534426  0.51807296 0.53131801\n",
      " 0.60824871 0.69710177 0.74992865 0.72774976 0.73742813 0.62683737]\n",
      "[0.62290049 0.54547459 0.50496954 0.50918531 0.61422956 0.68385607\n",
      " 0.70041007 0.73397511 0.74591476 0.68292779 0.6638875  0.63230532]\n",
      "[0.60261834 0.64689827 0.56938398 0.52273124 0.57473743 0.65293306\n",
      " 0.72577906 0.78718156 0.72544765 0.66706806 0.64296442 0.57588226]\n",
      "[0.56596613 0.50891489 0.52774733 0.54180419 0.55647272 0.62702864\n",
      " 0.63902122 0.70920259 0.701801   0.65863043 0.63906217 0.65059131]\n",
      "[0.50056463 0.50436389 0.54712051 0.63701671 0.63319188 0.62482125\n",
      " 0.68166029 0.75162125 0.75686598 0.65182251 0.58723611 0.49616447]\n",
      "[0.54452789 0.59100515 0.5760479  0.5549109  0.57135862 0.65784329\n",
      " 0.69870907 0.68282121 0.67279184 0.68240213 0.67303866 0.6109373 ]\n",
      "[0.49717343 0.49067187 0.55599248 0.55481786 0.56686682 0.65131909\n",
      " 0.6552515  0.76177019 0.74427325 0.72721612 0.68775213 0.58398867]\n",
      "[0.57402885 0.53108472 0.55827844 0.5597102  0.58919901 0.63605696\n",
      " 0.63894314 0.60862827 0.67188138 0.6982758  0.70858008 0.64052767]\n",
      "[0.44660878 0.46904576 0.48063555 0.49136037 0.59695596 0.70868421\n",
      " 0.67263418 0.64813608 0.63173598 0.56087315 0.49370417 0.45001581]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.52197486 0.54861063 0.59529793 0.64146894 0.66690129 0.69639486\n",
      " 0.71004254 0.74187988 0.72741407 0.66530466 0.64133286 0.54845649]\n",
      "[0.56683463 0.56130481 0.5660224  0.52345145 0.48799774 0.58171642\n",
      " 0.66721308 0.67370546 0.6830911  0.68700695 0.60831851 0.62454379]\n",
      "[0.69148487 0.68932152 0.56853116 0.55603218 0.51198351 0.50369138\n",
      " 0.60930634 0.68786412 0.64194334 0.64734751 0.71511656 0.74232942]\n",
      "[0.56569302 0.57545388 0.54101479 0.52157909 0.50506222 0.59630883\n",
      " 0.67890918 0.70971799 0.7320438  0.75265253 0.71378297 0.6384719 ]\n",
      "[0.61712438 0.56253022 0.5029363  0.47369525 0.47537765 0.57705367\n",
      " 0.62954754 0.5935477  0.63373834 0.64546251 0.65388387 0.62952286]\n",
      "[0.58569616 0.51421028 0.51320779 0.51723957 0.51740271 0.59988278\n",
      " 0.62092406 0.6671195  0.70020342 0.75476784 0.8384648  0.78036457]\n",
      "[0.53763062 0.52722651 0.53240049 0.58302301 0.60899901 0.66653359\n",
      " 0.64837986 0.63454294 0.61294597 0.69832563 0.75277567 0.64133853]\n",
      "[0.58164889 0.55097729 0.52840096 0.58021927 0.62352926 0.72625899\n",
      " 0.71191853 0.63174832 0.60539317 0.65002799 0.70949924 0.69570148]\n",
      "[0.45957115 0.44068438 0.42784727 0.44683328 0.49307862 0.59155327\n",
      " 0.56460941 0.55241889 0.60094768 0.6834926  0.65684611 0.53443611]\n",
      "[0.52064937 0.53851682 0.57410187 0.56842464 0.53686398 0.51960796\n",
      " 0.56201094 0.59583628 0.66827762 0.70563298 0.64654785 0.54854923]\n",
      "[0.50124031 0.59036303 0.55520719 0.49513206 0.50006509 0.61711162\n",
      " 0.736745   0.74317747 0.68179518 0.67731512 0.67784947 0.61820459]\n",
      "[0.62250632 0.60042316 0.50095963 0.47771233 0.58819294 0.68360412\n",
      " 0.68737483 0.62439412 0.65162879 0.70340508 0.68116707 0.57977009]\n",
      "[0.51968741 0.51913106 0.58211505 0.56773019 0.61460978 0.7049008\n",
      " 0.68045658 0.64021218 0.62905443 0.65490723 0.65312666 0.55531299]\n",
      "[0.56182635 0.53922278 0.53177243 0.49481413 0.48762462 0.53789455\n",
      " 0.61554027 0.67569959 0.69704354 0.68649828 0.80347729 0.70797956]\n",
      "[0.51203102 0.50783134 0.51731426 0.54030532 0.58291    0.57828969\n",
      " 0.58964086 0.63651311 0.66431659 0.66513377 0.66587758 0.58584028]\n",
      "[0.54579097 0.55479079 0.52711904 0.51889926 0.53958589 0.564776\n",
      " 0.55742896 0.55138153 0.59881765 0.66580451 0.75100571 0.6523121 ]\n",
      "[0.60844904 0.53905237 0.50613147 0.50724548 0.49647787 0.49049938\n",
      " 0.54263711 0.53870118 0.54743779 0.64482707 0.74683189 0.69086426]\n",
      "[0.45144129 0.52316076 0.49172765 0.48778036 0.44320345 0.49846789\n",
      " 0.54227853 0.51372409 0.56350845 0.62877375 0.65612406 0.55936182]\n",
      "[0.63164294 0.62716872 0.64458406 0.64899933 0.60907853 0.66704524\n",
      " 0.71779138 0.76168597 0.69467157 0.68556237 0.76147926 0.7315315 ]\n",
      "[0.58969349 0.5766241  0.54809368 0.5329417  0.59293538 0.65808624\n",
      " 0.69024158 0.68365228 0.68673712 0.70807493 0.70803785 0.65419126]\n",
      "[0.65131098 0.62129605 0.60053176 0.58718264 0.6105991  0.63781357\n",
      " 0.66600859 0.71067935 0.71845073 0.73665327 0.7490322  0.68170464]\n",
      "[0.66415823 0.55164397 0.5343892  0.55240673 0.65316772 0.70169878\n",
      " 0.62502277 0.60758621 0.64098656 0.73157692 0.76539332 0.734326  ]\n",
      "[0.58569688 0.59503835 0.62651569 0.5913679  0.62527323 0.65899271\n",
      " 0.68776965 0.68959212 0.72714555 0.67378026 0.65987611 0.60620874]\n",
      "[0.59998834 0.54596061 0.55487937 0.61800289 0.72622871 0.75688803\n",
      " 0.71271849 0.66670591 0.67197156 0.63541049 0.62927973 0.64462018]\n",
      "[0.52012128 0.4933688  0.51509821 0.56770802 0.6475389  0.7439298\n",
      " 0.71734911 0.67405045 0.64662462 0.67230892 0.66377258 0.60468292]\n",
      "[0.57125151 0.61350954 0.62830347 0.68603128 0.63047904 0.65306652\n",
      " 0.63916159 0.61841631 0.60805672 0.65337342 0.67348737 0.60776705]\n",
      "[0.67766565 0.70591789 0.66010725 0.6635763  0.70853907 0.74366295\n",
      " 0.71532565 0.75041062 0.79192698 0.81619245 0.7687155  0.74920648]\n",
      "[0.72418344 0.69442505 0.70693731 0.70318848 0.68217689 0.67959964\n",
      " 0.69371599 0.76613963 0.77471095 0.76041073 0.74461538 0.77293521]\n",
      "[0.64038163 0.65844506 0.66321373 0.69144928 0.72424012 0.74173719\n",
      " 0.74655837 0.75587749 0.79879177 0.81474751 0.81630176 0.73354143]\n",
      "[0.60690427 0.65330422 0.66193998 0.68181962 0.74612355 0.75413305\n",
      " 0.74571687 0.77026147 0.83053887 0.85191017 0.76321828 0.6391601 ]\n",
      "[0.64392018 0.67548281 0.7201336  0.72387016 0.76122808 0.78542453\n",
      " 0.84693432 0.87073278 0.84214652 0.81046122 0.76274198 0.67899418]\n",
      "[0.63007361 0.60999763 0.63961768 0.66077679 0.73252106 0.79514641\n",
      " 0.78454226 0.84200317 0.82647371 0.76747161 0.72803831 0.71391219]\n",
      "[0.62988353 0.67742413 0.70725971 0.76807755 0.74687797 0.76402992\n",
      " 0.7793172  0.82895249 0.81445259 0.75397021 0.7189998  0.71636289]\n",
      "[0.54616815 0.63370401 0.66005218 0.67278022 0.73704541 0.75975376\n",
      " 0.72696257 0.74825841 0.77532774 0.77171361 0.70816988 0.61049885]\n",
      "[0.60985732 0.64470106 0.66802794 0.72999722 0.81046462 0.84992522\n",
      " 0.77082837 0.76367611 0.77429616 0.76322341 0.73034936 0.67781919]\n",
      "[0.60987151 0.60928643 0.64119643 0.69081318 0.69136584 0.71350211\n",
      " 0.7424314  0.81703407 0.84043235 0.81847537 0.76033604 0.71070296]\n",
      "[0.62791282 0.61955637 0.64357311 0.70050919 0.69985801 0.76206023\n",
      " 0.78109711 0.76549858 0.77127707 0.79720867 0.76693857 0.72927624]\n",
      "[0.62129742 0.55538267 0.54955846 0.61747682 0.67797524 0.73518151\n",
      " 0.70965332 0.7093944  0.68952191 0.71916795 0.72241801 0.69165951]\n",
      "[0.64594114 0.68830973 0.68703973 0.77778351 0.79681045 0.75144762\n",
      " 0.70002085 0.71607667 0.71093225 0.71868914 0.75630057 0.69292051]\n",
      "[0.67752516 0.69120657 0.66251844 0.71155775 0.73185003 0.80285174\n",
      " 0.79073203 0.7563166  0.74261296 0.75176752 0.77499902 0.73137927]\n",
      "[0.68190366 0.64882904 0.66789824 0.72589922 0.6891157  0.65683192\n",
      " 0.68920386 0.74401766 0.77249002 0.79093122 0.81809062 0.78250003]\n",
      "[0.67330456 0.63435006 0.63957942 0.69596517 0.7431618  0.76265627\n",
      " 0.7005192  0.69327009 0.76599443 0.78688836 0.74839085 0.68720859]\n",
      "[0.55255866 0.61373401 0.6152162  0.65143019 0.67833573 0.69744736\n",
      " 0.64928496 0.63086164 0.61244017 0.65035081 0.67635405 0.59090149]\n",
      "[0.61024767 0.58012354 0.5993818  0.67932767 0.72879636 0.72787005\n",
      " 0.66388208 0.67349529 0.71870428 0.75013101 0.74807    0.71404439]\n",
      "[0.61762321 0.64396518 0.64348096 0.64042056 0.69370043 0.65656763\n",
      " 0.66214257 0.72919983 0.65609455 0.62661546 0.66139942 0.60341227]\n",
      "[0.60663211 0.61859781 0.64689708 0.66292894 0.69097745 0.71080899\n",
      " 0.66749942 0.69003224 0.70211846 0.69292241 0.69682747 0.70229852]\n",
      "[0.62707692 0.62485182 0.6240406  0.66348404 0.66822737 0.70972306\n",
      " 0.69191188 0.71402389 0.76768106 0.79562771 0.81480783 0.76978296]\n",
      "[0.63873965 0.66323429 0.70789278 0.71278983 0.72468567 0.74258012\n",
      " 0.74792111 0.81202    0.8351683  0.78826666 0.70479339 0.63263041]\n",
      "[0.63392287 0.664298   0.60917991 0.61818409 0.66550285 0.71404821\n",
      " 0.65627182 0.67035288 0.74724591 0.80996644 0.81041342 0.74025619]\n",
      "[0.60821807 0.63003457 0.60834616 0.6338113  0.65445316 0.68214124\n",
      " 0.68223029 0.72895658 0.84872669 0.83480811 0.75824088 0.68944198]\n",
      "[0.6556797  0.69476712 0.6846714  0.660559   0.69285363 0.69426405\n",
      " 0.7584129  0.78874099 0.78465527 0.73510015 0.71528733 0.65633619]\n",
      "[0.60329896 0.53117698 0.5749898  0.6563527  0.75090981 0.75573546\n",
      " 0.63198644 0.63032269 0.71278411 0.68403316 0.66470522 0.65302742]\n",
      "[0.63142532 0.659118   0.70898324 0.72023195 0.71212417 0.69385886\n",
      " 0.6461224  0.67938489 0.69546324 0.68607563 0.68421662 0.72251272]\n",
      "[0.64248693 0.66321176 0.64403158 0.67578518 0.72053075 0.7624023\n",
      " 0.70396549 0.69367862 0.6783812  0.70901185 0.74230206 0.70441699]\n",
      "[0.64569062 0.64324403 0.69439143 0.73838532 0.73162293 0.73019671\n",
      " 0.7145797  0.73861867 0.74784178 0.76380622 0.72399563 0.67420262]\n",
      "[0.64552093 0.6646294  0.68433064 0.69948655 0.71843606 0.77416861\n",
      " 0.75170165 0.70984995 0.75418293 0.78649819 0.74831253 0.7050131 ]\n",
      "[0.66241586 0.64679784 0.72188967 0.76939207 0.78175426 0.7052654\n",
      " 0.6885252  0.75638914 0.7848441  0.70706177 0.70528448 0.66315031]\n",
      "[0.61194962 0.64364719 0.66112411 0.68595707 0.71415263 0.70549375\n",
      " 0.70265353 0.70726424 0.71051377 0.72413123 0.70381421 0.68427598]\n",
      "[0.62741101 0.68233091 0.74126267 0.70115536 0.70479017 0.75101036\n",
      " 0.75245863 0.74300009 0.71982896 0.70460391 0.69876289 0.64324832]\n",
      "[0.66021097 0.73023129 0.74709654 0.70179379 0.69254363 0.65634388\n",
      " 0.61645389 0.65998852 0.65948683 0.58660567 0.6163404  0.62907672]\n",
      "[0.63512528 0.68657833 0.68356764 0.72697765 0.69225144 0.69241685\n",
      " 0.63373715 0.62758595 0.67418671 0.66444319 0.65899974 0.71377051]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.6287095  0.75396502 0.6977734  0.66176313 0.67226624 0.7174781\n",
      " 0.65553808 0.64839828 0.63208222 0.54893327 0.51598668 0.58084881]\n",
      "[0.64712256 0.74722743 0.74781668 0.77972293 0.77279943 0.73116457\n",
      " 0.70361352 0.74947274 0.78441525 0.74932861 0.71733409 0.69705403]\n",
      "[0.65878409 0.69251937 0.7183606  0.77442032 0.78954399 0.71483976\n",
      " 0.67731822 0.70946616 0.75800753 0.72066486 0.70611346 0.69528353]\n",
      "[0.63721842 0.67621768 0.70198357 0.72927946 0.77690417 0.76388186\n",
      " 0.73494059 0.73127365 0.71279442 0.72688937 0.7471332  0.71162987]\n",
      "[0.66339594 0.68479383 0.68907607 0.72965389 0.70658064 0.74255729\n",
      " 0.70536888 0.67378914 0.70829296 0.76587838 0.78760558 0.70789927]\n",
      "[0.54383147 0.62278008 0.62342471 0.71339047 0.7989096  0.67075282\n",
      " 0.58078712 0.61249858 0.69968069 0.72533637 0.60407829 0.52471054]\n",
      "[0.66919476 0.6534121  0.61263931 0.61094111 0.67434776 0.743141\n",
      " 0.69102448 0.73947227 0.737409   0.67887843 0.69330853 0.6579805 ]\n",
      "[0.62875336 0.63767165 0.6427055  0.67796165 0.70012087 0.69079381\n",
      " 0.65528804 0.71523243 0.67692763 0.62847078 0.66166055 0.62094414]\n",
      "[0.67614037 0.73222029 0.65216875 0.64045024 0.704261   0.71246547\n",
      " 0.64047456 0.67114866 0.67803955 0.64999855 0.62709731 0.65228885]\n",
      "[0.64003927 0.6614995  0.68340492 0.71386027 0.73183608 0.75475907\n",
      " 0.72359252 0.75559056 0.78015417 0.7637006  0.72366846 0.66532904]\n",
      "[0.66517836 0.66851711 0.66556859 0.70573276 0.74045175 0.72612762\n",
      " 0.7168873  0.75428641 0.75685972 0.75206441 0.72733653 0.66814822]\n",
      "[0.66437125 0.66549611 0.69789737 0.70082045 0.70956659 0.68816209\n",
      " 0.6945079  0.7574535  0.78827238 0.74805576 0.72956115 0.6672793 ]\n",
      "[0.68957704 0.71812451 0.67640233 0.68885708 0.72655278 0.72491848\n",
      " 0.65721965 0.69050086 0.73453337 0.7195307  0.72040111 0.71995652]\n",
      "[0.57578313 0.65974408 0.71635103 0.73862636 0.74464405 0.75303954\n",
      " 0.73280936 0.73373181 0.73423105 0.75575542 0.75103408 0.65495509]\n",
      "[0.5832544  0.66050446 0.6905179  0.65677202 0.64410049 0.69646281\n",
      " 0.70390272 0.72860396 0.76277369 0.74701089 0.72610492 0.64142263]\n",
      "[0.58361834 0.70438874 0.75779176 0.73226058 0.69999617 0.69666898\n",
      " 0.69369662 0.67440075 0.68755692 0.67646796 0.67512494 0.64260471]\n",
      "[0.54759932 0.61675352 0.72081834 0.73636097 0.68365598 0.71676421\n",
      " 0.74576777 0.69745004 0.65245515 0.72967476 0.71586984 0.60759658]\n",
      "[0.59713125 0.64371961 0.67790407 0.70865661 0.76415926 0.7590887\n",
      " 0.72993296 0.73728067 0.75292927 0.77405959 0.75574034 0.66734904]\n",
      "[0.63480067 0.67648292 0.67451084 0.71889263 0.72633159 0.72413057\n",
      " 0.67881989 0.65389287 0.69854605 0.76117784 0.80704808 0.72268999]\n",
      "[0.5695554  0.59694833 0.63691574 0.6642614  0.71111578 0.74986249\n",
      " 0.73496771 0.7659471  0.79785156 0.8082267  0.77024072 0.6502685 ]\n",
      "[0.57210344 0.67497551 0.70768976 0.71314311 0.69337982 0.6764493\n",
      " 0.6845755  0.70020968 0.75994974 0.79926729 0.6898402  0.60545182]\n",
      "[0.62116069 0.67291898 0.71537304 0.70872146 0.70157504 0.75736833\n",
      " 0.68816286 0.6817565  0.72501713 0.77218676 0.79301828 0.68772453]\n",
      "[0.67336708 0.70102018 0.66970432 0.70548719 0.76706409 0.73908997\n",
      " 0.68296242 0.70841235 0.74290031 0.70212954 0.70683062 0.68353254]\n",
      "[0.6561389  0.63911599 0.61917394 0.68094188 0.74251127 0.7812534\n",
      " 0.73050904 0.66928083 0.67822248 0.74025708 0.77156621 0.73491597]\n",
      "[0.70714909 0.70658082 0.63815027 0.65835643 0.69975668 0.69009066\n",
      " 0.70789385 0.74441767 0.7578215  0.73547345 0.74693298 0.72210789]\n",
      "[0.54035175 0.54236621 0.53719735 0.5341652  0.56641877 0.58544374\n",
      " 0.6247707  0.7244454  0.66641951 0.63063878 0.64505744 0.5969817 ]\n",
      "[0.5937894  0.56672829 0.56114697 0.6207571  0.66839832 0.68460977\n",
      " 0.74368    0.77657413 0.69104481 0.69104755 0.67863405 0.64705479]\n",
      "[0.5641824  0.55277425 0.52985549 0.53383219 0.62686515 0.68444479\n",
      " 0.68509108 0.66679722 0.67460907 0.73579472 0.7028119  0.60892963]\n",
      "[0.58629632 0.56701243 0.55319548 0.58274627 0.63606381 0.69187188\n",
      " 0.73893243 0.70124912 0.74043047 0.71839851 0.70412374 0.66185772]\n",
      "[0.60047066 0.5832718  0.59099215 0.56375062 0.59218502 0.60945773\n",
      " 0.63062096 0.69042885 0.69075745 0.66666234 0.65128213 0.66421658]\n",
      "[0.61399949 0.66780514 0.63155848 0.63096958 0.64562052 0.67066097\n",
      " 0.7275129  0.72318757 0.68041247 0.67682606 0.66525036 0.6464541 ]\n",
      "[0.5260219  0.52064359 0.53654319 0.54382265 0.55866319 0.58656371\n",
      " 0.57132012 0.68425089 0.74217969 0.73664469 0.71519589 0.6255399 ]\n",
      "[0.56431979 0.5610373  0.5724389  0.57168913 0.58457589 0.64777929\n",
      " 0.72660011 0.75412381 0.65753537 0.66880548 0.7285223  0.65959823]\n",
      "[0.51528519 0.46299872 0.45643407 0.49809781 0.55188996 0.59702885\n",
      " 0.6037308  0.58550394 0.61873406 0.63799828 0.63251913 0.57351494]\n",
      "[0.5831759  0.57346278 0.58875048 0.60587722 0.6506629  0.66960847\n",
      " 0.65459919 0.69335598 0.68855619 0.71284103 0.71194583 0.64228594]\n",
      "[0.58190101 0.58014482 0.60990083 0.61938465 0.63582051 0.6152336\n",
      " 0.60289782 0.68372697 0.65203905 0.65894449 0.69626659 0.65740126]\n",
      "[0.54471779 0.55616707 0.55950457 0.58574015 0.644961   0.65823042\n",
      " 0.63666004 0.62396502 0.64129359 0.70253193 0.74423712 0.63539279]\n",
      "[0.54554552 0.53394222 0.63272798 0.6798892  0.64018309 0.64487451\n",
      " 0.64504647 0.68669248 0.6740061  0.68568206 0.64219135 0.5997932 ]\n",
      "[0.58898968 0.6303637  0.67720389 0.72840327 0.6844247  0.72925729\n",
      " 0.72780287 0.67759109 0.64874959 0.70557868 0.68371528 0.6322242 ]\n",
      "[0.53212047 0.57947069 0.63824195 0.60821265 0.57209241 0.64370143\n",
      " 0.6298067  0.59625548 0.64416367 0.67134178 0.68849343 0.60637635]\n",
      "[0.57441032 0.5952493  0.60289699 0.60703838 0.62716615 0.67005187\n",
      " 0.7075485  0.67348748 0.62682998 0.68122166 0.73407656 0.66128522]\n",
      "[0.66667986 0.59401041 0.59484768 0.61732346 0.6427474  0.69962144\n",
      " 0.74952984 0.76620835 0.71449184 0.69826239 0.65763569 0.65918159]\n",
      "[0.58319288 0.55285299 0.58950019 0.63168937 0.65184689 0.69885701\n",
      " 0.69687301 0.67704761 0.68528658 0.65009665 0.62347639 0.619712  ]\n",
      "[0.57542247 0.56783056 0.57759839 0.62060165 0.60908222 0.59012508\n",
      " 0.56021994 0.59039205 0.572124   0.58243835 0.64395195 0.58570439]\n",
      "[0.52646285 0.5194754  0.51277196 0.53390318 0.56656718 0.59743202\n",
      " 0.59750623 0.62709326 0.63405645 0.71739435 0.7537536  0.59195268]\n",
      "[0.56756896 0.61964345 0.61116451 0.62902349 0.63462281 0.62566996\n",
      " 0.6198647  0.63222432 0.70324427 0.74571484 0.69705397 0.63202101]\n",
      "[0.59417981 0.60251582 0.6084941  0.58194727 0.5671404  0.63322902\n",
      " 0.62473661 0.656479   0.66500622 0.70949256 0.76383042 0.70912647]\n",
      "[0.54612941 0.51025492 0.55764282 0.67066813 0.65420359 0.65941805\n",
      " 0.66301239 0.67268503 0.72552115 0.74221838 0.75364399 0.67438042]\n",
      "[0.58209819 0.56489414 0.60035765 0.63105959 0.59779251 0.58669007\n",
      " 0.6126309  0.65778035 0.69174951 0.73112708 0.76908016 0.70451862]\n",
      "[0.54349017 0.53554559 0.4932591  0.54269964 0.61361361 0.68511379\n",
      " 0.6440711  0.64024097 0.64682561 0.65710455 0.63969988 0.59106767]\n",
      "[0.46474609 0.40684691 0.45020261 0.50098789 0.5542627  0.63078731\n",
      " 0.59444588 0.62493819 0.67906147 0.72874421 0.74287492 0.60345823]\n",
      "[0.55545652 0.54423189 0.57522857 0.61069131 0.67978185 0.70355678\n",
      " 0.65693015 0.6843785  0.66976792 0.63633537 0.63702124 0.58730054]\n",
      "[0.49911064 0.42978823 0.45827228 0.47227356 0.4359501  0.47087845\n",
      " 0.59557056 0.70771402 0.70159209 0.61815059 0.60822386 0.55636585]\n",
      "[0.59665704 0.58997101 0.60158223 0.61109453 0.62535411 0.68806309\n",
      " 0.70383167 0.69656873 0.6881516  0.66682929 0.69451737 0.65022707]\n",
      "[0.56856489 0.58649975 0.61191398 0.6186927  0.65005463 0.71565098\n",
      " 0.65004396 0.61299241 0.60756952 0.63478231 0.67961562 0.63161427]\n",
      "[0.61317849 0.57731265 0.5739336  0.65464044 0.68642658 0.6758278\n",
      " 0.6548484  0.68331444 0.75527841 0.72792602 0.73716319 0.71432543]\n",
      "[0.61952525 0.64190036 0.65874416 0.6411925  0.67346543 0.70328265\n",
      " 0.69576919 0.70377702 0.69344956 0.68508875 0.69800597 0.68194902]\n",
      "[0.61798924 0.61886448 0.59957194 0.60999608 0.67170888 0.67393911\n",
      " 0.635427   0.64837199 0.63248658 0.64906466 0.62024027 0.62232494]\n",
      "[0.65426987 0.68957764 0.69461346 0.67698073 0.70363253 0.68218309\n",
      " 0.6493898  0.68422616 0.66771281 0.69366461 0.71414322 0.69897121]\n",
      "[0.64450026 0.61996371 0.62939584 0.68292373 0.70883834 0.70432347\n",
      " 0.64286894 0.6496833  0.64901108 0.70157844 0.71414447 0.69652557]\n",
      "[0.53913867 0.54942703 0.54168576 0.59470397 0.59966213 0.65174073\n",
      " 0.61911106 0.60074097 0.62969196 0.62947059 0.61412686 0.55248642]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.62309051 0.61365569 0.57341725 0.60935068 0.67742598 0.70144033\n",
      " 0.65380144 0.68146479 0.70642674 0.72606182 0.71136391 0.66563845]\n",
      "[0.66228223 0.60184103 0.5482921  0.59283715 0.66076356 0.6686542\n",
      " 0.60107297 0.64802903 0.64703542 0.67274904 0.75259745 0.71438736]\n",
      "[0.62219036 0.57740915 0.57946002 0.63113582 0.67635947 0.69555765\n",
      " 0.64106637 0.66984653 0.65857553 0.72661716 0.82021886 0.7229296 ]\n",
      "[0.65135366 0.55717814 0.58706033 0.65302199 0.69805974 0.70266807\n",
      " 0.66902161 0.67408532 0.69620854 0.78125393 0.84251153 0.74442846]\n",
      "[0.47799325 0.43646941 0.43411258 0.45464262 0.50844467 0.52643842\n",
      " 0.60557896 0.72027707 0.76576364 0.79317683 0.71099877 0.58745235]\n",
      "[0.50794166 0.48022443 0.44587988 0.38873535 0.40090549 0.47899723\n",
      " 0.51773655 0.55686432 0.56653959 0.6306296  0.64874291 0.57164538]\n",
      "[0.58669686 0.52548015 0.48964155 0.5334751  0.53882498 0.55553108\n",
      " 0.61183739 0.599693   0.63987565 0.69209296 0.74419981 0.68936276]\n",
      "[0.53702909 0.47076139 0.50867301 0.50148463 0.47057942 0.51198614\n",
      " 0.55317241 0.61207891 0.63042694 0.69254696 0.69035214 0.64298457]\n",
      "[0.63318592 0.51749343 0.49839684 0.55088764 0.5709542  0.58837634\n",
      " 0.54340857 0.59579694 0.62996656 0.65781319 0.65886879 0.67887491]\n",
      "[0.64476919 0.65781552 0.69094199 0.70779836 0.6834805  0.68540037\n",
      " 0.65809304 0.7254765  0.7080515  0.72305083 0.7189672  0.70310092]\n",
      "[0.57770228 0.53998286 0.54629594 0.58640867 0.64291888 0.68028677\n",
      " 0.64699703 0.6392014  0.65598845 0.70068473 0.67783886 0.63884836]\n",
      "[0.60045761 0.59886312 0.63213348 0.66425383 0.7010076  0.72927743\n",
      " 0.72689515 0.73294294 0.71783835 0.75037265 0.7781508  0.70630568]\n",
      "[0.6071105  0.5640375  0.56163454 0.59471732 0.63664407 0.67486084\n",
      " 0.67240417 0.67830992 0.67220753 0.69969541 0.76157409 0.69080782]\n",
      "[0.54639781 0.56376082 0.58109462 0.55401456 0.60301924 0.67568296\n",
      " 0.68789041 0.70008278 0.6744833  0.65876997 0.61359215 0.55763239]\n",
      "[0.56458628 0.56971157 0.5976904  0.58854198 0.65417409 0.65992016\n",
      " 0.64628601 0.67998523 0.68871284 0.76136458 0.71939468 0.6478712 ]\n",
      "[0.56451607 0.57654744 0.57709867 0.60475862 0.68105334 0.66038674\n",
      " 0.5997926  0.58982944 0.64093649 0.71654153 0.73139036 0.65366882]\n",
      "[0.68684793 0.65753496 0.6302982  0.6452418  0.70110744 0.7113173\n",
      " 0.67896825 0.67403835 0.70870954 0.66694945 0.68299693 0.707142  ]\n",
      "[0.6234678  0.57884306 0.56083143 0.60102975 0.62376332 0.64751673\n",
      " 0.64903581 0.66656137 0.67512155 0.63652796 0.66150445 0.66180807]\n",
      "[0.61201006 0.6114434  0.62568235 0.67261249 0.74038064 0.76060998\n",
      " 0.70199293 0.71321774 0.70907474 0.7781077  0.74907601 0.69971031]\n",
      "[0.60228282 0.59642386 0.67918569 0.77331346 0.70382357 0.67493951\n",
      " 0.64566463 0.70303851 0.72200906 0.72412831 0.70290154 0.65534115]\n",
      "[0.55198324 0.55766213 0.56140161 0.57878804 0.64260024 0.71246916\n",
      " 0.67187357 0.66646326 0.62204999 0.67274767 0.63968718 0.59102005]\n",
      "[0.60506386 0.56496888 0.58345234 0.59983838 0.61955935 0.69268811\n",
      " 0.6774888  0.68249053 0.71203792 0.74555254 0.78831834 0.70059693]\n",
      "[0.65244335 0.60233104 0.5665614  0.58428901 0.64576417 0.70464951\n",
      " 0.67818356 0.70927989 0.69880199 0.74151152 0.78273749 0.75035   ]\n",
      "[0.61201286 0.5811429  0.57683891 0.56374854 0.6034165  0.69210804\n",
      " 0.69880331 0.67247492 0.63842231 0.64377463 0.71435362 0.69343585]\n",
      "[0.66748422 0.69670004 0.7359404  0.71230608 0.68861723 0.72526097\n",
      " 0.74444669 0.77747065 0.81440961 0.78602517 0.76617002 0.72602773]\n",
      "[0.68639094 0.71317059 0.72586757 0.70917875 0.72451931 0.69508809\n",
      " 0.71851403 0.78631592 0.79265708 0.81241459 0.76613766 0.73596615]\n",
      "[0.65767318 0.7106325  0.71723491 0.71361405 0.68955284 0.68098277\n",
      " 0.72459167 0.78901452 0.82045346 0.8157869  0.77662086 0.67287558]\n",
      "[0.6685313  0.70003712 0.7257607  0.72339076 0.72640091 0.70635384\n",
      " 0.73029906 0.75925344 0.79434168 0.84892792 0.79883444 0.75358546]\n",
      "[0.69124019 0.70357966 0.73893106 0.68684441 0.67589849 0.70658672\n",
      " 0.72851855 0.80418497 0.78603297 0.78946263 0.81793112 0.7699371 ]\n",
      "[0.72373891 0.73922271 0.72764325 0.72331065 0.68703496 0.69697762\n",
      " 0.7151764  0.75259829 0.80074877 0.79066378 0.78282511 0.76286036]\n",
      "[0.64718592 0.64232153 0.69717216 0.6997841  0.6824072  0.71519679\n",
      " 0.69065505 0.7499156  0.78666449 0.83301646 0.83787543 0.76881057]\n",
      "[0.69903189 0.67092669 0.71955276 0.70104212 0.69581479 0.74602002\n",
      " 0.7417711  0.78557473 0.81748605 0.82409155 0.85216367 0.8164798 ]\n",
      "[0.64179105 0.6934669  0.73368549 0.7181586  0.72567987 0.74457294\n",
      " 0.80369508 0.80795938 0.79045141 0.79834753 0.81150073 0.75642759]\n",
      "[0.65174973 0.65363753 0.73648655 0.7099272  0.68703878 0.71233851\n",
      " 0.75619704 0.81538963 0.78194904 0.74773186 0.73017383 0.76378566]\n",
      "[0.67517579 0.68461818 0.70473665 0.76775455 0.73150796 0.73924732\n",
      " 0.75623524 0.7606017  0.8157236  0.80265743 0.78423923 0.71314448]\n",
      "[0.66268778 0.62983412 0.70537913 0.71866214 0.71207988 0.69949752\n",
      " 0.71886945 0.79062283 0.77857065 0.76472259 0.77472794 0.751472  ]\n",
      "[0.61444491 0.65481853 0.75862586 0.75547355 0.69946402 0.69187248\n",
      " 0.74029392 0.75807971 0.77491474 0.81446302 0.83487731 0.74410427]\n",
      "[0.6048094  0.63662446 0.70659006 0.74038255 0.76777488 0.71092373\n",
      " 0.68575096 0.7217356  0.77191299 0.81230527 0.81839889 0.75922924]\n",
      "[0.64650023 0.68571287 0.70618731 0.70574659 0.74783665 0.76068372\n",
      " 0.74543953 0.73168021 0.77351487 0.78943795 0.77153993 0.7229054 ]\n",
      "[0.62553155 0.66920644 0.66996932 0.70514387 0.75620854 0.72466767\n",
      " 0.72634369 0.76413888 0.76136613 0.80044466 0.7972033  0.64698535]\n",
      "[0.61255503 0.59221011 0.63045239 0.67194825 0.66657364 0.71092689\n",
      " 0.745134   0.75334257 0.70958012 0.76604545 0.76972622 0.72977322]\n",
      "[0.58703005 0.58700764 0.61003786 0.6492281  0.71329993 0.68347579\n",
      " 0.65060496 0.68778777 0.72018933 0.72996855 0.73438078 0.64581746]\n",
      "[0.60790992 0.58999193 0.63662291 0.62150508 0.66307628 0.73501182\n",
      " 0.76667184 0.73125952 0.72154486 0.74265528 0.77247512 0.74135625]\n",
      "[0.62055033 0.62130463 0.64722639 0.6472472  0.70635635 0.7256217\n",
      " 0.70448315 0.69066608 0.71492922 0.78660405 0.83564848 0.7802797 ]\n",
      "[0.66676301 0.67043316 0.63776112 0.65041685 0.69995493 0.68284684\n",
      " 0.73362392 0.73642212 0.75306028 0.81418747 0.80440265 0.76618111]\n",
      "[0.58827162 0.58758461 0.62028813 0.64641654 0.65887725 0.67631167\n",
      " 0.6892823  0.66604048 0.76741368 0.83870411 0.80172145 0.69193417]\n",
      "[0.62229842 0.60491955 0.63792574 0.69897985 0.68514258 0.71787888\n",
      " 0.74893522 0.76073307 0.78548485 0.80844045 0.82278579 0.72504437]\n",
      "[0.61883879 0.60122937 0.58228207 0.59983277 0.63747311 0.64710367\n",
      " 0.66353637 0.73072189 0.75382298 0.76915264 0.74305385 0.69147408]\n",
      "[0.69863653 0.70151806 0.68069851 0.67447227 0.69674981 0.68592846\n",
      " 0.68278569 0.74250937 0.72608358 0.73660654 0.75701809 0.74055129]\n",
      "[0.69952655 0.65111756 0.64660335 0.61576658 0.63755858 0.65758771\n",
      " 0.67272937 0.74980605 0.71762747 0.71608633 0.78829896 0.76786429]\n",
      "[0.66760993 0.67596984 0.70868421 0.67960829 0.68404835 0.66957068\n",
      " 0.70722276 0.7436772  0.74060565 0.76502556 0.8036117  0.80232805]\n",
      "[0.56648314 0.58473253 0.63699538 0.6267032  0.66182995 0.69276905\n",
      " 0.71513742 0.73700845 0.7868616  0.80536395 0.80300581 0.68340379]\n",
      "[0.70453    0.69513661 0.69767624 0.66746742 0.66616112 0.68669748\n",
      " 0.71643823 0.72241694 0.71648192 0.74896991 0.79730672 0.77964526]\n",
      "[0.66673756 0.68031275 0.68879622 0.71552289 0.75353968 0.73178291\n",
      " 0.73259652 0.74583399 0.71630877 0.73583537 0.78061938 0.71658325]\n",
      "[0.63306171 0.71298188 0.70671326 0.71493542 0.69766045 0.68867075\n",
      " 0.71932125 0.71779019 0.7451331  0.79815143 0.76259261 0.6490826 ]\n",
      "[0.61268836 0.63298661 0.69593358 0.75132263 0.75501806 0.72987968\n",
      " 0.73885339 0.7564193  0.74495572 0.73482174 0.73185009 0.63812983]\n",
      "[0.56645882 0.6114403  0.68288279 0.72656989 0.79441184 0.72543782\n",
      " 0.65672946 0.64320707 0.67085147 0.68413538 0.67638105 0.61715305]\n",
      "[0.65032488 0.67280024 0.63854963 0.67413676 0.73257226 0.76012546\n",
      " 0.70296681 0.70051426 0.73246056 0.76790905 0.73535281 0.63524723]\n",
      "[0.61044353 0.69103187 0.74660492 0.71984136 0.70292932 0.71451002\n",
      " 0.65294796 0.64788735 0.65625727 0.6556173  0.65910989 0.65527469]\n",
      "[0.44971931 0.53608322 0.53989702 0.61138266 0.66610461 0.64973927\n",
      " 0.75423312 0.6121031  0.53063399 0.49240887 0.5420087  0.4833445 ]\n",
      "[0.66042292 0.63231963 0.64133316 0.66016096 0.71956325 0.68102133\n",
      " 0.68624544 0.74635929 0.79578716 0.79587585 0.8091293  0.76865971]\n",
      "[0.6957134  0.68176872 0.70338094 0.69485593 0.71066374 0.76430547\n",
      " 0.82368219 0.81072646 0.80654985 0.82877159 0.81252694 0.77235854]\n",
      "[0.68883008 0.68472606 0.73646903 0.75293726 0.76140738 0.70133173\n",
      " 0.7245034  0.72822011 0.73617589 0.77032804 0.79843658 0.79487288]\n",
      "[0.65696168 0.7212016  0.77867532 0.79075605 0.79948217 0.74325651\n",
      " 0.75881308 0.76170868 0.77161896 0.77426988 0.78083205 0.70797879]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.65418029 0.65766764 0.66707647 0.71472037 0.74858469 0.75740492\n",
      " 0.72659045 0.75964397 0.80400604 0.80870396 0.78392369 0.71672547]\n",
      "[0.66467887 0.67065191 0.65686041 0.7067185  0.73900336 0.72072029\n",
      " 0.69435567 0.70827508 0.73686671 0.76328909 0.74873418 0.69697022]\n",
      "[0.68340158 0.62911981 0.63929617 0.65234089 0.6852386  0.70639044\n",
      " 0.73539263 0.76241404 0.79489309 0.77761799 0.79027617 0.73944503]\n",
      "[0.69884425 0.72360462 0.72558028 0.70426607 0.69398928 0.6917854\n",
      " 0.73029691 0.7709474  0.79232723 0.81347549 0.80633903 0.77110159]\n",
      "[0.689417   0.72947556 0.72088474 0.70127159 0.71540987 0.73833615\n",
      " 0.79561508 0.82289922 0.78791332 0.77316862 0.8179003  0.75669408]\n",
      "[0.71098471 0.74021506 0.75823092 0.70639032 0.76704657 0.77229553\n",
      " 0.78178817 0.83352572 0.78935003 0.78049964 0.80739599 0.78096271]\n",
      "[0.65771699 0.68892759 0.68130451 0.72066677 0.76672935 0.76629764\n",
      " 0.80639976 0.80334926 0.79224813 0.80441463 0.80589783 0.73042244]\n",
      "[0.70674467 0.71672446 0.72234398 0.77234334 0.76663202 0.7493661\n",
      " 0.78307921 0.78942573 0.80353975 0.80787766 0.80394459 0.73569363]\n",
      "[0.68614626 0.67096394 0.66680497 0.67610455 0.70633113 0.70195621\n",
      " 0.75696307 0.78498209 0.83053046 0.86881214 0.82218015 0.75635642]\n",
      "[0.67205626 0.69849092 0.67672503 0.64587706 0.67347932 0.71886384\n",
      " 0.73282284 0.76682144 0.77301461 0.79461455 0.8322922  0.73127872]\n",
      "[0.66561919 0.68662882 0.70638937 0.72522324 0.73580134 0.72962284\n",
      " 0.74952197 0.79719359 0.80565852 0.7844035  0.7923497  0.75972867]\n",
      "[0.6866203  0.70280308 0.71079624 0.71290731 0.74342042 0.76602489\n",
      " 0.76276314 0.76331574 0.82413417 0.81421232 0.7792331  0.7192083 ]\n",
      "[0.63498145 0.67256242 0.67812848 0.6923669  0.71199542 0.71397883\n",
      " 0.7764979  0.79952741 0.76251841 0.75233275 0.73198223 0.67222255]\n",
      "[0.6276257  0.69105959 0.76243812 0.75643063 0.72159201 0.72187251\n",
      " 0.71917862 0.77304131 0.80966985 0.76218551 0.75047404 0.70168489]\n",
      "[0.64722335 0.68030632 0.7254709  0.70849615 0.70360768 0.73495793\n",
      " 0.73376429 0.75910944 0.81558049 0.83478498 0.80562937 0.7058363 ]\n",
      "[0.59209991 0.59129906 0.65843993 0.73788375 0.74335217 0.7390874\n",
      " 0.701509   0.75188166 0.81397527 0.77572072 0.72894537 0.66334438]\n",
      "[0.63715041 0.65110934 0.69603574 0.68994409 0.73935431 0.76374722\n",
      " 0.81912386 0.83381784 0.8023271  0.7941553  0.79316276 0.71767926]\n",
      "[0.69533658 0.682356   0.69288909 0.70965201 0.77299458 0.77977711\n",
      " 0.78424633 0.78025907 0.78689855 0.79987717 0.79625481 0.79765797]\n",
      "[0.65812343 0.6554203  0.69843203 0.7159155  0.73322904 0.73179716\n",
      " 0.75390327 0.82763302 0.82565397 0.79905468 0.78010863 0.74835932]\n",
      "[0.67615724 0.70399612 0.70248592 0.71175247 0.71390873 0.72351897\n",
      " 0.78961861 0.82035071 0.8052898  0.76236862 0.73222917 0.69111723]\n",
      "[0.46437743 0.48187023 0.494569   0.51189655 0.53229058 0.57725757\n",
      " 0.59794724 0.57795417 0.53615016 0.51326251 0.53240174 0.50780469]\n",
      "[0.51877487 0.51074481 0.51775801 0.51780975 0.51529896 0.56230348\n",
      " 0.59961933 0.64036191 0.6650269  0.62329721 0.60788399 0.58096385]\n",
      "[0.53696388 0.49895936 0.50874525 0.54689693 0.55122942 0.62069106\n",
      " 0.63152289 0.59532261 0.58303857 0.61074024 0.6844216  0.63025552]\n",
      "[0.51925874 0.45715532 0.44009629 0.50871354 0.56925672 0.56046706\n",
      " 0.50698292 0.53640813 0.57575238 0.56991643 0.54715049 0.55002028]\n",
      "[0.375792   0.3669267  0.41042221 0.42901999 0.52886593 0.49578634\n",
      " 0.53606099 0.59401357 0.49539176 0.41858935 0.40873399 0.45563084]\n",
      "[0.47258061 0.50406712 0.52201122 0.50034159 0.48662263 0.54574978\n",
      " 0.63698584 0.64620644 0.62184566 0.55427569 0.54056609 0.52259105]\n",
      "[0.51813984 0.50904793 0.48305616 0.43748528 0.41768751 0.48677015\n",
      " 0.51689887 0.50439286 0.49829859 0.50344992 0.50447387 0.49697602]\n",
      "[0.50314009 0.44586292 0.43645939 0.44653922 0.45239475 0.47590947\n",
      " 0.56593442 0.62249041 0.58218992 0.57060087 0.56225264 0.49383494]\n",
      "[0.52052486 0.49014705 0.48318872 0.50336164 0.55923241 0.64265817\n",
      " 0.63395077 0.61963326 0.61957121 0.60028988 0.56664205 0.55795914]\n",
      "[0.4396078  0.40602919 0.41436124 0.51919949 0.47547656 0.44920364\n",
      " 0.47861105 0.54939735 0.63511896 0.58281451 0.54167795 0.52013379]\n",
      "[0.55766433 0.57039338 0.54861891 0.52441657 0.54142296 0.65406507\n",
      " 0.61359179 0.57023209 0.60288632 0.59783745 0.55287862 0.55597419]\n",
      "[0.47109452 0.41530955 0.38876882 0.39685869 0.41609642 0.47183499\n",
      " 0.4802132  0.46719584 0.5093317  0.54560626 0.49198735 0.49345276]\n",
      "[0.66237307 0.66542548 0.67062145 0.62770182 0.6429643  0.66142637\n",
      " 0.74445546 0.78662735 0.72318804 0.67966849 0.63489974 0.65164608]\n",
      "[0.43503103 0.45753515 0.46696886 0.51408845 0.53495198 0.56493431\n",
      " 0.55627745 0.56889808 0.59231758 0.58938253 0.52419037 0.45941445]\n",
      "[0.59419703 0.59438092 0.55943787 0.58437365 0.57791436 0.5936023\n",
      " 0.58262295 0.60350603 0.62980258 0.57841849 0.58211643 0.59311968]\n",
      "[0.39958727 0.42834225 0.44511378 0.47405249 0.46879533 0.49971792\n",
      " 0.511702   0.49167529 0.48245618 0.46010956 0.46673879 0.44707504]\n",
      "[0.59431607 0.53464484 0.46387252 0.42945021 0.43989289 0.47114557\n",
      " 0.50989431 0.53060412 0.52748019 0.57215214 0.60602438 0.60648203]\n",
      "[0.51062602 0.48332238 0.4821254  0.48720855 0.50305068 0.53330314\n",
      " 0.57739633 0.63101304 0.57816321 0.58304858 0.6003961  0.54818642]\n",
      "[0.57065064 0.56772923 0.49516305 0.48835963 0.47842965 0.50353068\n",
      " 0.52514499 0.5767464  0.6115495  0.65359843 0.60452014 0.63022155]\n",
      "[0.54520822 0.52591306 0.45604298 0.41921568 0.42459723 0.45528311\n",
      " 0.47584569 0.50418979 0.54675871 0.55081046 0.5906902  0.59781951]\n",
      "[0.56773424 0.57719767 0.58214283 0.5527401  0.5775485  0.58385319\n",
      " 0.62721193 0.67991561 0.63548934 0.60423505 0.64155114 0.59244353]\n",
      "[0.55137759 0.5439353  0.52234149 0.52682334 0.55380601 0.55015892\n",
      " 0.56142056 0.60931987 0.6689707  0.61724102 0.56872642 0.53625458]\n",
      "[0.66121811 0.64457387 0.59345108 0.56322956 0.60498297 0.62594813\n",
      " 0.63950413 0.67196256 0.70673406 0.69777995 0.68394893 0.65963876]\n",
      "[0.6541788  0.62815535 0.58795381 0.58501333 0.59737664 0.61050779\n",
      " 0.62646157 0.70200628 0.73767072 0.74044007 0.68987739 0.64256233]\n",
      "[0.62915248 0.67523563 0.65204644 0.65509534 0.63685852 0.66627866\n",
      " 0.70414478 0.77670521 0.73285103 0.66390312 0.64445853 0.6363216 ]\n",
      "[0.57867485 0.5857833  0.55680519 0.58032525 0.59651649 0.63837618\n",
      " 0.61109716 0.6023789  0.58839852 0.61309922 0.61054629 0.58282983]\n",
      "[0.62762928 0.64659989 0.6343357  0.61943811 0.57929945 0.62885273\n",
      " 0.61256772 0.61289179 0.64360875 0.622271   0.64049083 0.60388064]\n",
      "[0.593229   0.57909095 0.61842597 0.61512333 0.58541894 0.57594579\n",
      " 0.55929995 0.58464456 0.58541161 0.59142607 0.58304054 0.57909155]\n",
      "[0.60718477 0.61211807 0.62135834 0.59588057 0.56752217 0.64332646\n",
      " 0.64528608 0.67976904 0.72624928 0.67971832 0.72679394 0.69333601]\n",
      "[0.54283488 0.56274229 0.54259944 0.52127761 0.50095105 0.54793918\n",
      " 0.58017087 0.61264056 0.57761329 0.53295094 0.54805696 0.55046409]\n",
      "[0.52407062 0.52106696 0.50007904 0.52470607 0.52340245 0.53856999\n",
      " 0.52768987 0.55788815 0.60488892 0.55170983 0.5505296  0.55849433]\n",
      "[0.55789608 0.50526893 0.48140615 0.52137887 0.53372341 0.5831061\n",
      " 0.61200184 0.60025167 0.63383877 0.60224557 0.59862703 0.61915869]\n",
      "[0.61727279 0.55312651 0.56711382 0.55992705 0.55224311 0.55183029\n",
      " 0.58925211 0.60895729 0.59583193 0.56500858 0.64045328 0.6729278 ]\n",
      "[0.56270474 0.56208444 0.58986712 0.51602095 0.53703183 0.57071674\n",
      " 0.61042613 0.60899228 0.57489717 0.5819506  0.60985118 0.57678533]\n",
      "[0.51735419 0.55186826 0.5900352  0.55245429 0.57954884 0.62968439\n",
      " 0.57853174 0.58549535 0.59212768 0.6215024  0.59455782 0.53048706]\n",
      "[0.64219284 0.6022613  0.62397182 0.62552881 0.63395882 0.68555641\n",
      " 0.66618305 0.67206991 0.72632217 0.74119765 0.73653013 0.7026211 ]\n",
      "[0.63517237 0.57395095 0.52084154 0.48602089 0.46894506 0.49459097\n",
      " 0.53739047 0.57250863 0.58217949 0.60385025 0.63781977 0.65457225]\n",
      "[0.6695258  0.64786518 0.56051457 0.56099492 0.52431267 0.52524775\n",
      " 0.52944964 0.57039911 0.63580096 0.65308386 0.63069075 0.64473885]\n",
      "[0.72798264 0.70638853 0.61086506 0.59979862 0.61111063 0.63618404\n",
      " 0.65705723 0.71767694 0.74345291 0.71377212 0.70735681 0.74924535]\n",
      "[0.65008163 0.67670709 0.597175   0.55468655 0.52974337 0.53290498\n",
      " 0.54350305 0.55338091 0.63258767 0.66205603 0.6674369  0.67952442]\n",
      "[0.52330232 0.55154639 0.61366051 0.61221772 0.47276706 0.42858663\n",
      " 0.42090183 0.4347311  0.50021291 0.56016988 0.54411656 0.50314289]\n",
      "[0.5985471  0.54560244 0.54301822 0.51982218 0.47729015 0.41465196\n",
      " 0.41893917 0.44557783 0.5209589  0.57591105 0.59855139 0.63538432]\n",
      "[0.62822825 0.55177236 0.40908065 0.36021861 0.39741066 0.41177899\n",
      " 0.41140309 0.49460271 0.54609424 0.56940192 0.59738922 0.62209064]\n",
      "[0.58010852 0.48873773 0.4108564  0.39879066 0.42206219 0.45700911\n",
      " 0.43881449 0.53088987 0.5795241  0.57792288 0.61448437 0.58815914]\n",
      "[0.61180729 0.61283255 0.58823532 0.5452916  0.51932943 0.57365847\n",
      " 0.6000818  0.59742093 0.60335606 0.61178005 0.62773871 0.62041903]\n",
      "[0.4843781  0.46252939 0.46194354 0.49491659 0.50026798 0.53547412\n",
      " 0.56516373 0.5537495  0.57676488 0.59206045 0.60460848 0.58423209]\n",
      "[0.5413779  0.53599757 0.54235381 0.53276658 0.53350806 0.55447316\n",
      " 0.55251014 0.5735231  0.61056626 0.61645782 0.61425066 0.60522193]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.60130823 0.6318562  0.62711054 0.65530789 0.59021312 0.55332154\n",
      " 0.56484085 0.5565992  0.56129456 0.5504728  0.54200882 0.56032944]\n",
      "[0.49327895 0.48690841 0.50078088 0.51055807 0.53645539 0.54645121\n",
      " 0.57040286 0.50625575 0.54547846 0.59330386 0.54591131 0.55500084]\n",
      "[0.5407939  0.51570213 0.52049124 0.56849504 0.57578331 0.59124714\n",
      " 0.60948259 0.62869376 0.61404693 0.61829323 0.61542523 0.60278898]\n",
      "[0.61116982 0.58735895 0.52151817 0.51039755 0.5014174  0.5297907\n",
      " 0.56218231 0.62616587 0.64695966 0.60375857 0.59021056 0.59208077]\n",
      "[0.47906941 0.496609   0.50029612 0.49670127 0.44274771 0.42418417\n",
      " 0.46551979 0.49173614 0.48033464 0.49057609 0.52207881 0.51330489]\n",
      "[0.52775997 0.52034521 0.50312418 0.55961102 0.58456349 0.64316291\n",
      " 0.61753428 0.59729105 0.59729809 0.58850902 0.56502301 0.53934884]\n",
      "[0.5142588  0.47557804 0.48063526 0.49357659 0.53013986 0.56354564\n",
      " 0.61149079 0.64322859 0.57969612 0.54330027 0.54846919 0.55371338]\n",
      "[0.57455111 0.57698512 0.54220158 0.57006264 0.61131138 0.61594385\n",
      " 0.61586982 0.61259258 0.6345408  0.60572058 0.59898561 0.58900428]\n",
      "[0.50202936 0.50829399 0.50565314 0.51801282 0.54326499 0.57707632\n",
      " 0.5566113  0.60180175 0.59672695 0.59921056 0.62301606 0.57511085]\n",
      "[0.6063658  0.57218742 0.5996424  0.61646402 0.63512063 0.63492793\n",
      " 0.64671934 0.71113831 0.71118218 0.67766792 0.68781275 0.64014006]\n",
      "[0.45838457 0.46581256 0.46339396 0.4688409  0.44767973 0.4730148\n",
      " 0.49049431 0.49427623 0.50450045 0.53422701 0.56672031 0.52984238]\n",
      "[0.55144334 0.49216247 0.50581551 0.51236099 0.49688956 0.51714468\n",
      " 0.53708643 0.57058519 0.60106462 0.60021281 0.60419279 0.5854277 ]\n",
      "[0.5256027  0.51369447 0.501966   0.50377095 0.52703154 0.562558\n",
      " 0.59249836 0.66001803 0.68315822 0.6511389  0.67237157 0.60784996]\n",
      "[0.6652211  0.72566301 0.75891066 0.72904873 0.70403939 0.72616708\n",
      " 0.71787256 0.77663326 0.72901314 0.74040264 0.76799107 0.76414359]\n",
      "[0.71754771 0.76080781 0.72404468 0.70882481 0.73302054 0.73241585\n",
      " 0.73643911 0.76371354 0.78699464 0.79337269 0.77518284 0.72135168]\n",
      "[0.68998533 0.6809296  0.70895553 0.75102603 0.74721825 0.73964894\n",
      " 0.72404408 0.74119765 0.73045677 0.77557862 0.799447   0.77756947]\n",
      "[0.66608816 0.68601346 0.71173728 0.70943886 0.67951602 0.68872529\n",
      " 0.72695136 0.73764288 0.7485413  0.77529168 0.77440864 0.76479119]\n",
      "[0.66043282 0.69381493 0.68309468 0.69943351 0.72770399 0.69550842\n",
      " 0.71722853 0.73352391 0.74443978 0.76221615 0.76090187 0.67134988]\n",
      "[0.61586654 0.70309812 0.67163026 0.66756618 0.67950249 0.70193285\n",
      " 0.69780606 0.70210606 0.68713486 0.71207285 0.71643114 0.64186555]\n",
      "[0.61771542 0.70996666 0.75200236 0.66545337 0.66588163 0.67254549\n",
      " 0.61892533 0.69594562 0.6940608  0.73125708 0.67675167 0.60716951]\n",
      "[0.67217463 0.71041727 0.72237545 0.72903728 0.73129904 0.71470672\n",
      " 0.69342893 0.73940456 0.75722605 0.78808749 0.78331339 0.72545803]\n",
      "[0.60899079 0.6279031  0.6968891  0.6826731  0.682181   0.72723854\n",
      " 0.71723831 0.72449058 0.72377038 0.78108519 0.78718096 0.72168052]\n",
      "[0.65863633 0.66492939 0.69334137 0.69530177 0.70759761 0.6970191\n",
      " 0.68576813 0.72211057 0.72208369 0.75476474 0.78081065 0.73083484]\n",
      "[0.64714181 0.71537465 0.69664729 0.67318034 0.60718054 0.60909092\n",
      " 0.65794063 0.65049243 0.62371361 0.68549144 0.69684905 0.68521661]\n",
      "[0.66609126 0.67715973 0.68796182 0.70568359 0.69415689 0.6737411\n",
      " 0.64676988 0.70293528 0.71253216 0.70875615 0.71457499 0.68976867]\n",
      "[0.69109046 0.71788013 0.71131963 0.74731022 0.73625571 0.75771695\n",
      " 0.76656425 0.79379696 0.76587385 0.7774353  0.72793412 0.75474161]\n",
      "[0.69074535 0.79232055 0.79501641 0.7162798  0.68344557 0.71258247\n",
      " 0.73945081 0.77426225 0.7413314  0.72969878 0.69321507 0.68295515]\n",
      "[0.61600256 0.65241849 0.65274608 0.68938518 0.69296104 0.66640979\n",
      " 0.65306634 0.69864231 0.73834515 0.81877297 0.7706396  0.69074821]\n",
      "[0.66375738 0.73666358 0.74967158 0.73353338 0.72986716 0.71956056\n",
      " 0.66506815 0.69732618 0.72636008 0.76777571 0.73255908 0.68433714]\n",
      "[0.62655145 0.70222861 0.75025165 0.69181329 0.65972275 0.65062189\n",
      " 0.68766981 0.75077903 0.74799001 0.71828991 0.66271955 0.61173278]\n",
      "[0.61892653 0.6746102  0.73210281 0.72165006 0.75100714 0.75455719\n",
      " 0.80488819 0.72698444 0.6981467  0.72291881 0.68649203 0.6146059 ]\n",
      "[0.64532191 0.61404222 0.63518393 0.65998536 0.69601125 0.72894824\n",
      " 0.70895022 0.74847132 0.74323219 0.72856718 0.70490372 0.68943924]\n",
      "[0.6680004  0.68941939 0.74717075 0.75051397 0.74098337 0.72282827\n",
      " 0.68456048 0.69702357 0.71528208 0.69831198 0.68425572 0.65574032]\n",
      "[0.71864486 0.7195012  0.69714606 0.65170592 0.62607062 0.64558172\n",
      " 0.66556984 0.69819188 0.70667171 0.70727968 0.72199535 0.74214792]\n",
      "[0.67825413 0.69041783 0.69415671 0.6706385  0.72305274 0.73755831\n",
      " 0.72491682 0.7792052  0.7505967  0.7949397  0.82975811 0.69839311]\n",
      "[0.68687999 0.7121141  0.72754705 0.69305867 0.689758   0.72239214\n",
      " 0.72906214 0.7585749  0.76716179 0.80664349 0.8296693  0.7526654 ]\n",
      "[0.6873287  0.72803915 0.65160984 0.62658978 0.61870581 0.63202351\n",
      " 0.64730805 0.69461536 0.70447171 0.72597837 0.71082145 0.65726882]\n",
      "[0.66804492 0.74648523 0.69024748 0.64676201 0.65928537 0.67518836\n",
      " 0.69676667 0.73992473 0.77257526 0.73564798 0.6862576  0.63570291]\n",
      "[0.70165724 0.76477581 0.77996355 0.72270083 0.68759507 0.70948124\n",
      " 0.71901131 0.75476909 0.79234946 0.81339759 0.75527912 0.73041904]\n",
      "[0.6286298  0.66653121 0.65462494 0.66826868 0.65416855 0.68040526\n",
      " 0.69372755 0.74692851 0.77440131 0.76565909 0.78204542 0.74588829]\n",
      "[0.71358299 0.68210173 0.68036985 0.68515056 0.67975694 0.67350668\n",
      " 0.72955352 0.77695847 0.78188944 0.78826404 0.77406734 0.74705857]\n",
      "[0.67906857 0.67807209 0.68699205 0.70687109 0.72077638 0.70660734\n",
      " 0.679735   0.7565887  0.7786265  0.73881149 0.67254317 0.68577814]\n",
      "[0.65919524 0.67603171 0.71272904 0.71112978 0.75843799 0.74348032\n",
      " 0.71381056 0.78751874 0.78136832 0.76072866 0.73737931 0.69423324]\n",
      "[0.6778397  0.69686007 0.72888887 0.74452782 0.75922674 0.74022698\n",
      " 0.7171365  0.75083852 0.73565823 0.76413739 0.80768889 0.74102455]\n",
      "[0.66875863 0.64455867 0.66856009 0.67932171 0.7425859  0.76552624\n",
      " 0.71973914 0.76639676 0.76128191 0.78543615 0.79030776 0.81114817]\n",
      "[0.59662324 0.64258754 0.70622331 0.64165103 0.62873161 0.63677967\n",
      " 0.64793354 0.69873029 0.70770282 0.72695547 0.73003286 0.65631151]\n",
      "[0.63214403 0.6422925  0.64713299 0.67934221 0.68525541 0.66659015\n",
      " 0.70131916 0.69364285 0.69112819 0.7185365  0.75559151 0.70241129]\n",
      "[0.62815088 0.72575974 0.70176625 0.74759072 0.76734006 0.76769686\n",
      " 0.73728597 0.76414937 0.76840621 0.77747303 0.76563787 0.67971891]\n",
      "[0.64998436 0.66644025 0.65094966 0.69805688 0.73828948 0.70855981\n",
      " 0.64081079 0.66806173 0.71650219 0.75488013 0.70207781 0.64684808]\n",
      "[0.70556092 0.71929443 0.67160833 0.68322194 0.70045149 0.69983059\n",
      " 0.75884026 0.78836733 0.78693604 0.76246983 0.75207394 0.74499446]\n",
      "[0.62264091 0.65453351 0.63017851 0.64892608 0.69745231 0.70504665\n",
      " 0.6962648  0.72109252 0.7810775  0.80326718 0.74726391 0.66881174]\n",
      "[0.62711787 0.68891072 0.78768361 0.79664975 0.74792713 0.71974581\n",
      " 0.68829131 0.69343311 0.75051206 0.82278007 0.83334351 0.74325693]\n",
      "[0.56140727 0.61976063 0.63082719 0.66189635 0.68466097 0.75662333\n",
      " 0.70254803 0.73203212 0.79972064 0.85021818 0.79863036 0.63322014]\n",
      "[0.6357134  0.70923936 0.6817739  0.64189124 0.67663479 0.78157318\n",
      " 0.77262491 0.73904026 0.76221311 0.75230527 0.74473763 0.66951251]\n",
      "[0.57217395 0.60304028 0.61721879 0.66100162 0.65430868 0.69043088\n",
      " 0.69057047 0.72417223 0.81507164 0.8193385  0.73223275 0.63923407]\n",
      "[0.6273278  0.71083152 0.68136466 0.64803696 0.65288812 0.68591797\n",
      " 0.66448218 0.73253119 0.81342614 0.79523516 0.74896652 0.63774866]\n",
      "[0.59853792 0.69022322 0.65791833 0.6469481  0.68069822 0.76292819\n",
      " 0.76887935 0.78105265 0.7297495  0.72256726 0.67467558 0.55819619]\n",
      "[0.56756586 0.61272317 0.57842052 0.60304874 0.6603781  0.63613296\n",
      " 0.62899631 0.64494437 0.68493885 0.72003525 0.7470057  0.64855069]\n",
      "[0.59513754 0.5979706  0.57881093 0.6303398  0.72317922 0.73192275\n",
      " 0.67845416 0.70081657 0.73517996 0.75128096 0.76505268 0.66879499]\n",
      "[0.51302421 0.53408891 0.55596858 0.56829929 0.61530513 0.67507339\n",
      " 0.64420515 0.70125175 0.77031851 0.75966901 0.74179137 0.6203385 ]\n",
      "[0.56452167 0.567258   0.6350733  0.62179327 0.62733305 0.61127716\n",
      " 0.61502528 0.67514133 0.72302276 0.76243651 0.76918459 0.66509199]\n",
      "[0.66361815 0.68213338 0.65763479 0.64681816 0.67578852 0.72919416\n",
      " 0.71371901 0.73073435 0.73781419 0.75086212 0.73929721 0.70313156]\n",
      "[0.64430606 0.65420163 0.67789501 0.70920116 0.68827397 0.71714377\n",
      " 0.67165941 0.70954907 0.78386557 0.71395934 0.6425631  0.60908478]\n",
      "[0.69161838 0.70419723 0.71123302 0.76000226 0.7385444  0.67767406\n",
      " 0.69423866 0.73612219 0.76177394 0.74734437 0.7070452  0.69550931]\n",
      "[0.62133789 0.61214358 0.60131931 0.5975318  0.66978872 0.66423959\n",
      " 0.61790913 0.64400959 0.7042979  0.7841773  0.80694109 0.72882849]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.63031375 0.69488859 0.67918217 0.67256397 0.73998058 0.73741859\n",
      " 0.69332147 0.69065905 0.74239439 0.77327371 0.74232531 0.6758917 ]\n",
      "[0.70404571 0.71459502 0.69939017 0.68421555 0.70051318 0.69963837\n",
      " 0.72924513 0.77186531 0.74913287 0.72756803 0.72224057 0.69748843]\n",
      "[0.61153084 0.64652485 0.66014171 0.63433272 0.66328132 0.74842399\n",
      " 0.71729535 0.74554902 0.78415662 0.8076697  0.80611211 0.70148897]\n",
      "[0.66188443 0.68556762 0.67755985 0.65083939 0.65610337 0.67637885\n",
      " 0.65813589 0.71547663 0.76576972 0.78667831 0.79301    0.73010045]\n",
      "[0.6598047  0.66011083 0.66295946 0.66425174 0.70606828 0.80788314\n",
      " 0.75481892 0.7578482  0.77127814 0.78283703 0.74287933 0.70941418]\n",
      "[0.70020902 0.72137535 0.70183939 0.72280908 0.73550195 0.74536175\n",
      " 0.77171427 0.80821395 0.80021995 0.7957204  0.7743721  0.7227754 ]\n",
      "[0.70679843 0.76314723 0.74872983 0.70046687 0.67502081 0.70327854\n",
      " 0.70170027 0.76892239 0.8076604  0.81014448 0.77480686 0.73745018]\n",
      "[0.62659997 0.65836769 0.66632223 0.66622806 0.65592265 0.69679332\n",
      " 0.6859951  0.70998645 0.70517063 0.73559278 0.75275677 0.71502054]\n",
      "[0.49505839 0.50474727 0.51578438 0.5452258  0.58358234 0.57843769\n",
      " 0.5644297  0.55977863 0.58882546 0.60371882 0.560543   0.52966338]\n",
      "[0.54326332 0.51601249 0.49463516 0.48014989 0.47921801 0.55700451\n",
      " 0.60448992 0.62955832 0.58802521 0.5981344  0.60940355 0.5930056 ]\n",
      "[0.60443246 0.54892004 0.54754359 0.5336023  0.60849273 0.62267566\n",
      " 0.63578355 0.66650766 0.66702884 0.65221173 0.64607024 0.62672591]\n",
      "[0.50188881 0.52213961 0.51980245 0.53500128 0.53265083 0.54472238\n",
      " 0.5787096  0.56922895 0.5845871  0.64087445 0.57732344 0.5184983 ]\n",
      "[0.47691217 0.43221724 0.43478003 0.48612255 0.5714131  0.5338701\n",
      " 0.48261711 0.49085718 0.54995024 0.53121227 0.48836154 0.49189141]\n",
      "[0.52492249 0.48286223 0.43402556 0.46016395 0.44892383 0.45645568\n",
      " 0.46665186 0.46501312 0.46932328 0.53557467 0.49714518 0.48421544]\n",
      "[0.46611041 0.47681481 0.56261593 0.60009748 0.5254795  0.52556258\n",
      " 0.58274454 0.58717108 0.59864789 0.59225088 0.58605009 0.5399825 ]\n",
      "[0.55880493 0.52333617 0.49180582 0.48216751 0.48855585 0.5256238\n",
      " 0.55689657 0.57565588 0.58252579 0.59601599 0.53332156 0.53140211]\n",
      "[0.56845844 0.58696729 0.61606699 0.58495903 0.61781096 0.61169636\n",
      " 0.61671996 0.62220514 0.6183117  0.6344825  0.65273327 0.61278498]\n",
      "[0.48656821 0.49405012 0.44635904 0.47809127 0.55866778 0.57140559\n",
      " 0.59122574 0.59362411 0.63164937 0.724141   0.6787852  0.55716354]\n",
      "[0.43524206 0.43877858 0.47513598 0.43153369 0.41586009 0.44571796\n",
      " 0.47531128 0.52678019 0.51917887 0.5137769  0.51522118 0.51278555]\n",
      "[0.5378229  0.51151627 0.59263259 0.61382735 0.5286172  0.5284825\n",
      " 0.53783864 0.63609338 0.59432161 0.58206969 0.57837027 0.58542323]\n",
      "[0.5201714  0.47031009 0.47647831 0.44766799 0.41058636 0.42817202\n",
      " 0.54540312 0.58826613 0.54933554 0.53641993 0.53186244 0.53458428]\n",
      "[0.69927043 0.6157006  0.60327172 0.62963974 0.64014322 0.64972496\n",
      " 0.57764041 0.58115464 0.62528992 0.68515843 0.71147019 0.75880206]\n",
      "[0.65865272 0.62700295 0.64544511 0.58729857 0.6219005  0.62968707\n",
      " 0.67716032 0.71409518 0.64090443 0.64222652 0.70266682 0.70041507]\n",
      "[0.47286949 0.44786069 0.47368488 0.4667027  0.48094913 0.52487445\n",
      " 0.57748389 0.58712119 0.55726635 0.57942396 0.62721699 0.55043757]\n",
      "[0.5573712  0.46732891 0.46928763 0.50553566 0.5344004  0.51424021\n",
      " 0.49729857 0.4886322  0.52485794 0.55341333 0.61468965 0.59488523]\n",
      "[0.72983545 0.66749316 0.61929452 0.60061973 0.64591217 0.6424883\n",
      " 0.6798166  0.68833625 0.6525529  0.65013641 0.68079585 0.72172266]\n",
      "[0.6098004  0.58392531 0.53334743 0.51808387 0.57421118 0.6292491\n",
      " 0.62946063 0.59736735 0.62189966 0.65618753 0.71990651 0.69401723]\n",
      "[0.6160928  0.55487376 0.47950587 0.43303508 0.46163952 0.54345936\n",
      " 0.57868689 0.62241381 0.61727291 0.59979773 0.68254441 0.71749049]\n",
      "[0.56308502 0.49817842 0.49097458 0.54114121 0.57448775 0.55515963\n",
      " 0.58153933 0.61335117 0.62245262 0.62377846 0.7114554  0.69657183]\n",
      "[0.54967958 0.53656286 0.52111012 0.55995977 0.56155562 0.55629897\n",
      " 0.59184301 0.60343391 0.53702092 0.53140128 0.57695115 0.60150504]\n",
      "[0.52112025 0.47473812 0.46344933 0.46861848 0.52525955 0.56640869\n",
      " 0.56242871 0.58680558 0.59520096 0.62650549 0.64223367 0.65915525]\n",
      "[0.39145187 0.35885751 0.37126774 0.34499896 0.39032823 0.48592511\n",
      " 0.44187298 0.50769031 0.50477916 0.47360468 0.50069058 0.4686085 ]\n",
      "[0.63964617 0.5306834  0.44056222 0.43383095 0.43500066 0.47227886\n",
      " 0.46982709 0.50895315 0.51315039 0.60502207 0.64442819 0.67238122]\n",
      "[0.56986994 0.4957791  0.45289648 0.40029356 0.38141662 0.40078181\n",
      " 0.42447862 0.45769009 0.4847787  0.59527242 0.63094902 0.63987923]\n",
      "[0.61886269 0.64802539 0.60232073 0.54771847 0.56169724 0.57257628\n",
      " 0.55353105 0.61040056 0.65022171 0.73825788 0.77877927 0.66988963]\n",
      "[0.79407316 0.65456438 0.58349335 0.52253199 0.46155846 0.47636521\n",
      " 0.50235128 0.59262711 0.59877306 0.64779174 0.67678845 0.78208339]\n",
      "[0.49053788 0.45973235 0.43070337 0.4393113  0.51322287 0.54878843\n",
      " 0.53914547 0.51994151 0.53534055 0.56907719 0.58046615 0.55044109]\n",
      "[0.61910456 0.56065685 0.51616657 0.51098496 0.53067267 0.60551971\n",
      " 0.63977647 0.65029198 0.70529312 0.74711269 0.72788566 0.67458981]\n",
      "[0.44188762 0.44689989 0.48058102 0.50234824 0.55600613 0.62026423\n",
      " 0.57261103 0.60374451 0.6639508  0.63986403 0.6338349  0.54288757]\n",
      "[0.56793588 0.51975483 0.50162363 0.53145981 0.56522405 0.59674257\n",
      " 0.64481431 0.64896935 0.68510282 0.69938499 0.66362238 0.60653359]\n",
      "[0.59534889 0.59571934 0.51029265 0.51179314 0.52057898 0.57959688\n",
      " 0.55385453 0.52355427 0.55787307 0.58891857 0.58026129 0.56597251]\n",
      "[0.56254238 0.54243082 0.49678317 0.51384223 0.54369187 0.60694414\n",
      " 0.61069673 0.56335187 0.59009004 0.60487753 0.58002448 0.57599902]\n",
      "[0.52071834 0.48587716 0.46546021 0.4600884  0.47664204 0.53550225\n",
      " 0.57326412 0.5645963  0.57913893 0.62929368 0.59649467 0.53797674]\n",
      "[0.53408402 0.53473407 0.50128281 0.49139819 0.45348364 0.50158316\n",
      " 0.57913363 0.56729293 0.64293993 0.67530626 0.61957341 0.57051855]\n",
      "[0.59727108 0.51359403 0.48759082 0.47086594 0.44577304 0.48789126\n",
      " 0.48440593 0.52172583 0.56157702 0.60555589 0.60750806 0.60725528]\n",
      "[0.50972718 0.45611161 0.44076121 0.36697215 0.37194508 0.41974497\n",
      " 0.44527724 0.48528051 0.52623355 0.56323159 0.53743482 0.53711462]\n",
      "[0.63264406 0.55474019 0.52868849 0.5205549  0.50329477 0.53842604\n",
      " 0.60721034 0.60964864 0.71613097 0.7514202  0.7539503  0.75643742]\n",
      "[0.48188803 0.44538811 0.42209086 0.39089236 0.38412413 0.43723336\n",
      " 0.45281625 0.48196489 0.53025413 0.58747953 0.57756215 0.53681982]\n",
      "[0.50859582 0.39570177 0.39467302 0.43971464 0.43122584 0.44014505\n",
      " 0.48118022 0.57182926 0.62681353 0.63997579 0.66951048 0.64049399]\n",
      "[0.6434989  0.56423193 0.48464382 0.39282498 0.3987942  0.49171919\n",
      " 0.56320924 0.6280781  0.68956017 0.73572671 0.72931761 0.70963359]\n",
      "[0.58083218 0.49003425 0.43928257 0.44200748 0.45160997 0.49656436\n",
      " 0.5613358  0.60338676 0.68039918 0.74620622 0.70208699 0.67816025]\n",
      "[0.53978342 0.48064122 0.45527658 0.45655483 0.48293617 0.53896677\n",
      " 0.53634495 0.58120376 0.56410199 0.52498788 0.5338068  0.56771207]\n",
      "[0.53943962 0.49307632 0.45446616 0.43658265 0.44794118 0.5244953\n",
      " 0.55810136 0.57573324 0.5778656  0.54955852 0.54745555 0.55380774]\n",
      "[0.46824077 0.47010013 0.43716484 0.44606417 0.464351   0.50753695\n",
      " 0.5379355  0.50397652 0.48271814 0.50430071 0.46671602 0.43930289]\n",
      "[0.58804661 0.55255598 0.52130473 0.49993727 0.4994846  0.52621263\n",
      " 0.55311686 0.63313621 0.65051615 0.61345667 0.58249331 0.59899813]\n",
      "[0.5693993  0.54280323 0.55789793 0.54863518 0.5672667  0.66432202\n",
      " 0.69243473 0.76489311 0.77187544 0.76635766 0.6909681  0.61851895]\n",
      "[0.49226734 0.41649356 0.43260679 0.44363946 0.45172709 0.50719696\n",
      " 0.5327509  0.56013983 0.56830084 0.58644331 0.62558794 0.60543507]\n",
      "[0.52682209 0.50793028 0.49723217 0.5205366  0.63431978 0.64760393\n",
      " 0.61229324 0.68330824 0.71670872 0.68885928 0.63151509 0.58247739]\n",
      "[0.54299945 0.49630603 0.47293699 0.44690922 0.46974921 0.54367012\n",
      " 0.55878907 0.5955019  0.62347609 0.6500327  0.66049635 0.60331035]\n",
      "[0.51363707 0.46769717 0.43405205 0.45847991 0.52537674 0.5190962\n",
      " 0.468016   0.45841298 0.5397408  0.54008961 0.49834111 0.52735126]\n",
      "[0.590406   0.57913011 0.58801287 0.61422533 0.64363474 0.71337485\n",
      " 0.71983367 0.71649784 0.70248526 0.66361654 0.65072316 0.65848613]\n",
      "[0.55670011 0.53834355 0.54353398 0.54694188 0.52920777 0.54989791\n",
      " 0.5470742  0.5593316  0.55560362 0.56227869 0.61254573 0.64440012]\n",
      "[0.50944179 0.45044395 0.48144341 0.51279092 0.55560833 0.60449457\n",
      " 0.55633944 0.53943849 0.55195391 0.59127438 0.64146769 0.57770681]\n",
      "[0.58563888 0.50910056 0.4626452  0.46767476 0.51247078 0.57731986\n",
      " 0.60806829 0.60250729 0.5483942  0.55420715 0.56987351 0.61864215]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.54842043 0.49251992 0.48845106 0.52921176 0.50338113 0.52701151\n",
      " 0.54720825 0.54662877 0.50245327 0.52512437 0.55541074 0.60687655]\n",
      "[0.68739891 0.70730025 0.71556276 0.66850609 0.64843398 0.69805449\n",
      " 0.67206705 0.68942326 0.6952256  0.72653133 0.7177195  0.71995747]\n",
      "[0.59067291 0.5129388  0.47082609 0.48381606 0.53367174 0.56007236\n",
      " 0.54372889 0.55560273 0.51300663 0.50747865 0.54899609 0.61235863]\n",
      "[0.63723695 0.66676658 0.66721982 0.67634302 0.65249151 0.69315982\n",
      " 0.7157234  0.78284127 0.83667713 0.76904035 0.73767877 0.69209504]\n",
      "[0.69483113 0.71142817 0.69785482 0.70320559 0.70693964 0.73011047\n",
      " 0.77127969 0.80438644 0.81807536 0.80958676 0.77209896 0.74514949]\n",
      "[0.69678551 0.67082959 0.62637192 0.64534473 0.70898044 0.7118361\n",
      " 0.74093527 0.77417982 0.79569852 0.77262253 0.74419814 0.72315586]\n",
      "[0.57877153 0.61927271 0.60187399 0.66922283 0.71960074 0.71060437\n",
      " 0.69206423 0.75593215 0.81357121 0.76406777 0.70594907 0.61670727]\n",
      "[0.64534336 0.68678963 0.7318179  0.75062448 0.69907898 0.72441649\n",
      " 0.72917217 0.803469   0.83914441 0.75132084 0.70752394 0.69680923]\n",
      "[0.64950544 0.70312738 0.70027059 0.64181846 0.65851092 0.73067063\n",
      " 0.70505983 0.69577307 0.73446417 0.78217423 0.79620576 0.70321983]\n",
      "[0.68966031 0.69982851 0.66561621 0.62953138 0.6623736  0.71485853\n",
      " 0.70338488 0.7578876  0.75819278 0.7444557  0.76039392 0.73099595]\n",
      "[0.65327954 0.6698311  0.64674354 0.60866868 0.66361082 0.67367256\n",
      " 0.66822642 0.74753809 0.76825893 0.74183023 0.71805888 0.68299484]\n",
      "[0.68167436 0.59022975 0.58298737 0.61706299 0.64086282 0.69581628\n",
      " 0.66639882 0.68908042 0.7430082  0.76096523 0.75177598 0.7736792 ]\n",
      "[0.64715606 0.66329992 0.67842728 0.67423558 0.71116334 0.70321053\n",
      " 0.72763765 0.79910988 0.78367025 0.78694576 0.77610034 0.72025311]\n",
      "[0.69859803 0.66590172 0.61807632 0.60822606 0.62867165 0.66916442\n",
      " 0.6807037  0.75311184 0.77185416 0.76143044 0.75419444 0.74702573]\n",
      "[0.61377823 0.60969943 0.62651056 0.64169538 0.6725949  0.70427787\n",
      " 0.66298938 0.70725131 0.83335423 0.82380402 0.74615705 0.62718374]\n",
      "[0.59218246 0.60839552 0.65330642 0.68323487 0.736431   0.75330687\n",
      " 0.74679178 0.76301652 0.7752896  0.68869859 0.66386706 0.61663014]\n",
      "[0.55201119 0.55941153 0.53738254 0.55962574 0.60822809 0.69473261\n",
      " 0.66561782 0.67608297 0.76060981 0.74511397 0.66620374 0.63193542]\n",
      "[0.66416329 0.67667067 0.66855383 0.70094788 0.77924681 0.78119421\n",
      " 0.7318396  0.75501502 0.7884627  0.79165435 0.77919233 0.70844001]\n",
      "[0.65466309 0.68107653 0.64989644 0.6635437  0.7201184  0.71766591\n",
      " 0.72470516 0.76220435 0.74487108 0.68146932 0.6740061  0.68047458]\n",
      "[0.56181324 0.60552108 0.64356536 0.68413281 0.73163724 0.68106955\n",
      " 0.64356261 0.6835584  0.69348603 0.6988337  0.653045   0.58160752]\n",
      "[0.60771483 0.63745522 0.63916463 0.71040601 0.73838639 0.69825912\n",
      " 0.68074483 0.75339961 0.73642558 0.70971847 0.68863618 0.62210763]\n",
      "[0.62097532 0.67872387 0.63842028 0.69977713 0.74646372 0.72395891\n",
      " 0.67791629 0.6955629  0.73857993 0.73294514 0.72300613 0.6425429 ]\n",
      "[0.62833345 0.64405221 0.61563289 0.66171616 0.71378583 0.69517726\n",
      " 0.67057973 0.69406211 0.69091284 0.67805248 0.67946362 0.62337691]\n",
      "[0.57504857 0.6476512  0.62422293 0.69490987 0.80180579 0.74033958\n",
      " 0.69074261 0.71325117 0.73536855 0.71753526 0.66545331 0.62168258]\n",
      "[0.66130507 0.69014949 0.7041316  0.69520628 0.74802202 0.74905384\n",
      " 0.70579708 0.7427693  0.73976719 0.74357784 0.73745728 0.71228087]\n",
      "[0.67551631 0.63359678 0.61397898 0.61948603 0.66919363 0.65628248\n",
      " 0.63547081 0.71046436 0.77735364 0.73367292 0.75216287 0.73796481]\n",
      "[0.59516507 0.636621   0.6243422  0.68092638 0.6992743  0.65459096\n",
      " 0.6491338  0.71879536 0.81125528 0.73849428 0.66396314 0.59442693]\n",
      "[0.6219306  0.62929451 0.61959904 0.64667201 0.70395172 0.70751595\n",
      " 0.75215143 0.81045777 0.8209517  0.73890609 0.66371059 0.61861628]\n",
      "[0.67430371 0.66753882 0.65584946 0.66234392 0.72365254 0.74331564\n",
      " 0.74670011 0.78619891 0.80242455 0.77776045 0.74942738 0.71392679]\n",
      "[0.59284443 0.644593   0.63191384 0.64316595 0.70864856 0.71440548\n",
      " 0.70057219 0.7914058  0.79897785 0.77886647 0.7034322  0.6389333 ]\n",
      "[0.59589273 0.64039224 0.62425196 0.63259208 0.6844703  0.68064165\n",
      " 0.71774101 0.7902211  0.80375308 0.79893464 0.72176689 0.6249364 ]\n",
      "[0.7252546  0.70770782 0.69100708 0.71204078 0.72832257 0.74694115\n",
      " 0.77660805 0.80665427 0.77101171 0.78290814 0.81000555 0.76204413]\n",
      "[0.64221764 0.61866766 0.61645913 0.62098587 0.64411777 0.69658625\n",
      " 0.75828946 0.78279865 0.8074578  0.82253706 0.7758339  0.70092499]\n",
      "[0.69501513 0.68233961 0.71723008 0.7216047  0.71050018 0.68033987\n",
      " 0.68811959 0.76241446 0.79880673 0.80402231 0.79888576 0.75745475]\n",
      "[0.70599866 0.66024137 0.62158376 0.61236268 0.64278334 0.65476096\n",
      " 0.64521754 0.73228198 0.79830319 0.8023622  0.80773395 0.80154705]\n",
      "[0.67820644 0.6498217  0.63390887 0.69127226 0.70793778 0.68859994\n",
      " 0.64910877 0.67607993 0.75385869 0.7411496  0.74252361 0.76176977]\n",
      "[0.6935457  0.69422776 0.68782812 0.67618918 0.63990158 0.66522104\n",
      " 0.70266312 0.81168687 0.82208568 0.77216089 0.76805866 0.71791476]\n",
      "[0.67237276 0.64030725 0.70296991 0.72026759 0.74114013 0.73389554\n",
      " 0.69509339 0.77705121 0.8022005  0.76170558 0.7185294  0.72483969]\n",
      "[0.67717707 0.6333878  0.60463917 0.65365404 0.68749499 0.71753192\n",
      " 0.70489645 0.7360732  0.74460739 0.75225317 0.75929081 0.76887846]\n",
      "[0.61706066 0.58465481 0.52235359 0.60314536 0.714948   0.75768405\n",
      " 0.73363763 0.7819283  0.75399411 0.69701141 0.65108258 0.66125566]\n",
      "[0.64389932 0.6520009  0.65399259 0.68889964 0.73812187 0.75672621\n",
      " 0.80254537 0.7983644  0.75333071 0.72291148 0.67021883 0.62798911]\n",
      "[0.64547223 0.60129225 0.60607088 0.66532153 0.72070622 0.68241483\n",
      " 0.72426802 0.81729519 0.78589523 0.75650173 0.70610946 0.66914922]\n",
      "[0.66422236 0.65492845 0.74136537 0.77038282 0.75030279 0.71606839\n",
      " 0.72931272 0.80556548 0.79702002 0.79753274 0.78307241 0.70743811]\n",
      "[0.60266113 0.63952696 0.67535669 0.73161429 0.71151084 0.67538619\n",
      " 0.69903678 0.7777254  0.76771486 0.78535426 0.7568748  0.66565382]\n",
      "[0.68629205 0.71859163 0.66329163 0.67562264 0.72379988 0.75018245\n",
      " 0.75620574 0.79417056 0.76772565 0.73531705 0.77420044 0.72208136]\n",
      "[0.66774631 0.6476661  0.65645987 0.69033414 0.79049045 0.76906168\n",
      " 0.73593664 0.79828876 0.77685142 0.75944465 0.77248919 0.70685655]\n",
      "[0.61215377 0.57409465 0.61986524 0.71469706 0.82535166 0.74920452\n",
      " 0.69573194 0.78091407 0.79379427 0.75475556 0.73193717 0.6640746 ]\n",
      "[0.68454224 0.72773075 0.6730783  0.67689985 0.69280761 0.72083962\n",
      " 0.73361993 0.78909403 0.80672103 0.76078218 0.75449765 0.72006607]\n",
      "[0.69564563 0.71837223 0.72110391 0.71854651 0.72097844 0.7355485\n",
      " 0.7527917  0.80649924 0.81799245 0.79756689 0.80564761 0.72435272]\n",
      "[0.64712852 0.64085519 0.61439514 0.619712   0.64846033 0.67294562\n",
      " 0.71931905 0.7994985  0.79687661 0.7922163  0.81995469 0.76198679]\n",
      "[0.65943199 0.66366053 0.66151422 0.68357652 0.70020759 0.70885098\n",
      " 0.70303571 0.77508801 0.84552324 0.83119923 0.83847147 0.73741138]\n",
      "[0.69657463 0.6783675  0.66347641 0.68290108 0.68533361 0.68621784\n",
      " 0.67990315 0.7230854  0.78150231 0.80212152 0.82000726 0.77270174]\n",
      "[0.64729226 0.64517188 0.60253131 0.60131413 0.6278888  0.6447295\n",
      " 0.61947036 0.67723131 0.78365082 0.78034282 0.73695087 0.6964308 ]\n",
      "[0.57136595 0.60024548 0.5917061  0.62121552 0.63984156 0.63677585\n",
      " 0.68080592 0.77424026 0.83778286 0.81382751 0.77466583 0.6587894 ]\n",
      "[0.58597207 0.59700984 0.56239569 0.56282353 0.59672481 0.6327529\n",
      " 0.67172813 0.73347741 0.77212805 0.77314878 0.77909374 0.68112648]\n",
      "[0.69625992 0.72584838 0.71065116 0.69623941 0.68889302 0.6902011\n",
      " 0.71238756 0.77983189 0.78489369 0.7503047  0.75810009 0.73928303]\n",
      "[0.66569006 0.67705631 0.69471914 0.71763426 0.69239485 0.73662823\n",
      " 0.74804664 0.73552847 0.74213403 0.72124678 0.72728372 0.69394976]\n",
      "[0.63620615 0.66259283 0.71275908 0.76376128 0.77266824 0.70783573\n",
      " 0.69827753 0.74327201 0.75515109 0.75039917 0.7506339  0.69419402]\n",
      "[0.64402348 0.62757063 0.60405892 0.63520116 0.69267702 0.72756231\n",
      " 0.67731863 0.70614195 0.77562153 0.78667724 0.73725218 0.7194317 ]\n",
      "[0.60619015 0.62576663 0.69610232 0.74367565 0.76151818 0.72050488\n",
      " 0.73780251 0.78207546 0.78253406 0.78768873 0.78460383 0.71774036]\n",
      "[0.65119356 0.65561163 0.69298124 0.74763161 0.74646461 0.71750927\n",
      " 0.71835101 0.73285651 0.73097485 0.72405815 0.71520472 0.67271793]\n",
      "[0.63287348 0.69162714 0.7127049  0.69829291 0.70794207 0.74939722\n",
      " 0.74074513 0.7536059  0.76307863 0.76723915 0.74438524 0.66253436]\n",
      "[0.6277048  0.67514378 0.71229851 0.73343354 0.76543546 0.6837669\n",
      " 0.67306066 0.70945954 0.72614217 0.72326869 0.66390169 0.62492198]\n",
      "[0.46625194 0.41032586 0.36319205 0.39669463 0.49083778 0.73007488\n",
      " 0.65051496 0.52251089 0.48236045 0.4889752  0.45791271 0.44527656]\n",
      "[0.44278723 0.41827556 0.42683792 0.44886529 0.58467728 0.68466014\n",
      " 0.61194491 0.54693139 0.51580125 0.47219104 0.46532375 0.49910766]\n",
      "[0.36174339 0.35854539 0.38874096 0.45471066 0.41301259 0.41341594\n",
      " 0.46877727 0.52520835 0.54489487 0.43574181 0.41573152 0.42309898]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.48895109 0.48598468 0.50394064 0.54717129 0.59266454 0.67660409\n",
      " 0.63665855 0.58691114 0.55544829 0.53635001 0.54270345 0.53391379]\n",
      "[0.46291476 0.46720961 0.46121764 0.50358826 0.52960932 0.53698492\n",
      " 0.56203324 0.58844554 0.61970335 0.67116177 0.63988686 0.57610357]\n",
      "[0.46973088 0.45267951 0.5005917  0.56432778 0.63964778 0.6334846\n",
      " 0.60392004 0.61985368 0.64207488 0.62152922 0.59796691 0.56119549]\n",
      "[0.36951065 0.38774031 0.40873832 0.42916444 0.45447421 0.53005159\n",
      " 0.51983851 0.47421467 0.48903319 0.54312426 0.52458221 0.41881526]\n",
      "[0.47718078 0.41267732 0.42891183 0.4724952  0.51808697 0.54736996\n",
      " 0.50202364 0.5070408  0.5387792  0.59575707 0.54848617 0.54776829]\n",
      "[0.38600072 0.39979461 0.41284877 0.47677159 0.4741593  0.49076492\n",
      " 0.53704995 0.55637991 0.61404687 0.523193   0.46892551 0.42904299]\n",
      "[0.41313037 0.42010778 0.47698331 0.52348018 0.54375458 0.53762239\n",
      " 0.51036221 0.55058628 0.59110498 0.5269224  0.50762874 0.48751017]\n",
      "[0.34606627 0.38501704 0.44567329 0.45191279 0.48163173 0.48879108\n",
      " 0.49672291 0.54937893 0.53661919 0.48040038 0.45705444 0.40289348]\n",
      "[0.41539207 0.38849616 0.44762209 0.47304124 0.48470023 0.50159621\n",
      " 0.50089991 0.53120404 0.54685587 0.51819342 0.4765875  0.44236729]\n",
      "[0.57402426 0.55202526 0.54421997 0.62166381 0.64321095 0.59776109\n",
      " 0.60498148 0.63448912 0.66122919 0.67764252 0.67047596 0.66623187]\n",
      "[0.5624246  0.57843888 0.55865657 0.49488261 0.47203934 0.50047654\n",
      " 0.57035679 0.64110291 0.64678919 0.57339966 0.54575026 0.56928021]\n",
      "[0.53199464 0.51679641 0.56307322 0.55556393 0.5726518  0.60533482\n",
      " 0.61747056 0.63640428 0.62991035 0.63479775 0.64901745 0.61791694]\n",
      "[0.55281913 0.57438004 0.57016873 0.56582654 0.59676367 0.60758913\n",
      " 0.57534426 0.59635729 0.61805087 0.67146879 0.73805624 0.63961661]\n",
      "[0.55806124 0.47700706 0.39705151 0.41647995 0.46542895 0.51966912\n",
      " 0.60313839 0.64504808 0.61111826 0.63177961 0.66689372 0.64635009]\n",
      "[0.52657312 0.43433946 0.42138979 0.45763817 0.52911085 0.60445356\n",
      " 0.61356884 0.66450703 0.68284124 0.70734137 0.70024109 0.60224015]\n",
      "[0.50029606 0.4235113  0.39448047 0.37621087 0.39480513 0.49438834\n",
      " 0.52688324 0.5404309  0.58777952 0.6236555  0.65861255 0.62060487]\n",
      "[0.53413016 0.42270833 0.36390236 0.37202457 0.43866265 0.4756408\n",
      " 0.51293695 0.57350177 0.61601651 0.61395627 0.60411495 0.56046277]\n",
      "[0.49478605 0.52680308 0.44052956 0.41338283 0.44827765 0.45479009\n",
      " 0.42353961 0.4498933  0.46716788 0.51257628 0.52348763 0.47330186]\n",
      "[0.47005543 0.47466201 0.48106298 0.46848062 0.50938517 0.55252743\n",
      " 0.52740335 0.54449594 0.54549778 0.56656873 0.58826041 0.51904422]\n",
      "[0.48610151 0.51569295 0.51912296 0.52413225 0.57248569 0.59498203\n",
      " 0.62374681 0.64538753 0.69986314 0.71842915 0.65517306 0.58827865]\n",
      "[0.38550565 0.38850588 0.42949975 0.42797244 0.49220321 0.5179196\n",
      " 0.50628376 0.57012296 0.59984285 0.5935322  0.58450896 0.50138319]\n",
      "[0.5142383  0.45945579 0.47330868 0.4789263  0.5141142  0.57158673\n",
      " 0.67121404 0.70654583 0.62412268 0.56380129 0.62572449 0.62696761]\n",
      "[0.48291528 0.4568947  0.47003347 0.50988775 0.59830129 0.63292688\n",
      " 0.69530332 0.65931588 0.61216336 0.64440215 0.65043271 0.58945614]\n",
      "[0.41709432 0.39986712 0.45797998 0.60198706 0.63845378 0.59319252\n",
      " 0.65800238 0.68442613 0.623402   0.58918601 0.60362941 0.48284158]\n",
      "[0.42284909 0.40617311 0.4630743  0.55725825 0.62387872 0.6489805\n",
      " 0.63668156 0.62983453 0.59511274 0.5937767  0.6354472  0.58955413]\n",
      "[0.44464374 0.45967916 0.5725165  0.6052869  0.59759539 0.59960794\n",
      " 0.59872103 0.65292186 0.64867115 0.60652792 0.59664005 0.51917797]\n",
      "[0.55916935 0.57104826 0.5718863  0.62867069 0.65513766 0.67108256\n",
      " 0.70940715 0.74157709 0.75098497 0.70000231 0.6295687  0.61290079]\n",
      "[0.59986502 0.60251373 0.58045202 0.5837245  0.64979488 0.69200838\n",
      " 0.71327299 0.70896822 0.67447907 0.66152573 0.69133818 0.64369327]\n",
      "[0.51814061 0.50682348 0.51855236 0.54046172 0.57919192 0.65179926\n",
      " 0.61082566 0.54348218 0.56703842 0.61028212 0.6071167  0.61798322]\n",
      "[0.59603077 0.54471523 0.56844163 0.64763212 0.68175441 0.66384053\n",
      " 0.65713131 0.63886774 0.63813907 0.71266705 0.83995616 0.75326914]\n",
      "[0.66749966 0.61816049 0.58300966 0.50191104 0.52357799 0.50734997\n",
      " 0.55735976 0.59355664 0.57359755 0.57484132 0.61159509 0.67646581]\n",
      "[0.71056086 0.57685363 0.50776201 0.51843989 0.53427148 0.50282258\n",
      " 0.56878126 0.6763435  0.62355357 0.6260792  0.63376504 0.69046962]\n",
      "[0.59006006 0.51996499 0.48600206 0.48751837 0.46016705 0.45567304\n",
      " 0.54584301 0.60257435 0.58712494 0.56766331 0.57371873 0.67603755]\n",
      "[0.56246752 0.61568922 0.62591988 0.59792131 0.62293667 0.59990668\n",
      " 0.62725335 0.62407428 0.60185504 0.61644208 0.63867903 0.56016004]\n",
      "[0.53726757 0.57819045 0.56994432 0.57631147 0.55189091 0.53401798\n",
      " 0.536524   0.59127438 0.53508043 0.48411852 0.52619797 0.56364191]\n",
      "[0.52445328 0.50487643 0.44823173 0.40853477 0.42438972 0.46382329\n",
      " 0.49748647 0.50610679 0.52311081 0.48731312 0.54523033 0.61317044]\n",
      "[0.60293382 0.53398138 0.5218659  0.55337059 0.59229004 0.63197833\n",
      " 0.69406474 0.65167427 0.61998385 0.64194036 0.70378047 0.70586509]\n",
      "[0.46774089 0.48671058 0.48348618 0.54783499 0.69373578 0.77439326\n",
      " 0.80780447 0.77355528 0.73056585 0.70638472 0.68168229 0.54115832]\n",
      "[0.49443042 0.50227517 0.47771016 0.49893853 0.59532416 0.70645523\n",
      " 0.7881428  0.7524069  0.65121049 0.58177543 0.60002673 0.53585571]\n",
      "[0.422993   0.44216758 0.47268632 0.49219567 0.61383492 0.71016562\n",
      " 0.70136118 0.66428614 0.63440514 0.59496933 0.5558297  0.4580645 ]\n",
      "[0.44640142 0.48553684 0.48243654 0.54049987 0.67857087 0.73027283\n",
      " 0.76665378 0.72224736 0.65080792 0.59498143 0.59708685 0.49232551]\n",
      "[0.5830794  0.61748731 0.58226556 0.58089268 0.60572767 0.66169226\n",
      " 0.69541985 0.71216005 0.70922071 0.66037279 0.64854854 0.63213998]\n",
      "[0.4935424  0.51783133 0.51329505 0.53879499 0.53949213 0.5659703\n",
      " 0.58887833 0.60133266 0.63548326 0.60910368 0.56762475 0.54956365]\n",
      "[0.51673681 0.52743977 0.51340884 0.52154267 0.5085218  0.52280211\n",
      " 0.57177365 0.57833862 0.62203783 0.57663679 0.54257572 0.56956005]\n",
      "[0.49666044 0.45595247 0.45433277 0.46350193 0.47724593 0.52838111\n",
      " 0.54966253 0.56677324 0.54945976 0.54632419 0.56473255 0.5670383 ]\n",
      "[0.51831907 0.51775557 0.4825246  0.5253551  0.6274516  0.68763357\n",
      " 0.76663941 0.75931388 0.67837751 0.57786918 0.55960643 0.53218478]\n",
      "[0.52621335 0.51961243 0.51568127 0.50677162 0.55804813 0.64162475\n",
      " 0.60859144 0.55153435 0.5366255  0.538369   0.52137852 0.48983994]\n",
      "[0.57002187 0.56148493 0.5569427  0.58097655 0.63247639 0.676027\n",
      " 0.66705173 0.62580597 0.62156904 0.61730891 0.625615   0.60794461]\n",
      "[0.58205467 0.56885493 0.60009688 0.57743496 0.61050326 0.64725149\n",
      " 0.7159847  0.75689262 0.68268043 0.65915108 0.63164526 0.57513154]\n",
      "[0.55634755 0.53778309 0.48682365 0.47647262 0.48143703 0.52267623\n",
      " 0.57876611 0.65567011 0.63372278 0.61149561 0.65860587 0.64935327]\n",
      "[0.5596475  0.52536845 0.53382868 0.54262006 0.59106714 0.60599428\n",
      " 0.63656503 0.63116217 0.63371688 0.65039754 0.69361842 0.64693689]\n",
      "[0.59051847 0.54009372 0.51810992 0.57042223 0.5863623  0.63122195\n",
      " 0.64140362 0.63457996 0.66031069 0.63731301 0.66031367 0.68527842]\n",
      "[0.54536742 0.51334226 0.50286323 0.50728661 0.54927224 0.59570992\n",
      " 0.60704744 0.60609913 0.623227   0.63877958 0.68620032 0.61132425]\n",
      "[0.52610093 0.52684098 0.53677732 0.58020669 0.59831035 0.57983524\n",
      " 0.58238006 0.6007365  0.576011   0.57602918 0.63799167 0.55620533]\n",
      "[0.53158683 0.55042791 0.55707908 0.59815347 0.56727606 0.55577338\n",
      " 0.56754565 0.58305925 0.57729954 0.57804292 0.62745661 0.57260597]\n",
      "[0.54100412 0.49670115 0.4642714  0.48709357 0.50163752 0.55303621\n",
      " 0.56360012 0.54256153 0.52806753 0.53541321 0.58522218 0.5783878 ]\n",
      "[0.59553504 0.63621855 0.60101926 0.576244   0.60211867 0.62704581\n",
      " 0.61730182 0.58784753 0.60608619 0.63400072 0.66436267 0.66150117]\n",
      "[0.62976515 0.63910317 0.68083549 0.64785236 0.61460555 0.67664587\n",
      " 0.69181192 0.66873723 0.74401784 0.75046098 0.78001291 0.77065104]\n",
      "[0.61931121 0.59328604 0.62116629 0.62639952 0.62647152 0.61568362\n",
      " 0.67503774 0.68869132 0.70572644 0.78070074 0.69722348 0.65491921]\n",
      "[0.55664635 0.56545919 0.64705664 0.63331437 0.60699201 0.67553675\n",
      " 0.71692461 0.72585362 0.81551337 0.77936655 0.73781896 0.64268726]\n",
      "[0.59139621 0.57833761 0.64385533 0.64217383 0.6730454  0.6692394\n",
      " 0.68226993 0.74576002 0.83743447 0.79622567 0.75176167 0.7169981 ]\n",
      "[0.58091795 0.63224006 0.62263036 0.61713457 0.67490923 0.7141183\n",
      " 0.75754482 0.71538901 0.68712604 0.71366531 0.678563   0.58358157]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.54737598 0.58757973 0.62937516 0.61164469 0.67949134 0.72043651\n",
      " 0.7125966  0.64843279 0.68728429 0.81137419 0.79889816 0.66294175]\n",
      "[0.57123351 0.55367666 0.57310605 0.61037815 0.66098267 0.68749082\n",
      " 0.6623317  0.67271912 0.77455008 0.83615208 0.78677726 0.67309576]\n",
      "[0.60138148 0.60372227 0.63210392 0.67128587 0.70217901 0.74190456\n",
      " 0.76586449 0.8063876  0.81659883 0.72389269 0.65705293 0.64934695]\n",
      "[0.50342214 0.43464702 0.46073335 0.57182842 0.58947039 0.59515917\n",
      " 0.61016876 0.56449777 0.59808701 0.62621009 0.67457277 0.7222361 ]\n",
      "[0.65313232 0.6551671  0.610636   0.56325942 0.58816409 0.62046599\n",
      " 0.61779326 0.64550656 0.73982286 0.77652782 0.75345659 0.70977968]\n",
      "[0.53206372 0.55275506 0.5673359  0.57451159 0.61666608 0.70655054\n",
      " 0.77527755 0.7602036  0.74371946 0.7060622  0.62088895 0.57684308]\n",
      "[0.54361421 0.54452926 0.54728073 0.57258606 0.59630799 0.55698287\n",
      " 0.58112901 0.64487159 0.75773162 0.78026325 0.65415168 0.59062135]\n",
      "[0.55684429 0.58367819 0.67038655 0.68960208 0.7008546  0.70122367\n",
      " 0.66279161 0.68071657 0.68661231 0.65906483 0.66495669 0.60837126]\n",
      "[0.55850071 0.54918706 0.62378901 0.66097367 0.68907857 0.74014086\n",
      " 0.72693455 0.74968803 0.75531787 0.72977245 0.68452942 0.64020264]\n",
      "[0.64499474 0.66473114 0.69481891 0.69144309 0.70082867 0.75869578\n",
      " 0.75742769 0.683792   0.71757197 0.73683935 0.75868094 0.73462343]\n",
      "[0.6094147  0.55154961 0.68796611 0.76622361 0.7807008  0.79771459\n",
      " 0.72034484 0.68475926 0.6814881  0.7210201  0.72733152 0.72844493]\n",
      "[0.56926489 0.6305486  0.64170885 0.618994   0.64795703 0.69324058\n",
      " 0.68470669 0.71397096 0.69538164 0.64934629 0.62409514 0.56123751]\n",
      "[0.54691619 0.60490441 0.58109307 0.50866413 0.55886012 0.59076262\n",
      " 0.58872837 0.65810156 0.7374422  0.67533517 0.61985576 0.58248389]\n",
      "[0.53279608 0.53782076 0.583076   0.60603434 0.58249414 0.64616823\n",
      " 0.68632799 0.66082394 0.69994193 0.73816478 0.65177649 0.59840798]\n",
      "[0.53364158 0.57279593 0.62147343 0.64507169 0.66918725 0.69477224\n",
      " 0.68482327 0.69849437 0.71555954 0.72568893 0.72489715 0.61385924]\n",
      "[0.63686019 0.68549854 0.70008123 0.67663419 0.67520589 0.69839936\n",
      " 0.74145436 0.72250164 0.73138928 0.78232044 0.76474088 0.68007481]\n",
      "[0.62861705 0.66210115 0.69212472 0.67824256 0.6851601  0.71856922\n",
      " 0.73988098 0.80676913 0.83828896 0.837093   0.80421412 0.69334376]\n",
      "[0.57606298 0.56811887 0.56526697 0.57214075 0.60603732 0.66289949\n",
      " 0.71916854 0.78993911 0.81379336 0.75497305 0.69157732 0.66211164]\n",
      "[0.5983094  0.61508596 0.6279915  0.62026256 0.63622886 0.70116484\n",
      " 0.76909935 0.76269513 0.68729216 0.65818572 0.70698142 0.69739568]\n",
      "[0.60440946 0.5885722  0.57794231 0.60162657 0.70372659 0.76687527\n",
      " 0.74657875 0.74994183 0.76681507 0.75263464 0.73137742 0.67862183]\n",
      "[0.66315222 0.67746139 0.68470675 0.70205557 0.74565965 0.77696651\n",
      " 0.77186716 0.75434953 0.77305341 0.74230593 0.7275269  0.74433672]\n",
      "[0.68726087 0.69884413 0.64383203 0.61226481 0.66086709 0.70894563\n",
      " 0.684618   0.64914763 0.6658929  0.65888292 0.71668899 0.75056088]\n",
      "[0.65864307 0.65734982 0.59766936 0.56411755 0.60060251 0.72210068\n",
      " 0.76036185 0.71416026 0.66891742 0.66226059 0.65019637 0.65278345]\n",
      "[0.54414451 0.61095536 0.65288883 0.64332283 0.69311857 0.70391953\n",
      " 0.67498291 0.70556349 0.6863336  0.62964141 0.62143624 0.57215625]\n",
      "[0.55305737 0.54246569 0.56748831 0.59200788 0.64084852 0.71611762\n",
      " 0.76321244 0.78355575 0.77684414 0.69561005 0.61206168 0.59229338]\n",
      "[0.57717472 0.5480209  0.58289754 0.59392506 0.60148168 0.68595338\n",
      " 0.81435388 0.83778542 0.80668986 0.7376616  0.6826604  0.64424318]\n",
      "[0.59639394 0.60855091 0.68378299 0.70021909 0.71469784 0.71659774\n",
      " 0.73517925 0.79127085 0.78235596 0.73523378 0.67310983 0.65323335]\n",
      "[0.56442922 0.59108263 0.6453827  0.66731936 0.70007259 0.6901812\n",
      " 0.74337745 0.75462693 0.73682791 0.71749777 0.69075495 0.67887396]\n",
      "[0.46314502 0.48078924 0.56060773 0.60406011 0.70587671 0.74340039\n",
      " 0.70418912 0.70969164 0.69380486 0.66118604 0.5718646  0.50177824]\n",
      "[0.51942968 0.55647826 0.61911756 0.64995581 0.70569664 0.69967639\n",
      " 0.72152662 0.6966607  0.70900983 0.65777349 0.56735063 0.54930514]\n",
      "[0.59105301 0.56938666 0.62215292 0.66074473 0.69655681 0.71477103\n",
      " 0.76066631 0.76835322 0.73134798 0.70404589 0.63423342 0.60993725]\n",
      "[0.53947812 0.57620609 0.59653801 0.61121815 0.6023829  0.66018021\n",
      " 0.67033201 0.68921322 0.74861026 0.7360785  0.68452847 0.60876065]\n",
      "[0.63583177 0.59370524 0.60332996 0.61097419 0.63061422 0.654127\n",
      " 0.70299333 0.68345714 0.77636385 0.73743957 0.69692427 0.7140066 ]\n",
      "[0.57578021 0.57558781 0.57828701 0.59088445 0.58650815 0.65981448\n",
      " 0.68135238 0.68123627 0.75684541 0.74441808 0.65896934 0.61360645]\n",
      "[0.7000441  0.61277258 0.58554155 0.65878749 0.68674195 0.73764229\n",
      " 0.71617359 0.68163657 0.69033688 0.72498107 0.69301063 0.72627079]\n",
      "[0.46136078 0.53403693 0.58983928 0.58067024 0.51288784 0.52328402\n",
      " 0.5946961  0.60848814 0.68139517 0.69346309 0.5469116  0.45501372]\n",
      "[0.58381075 0.59581065 0.54166216 0.55714619 0.63468915 0.66013688\n",
      " 0.5760783  0.62315178 0.6236313  0.56111825 0.56414521 0.6056962 ]\n",
      "[0.5867216  0.60799879 0.65851033 0.58564329 0.56668746 0.65553033\n",
      " 0.65449458 0.64914805 0.70300305 0.68895608 0.62121493 0.6468938 ]\n",
      "[0.51447439 0.52781516 0.49436331 0.45351452 0.4841018  0.63936979\n",
      " 0.73720884 0.71974087 0.72073609 0.65159619 0.57717979 0.56277883]\n",
      "[0.64648694 0.64942187 0.69595587 0.70018715 0.66510266 0.69901878\n",
      " 0.73100156 0.75082487 0.75785226 0.74799782 0.74315012 0.71434295]\n",
      "[0.63452196 0.62750643 0.64258081 0.66346514 0.68166429 0.73319346\n",
      " 0.75812227 0.76143789 0.73959625 0.71372688 0.72145844 0.68588263]\n",
      "[0.61991227 0.63975108 0.63972431 0.64978886 0.66048795 0.67817885\n",
      " 0.69521588 0.74774987 0.8169136  0.82079387 0.72917366 0.65196633]\n",
      "[0.55425537 0.56887162 0.61858791 0.64751732 0.6913057  0.77156246\n",
      " 0.76132965 0.69499969 0.6717419  0.65602863 0.65560931 0.62565279]\n",
      "[0.55360919 0.57911438 0.61603916 0.60028237 0.64193898 0.75161844\n",
      " 0.76547039 0.75981086 0.71621853 0.64571476 0.69202006 0.6722371 ]\n",
      "[0.60786074 0.61536026 0.64504081 0.62046945 0.66838861 0.70440179\n",
      " 0.6843999  0.70751369 0.69538319 0.65989506 0.65794545 0.62421358]\n",
      "[0.59853446 0.64269412 0.66715688 0.67431653 0.7058841  0.75189298\n",
      " 0.7165817  0.68409675 0.64261657 0.63185197 0.67774647 0.65057349]\n",
      "[0.54561323 0.57071686 0.62176317 0.6382224  0.73859274 0.77826077\n",
      " 0.74816114 0.73487455 0.74060136 0.687527   0.62809199 0.5644328 ]\n",
      "[0.59824008 0.61505616 0.66762978 0.67981327 0.71553457 0.73004228\n",
      " 0.71666038 0.74254513 0.7601437  0.74679154 0.75567061 0.70748502]\n",
      "[0.61707854 0.66061312 0.70622617 0.68028426 0.66948384 0.73624182\n",
      " 0.75711209 0.72829872 0.69957542 0.66187704 0.66976404 0.65643322]\n",
      "[0.62805635 0.62381339 0.6124692  0.6287266  0.68727225 0.73125267\n",
      " 0.70476258 0.71562541 0.76366526 0.79446119 0.79215831 0.69463056]\n",
      "[0.66971874 0.70654839 0.7326203  0.73304731 0.75161099 0.79267627\n",
      " 0.79273391 0.8005569  0.82235831 0.79070383 0.77692634 0.73258257]\n",
      "[0.65999025 0.62266642 0.6142543  0.66844511 0.68171179 0.69171286\n",
      " 0.67499059 0.68640023 0.7406894  0.74106103 0.75015205 0.7399267 ]\n",
      "[0.56575054 0.57886058 0.61756337 0.66696703 0.65750796 0.66855061\n",
      " 0.69773209 0.76640981 0.78250468 0.723436   0.67446274 0.61195952]\n",
      "[0.70712936 0.6754638  0.6318118  0.62047201 0.63726282 0.69817442\n",
      " 0.68830526 0.75830543 0.80818027 0.82102185 0.81975853 0.76666737]\n",
      "[0.7026211  0.66305083 0.70611024 0.71813649 0.75064892 0.78277975\n",
      " 0.77727664 0.80794513 0.83640015 0.82156563 0.80764991 0.80079043]\n",
      "[0.4487263  0.48036769 0.48057738 0.46169621 0.45605794 0.48623741\n",
      " 0.49055594 0.53840846 0.55093521 0.55936325 0.52729475 0.5251103 ]\n",
      "[0.51244068 0.44240949 0.4632366  0.47045183 0.50290525 0.54893959\n",
      " 0.52546537 0.54149044 0.52115285 0.50438726 0.53879124 0.57820755]\n",
      "[0.52683407 0.47263628 0.43571439 0.43819439 0.49321866 0.52484792\n",
      " 0.48424384 0.48392892 0.49616465 0.52014917 0.49776676 0.51956016]\n",
      "[0.49729365 0.43670782 0.44150186 0.47780168 0.48587418 0.51385856\n",
      " 0.51422918 0.53846604 0.526115   0.53126281 0.49800935 0.50563574]\n",
      "[0.41780129 0.41646987 0.46215212 0.44008127 0.45966992 0.47794059\n",
      " 0.52280992 0.61121589 0.59246367 0.5478369  0.52147102 0.49821234]\n",
      "[0.41384655 0.43035498 0.42965794 0.39319596 0.41028544 0.43896204\n",
      " 0.54117596 0.58271652 0.52919543 0.49193591 0.48320574 0.4562273 ]\n",
      "[0.46402889 0.48887339 0.49344704 0.46568802 0.46583605 0.51152015\n",
      " 0.59430474 0.61530596 0.56612086 0.52307683 0.53937632 0.53704029]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.38018438 0.38126591 0.42708644 0.41678768 0.43040696 0.47715336\n",
      " 0.52457583 0.57929766 0.56776291 0.53843355 0.53245533 0.50833803]\n",
      "[0.43120599 0.37436038 0.44754413 0.50903177 0.44685116 0.44272918\n",
      " 0.45088351 0.58710736 0.57600409 0.49682653 0.48515788 0.50498003]\n",
      "[0.40777501 0.41298378 0.4233321  0.41103351 0.45070893 0.49857053\n",
      " 0.5509671  0.58482903 0.56595474 0.57413131 0.51729459 0.46346626]\n",
      "[0.44588518 0.43978375 0.46776152 0.50421762 0.49740928 0.5171091\n",
      " 0.54755932 0.61129355 0.66873205 0.60533392 0.54991192 0.51674777]\n",
      "[0.37733614 0.3684921  0.44410628 0.4484522  0.43077487 0.46961343\n",
      " 0.51297075 0.62021732 0.58861363 0.55757546 0.52084315 0.48417827]\n",
      "[0.49410564 0.49850091 0.47524136 0.50286043 0.51435161 0.53033769\n",
      " 0.48066199 0.46843386 0.492661   0.49736077 0.52209014 0.52491641]\n",
      "[0.43163726 0.42044863 0.47180623 0.51096988 0.47303289 0.50438875\n",
      " 0.46774426 0.47266445 0.46041012 0.4384295  0.46889624 0.49337   ]\n",
      "[0.60393918 0.53532284 0.55073494 0.58215326 0.55044192 0.61191726\n",
      " 0.66618913 0.62675208 0.6141184  0.64315999 0.72436237 0.7717461 ]\n",
      "[0.42165068 0.46390328 0.46154624 0.50067466 0.55344504 0.505207\n",
      " 0.5277133  0.55424762 0.50233376 0.47081044 0.50386423 0.44718713]\n",
      "[0.46729305 0.45067433 0.42847556 0.44302899 0.47456849 0.51241434\n",
      " 0.50424105 0.50670093 0.53764796 0.53623217 0.52375835 0.53160822]\n",
      "[0.52081954 0.48780748 0.49470058 0.46080717 0.40154085 0.44840422\n",
      " 0.5017342  0.56126511 0.61841404 0.60619408 0.59382814 0.56661791]\n",
      "[0.47694445 0.45789197 0.46059033 0.496025   0.53601199 0.53953636\n",
      " 0.56743366 0.56982958 0.52295023 0.5466761  0.5586313  0.52281338]\n",
      "[0.53174508 0.5494349  0.64298904 0.67289275 0.67255533 0.68079829\n",
      " 0.64330178 0.55972046 0.56953698 0.61373562 0.70610219 0.63457859]\n",
      "[0.53753334 0.56196791 0.57519686 0.54827678 0.55713993 0.59924781\n",
      " 0.68760675 0.76608932 0.72655296 0.66880834 0.65876037 0.61085886]\n",
      "[0.42291194 0.43363658 0.44235593 0.4354547  0.46760902 0.49384436\n",
      " 0.56505489 0.61540055 0.59366906 0.54996961 0.55286294 0.48941621]\n",
      "[0.46371996 0.47072166 0.49186644 0.47506836 0.47174749 0.51047385\n",
      " 0.54073703 0.60321915 0.58836633 0.53355461 0.51249951 0.49362251]\n",
      "[0.4989475  0.48917326 0.51406133 0.53635168 0.50348419 0.5270794\n",
      " 0.57847732 0.70427144 0.7275666  0.64480096 0.59506154 0.52962959]\n",
      "[0.53070921 0.58879691 0.54470801 0.52206743 0.47399151 0.50674009\n",
      " 0.53110534 0.53033042 0.59309477 0.6279009  0.65360266 0.57628655]\n",
      "[0.50277138 0.51326495 0.52295262 0.50586003 0.51330829 0.56662238\n",
      " 0.63395494 0.66162711 0.68207991 0.70614249 0.67865515 0.5621475 ]\n",
      "[0.49195033 0.5051282  0.50207806 0.46833947 0.49192935 0.52208138\n",
      " 0.51052028 0.49066538 0.51098728 0.55956459 0.5573144  0.54390621]\n",
      "[0.48383844 0.47206917 0.47979838 0.47581938 0.52693611 0.52534205\n",
      " 0.52156758 0.57562137 0.57204771 0.54810405 0.50864148 0.49307826]\n",
      "[0.44088331 0.43461066 0.44104004 0.45292395 0.50845629 0.54413176\n",
      " 0.5190869  0.5512352  0.54090297 0.49645999 0.46414337 0.45949894]\n",
      "[0.4525353  0.46597326 0.49006075 0.49146286 0.52704716 0.55836761\n",
      " 0.55278563 0.49488792 0.43914914 0.43963718 0.45014518 0.42207086]\n",
      "[0.49931952 0.51770478 0.57469112 0.58185184 0.5924471  0.64947504\n",
      " 0.63855237 0.62527299 0.66171837 0.66250181 0.65542537 0.58459765]\n",
      "[0.57485616 0.58525461 0.59313422 0.60967529 0.66479754 0.70450526\n",
      " 0.67729646 0.6617285  0.67169386 0.65736288 0.66202205 0.66847736]\n",
      "[0.68546104 0.67803615 0.68564212 0.71733665 0.68443114 0.68515104\n",
      " 0.73193526 0.7243827  0.69428647 0.72136819 0.76076442 0.7351023 ]\n",
      "[0.55377382 0.52638555 0.55708748 0.59920615 0.64377904 0.6971162\n",
      " 0.75074756 0.71002638 0.66636252 0.64453441 0.62857068 0.60204989]\n",
      "[0.45070335 0.46741635 0.49174023 0.53533441 0.56343228 0.58935481\n",
      " 0.60676074 0.57700497 0.58689201 0.60515577 0.54941499 0.49851647]\n",
      "[0.60368389 0.59463876 0.60511464 0.65520883 0.66551614 0.75165045\n",
      " 0.80985427 0.77669454 0.73088717 0.707012   0.67493218 0.63791978]\n",
      "[0.58390212 0.52519631 0.49588379 0.45844817 0.4654949  0.49481493\n",
      " 0.46776396 0.47738105 0.49022219 0.49407348 0.55380565 0.56334591]\n",
      "[0.59901172 0.56273669 0.53858882 0.49663192 0.4802902  0.47889298\n",
      " 0.48863021 0.48764268 0.47491956 0.49883923 0.52690828 0.56790882]\n",
      "[0.56034625 0.51378721 0.447211   0.4488008  0.45816925 0.45825386\n",
      " 0.47077057 0.50819552 0.49885836 0.52761066 0.54816449 0.56024724]\n",
      "[0.50117671 0.4603917  0.47113261 0.45308396 0.43583423 0.44808844\n",
      " 0.45175475 0.49257103 0.52628428 0.58957899 0.62425649 0.60199887]\n",
      "[0.5893442  0.56474042 0.56985426 0.58225203 0.59216475 0.55091643\n",
      " 0.58301055 0.66876143 0.68236387 0.68413442 0.68781942 0.65914828]\n",
      "[0.5968644  0.55857635 0.5776512  0.60430533 0.58297348 0.55208498\n",
      " 0.58840954 0.62535107 0.60490239 0.59456623 0.67230666 0.67919981]\n",
      "[0.56401622 0.52414793 0.54326177 0.5572837  0.54005861 0.60368049\n",
      " 0.6839034  0.73387086 0.68642879 0.660873   0.71138519 0.68230754]\n",
      "[0.46031117 0.46407601 0.48966438 0.5197382  0.55472201 0.55772221\n",
      " 0.56124252 0.56047171 0.57372391 0.57853234 0.62297004 0.5905689 ]\n",
      "[0.47217113 0.49453247 0.49059024 0.48473671 0.48684126 0.52903962\n",
      " 0.53115863 0.53074431 0.53061444 0.55080402 0.57554203 0.53784758]\n",
      "[0.59258974 0.59734762 0.59607095 0.58078891 0.55556601 0.57344699\n",
      " 0.60456866 0.62064934 0.59918368 0.61627358 0.65667218 0.6388455 ]\n",
      "[0.46672636 0.45301822 0.48112929 0.50000858 0.50883675 0.51256323\n",
      " 0.54540575 0.56794405 0.60312694 0.61261463 0.59560657 0.57935095]\n",
      "[0.50080961 0.45241404 0.45922896 0.50359458 0.49535337 0.54922998\n",
      " 0.51963127 0.49334702 0.54688495 0.61159474 0.65041655 0.62277639]\n",
      "[0.52731013 0.52440441 0.52689284 0.5302887  0.56436527 0.57714438\n",
      " 0.57200241 0.59190124 0.56197077 0.56059611 0.57253522 0.53350681]\n",
      "[0.61959106 0.6117816  0.57452589 0.57650936 0.59743971 0.63288432\n",
      " 0.59095746 0.5614711  0.56319952 0.5683502  0.58627009 0.59625703]\n",
      "[0.46183199 0.45726293 0.50857705 0.53204304 0.57369977 0.64334017\n",
      " 0.61100805 0.61389458 0.64641643 0.64689898 0.64381933 0.57626396]\n",
      "[0.49938053 0.47430378 0.46852717 0.53144211 0.59389532 0.6660803\n",
      " 0.64384776 0.64315003 0.61485887 0.59284747 0.57238513 0.5372203 ]\n",
      "[0.49555328 0.47696719 0.48395702 0.51573479 0.47015104 0.47475746\n",
      " 0.49918824 0.4688459  0.4599185  0.48251295 0.49251166 0.54371786]\n",
      "[0.46606854 0.46408769 0.51000595 0.53499919 0.53810728 0.57797533\n",
      " 0.57801896 0.60610884 0.55877405 0.49442938 0.51731223 0.50121963]\n",
      "[0.45139584 0.44675237 0.46544605 0.48516026 0.53769553 0.53575993\n",
      " 0.54737073 0.57431155 0.57315433 0.52065945 0.52007681 0.47928563]\n",
      "[0.48690683 0.46136755 0.46574327 0.41293457 0.45709455 0.46196607\n",
      " 0.4905389  0.53834146 0.51230401 0.48044363 0.5012576  0.52864993]\n",
      "[0.52989286 0.47464523 0.44380334 0.44928512 0.5141713  0.52273935\n",
      " 0.53721112 0.59388846 0.63197851 0.60215503 0.6325869  0.60826719]\n",
      "[0.54673594 0.55107349 0.53736532 0.56499916 0.57095891 0.55701572\n",
      " 0.5940699  0.60218108 0.57785922 0.56340593 0.5949809  0.57449293]\n",
      "[0.59573007 0.55430698 0.51583803 0.54537165 0.60588491 0.64840472\n",
      " 0.67289764 0.68737221 0.66474473 0.67606175 0.69122398 0.69839519]\n",
      "[0.54643458 0.51735115 0.54813981 0.53329033 0.51750994 0.51900017\n",
      " 0.526182   0.54653949 0.57991523 0.59806281 0.63957399 0.57214636]\n",
      "[0.64204884 0.62145752 0.62918544 0.65343124 0.65260881 0.74293005\n",
      " 0.73872191 0.78239143 0.80995697 0.77235955 0.72162288 0.69494426]\n",
      "[0.61037546 0.64750022 0.65621066 0.64856064 0.68592012 0.71128136\n",
      " 0.73307133 0.79989743 0.74053162 0.7167356  0.71344727 0.64004344]\n",
      "[0.57969379 0.64938223 0.66632557 0.65482622 0.68897462 0.74125731\n",
      " 0.79337347 0.80960035 0.74738073 0.70999205 0.72596467 0.61939025]\n",
      "[0.61601061 0.67982167 0.68948871 0.65425217 0.71414578 0.74020892\n",
      " 0.77885956 0.79722047 0.7391116  0.70791411 0.72700483 0.64965481]\n",
      "[0.57961625 0.6174131  0.60249865 0.67964494 0.6957826  0.75786549\n",
      " 0.73970026 0.72540605 0.73316735 0.78073901 0.72947943 0.57636911]\n",
      "[0.58206445 0.63542879 0.61071801 0.64533329 0.65570706 0.6927495\n",
      " 0.73797703 0.73442727 0.7214272  0.73378021 0.71460313 0.57726222]\n",
      "[0.62951934 0.65662235 0.64679617 0.66042954 0.67751044 0.75692052\n",
      " 0.76191241 0.73120701 0.70885998 0.76528341 0.75220156 0.66239339]\n",
      "[0.56727004 0.64748138 0.63758022 0.61958975 0.68257427 0.69279814\n",
      " 0.73800945 0.76306283 0.74971467 0.7060982  0.73034173 0.61429864]\n",
      "[0.55326587 0.6206972  0.58779979 0.6269421  0.66718906 0.729931\n",
      " 0.77541304 0.75335407 0.74547058 0.7582112  0.72516882 0.59995174]\n",
      "[0.50929534 0.58220249 0.57200527 0.61174625 0.66328025 0.68060738\n",
      " 0.76491082 0.7521202  0.77919811 0.76546597 0.70253128 0.59921187]\n",
      "[0.51552135 0.50194299 0.59404278 0.62452966 0.63283014 0.64780819\n",
      " 0.70011199 0.7682327  0.76075351 0.75656044 0.73786873 0.68538421]\n",
      "[0.55500799 0.59792316 0.62013942 0.61877805 0.66894919 0.72211641\n",
      " 0.74863583 0.72729033 0.69982088 0.74679703 0.73562771 0.59433216]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.57576239 0.60248953 0.58943897 0.63144773 0.66925597 0.64576256\n",
      " 0.70610452 0.755952   0.80162281 0.8087762  0.70748264 0.60453707]\n",
      "[0.55468768 0.58189321 0.62660956 0.61606354 0.64806706 0.71447814\n",
      " 0.71997225 0.76284897 0.75500524 0.73156005 0.76916987 0.66078764]\n",
      "[0.57261187 0.56836456 0.57151175 0.61112535 0.64901048 0.59821445\n",
      " 0.60618824 0.64021069 0.72040927 0.8287459  0.73559296 0.67031133]\n",
      "[0.56876129 0.56267041 0.60508418 0.63919747 0.63557011 0.67651403\n",
      " 0.69040596 0.69031519 0.71409208 0.7422629  0.78706187 0.69262451]\n",
      "[0.57669944 0.58680558 0.59823996 0.63723129 0.61480033 0.64068633\n",
      " 0.66760385 0.71987039 0.7643894  0.78083313 0.77222556 0.67017484]\n",
      "[0.62907046 0.60101777 0.61071098 0.60306799 0.64559454 0.70630348\n",
      " 0.68094069 0.68278253 0.73755836 0.77534235 0.78941298 0.75200123]\n",
      "[0.6070447  0.6049211  0.58896697 0.62455744 0.66889697 0.69355267\n",
      " 0.661623   0.69151247 0.74535608 0.78486902 0.77353901 0.66820753]\n",
      "[0.58175921 0.58526134 0.62576216 0.64639634 0.66157275 0.68526882\n",
      " 0.66184223 0.68293732 0.73363459 0.76354504 0.70570892 0.63438916]\n",
      "[0.58328891 0.61099178 0.63047272 0.69338697 0.74507797 0.76123101\n",
      " 0.77808219 0.80983168 0.76995313 0.75212276 0.73729956 0.63985205]\n",
      "[0.675309   0.70933324 0.64030308 0.68197745 0.73737323 0.76508075\n",
      " 0.76998591 0.76856041 0.77286792 0.7913394  0.76052505 0.66104299]\n",
      "[0.64417559 0.65380073 0.64331287 0.68749523 0.7332781  0.78568715\n",
      " 0.77920419 0.74611908 0.81106585 0.84191048 0.7148546  0.65919644]\n",
      "[0.6097092  0.64571708 0.67200947 0.69052601 0.72848767 0.75716233\n",
      " 0.81054312 0.86062908 0.79716474 0.74513787 0.7382217  0.66796935]\n",
      "[0.57386851 0.58842653 0.63133216 0.65927237 0.67224079 0.7403025\n",
      " 0.73907477 0.72680205 0.71583372 0.74576193 0.70036197 0.58471876]\n",
      "[0.52236819 0.50474107 0.5566299  0.65491205 0.69007456 0.72384268\n",
      " 0.70072812 0.69171178 0.67412955 0.70283568 0.70089519 0.57082117]\n",
      "[0.6058706  0.56844115 0.58444667 0.65441197 0.6721682  0.69416583\n",
      " 0.71096539 0.78498429 0.79234797 0.74446607 0.6925475  0.62449807]\n",
      "[0.61072624 0.65335852 0.63164186 0.64962196 0.6800375  0.70874953\n",
      " 0.75659084 0.7900694  0.76587242 0.73831969 0.69992608 0.61235601]\n",
      "[0.55435336 0.58353865 0.58027679 0.61760736 0.68668956 0.73339224\n",
      " 0.75584048 0.74996936 0.7429117  0.7084446  0.68173611 0.60384774]\n",
      "[0.56992269 0.58294076 0.59777945 0.66297323 0.73269987 0.77708554\n",
      " 0.73843217 0.76378489 0.77733475 0.72623479 0.68054777 0.6002841 ]\n",
      "[0.53165036 0.54993457 0.57059819 0.60072112 0.65723509 0.72347558\n",
      " 0.71394181 0.74365044 0.77453333 0.78320819 0.71936494 0.63657087]\n",
      "[0.51793969 0.58133805 0.58807683 0.67129999 0.74967009 0.76655447\n",
      " 0.77128929 0.77067846 0.75919944 0.73379195 0.65339851 0.51180583]\n",
      "[0.44138652 0.46976271 0.48982981 0.56739157 0.65583843 0.71066242\n",
      " 0.68394673 0.66827446 0.74835527 0.78232437 0.67851812 0.50746101]\n",
      "[0.51144111 0.5217638  0.52776921 0.53752607 0.59874946 0.66117507\n",
      " 0.69422454 0.72042435 0.76626426 0.79320616 0.7163372  0.6063773 ]\n",
      "[0.50455624 0.53556949 0.53714287 0.57440847 0.62083435 0.69845343\n",
      " 0.74860168 0.80574673 0.82461512 0.71479547 0.62283111 0.52217603]\n",
      "[0.54056263 0.53343302 0.55387026 0.60850483 0.70919406 0.7739979\n",
      " 0.72041136 0.73715985 0.76997018 0.74565583 0.71220398 0.58316678]\n",
      "[0.62023324 0.60925752 0.54463744 0.56910878 0.62075222 0.67081779\n",
      " 0.73962069 0.78243738 0.811243   0.77108765 0.67892051 0.61969924]\n",
      "[0.5843541  0.59783489 0.62978387 0.63891989 0.64981389 0.68136388\n",
      " 0.72981119 0.78661525 0.82996517 0.79725409 0.72785258 0.63798064]\n",
      "[0.60597056 0.59945703 0.56987631 0.5832361  0.60008973 0.67774642\n",
      " 0.70982146 0.74718851 0.81483221 0.8566218  0.77875334 0.65977317]\n",
      "[0.6146     0.62304842 0.59601969 0.63155669 0.68857247 0.70242727\n",
      " 0.71033132 0.77769679 0.81140661 0.79868507 0.76836866 0.65015888]\n",
      "[0.64553648 0.65129578 0.60238808 0.61592597 0.66148585 0.69307601\n",
      " 0.72339296 0.74510705 0.751881   0.79419643 0.80748886 0.71834058]\n",
      "[0.59712654 0.57799214 0.63013721 0.64313906 0.63132304 0.68243152\n",
      " 0.73662108 0.74159563 0.71553665 0.73405784 0.74036598 0.68392354]\n",
      "[0.64841419 0.64053446 0.64336121 0.67493385 0.66262758 0.67664766\n",
      " 0.71018177 0.78125775 0.78224605 0.76985747 0.74286664 0.68408895]\n",
      "[0.63493001 0.63744402 0.64356214 0.69276732 0.6453011  0.62057781\n",
      " 0.66395235 0.72440416 0.76454341 0.77615225 0.81223738 0.74014324]\n",
      "[0.62163305 0.62810177 0.61746156 0.65302789 0.74133724 0.82364446\n",
      " 0.8226884  0.82329339 0.80696398 0.75241482 0.7185244  0.66350621]\n",
      "[0.59587979 0.5747627  0.58696651 0.68819451 0.76800281 0.8368277\n",
      " 0.81575012 0.82982284 0.82210904 0.72025818 0.67546391 0.62917292]\n",
      "[0.61318594 0.61367416 0.584216   0.61981195 0.72787797 0.80645734\n",
      " 0.77260298 0.80362684 0.80067444 0.73749423 0.70266855 0.6311034 ]\n",
      "[0.62681329 0.63719797 0.64187199 0.70615423 0.79603392 0.83728838\n",
      " 0.80982876 0.81596375 0.8002435  0.79205126 0.73820138 0.64087504]\n",
      "[0.59704745 0.61953783 0.60595113 0.66050482 0.75468773 0.81171513\n",
      " 0.80478686 0.82002974 0.80526841 0.74683678 0.71452069 0.64192331]\n",
      "[0.63049716 0.63373327 0.64568335 0.6909647  0.75763631 0.80527174\n",
      " 0.81753433 0.79529166 0.76521581 0.74697834 0.72249967 0.66542757]\n",
      "[0.58700854 0.59156013 0.62300825 0.68027198 0.7680524  0.81304687\n",
      " 0.77257591 0.79743499 0.79913568 0.73689145 0.70824009 0.63443071]\n",
      "[0.63812584 0.6197148  0.61401999 0.69685143 0.79912859 0.83002222\n",
      " 0.78096974 0.78316307 0.76425129 0.73320669 0.69146031 0.63716996]\n",
      "[0.579207   0.61811996 0.63442022 0.65110064 0.65016466 0.73655653\n",
      " 0.77620655 0.84998995 0.83237499 0.77623141 0.7251327  0.62344456]\n",
      "[0.64275604 0.66734344 0.64818287 0.66254723 0.69318432 0.74113268\n",
      " 0.75508839 0.83508104 0.83574098 0.79826367 0.77769142 0.68670011]\n",
      "[0.64163733 0.66315258 0.63680387 0.64415848 0.64600396 0.69865584\n",
      " 0.74369931 0.76386422 0.81299901 0.81873006 0.8164162  0.71104401]\n",
      "[0.63389152 0.6510523  0.65666163 0.62139779 0.63698351 0.68018132\n",
      " 0.73497707 0.80292881 0.83007675 0.8023209  0.76049316 0.73037928]\n",
      "[0.61184287 0.59228522 0.57473034 0.62278873 0.68350434 0.70739192\n",
      " 0.73070735 0.79655617 0.83973891 0.80362701 0.73377162 0.63550282]\n",
      "[0.67280471 0.67902166 0.65048939 0.64803076 0.68219095 0.70658344\n",
      " 0.72366023 0.77426183 0.80239326 0.78079802 0.78714925 0.75207174]\n",
      "[0.55209178 0.56425786 0.59053665 0.65779167 0.64704001 0.6971395\n",
      " 0.74735212 0.8190456  0.82992983 0.78265423 0.73476177 0.64128244]\n",
      "[0.67077762 0.61050862 0.603719   0.6344955  0.67269206 0.69153327\n",
      " 0.74668741 0.79807681 0.83813417 0.831981   0.79944491 0.74890709]\n",
      "[0.59556693 0.54802454 0.60386771 0.57440931 0.63514769 0.69670868\n",
      " 0.65344393 0.69135892 0.67962205 0.68330389 0.68223733 0.68868166]\n",
      "[0.54788899 0.53749228 0.55677491 0.5736441  0.66962594 0.68793899\n",
      " 0.65076202 0.70983976 0.7444858  0.70086187 0.69781899 0.6947642 ]\n",
      "[0.54964513 0.54490429 0.55636334 0.63704967 0.62336439 0.69286555\n",
      " 0.66679555 0.74871814 0.73769414 0.71406066 0.74955124 0.65752912]\n",
      "[0.4979516  0.4992699  0.53148741 0.56655324 0.62082726 0.64861417\n",
      " 0.67877424 0.77791107 0.77138907 0.69303834 0.69663632 0.63278884]\n",
      "[0.57065529 0.54516411 0.59047759 0.59733653 0.58660609 0.64956045\n",
      " 0.67518902 0.69706309 0.6884793  0.70707166 0.72217858 0.66334218]\n",
      "[0.57724816 0.59806573 0.63829243 0.64303011 0.62102693 0.64136297\n",
      " 0.69151825 0.76637411 0.72834724 0.67713201 0.69088924 0.64227951]\n",
      "[0.55790162 0.5493902  0.56929594 0.6041292  0.56708139 0.58070695\n",
      " 0.65903515 0.69701546 0.76015443 0.74210346 0.72239876 0.67708552]\n",
      "[0.52239043 0.52038991 0.59143758 0.56452405 0.58074552 0.64967686\n",
      " 0.68261367 0.7517643  0.76482272 0.72639394 0.67879027 0.65475124]\n",
      "[0.51087302 0.56553525 0.57173896 0.61611009 0.69415957 0.66217947\n",
      " 0.67117906 0.68362635 0.69211215 0.70797753 0.75994927 0.63204944]\n",
      "[0.61293697 0.56613296 0.60038739 0.61409342 0.62847847 0.6592893\n",
      " 0.66393292 0.68029869 0.66474849 0.68763244 0.76035637 0.76659697]\n",
      "[0.48638096 0.51722014 0.57190651 0.62129378 0.62346953 0.6715709\n",
      " 0.69323695 0.75662959 0.76793474 0.7116974  0.67542082 0.55948597]\n",
      "[0.53619885 0.54468584 0.57862103 0.54617453 0.58309317 0.6244666\n",
      " 0.66663533 0.7570588  0.76532841 0.70327604 0.64795661 0.61924028]\n",
      "[0.57922637 0.56647247 0.54276377 0.5440349  0.59178907 0.61807698\n",
      " 0.63944602 0.71589935 0.76884967 0.74547559 0.68645895 0.66519785]\n",
      "[0.55968928 0.62708497 0.63479561 0.54093003 0.59972578 0.67137235\n",
      " 0.697837   0.70359349 0.75690311 0.78164816 0.71030706 0.63479394]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.59003872 0.55168802 0.56258786 0.55335265 0.63255125 0.72668707\n",
      " 0.7092312  0.69722754 0.73183566 0.77264142 0.74558961 0.73136675]\n",
      "[0.60223168 0.55742127 0.59997869 0.62352854 0.63087559 0.69273472\n",
      " 0.69811088 0.71575087 0.70554149 0.71221745 0.71426678 0.69897276]\n",
      "[0.55781406 0.56802392 0.58270293 0.55358869 0.59179783 0.69740015\n",
      " 0.74658406 0.75145978 0.73328453 0.69732273 0.64405978 0.61836219]\n",
      "[0.56846517 0.61403245 0.57985425 0.58070076 0.71194023 0.72416395\n",
      " 0.69379222 0.70449841 0.702196   0.63653976 0.60781753 0.59264356]\n",
      "[0.54163039 0.57593143 0.62512887 0.60480785 0.61831373 0.74499822\n",
      " 0.77917576 0.73096806 0.68932974 0.69815123 0.66816515 0.61296749]\n",
      "[0.58194548 0.61021996 0.57125139 0.52875322 0.62921268 0.71421427\n",
      " 0.71582121 0.701509   0.69167811 0.67573082 0.63502568 0.64269233]\n",
      "[0.65510249 0.55122852 0.54652125 0.55872244 0.63476866 0.72033018\n",
      " 0.67595339 0.71128088 0.69282043 0.68097299 0.68512982 0.70418274]\n",
      "[0.54440767 0.59315926 0.49619335 0.51638299 0.57897002 0.64268988\n",
      " 0.7349692  0.69271159 0.68272692 0.68034297 0.66869962 0.58421278]\n",
      "[0.47773567 0.50735492 0.49965832 0.49834996 0.54120427 0.62159234\n",
      " 0.77829874 0.79572237 0.70367545 0.65997863 0.66522807 0.60634452]\n",
      "[0.42066002 0.50875664 0.49948478 0.51436907 0.56154686 0.63425773\n",
      " 0.75387114 0.76228285 0.74342477 0.68051827 0.70391959 0.5201661 ]\n",
      "[0.49599895 0.53891534 0.5543527  0.5119366  0.56540155 0.62222588\n",
      " 0.71448803 0.79413468 0.70225966 0.71375406 0.7078014  0.59627229]\n",
      "[0.48501733 0.50929904 0.57795882 0.55016881 0.54774779 0.59399635\n",
      " 0.65362209 0.76815516 0.7455979  0.69576609 0.70437127 0.60510683]\n",
      "[0.43707412 0.44351092 0.50503141 0.53782195 0.54620677 0.58397931\n",
      " 0.60634822 0.7312032  0.8159498  0.7362206  0.66806382 0.54687017]\n",
      "[0.50977618 0.55212414 0.55980766 0.58727998 0.58195412 0.61576849\n",
      " 0.69290739 0.7947033  0.82878417 0.71136153 0.6223923  0.50621545]\n",
      "[0.57430923 0.53013599 0.52429777 0.60624814 0.60497975 0.6766966\n",
      " 0.66198885 0.70270669 0.71920335 0.69170201 0.68846893 0.61972165]\n",
      "[0.60412294 0.66863382 0.60855013 0.59534293 0.6671313  0.68244773\n",
      " 0.71989799 0.70296246 0.66148347 0.69697011 0.72247672 0.64638257]\n",
      "[0.57371098 0.58884776 0.60887575 0.60583937 0.63299984 0.68957508\n",
      " 0.71437544 0.70592576 0.66427797 0.67709661 0.68354362 0.60588002]\n",
      "[0.55098253 0.55997407 0.51268584 0.56929374 0.65176505 0.73185593\n",
      " 0.74138927 0.69952345 0.70406103 0.68077773 0.68817168 0.58339643]\n",
      "[0.66287184 0.56223333 0.57823348 0.57669747 0.613038   0.66802984\n",
      " 0.67669392 0.70288897 0.66565281 0.71449393 0.7321189  0.78012615]\n",
      "[0.58214116 0.5907017  0.63647795 0.61291802 0.64080882 0.65937221\n",
      " 0.66472465 0.66093123 0.68938774 0.67153263 0.7318182  0.7250104 ]\n",
      "[0.54993963 0.54987353 0.53990966 0.56269133 0.61322439 0.69754153\n",
      " 0.73098016 0.68464112 0.63690996 0.65047336 0.72072804 0.64320093]\n",
      "[0.63245386 0.62839252 0.57362151 0.56499231 0.62050724 0.67613834\n",
      " 0.69977874 0.71779436 0.69547206 0.66139567 0.67671418 0.66793269]\n",
      "[0.62104219 0.51705784 0.49953261 0.54417062 0.55549419 0.53890377\n",
      " 0.57349068 0.68910867 0.74113894 0.75856256 0.70482814 0.68930286]\n",
      "[0.58686066 0.54720247 0.53891307 0.59774286 0.57124013 0.57223302\n",
      " 0.6349445  0.71039027 0.78899115 0.72968501 0.67664379 0.6707744 ]\n",
      "[0.46549141 0.47900906 0.53956652 0.55724573 0.57388616 0.616602\n",
      " 0.65063721 0.74107814 0.78533405 0.79498315 0.72393304 0.57784986]\n",
      "[0.54360336 0.51594508 0.53211188 0.53299576 0.57268143 0.59403068\n",
      " 0.64161903 0.74157035 0.80232269 0.78518492 0.74026972 0.68491125]\n",
      "[0.50322717 0.52522022 0.57019711 0.65097624 0.70405114 0.74773985\n",
      " 0.7171849  0.62789738 0.61700523 0.67207825 0.666421   0.59266508]\n",
      "[0.64406461 0.60527122 0.66639173 0.68641871 0.65531784 0.6213308\n",
      " 0.60379165 0.67855823 0.67160851 0.68843597 0.71733981 0.67588294]\n",
      "[0.60578573 0.63300747 0.69396389 0.71323329 0.67866504 0.61238813\n",
      " 0.63343233 0.63980001 0.68244314 0.72259992 0.74304199 0.68434924]\n",
      "[0.54303885 0.50965035 0.56782937 0.57847136 0.64328766 0.67195529\n",
      " 0.68909997 0.75015128 0.68295532 0.71676129 0.68715066 0.66088814]\n",
      "[0.63060802 0.59482133 0.61583972 0.62905294 0.74199849 0.75290477\n",
      " 0.66113853 0.70024002 0.73013133 0.70970982 0.71350676 0.69857985]\n",
      "[0.64888424 0.6405158  0.63651896 0.62659258 0.71102703 0.7610653\n",
      " 0.664545   0.68944609 0.68785048 0.66062969 0.67204052 0.68537468]\n",
      "[0.63686532 0.64398021 0.60977107 0.61116278 0.68047339 0.72931701\n",
      " 0.7240262  0.67133605 0.66103512 0.67867112 0.71808022 0.69486737]\n",
      "[0.61745781 0.57255584 0.5591265  0.58666056 0.69644809 0.79709017\n",
      " 0.71296746 0.70217913 0.68377978 0.6928795  0.66186595 0.63619018]\n",
      "[0.57372063 0.52862573 0.52593547 0.56372309 0.62583792 0.66904205\n",
      " 0.68776256 0.71864825 0.75185901 0.7400586  0.68810856 0.61795753]\n",
      "[0.5949738  0.57560223 0.56738979 0.61725801 0.66902161 0.68068284\n",
      " 0.65589213 0.69096464 0.69406766 0.69234246 0.68105274 0.60079277]\n",
      "[0.57062066 0.54758012 0.55074978 0.54642171 0.58841765 0.65307266\n",
      " 0.66294891 0.70128059 0.73093432 0.71914321 0.72985232 0.69139582]\n",
      "[0.51790994 0.53601152 0.5816654  0.61391222 0.64909351 0.65970582\n",
      " 0.6703881  0.74839377 0.76074564 0.68252951 0.6703462  0.60517943]\n",
      "[0.55239415 0.54002512 0.61022186 0.66844338 0.68312222 0.67511338\n",
      " 0.69882572 0.78904653 0.82726389 0.80883187 0.77202016 0.70490348]\n",
      "[0.58670104 0.61367989 0.6384536  0.66824657 0.68489039 0.69505006\n",
      " 0.7248441  0.79188168 0.80861855 0.7880398  0.76353502 0.70102942]\n",
      "[0.55970234 0.540043   0.56954098 0.61688954 0.66980219 0.72878987\n",
      " 0.75102729 0.78698856 0.78136057 0.78143364 0.76712793 0.67601544]\n",
      "[0.53217947 0.55710876 0.53777581 0.55019069 0.66791332 0.68484527\n",
      " 0.706155   0.70552349 0.75424331 0.76570165 0.74401617 0.66603041]\n",
      "[0.59150821 0.55712366 0.58454663 0.63207734 0.69355303 0.70175159\n",
      " 0.72624427 0.76785076 0.81884497 0.77285856 0.71558011 0.70717824]\n",
      "[0.55912691 0.54398221 0.56953752 0.58581901 0.67703533 0.68891466\n",
      " 0.66668141 0.71089667 0.75743586 0.8044259  0.74520558 0.66740197]\n",
      "[0.54138482 0.53500867 0.54375631 0.55772418 0.67635638 0.71303254\n",
      " 0.72403014 0.74381322 0.72683817 0.74171126 0.71200508 0.61022848]\n",
      "[0.53481066 0.55055773 0.60033029 0.62157649 0.66864842 0.71223819\n",
      " 0.7036652  0.69407296 0.71346825 0.74471664 0.68654597 0.62684834]\n",
      "[0.71206665 0.7284022  0.68863738 0.62582797 0.64340079 0.66889822\n",
      " 0.66241568 0.68778241 0.72575116 0.66424668 0.66790575 0.71038592]\n",
      "[0.66000307 0.67201501 0.68060833 0.72467488 0.75882828 0.74377441\n",
      " 0.6867063  0.67837226 0.69068289 0.65317732 0.6974737  0.71174085]\n",
      "[0.61583072 0.69077194 0.69442874 0.73962355 0.74199581 0.73583102\n",
      " 0.70123035 0.69760233 0.71469444 0.67707419 0.71717095 0.64556831]\n",
      "[0.53253448 0.56776887 0.59444195 0.63933331 0.64820766 0.64123124\n",
      " 0.5992859  0.61881924 0.61832708 0.60807037 0.63697076 0.57533073]\n",
      "[0.6539228  0.64787394 0.67493904 0.69237006 0.71280283 0.727566\n",
      " 0.74795264 0.74465692 0.75466084 0.75895625 0.79803425 0.72160053]\n",
      "[0.62156898 0.60324031 0.59483081 0.63680196 0.66042721 0.71369183\n",
      " 0.68075281 0.67384094 0.67508346 0.69923866 0.69154686 0.64754152]\n",
      "[0.58956432 0.61913341 0.61203963 0.62900281 0.65161228 0.67699862\n",
      " 0.67849207 0.70372832 0.71386242 0.70980197 0.71508288 0.64672756]\n",
      "[0.61063337 0.63175404 0.62977737 0.64314359 0.66677326 0.66474992\n",
      " 0.69187135 0.70875943 0.6941101  0.70882046 0.75825948 0.68031669]\n",
      "[0.62766951 0.62194067 0.63810426 0.71563131 0.75384319 0.7251516\n",
      " 0.67252439 0.70978791 0.71787006 0.70926172 0.71298671 0.71326029]\n",
      "[0.62904674 0.6494177  0.66166359 0.67368615 0.71828622 0.74441493\n",
      " 0.68404514 0.72789574 0.74228919 0.71230114 0.72346181 0.66515756]\n",
      "[0.67812109 0.66189331 0.65560275 0.69717532 0.7146548  0.74225235\n",
      " 0.76205343 0.75251889 0.74586624 0.71333885 0.74213791 0.75340873]\n",
      "[0.60850847 0.61373681 0.61927134 0.65052003 0.68964785 0.71739709\n",
      " 0.71958488 0.71700454 0.71779352 0.71302617 0.72826868 0.68624353]\n",
      "[0.56627041 0.57132477 0.54553956 0.53743106 0.5791108  0.63900721\n",
      " 0.58810383 0.58667821 0.62879634 0.62850988 0.62799048 0.60775763]\n",
      "[0.67924827 0.67540652 0.68208545 0.7150085  0.69835472 0.70038015\n",
      " 0.68054193 0.69436038 0.72733468 0.74853677 0.74446869 0.75378102]\n",
      "[0.51060569 0.51979792 0.5701642  0.60015738 0.59346044 0.63270128\n",
      " 0.64225388 0.69362748 0.71223217 0.62333375 0.54839712 0.53825486]\n",
      "[0.54880852 0.55558079 0.56687754 0.61564994 0.63403201 0.63688391\n",
      " 0.64224452 0.63631707 0.59843606 0.55579305 0.56521541 0.52675045]\n",
      "[0.62520754 0.63303465 0.69036669 0.69855052 0.67906618 0.66937292\n",
      " 0.65346891 0.62629777 0.61438024 0.63665503 0.67785758 0.6643368 ]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.47388759 0.50354135 0.55104017 0.52055383 0.53455251 0.51525992\n",
      " 0.47800216 0.44585606 0.45885906 0.50210011 0.48982733 0.47604918]\n",
      "[0.45063087 0.43466109 0.5054931  0.53714156 0.49788457 0.47585064\n",
      " 0.47542107 0.47634935 0.49811974 0.49085793 0.48966438 0.46623299]\n",
      "[0.55321681 0.58658177 0.6228314  0.57904738 0.55902857 0.56575203\n",
      " 0.60288239 0.61939067 0.61654139 0.61491168 0.59065944 0.58383495]\n",
      "[0.46342653 0.44025853 0.44036135 0.49162745 0.53384805 0.54084897\n",
      " 0.5074321  0.49509165 0.47399005 0.50539166 0.61432195 0.58360952]\n",
      "[0.4618063  0.45842308 0.46785262 0.51973653 0.57264525 0.59661651\n",
      " 0.5516206  0.52194929 0.52254826 0.58759272 0.66799134 0.56716806]\n",
      "[0.68601805 0.66493052 0.65780747 0.69709206 0.73703766 0.72690743\n",
      " 0.71952885 0.72293621 0.71776581 0.69714129 0.71058732 0.7496748 ]\n",
      "[0.59540921 0.55135554 0.56845266 0.63547772 0.58091301 0.55274236\n",
      " 0.57071698 0.60437912 0.56875378 0.55027974 0.56863451 0.63512874]\n",
      "[0.51995641 0.55293924 0.58483756 0.67403096 0.64806557 0.64012945\n",
      " 0.68357986 0.7055704  0.71054149 0.68986493 0.70983458 0.60206413]\n",
      "[0.63746679 0.65686536 0.66088694 0.67629433 0.67819291 0.73292851\n",
      " 0.77866489 0.77842933 0.7386809  0.67426962 0.67391497 0.66245854]\n",
      "[0.51015782 0.50263298 0.55562395 0.60326189 0.67131472 0.71130574\n",
      " 0.6587764  0.63081241 0.59608591 0.58354175 0.57891071 0.50282353]\n",
      "[0.55385441 0.59884042 0.60732681 0.67055964 0.67453682 0.70799875\n",
      " 0.77702594 0.70637381 0.67516452 0.71583301 0.71463972 0.64235073]\n",
      "[0.634547   0.6286158  0.6559884  0.68500584 0.71153694 0.69972789\n",
      " 0.66607815 0.68445486 0.7238366  0.75071752 0.76936251 0.73163635]\n",
      "[0.44103834 0.42597452 0.46568137 0.52964675 0.54550421 0.54138064\n",
      " 0.50469798 0.48966664 0.52927303 0.56663585 0.57507342 0.54952538]\n",
      "[0.64572394 0.64053351 0.62779075 0.66761726 0.70422649 0.69044459\n",
      " 0.61913258 0.62677652 0.6567623  0.70755219 0.73248351 0.74387819]\n",
      "[0.63484061 0.60557014 0.58691394 0.62831378 0.64347416 0.6237343\n",
      " 0.64498913 0.64860064 0.66744202 0.66538632 0.69841844 0.69475532]\n",
      "[0.59872305 0.66802657 0.73941976 0.75529939 0.71512854 0.66981387\n",
      " 0.66305226 0.6900174  0.67808509 0.68861532 0.69802743 0.63416666]\n",
      "[0.66376829 0.67030597 0.71522498 0.71456039 0.72079879 0.73961812\n",
      " 0.68085539 0.70852339 0.67907012 0.67128211 0.69290137 0.71794897]\n",
      "[0.65240878 0.68923795 0.69009113 0.70614195 0.71269381 0.73839766\n",
      " 0.73568964 0.72352391 0.71699339 0.72978497 0.71009833 0.67280024]\n",
      "[0.61883706 0.68050277 0.63765919 0.68148798 0.72216934 0.70375323\n",
      " 0.67475384 0.70171744 0.73329538 0.68547177 0.69690478 0.66580546]\n",
      "[0.58739072 0.60740525 0.6578269  0.69234085 0.68047827 0.66705847\n",
      " 0.65665025 0.67442501 0.66691673 0.66297829 0.6533404  0.65051043]\n",
      "[0.60248959 0.61857128 0.71347809 0.76014262 0.74023271 0.73715293\n",
      " 0.74168366 0.71401691 0.69147134 0.7117027  0.69995946 0.66844505]\n",
      "[0.64345068 0.6704343  0.72224712 0.6828689  0.68365514 0.7078132\n",
      " 0.67981964 0.69227225 0.72646302 0.73580176 0.69102693 0.68463391]\n",
      "[0.47473755 0.51737231 0.56260526 0.51393282 0.53663981 0.52815771\n",
      " 0.48606482 0.48571089 0.52327168 0.54797637 0.50119323 0.50210297]\n",
      "[0.58285272 0.61644298 0.60203081 0.64398706 0.6245209  0.71280718\n",
      " 0.73074514 0.69639736 0.75218719 0.69883662 0.62446111 0.55410993]\n",
      "[0.46852666 0.46966609 0.52478516 0.53662503 0.52925384 0.60339516\n",
      " 0.58245015 0.66993248 0.63945389 0.51989216 0.47358328 0.48479861]\n",
      "[0.53357935 0.59338826 0.62751621 0.63505906 0.69796735 0.74335086\n",
      " 0.74282354 0.77929455 0.7756238  0.69918495 0.65315259 0.55387014]\n",
      "[0.40387046 0.43338439 0.47162968 0.51284647 0.5718134  0.66135567\n",
      " 0.61456025 0.58698231 0.54671431 0.49589097 0.46547449 0.43822765]\n",
      "[0.63797611 0.60147965 0.60768664 0.69049019 0.7168802  0.68136328\n",
      " 0.69940406 0.66579437 0.63289058 0.62836462 0.61075306 0.62590349]\n",
      "[0.70737988 0.70691586 0.67249811 0.69445169 0.73137861 0.74668491\n",
      " 0.76536924 0.77145523 0.78249747 0.76984733 0.75318372 0.71752197]\n",
      "[0.62681776 0.66530603 0.63588929 0.66444421 0.71311462 0.71188313\n",
      " 0.76990509 0.76244676 0.73375142 0.70496005 0.71092641 0.64525414]\n",
      "[0.68542671 0.65519023 0.68538374 0.75146157 0.77632082 0.74132973\n",
      " 0.77551579 0.79239255 0.74800414 0.72397888 0.76233476 0.71276122]\n",
      "[0.52290624 0.52061445 0.53487265 0.60072511 0.65836525 0.6729942\n",
      " 0.62728643 0.61202639 0.59700251 0.60478771 0.58082902 0.57626384]\n",
      "[0.53941697 0.54484838 0.55714333 0.59425569 0.65093458 0.75678641\n",
      " 0.72322631 0.65904057 0.65081173 0.66421711 0.64660209 0.61658037]\n",
      "[0.54534853 0.53741306 0.58225173 0.67108035 0.72787178 0.71146727\n",
      " 0.70098525 0.69955462 0.68934906 0.68509483 0.68369317 0.61892062]\n",
      "[0.47341606 0.47073197 0.48592639 0.55511594 0.62334955 0.62868536\n",
      " 0.61738873 0.60693926 0.5870294  0.56432444 0.54109377 0.50524396]\n",
      "[0.63621217 0.64034325 0.6943838  0.69103664 0.7070536  0.75373566\n",
      " 0.70964855 0.70205432 0.71238017 0.72245735 0.70793205 0.66232789]\n",
      "[0.66403055 0.70757031 0.73113114 0.71824384 0.72366959 0.71664256\n",
      " 0.75204581 0.77600956 0.81022167 0.76158839 0.74464178 0.70572686]\n",
      "[0.6264376  0.68679374 0.74656165 0.83691323 0.83723915 0.7753191\n",
      " 0.73873812 0.74025357 0.72828168 0.6900658  0.72999626 0.6759209 ]\n",
      "[0.64259332 0.68432796 0.74169958 0.78531361 0.79325765 0.76316142\n",
      " 0.74172473 0.76611716 0.76783746 0.74639714 0.71996552 0.70512128]\n",
      "[0.64557016 0.62709719 0.64880323 0.70756292 0.77433652 0.82239193\n",
      " 0.71835822 0.70630825 0.73401552 0.73538959 0.69545537 0.70250601]\n",
      "[0.62638068 0.62515634 0.66201192 0.71186125 0.7450521  0.7765407\n",
      " 0.71786404 0.71019602 0.7391212  0.73259592 0.75653517 0.71164078]\n",
      "[0.58143091 0.59160125 0.60217959 0.6533829  0.73055941 0.76407033\n",
      " 0.63154942 0.608307   0.6425373  0.63499022 0.63804001 0.61512053]\n",
      "[0.63275915 0.67584199 0.74614012 0.78772843 0.79839939 0.76762635\n",
      " 0.7131049  0.72238386 0.71957767 0.701949   0.72584569 0.67895007]\n",
      "[0.49498731 0.44949976 0.42531893 0.44979838 0.4357554  0.42549813\n",
      " 0.44430199 0.51348841 0.5677141  0.59788746 0.59031105 0.51640707]\n",
      "[0.54316789 0.5143137  0.47518611 0.49955645 0.50131965 0.5006814\n",
      " 0.51664907 0.57111299 0.64003974 0.63241839 0.61756206 0.57349247]\n",
      "[0.50747979 0.47627002 0.51144779 0.52408969 0.47968528 0.51630151\n",
      " 0.52755046 0.50292432 0.59208101 0.58482897 0.60868275 0.57791722]\n",
      "[0.4981097  0.52072859 0.53184766 0.51984352 0.49558762 0.53177798\n",
      " 0.53645205 0.5648483  0.61739981 0.60195863 0.68053788 0.61385208]\n",
      "[0.43102837 0.44387725 0.55657852 0.55216461 0.48242494 0.48153704\n",
      " 0.56700712 0.69180363 0.65613222 0.56384653 0.50164884 0.44462752]\n",
      "[0.42018136 0.43013057 0.4757528  0.44564542 0.40658188 0.3937929\n",
      " 0.4086962  0.53978914 0.58464807 0.55728585 0.50213337 0.44054821]\n",
      "[0.50208986 0.47278833 0.4583835  0.51610374 0.61183596 0.59804118\n",
      " 0.56399935 0.59253627 0.72533923 0.86565095 0.72504711 0.5929752 ]\n",
      "[0.50109744 0.46102405 0.50526464 0.50906503 0.53664565 0.53112221\n",
      " 0.56002212 0.73008627 0.77203637 0.69552398 0.6433441  0.59904462]\n",
      "[0.45641878 0.49040124 0.5375942  0.46163177 0.45307574 0.45584401\n",
      " 0.48408648 0.63947821 0.64253873 0.57973564 0.51553804 0.4767932 ]\n",
      "[0.41303331 0.42355984 0.45083719 0.4437319  0.45617759 0.42725104\n",
      " 0.4401817  0.5285033  0.53071958 0.58241254 0.52165091 0.43645605]\n",
      "[0.49509588 0.46126002 0.48844725 0.50643492 0.47130617 0.4954682\n",
      " 0.56840372 0.64077586 0.71316522 0.61219823 0.57212383 0.5510729 ]\n",
      "[0.53107417 0.52737194 0.56516981 0.53742683 0.51080012 0.59557825\n",
      " 0.66133034 0.71240044 0.71086484 0.62374455 0.63786811 0.59159428]\n",
      "[0.47396916 0.47327498 0.52543014 0.5310517  0.48087117 0.50861758\n",
      " 0.55936271 0.54536331 0.57415158 0.56028056 0.61722964 0.53106034]\n",
      "[0.487578   0.49763101 0.53086984 0.51910216 0.49050853 0.51267707\n",
      " 0.55466944 0.54275233 0.59366888 0.59679055 0.63081753 0.57395083]\n",
      "[0.5397352  0.51447284 0.52145004 0.53231996 0.49089614 0.52892929\n",
      " 0.55067974 0.59828305 0.57013798 0.59139973 0.62431681 0.60181057]\n",
      "[0.60232556 0.53587025 0.50226569 0.56016952 0.53957218 0.51632011\n",
      " 0.54554445 0.58412361 0.64110655 0.61425656 0.53319693 0.57995701]\n",
      "[0.56008929 0.62085932 0.59100658 0.55067265 0.53288114 0.60353714\n",
      " 0.55427593 0.52997929 0.60893029 0.6098029  0.61838889 0.57798839]\n",
      "[0.56178898 0.46481079 0.44118834 0.53207874 0.50277495 0.47074851\n",
      " 0.5127511  0.53580368 0.58187383 0.52150518 0.51993412 0.53663081]\n",
      "[0.54739857 0.553298   0.55262399 0.54721183 0.5432846  0.51753151\n",
      " 0.51890045 0.58048058 0.57610953 0.55411881 0.55802411 0.54022533]\n",
      "[0.54358298 0.48328882 0.47029242 0.48358169 0.54439062 0.63995928\n",
      " 0.62675434 0.63054341 0.53914201 0.46088755 0.48421329 0.53748178]\n",
      "[0.53347355 0.53205335 0.49020663 0.44212702 0.45524722 0.50121164\n",
      " 0.65091735 0.64280009 0.5665434  0.5731526  0.59483731 0.54683971]\n"
     ]
    },
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "[0.43285865 0.38481021 0.46049687 0.47176099 0.42780188 0.41910955\n",
      " 0.5080502  0.68164837 0.71267861 0.60548711 0.53529418 0.49008164]\n",
      "[0.44336054 0.45390695 0.48934594 0.44064698 0.43558517 0.46321636\n",
      " 0.51148427 0.63235545 0.56552058 0.50844318 0.51522273 0.48946813]\n",
      "[0.51357001 0.48090708 0.47282341 0.52945656 0.53516781 0.52839375\n",
      " 0.53286207 0.58215833 0.65543908 0.62240452 0.599244   0.56996322]\n",
      "[0.62046576 0.63082463 0.61459768 0.57972914 0.56826234 0.62414688\n",
      " 0.64103043 0.6423893  0.66445851 0.63106352 0.64299661 0.63473839]\n",
      "[0.63806993 0.58252144 0.53972799 0.5470311  0.5440414  0.56963265\n",
      " 0.60472792 0.61144584 0.60644442 0.63646579 0.66665769 0.66616052]\n",
      "[0.66482639 0.60815841 0.51403826 0.51711053 0.51499248 0.5890345\n",
      " 0.61013508 0.61738694 0.60587031 0.58360398 0.6269781  0.65856636]\n",
      "[0.56996948 0.49283612 0.42336458 0.42563617 0.43448171 0.46511111\n",
      " 0.52408999 0.51871514 0.47879604 0.48238802 0.53641981 0.60398263]\n",
      "[0.51807743 0.4978869  0.44222555 0.42721584 0.43076718 0.47813877\n",
      " 0.48743349 0.47891682 0.49638924 0.47685021 0.52515    0.57709402]\n",
      "[0.53424937 0.57582748 0.54389721 0.51429874 0.48894626 0.49235007\n",
      " 0.50767958 0.50150305 0.5568915  0.54777765 0.56893569 0.56282711]\n",
      "[0.54973763 0.47895041 0.49250433 0.57231808 0.54666144 0.57875955\n",
      " 0.65773141 0.59526587 0.6480059  0.64067525 0.5925526  0.60646182]\n",
      "[0.57334512 0.55243427 0.49627465 0.47199455 0.53225601 0.59607047\n",
      " 0.6195823  0.69028413 0.65353972 0.66370296 0.68177998 0.62404263]\n",
      "[0.52737528 0.58411068 0.55348736 0.47940069 0.46421716 0.47812158\n",
      " 0.49884096 0.51049834 0.52948612 0.52046049 0.53464383 0.54631263]\n",
      "[0.56218886 0.58917922 0.49492428 0.44101039 0.44174582 0.48865989\n",
      " 0.4812535  0.47687536 0.52771318 0.48418236 0.50281692 0.54153359]\n",
      "[0.62677485 0.6265834  0.56850368 0.56325465 0.52231002 0.56501043\n",
      " 0.61768574 0.56973755 0.55073225 0.55132556 0.59570026 0.68440789]\n",
      "[0.57106936 0.56656957 0.50463569 0.53531498 0.50090379 0.53194636\n",
      " 0.55737627 0.52917051 0.55147648 0.52624834 0.53195703 0.58564067]\n",
      "[0.61022198 0.52598965 0.55813175 0.54612744 0.53719652 0.55710894\n",
      " 0.55904478 0.57698286 0.54602772 0.6018998  0.60625917 0.64278281]\n",
      "[0.45824474 0.44448194 0.45709929 0.50515121 0.50243044 0.55587161\n",
      " 0.59468341 0.63852203 0.60636461 0.55843151 0.53562909 0.49009079]\n",
      "[0.5909881  0.56088126 0.5793153  0.65111321 0.61218214 0.57141203\n",
      " 0.57515645 0.56673485 0.55474418 0.59081763 0.64550734 0.67371285]\n",
      "[0.57896888 0.55863047 0.63462019 0.70977545 0.70017833 0.632806\n",
      " 0.62168515 0.62516475 0.61054754 0.62346685 0.65782529 0.61276037]\n",
      "[0.49084541 0.48495844 0.52193952 0.53742087 0.53211856 0.49476883\n",
      " 0.47631398 0.47207886 0.47186226 0.48045638 0.51241332 0.53407556]\n",
      "[0.62677127 0.55256319 0.5532276  0.49562731 0.50598937 0.56048006\n",
      " 0.60645354 0.59312093 0.6105094  0.67583996 0.73569709 0.68938434]\n",
      "[0.51940131 0.51666951 0.5472666  0.53080755 0.45137867 0.40353549\n",
      " 0.41329166 0.44190574 0.48464209 0.50985754 0.52324796 0.53650784]\n",
      "[0.39017877 0.43062571 0.55144721 0.59321237 0.50040001 0.35891452\n",
      " 0.34207392 0.37071764 0.42096528 0.49098119 0.51345831 0.4676432 ]\n",
      "[0.4613243  0.44335365 0.44595829 0.42002413 0.43025386 0.41966552\n",
      " 0.51548862 0.51377332 0.46680912 0.45480379 0.48280346 0.47084603]\n",
      "[0.55682707 0.51723552 0.51653063 0.48695627 0.5029614  0.55806959\n",
      " 0.59866166 0.65812206 0.63889438 0.63549721 0.64913619 0.67443013]\n",
      "[0.56141156 0.5483821  0.52363461 0.51809317 0.4878715  0.52266258\n",
      " 0.61203277 0.63217908 0.6349737  0.66120183 0.6820628  0.66212136]\n",
      "[0.52087802 0.50670826 0.56046563 0.56840932 0.58318055 0.62642986\n",
      " 0.65411025 0.70718563 0.68014181 0.67491192 0.64178056 0.59044939]\n",
      "[0.6718207  0.68142867 0.62921262 0.63328642 0.65065682 0.68423641\n",
      " 0.70952988 0.73863852 0.74982834 0.71697509 0.70669699 0.69989049]\n",
      "[0.66723055 0.61284465 0.52880663 0.5004887  0.50922823 0.5607335\n",
      " 0.55999386 0.6116522  0.63693756 0.63874495 0.66611081 0.64503819]\n",
      "[0.57303327 0.5651505  0.57815045 0.60258913 0.59561872 0.61265832\n",
      " 0.60771567 0.60339016 0.65199715 0.6376785  0.68124515 0.68150592]\n",
      "[0.66138977 0.6301704  0.57299906 0.49207821 0.48296368 0.48388195\n",
      " 0.52955449 0.5779373  0.59594935 0.60179782 0.6065864  0.62000126]\n",
      "[0.49883842 0.49582058 0.41769743 0.44647652 0.46251982 0.51112109\n",
      " 0.5364027  0.5342201  0.54143566 0.50709009 0.50287616 0.47915548]\n",
      "[0.5564239  0.49441174 0.45773652 0.49543285 0.51802236 0.53631186\n",
      " 0.54325962 0.57993388 0.64846832 0.64800781 0.67778391 0.65714228]\n",
      "[0.5182851  0.52889317 0.57382315 0.59876567 0.58734006 0.60132837\n",
      " 0.5748266  0.5891183  0.56907177 0.55555791 0.61130756 0.53727871]\n",
      "[0.5989874  0.52163619 0.49167192 0.53039473 0.56011939 0.57077157\n",
      " 0.62094152 0.62388098 0.63167059 0.62778258 0.63064289 0.6341247 ]\n",
      "[0.66902173 0.68618369 0.63999361 0.61096489 0.5948379  0.56283849\n",
      " 0.56131279 0.61008722 0.61790538 0.6125353  0.6244415  0.67839438]\n",
      "[0.63274485 0.61236858 0.54801112 0.53690648 0.53750098 0.58149219\n",
      " 0.58611757 0.56129187 0.57925665 0.59365302 0.62238067 0.6332137 ]\n",
      "[0.57457691 0.54491788 0.5432694  0.53229862 0.53965676 0.59964514\n",
      " 0.62371087 0.5972535  0.56734878 0.59137255 0.60688305 0.61414635]\n",
      "[0.61741143 0.56550747 0.56870848 0.56034344 0.59349716 0.63447881\n",
      " 0.66016644 0.68985862 0.67245799 0.68776888 0.71391881 0.67929125]\n",
      "(1078, 360)\n",
      "Features extracted: 12\n"
     ]
    }
   ],
   "source": [
    "#Load the data and extract features for each sound file\n",
    "mfcc,mel,chroma = get_user_feat()\n",
    "def load_data(test_size=0.25):\n",
    "    x,y=[],[]\n",
    "    warnings.filterwarnings(\"ignore\")\n",
    "    for file in glob.glob(\"D://ravdess data//Actor_*//*.wav\"):\n",
    "        file_name=os.path.basename(file)\n",
    "        emotion=emotions[file_name.split(\"-\")[2]]\n",
    "        if emotion not in observed_emotions:\n",
    "            continue\n",
    "        feature=extract_feature(file, mfcc, chroma, mel)\n",
    "        x.append(feature)\n",
    "        y.append(emotion)\n",
    "    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)\n",
    "\n",
    "#Split the dataset\n",
    "x_train,x_test,y_train,y_test=load_data(test_size=0.25)\n",
    "\n",
    "x_train\n",
    "\n",
    "#Get the shape of the training and testing datasets\n",
    "print((x_train.shape[0], x_test.shape[0]))\n",
    "\n",
    "#Get the number of features extracted\n",
    "print(f'Features extracted: {x_train.shape[1]}')\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "metadata": {},
   "outputs": [],
   "source": [
    "#Initialize the Multi Layer Perceptron Classifier\n",
    "model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)\n",
    "\n",
    "#Train the model\n",
    "model.fit(x_train,y_train)\n",
    "\n",
    "#Predict for the test set\n",
    "y_pred=model.predict(x_test)\n",
    "\n",
    "y_pred\n",
    "\n",
    "from sklearn.metrics import accuracy_score, f1_score\n",
    "\n",
    "f1_score(y_test, y_pred,average=None)\n",
    "\n",
    "import pandas as pd\n",
    "df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})\n",
    "df.head(20)\n",
    "\n",
    "import pickle\n",
    "\n",
    "\n",
    "# Writing different model files to file\n",
    "with open( 'modelForPrediction1.sav', 'wb') as f:\n",
    "    pickle.dump(model,f)\n",
    "\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {
    "scrolled": True
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Enter the test file.... testfile04\n",
      "C:/Users/SRIDHAR/Desktop/test/testfile04.wav\n",
      "Which Feature Extraction Processes do you want to apply: \n",
      "1. MFCC \n",
      "2. MEL \n",
      "3. Chroma \n",
      "\n",
      "Enter your options(e.g. 23): 3\n",
      "Processing. Please wait.....\n",
      "Playing the audio file\n",
      "[0.63806993 0.58252144 0.53972799 0.5470311  0.5440414  0.56963265\n",
      " 0.60472792 0.61144584 0.60644442 0.63646579 0.66665769 0.66616052]\n",
      "The given test file is predicted to be  fearful\n"
     ]
    }
   ],
   "source": [
    "# Predict for user input audio files\n",
    "filename = 'modelForPrediction1.sav'\n",
    "loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage\n",
    "testpath=\"C:/Users/SRIDHAR/Desktop/test/\"\n",
    "while(True):\n",
    "    testfile = input(\"Enter the test file.... \")\n",
    "    test=testpath+testfile+\".wav\"\n",
    "    print(test)\n",
    "    mfcc,mel,chroma =get_user_feat()\n",
    "    print(\"Playing the audio file\")\n",
    "    playsound(test)\n",
    "    feature=extract_feature(test, mfcc, chroma, mel)\n",
    "    feature=feature.reshape(1,-1)\n",
    "    prediction=loaded_model.predict(feature)\n",
    "    print(\"The given test file is predicted to be \",prediction[0])\n",
    "\n",
    "feature"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": []
  },
  {
   "cell_type": "code",
   "execution_count": None,
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.10.4"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}

{'cells': [{'cell_type': 'markdown',
   'metadata': {},
   'source': ['# I. Importing the required libraries']},
  {'cell_type': 'code',
   'execution_count': 106,
   'metadata': {},
   'outputs': [{'data': {'text/html': ['        <script type="text/javascript">\n',
       "        window.PlotlyConfig = {MathJaxConfig: 'local'};\n",
       '        if (window.MathJax && window.MathJax.Hub && window.MathJax.Hub.Config) {window.MathJax.Hub.Config({SVG: {font: "STIX-Web"}});}\n',
       "        if (typeof require !== 'undefined') {\n",
       '        require.undef("plotly");\n',
       '        requirejs.config({\n',
       '            paths: {\n',
       "                'plotly': ['https://cdn.plot.ly/plotly-2.12.1.min']\n",
       '            }\n',
       '        });\n',
       "        require(['plotly'], function(Plotly) {\n",
       '            window._Plotly = Plotly;\n',
       '        });\n',
       '        }\n',
       '        </script>\n',
       '        ']},
     'me